<a href="https://colab.research.google.com/github/digitaldaimyo/ASA/blob/main/building_blocks/ASAResearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paper

In [ ]:

#@title Addressed State Attention

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Dict, Tuple

# -------------------------
# RoPE helper (rotate-half)
# -------------------------
def _rotate_half(x: torch.Tensor) -> torch.Tensor:
    x1 = x[..., ::2]
    x2 = x[..., 1::2]
    return torch.stack((-x2, x1), dim=-1).flatten(-2)

class RotaryEmbedding(nn.Module):
    def __init__(self, dim: int, base: float = 10000.0):
        super().__init__()
        assert dim % 2 == 0, "RoPE requires even dim"
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)
        self._cos_cached = None
        self._sin_cached = None
        self._t_cached = None
        self._device_cached = None

    def get_cos_sin(self, T: int, device, dtype):
        if (
            self._t_cached == T
            and self._cos_cached is not None
            and self._device_cached == device
        ):
            return self._cos_cached.to(dtype=dtype), self._sin_cached.to(dtype=dtype)

        t = torch.arange(T, device=device, dtype=self.inv_freq.dtype)
        freqs = torch.einsum("t,f->tf", t, self.inv_freq)  # [T, d/2]
        emb = torch.cat([freqs, freqs], dim=-1)            # [T, d]
        cos = emb.cos()[None, None, :, :]                  # [1,1,T,d]
        sin = emb.sin()[None, None, :, :]                  # [1,1,T,d]

        self._t_cached = T
        self._device_cached = device
        self._cos_cached = cos
        self._sin_cached = sin
        return cos.to(dtype=dtype), sin.to(dtype=dtype)

def apply_rope(x: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor) -> torch.Tensor:
    return (x * cos) + (_rotate_half(x) * sin)

# -------------------------
# ALiBi slopes helper
# -------------------------
def alibi_slopes(num_heads: int, device=None, dtype=torch.float32) -> torch.Tensor:
    def get_slopes(n):
        def power_of_2_slopes(n):
            start = 2.0 ** (-(2.0 ** -(math.log2(n) - 3)))
            ratio = start
            return [start * (ratio ** i) for i in range(n)]
        if math.log2(n).is_integer():
            return power_of_2_slopes(n)
        closest = 2 ** math.floor(math.log2(n))
        return power_of_2_slopes(closest) + get_slopes(2 * closest)[0::2][: n - closest]
    return torch.tensor(get_slopes(num_heads), device=device, dtype=dtype)  # [H]

# -------------------------
# softplus init helpers
# -------------------------
def _inv_softplus(y: torch.Tensor) -> torch.Tensor:
    return torch.log(torch.expm1(y))

# -------------------------
# Linear attention feature map (Performer-style)
# -------------------------
def phi(x: torch.Tensor) -> torch.Tensor:
    return F.elu(x) + 1.0


class AddressedStateAttention(nn.Module):
    """
    Addressed State Attention (ASA):
      - prefix-softmax WRITE into slots (O(T))
      - READ routing from tokens -> slots (softmax over slots)
      - content-conditioned READ term (gamma)
      - RoPE on write keys (geometry)
      - ALiBi bias on write logits (prefix-friendly)

    slot-space refinement:
      - causal linear attention in a low-dim slot-address coordinate space
      - produces per-token signed weights over slots
      - decoded through the same streaming slot-state basis
      - gated by learnable slotspace_gate (softplus)

    PERF (behavior-preserving):
      - Streaming prefix write states in chunks (no [B,H,K,T,d] materialization)
      - Slot-space prefix scan is chunked (exact)
    """

    def __init__(
        self,
        embed_dim: int,
        num_heads: int = 8,
        num_slots: int = 8,
        dropout: float = 0.1,

        # temperatures / numerics
        read_temperature: float = 1.0,
        write_temperature: float = 1.0,
        state_fp32: bool = True,
        slot_dropout: float = 0.0,
        normalize_k: bool = False,

        # positions (write geometry)
        use_rope_keys: bool = True,
        rope_base: float = 10000.0,

        # write bias (ALiBi)
        use_alibi_write: bool = True,
        alibi_strength_init: float = 0.1,
        learn_alibi_strength: bool = True,
        min_strength: float = 0.0,

        # content-conditioned read term
        use_content_read: bool = True,
        content_read_init: float = -4.0,
        content_read_max_gamma: float = 3.0,

        # slot-space refinement
        use_slotspace_refine: bool = True,
        slotspace_dim: int = 32,
        slotspace_gate_init: float = -4.0,
        slotspace_dropout: float = 0.05,
        slotspace_signed_weights: bool = True,

        # RoPE in slot-space matcher (Q/K only)
        use_rope_slotspace: bool = True,
        rope_base_slotspace: float = 100000.0,

        # perf knobs (no behavior change)
        write_chunk_size: int = 128,
        slotspace_chunk_size: int = 128,
    ):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_slots = num_slots
        self.head_dim = embed_dim // num_heads

        self.dropout = nn.Dropout(dropout)

        self.read_temperature = float(read_temperature)
        self.write_temperature = float(write_temperature)
        self.state_fp32 = bool(state_fp32)
        self.slot_dropout = float(slot_dropout)
        self.normalize_k = bool(normalize_k)
        self.routing_override = None

        self.use_rope_keys = bool(use_rope_keys)
        self.use_alibi_write = bool(use_alibi_write)
        self.learn_alibi_strength = bool(learn_alibi_strength)
        self.min_strength = float(min_strength)

        self.use_content_read = bool(use_content_read)
        self.content_read_max_gamma = float(content_read_max_gamma)

        self.use_slotspace_refine = bool(use_slotspace_refine)
        self.slotspace_dim = int(slotspace_dim)
        self.slotspace_dropout = nn.Dropout(float(slotspace_dropout))
        self.slotspace_signed_weights = bool(slotspace_signed_weights)

        self.write_chunk_size = int(write_chunk_size)
        self.slotspace_chunk_size = int(slotspace_chunk_size)

        # Learned slot keys per head: [H,K,d]
        self.slot_keys = nn.Parameter(
            torch.randn(num_heads, num_slots, self.head_dim) / math.sqrt(self.head_dim)
        )

        # Projections
        self.Wk_write = nn.Linear(embed_dim, embed_dim, bias=False)
        self.Wv_write = nn.Linear(embed_dim, embed_dim, bias=False)
        self.Wq_read  = nn.Linear(embed_dim, embed_dim, bias=False)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=False)

        # RoPE (write geometry)
        self.rope = RotaryEmbedding(self.head_dim, base=rope_base) if self.use_rope_keys else None

        # ALiBi slopes (buffer)
        if self.use_alibi_write:
            self.register_buffer("_alibi_slopes", alibi_slopes(num_heads), persistent=False)  # [H]
        else:
            self.register_buffer("_alibi_slopes", torch.zeros(num_heads), persistent=False)

        # Learnable ALiBi strength (positive via softplus)
        if self.use_alibi_write and self.learn_alibi_strength:
            init = torch.tensor(float(alibi_strength_init) - self.min_strength).clamp_min(1e-8)
            self._alibi_strength_param = nn.Parameter(_inv_softplus(init))
        else:
            self._alibi_strength_param = None
            self.alibi_strength = float(alibi_strength_init)

        # Content read gamma (>=0 via softplus)
        if self.use_content_read:
            self._content_read_gamma_raw = nn.Parameter(torch.tensor(float(content_read_init)))
        else:
            self._content_read_gamma_raw = None

        # -------------------------
        # slot-space refinement
        # -------------------------
        self.use_rope_slotspace = bool(use_rope_slotspace) and bool(self.use_slotspace_refine)
        if self.use_slotspace_refine:
            self.slot_in  = nn.Linear(num_slots, self.slotspace_dim, bias=False)
            self.slot_q   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_k   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_v   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_out = nn.Linear(self.slotspace_dim, num_slots, bias=False)
            self._slotspace_gate_raw = nn.Parameter(torch.tensor(float(slotspace_gate_init)))

            if self.use_rope_slotspace:
                assert (self.slotspace_dim % 2) == 0, "use_rope_slotspace requires even slotspace_dim"
                self.rope_slotspace = RotaryEmbedding(self.slotspace_dim, base=float(rope_base_slotspace))
            else:
                self.rope_slotspace = None
        else:
            self.slot_in = None
            self.slot_q = self.slot_k = self.slot_v = None
            self.slot_out = None
            self._slotspace_gate_raw = None
            self.rope_slotspace = None

    def _alibi_strength(self, dtype, device) -> torch.Tensor:
        if not (self.use_alibi_write and self.learn_alibi_strength):
            return torch.tensor(self.alibi_strength, dtype=dtype, device=device)
        return (F.softplus(self._alibi_strength_param) + self.min_strength).to(dtype=dtype, device=device)

    def _content_read_gamma(self, dtype, device) -> torch.Tensor:
        if not self.use_content_read:
            return torch.tensor(0.0, dtype=dtype, device=device)
        g = F.softplus(self._content_read_gamma_raw)  # >= 0
        if self.content_read_max_gamma is not None and self.content_read_max_gamma > 0:
            g = g.clamp(max=self.content_read_max_gamma)
        return g.to(dtype=dtype, device=device)

    def _slotspace_gate(self, dtype, device) -> torch.Tensor:
        if not self.use_slotspace_refine:
            return torch.tensor(0.0, dtype=dtype, device=device)
        return F.softplus(self._slotspace_gate_raw).to(dtype=dtype, device=device)

    @staticmethod
    def _safe_exp_sub_max(s: torch.Tensor, m: torch.Tensor) -> torch.Tensor:
        diff = s - m
        diff = diff.masked_fill(~torch.isfinite(m), float("-inf"))
        return torch.exp(diff)

    #@torch.compile
    def forward(
        self,
        x: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        return_info: bool = False,
        routing_mode: str = "softmax",           # "softmax" | "top1" | "topk" | "external"
        routing_topk: int = 2,                   # used if routing_mode=="topk"
        read_weights_override: Optional[torch.Tensor] = None,  # [B,H,T,K] or [B,H,L,K]
        routing_noise: Optional[str] = None,     # None | "gumbel" | "gaussian"
        routing_noise_scale: float = 1.0,

    ) -> Tuple[torch.Tensor, Optional[Dict[str, torch.Tensor]]]:
        B, T, C = x.shape
        H, K, d = self.num_heads, self.num_slots, self.head_dim

        # Project (write K/V, read Q)
        k_write = self.Wk_write(x).view(B, T, H, d).transpose(1, 2)  # [B,H,T,d]
        v_write = self.Wv_write(x).view(B, T, H, d).transpose(1, 2)  # [B,H,T,d]
        q_read  = self.Wq_read(x).view(B, T, H, d).transpose(1, 2)   # [B,H,T,d]

        if self.normalize_k:
            k_write = F.normalize(k_write, dim=-1, eps=1e-8)

        # RoPE on write keys
        if self.use_rope_keys:
            cos, sin = self.rope.get_cos_sin(T, device=x.device, dtype=k_write.dtype)
            k_write = apply_rope(k_write, cos, sin)

        # Slot dropout
        slot_keys = self.slot_keys
        if self.training and self.slot_dropout > 0.0:
            drop = (torch.rand((H, K), device=x.device) < self.slot_dropout)
            slot_keys = slot_keys * (~drop).to(slot_keys.dtype).unsqueeze(-1)

        # WRITE logits: [B,H,K,T]
        write_logits_raw = torch.einsum("hkd,bhtd->bhkt", slot_keys, k_write) / math.sqrt(d)

        # Stable dtype for prefix-softmax math
        state_dtype = torch.float32 if (self.state_fp32 and x.dtype != torch.float32) else x.dtype
        write_logits = write_logits_raw.to(state_dtype)

        # Write temperature
        wtemp = max(1e-6, self.write_temperature)
        write_logits = write_logits / wtemp

        # ALiBi distance bias (prefix-friendly)
        alibi_bias_applied = None
        if self.use_alibi_write:
            strength = self._alibi_strength(dtype=state_dtype, device=x.device)  # scalar
            slopes = self._alibi_slopes.to(device=x.device, dtype=state_dtype) * strength  # [H]
            pos_i = torch.arange(T, device=x.device, dtype=state_dtype)  # [T]
            alibi_bias = slopes.view(1, H, 1, 1) * pos_i.view(1, 1, 1, T) # [1,H,1,T]
            write_logits = write_logits + alibi_bias
            alibi_bias_applied = alibi_bias

        # Key padding mask (mask positions that are padding)
        if attention_mask is not None:
            valid = attention_mask.to(dtype=torch.bool)
            write_logits = write_logits.masked_fill(~valid.view(B, 1, 1, T), float("-inf"))
        else:
            valid = None

        # =====================================================
        # STREAMING WRITE + READ (no [B,H,K,T,d] slot states)
        # =====================================================
        content_read_gamma = self._content_read_gamma(dtype=q_read.dtype, device=x.device)
        rtemp = max(1e-6, self.read_temperature)

        out_h = torch.empty((B, H, T, d), device=x.device, dtype=state_dtype)
        read_weights = torch.empty((B, H, T, K), device=x.device, dtype=q_read.dtype)

        # Optional analytics: [B,H,T,K] (later permuted to [B,H,K,T])
        slot_state_norm_t = torch.empty((B, H, T, K), device=x.device, dtype=torch.float32) if return_info else None

        denom_state = torch.zeros((B, H, K), device=x.device, dtype=state_dtype)
        numer_state = torch.zeros((B, H, K, d), device=x.device, dtype=state_dtype)
        m_state = torch.full((B, H, K), float("-inf"), device=x.device, dtype=state_dtype)

        # Optional analytics: full read logits across T (drop-in replacement)
        if return_info:
            read_logits_full = torch.empty((B, H, T, K), device=x.device, dtype=state_dtype)
            read_logits_key_full = torch.empty((B, H, T, K), device=x.device, dtype=state_dtype)
            read_logits_content_full = (
                torch.empty((B, H, T, K), device=x.device, dtype=state_dtype) if self.use_content_read else None
            )
        else:
            read_logits_full = None
            read_logits_key_full = None
            read_logits_content_full = None

        WRITE_CHUNK = self.write_chunk_size

        for t0 in range(0, T, WRITE_CHUNK):
            t1 = min(T, t0 + WRITE_CHUNK)
            L = t1 - t0

            wlog_c = write_logits[:, :, :, t0:t1]  # [B,H,K,L]

            # streaming cummax
            m_c, _ = torch.cummax(wlog_c, dim=-1)  # [B,H,K,L]
            m_new = torch.maximum(m_state.unsqueeze(-1), m_c)  # [B,H,K,L]

            # rescale old prefix state to new max reference
            scale = torch.exp(m_state.unsqueeze(-1) - m_new)  # [B,H,K,L] (exp(-inf)=0)

            denom_c = denom_state.unsqueeze(-1) * scale                  # [B,H,K,L]
            numer_c = numer_state.unsqueeze(-2) * scale.unsqueeze(-1)    # [B,H,K,L,d]

            # new weights
            w_new = self._safe_exp_sub_max(wlog_c, m_new)  # [B,H,K,L]

            # accumulate within chunk
            denom_c = denom_c + torch.cumsum(w_new, dim=-1)  # [B,H,K,L]
            v_c = v_write[:, :, t0:t1, :].to(state_dtype)    # [B,H,L,d]
            add = torch.cumsum(w_new.unsqueeze(-1) * v_c.unsqueeze(2), dim=-2)  # [B,H,K,L,d]
            numer_c = numer_c + add

            # per-token slot state for this chunk only
            slot_state_c = numer_c / denom_c.clamp_min(1e-8).unsqueeze(-1)  # [B,H,K,L,d]
            slot_state_t = slot_state_c.permute(0, 1, 3, 2, 4).contiguous() # [B,H,L,K,d]

            # READ routing logits
            q_read_c = q_read[:, :, t0:t1, :]  # [B,H,L,d]

            # base (key) term
            read_logits_key = torch.einsum("bhld,hkd->bhlk", q_read_c, slot_keys) / math.sqrt(d)

            # optional content term
            read_logits_content = None
            read_logits = read_logits_key
            if self.use_content_read:
                read_logits_content = torch.einsum(
                    "bhld,bhlkd->bhlk",
                    q_read_c,
                    slot_state_t.to(q_read_c.dtype)
                ) / math.sqrt(d)
                read_logits = read_logits + content_read_gamma.to(read_logits.dtype) * read_logits_content

            if return_info:
                read_logits_full[:, :, t0:t1, :] = read_logits.to(state_dtype)
                read_logits_key_full[:, :, t0:t1, :] = read_logits_key.to(state_dtype)
                if self.use_content_read:
                    read_logits_content_full[:, :, t0:t1, :] = read_logits_content.to(state_dtype)

            # Optional: noise on logits to probe routing stability (off by default)
            # You can plumb these as forward() kwargs; see signature snippet below.
            if routing_noise is not None:
                if routing_noise == "gumbel":
                    # gumbel(0,1) noise; scale by routing_noise_scale
                    u = torch.rand_like(read_logits)
                    g = -torch.log(-torch.log(u.clamp_min(1e-8)).clamp_min(1e-8))
                    read_logits = read_logits + routing_noise_scale * g
                elif routing_noise == "gaussian":
                    read_logits = read_logits + routing_noise_scale * torch.randn_like(read_logits)
                else:
                    raise ValueError(f"Unknown routing_noise={routing_noise}")


            if self.routing_override is not None:
                if callable(self.routing_override):
                    ctx = {
                        "t0": t0,
                        "t1": t1,
                        "B": B, "H": H, "T": T, "K": K, "d": d,
                        "rtemp": rtemp,
                        "state_dtype": state_dtype,
                        "q_read_c": q_read_c,          # [B,H,L,d]
                        "slot_keys": slot_keys,        # [H,K,d]
                        "slot_state_t": slot_state_t,  # [B,H,L,K,d] (current prefix slot states)
                        "valid": valid,                # [B,T] or None
                    }



                    # must return [B,H,L,K]
                    read_w_c = self.routing_override(
                        t0, t1, read_logits,    # [B,H,L,K] full (key + content + noise if applied),
                        read_logits_key,        # [B,H,L,K] key-only
                        read_logits_content,    # [B,H,L,K] or None
                        ctx,
                    )
                else:
                    # tensor override: [B,H,T,K]
                    read_w_c = self.routing_override[:, :, t0:t1, :].to(read_logits.dtype)

                # safety: ensure finite + normalize
                read_w_c = torch.nan_to_num(read_w_c, nan=0.0, posinf=0.0, neginf=0.0)
                read_w_c = read_w_c.clamp_min(0.0)
                read_w_c = read_w_c / read_w_c.sum(dim=-1, keepdim=True).clamp_min(1e-8)

            else:

                # Routing mode
                if routing_mode == "softmax":
                    read_w_c = torch.softmax(read_logits / rtemp, dim=-1)  # [B,H,L,K]

                elif routing_mode == "top1":
                    # hard one-hot on argmax
                    top = read_logits.argmax(dim=-1)  # [B,H,L]
                    read_w_c = F.one_hot(top, num_classes=K).to(read_logits.dtype)

                elif routing_mode == "topk":
                    kk = int(routing_topk)
                    kk = max(1, min(K, kk))
                    # mask out everything except top-k then renormalize with softmax-like
                    vals, idx = torch.topk(read_logits, k=kk, dim=-1)
                    masked = torch.full_like(read_logits, float("-inf"))
                    masked.scatter_(-1, idx, vals)
                    read_w_c = torch.softmax(masked / rtemp, dim=-1)

                elif routing_mode == "external":
                    if read_weights_override is None:
                        raise ValueError("routing_mode='external' requires read_weights_override")
                    # accept either full [B,H,T,K] or chunk [B,H,L,K]
                    if read_weights_override.shape[-2] == T:
                        read_w_c = read_weights_override[:, :, t0:t1, :]
                    else:
                        read_w_c = read_weights_override
                    # safety: renormalize
                    read_w_c = read_w_c / read_w_c.sum(dim=-1, keepdim=True).clamp_min(1e-8)

                else:
                    raise ValueError(f"Unknown routing_mode={routing_mode}")

            read_weights[:, :, t0:t1, :] = read_w_c

            # token output
            out_h[:, :, t0:t1, :] = torch.einsum(
                "bhlk,bhlkd->bhld",
                read_w_c.to(state_dtype),
                slot_state_t.to(state_dtype),
            )

            if return_info:
                slot_state_norm_t[:, :, t0:t1, :] = slot_state_t.to(torch.float32).norm(dim=-1)

            # update running states to end-of-chunk
            m_state = m_new[:, :, :, -1]
            denom_state = denom_c[:, :, :, -1]
            numer_state = numer_c[:, :, :, -1, :]

        # =====================================================
        # causal linear attention in slot-space (CHUNKED prefix scan)
        # =====================================================
        slotspace_delta_norm_mean = None
        if self.use_slotspace_refine:
            slotspace_dtype = state_dtype
            M = self.slotspace_dim

            # Encode read weights into slot-space coordinates
            u = self.slot_in(read_weights.to(slotspace_dtype))  # [B,H,T,M]
            q_s  = self.slot_q(u)
            k_s  = self.slot_k(u)
            v_s  = self.slot_v(u)

            # RoPE in slot-space matcher (Q/K only)
            if self.use_rope_slotspace:
                cos_s, sin_s = self.rope_slotspace.get_cos_sin(T, device=x.device, dtype=q_s.dtype)
                q_s = apply_rope(q_s, cos_s, sin_s)
                k_s = apply_rope(k_s, cos_s, sin_s)

            qf = phi(q_s)
            kf = phi(k_s)

            if valid is not None:
                mask = valid.view(B, 1, T, 1).to(slotspace_dtype)
                qf = qf * mask
                kf = kf * mask
                v_s = v_s * mask

            u2 = torch.empty((B, H, T, M), device=x.device, dtype=slotspace_dtype)

            S_state = torch.zeros((B, H, M, M), device=x.device, dtype=slotspace_dtype)
            Z_state = torch.zeros((B, H, M), device=x.device, dtype=slotspace_dtype)

            SS_CHUNK = self.slotspace_chunk_size

            for t0 in range(0, T, SS_CHUNK):
                t1 = min(T, t0 + SS_CHUNK)
                L = t1 - t0

                qf_c = qf[:, :, t0:t1, :]   # [B,H,L,M]
                kf_c = kf[:, :, t0:t1, :]   # [B,H,L,M]
                v_c  = v_s[:, :, t0:t1, :]  # [B,H,L,M]

                kv = torch.einsum("bhlm,bhln->bhlmn", kf_c, v_c)  # [B,H,L,M,M]
                S_c = torch.cumsum(kv, dim=2)
                Z_c = torch.cumsum(kf_c, dim=2)

                S_c = S_c + S_state.unsqueeze(2)
                Z_c = (Z_c + Z_state.unsqueeze(2)).clamp_min(1e-8)

                num = torch.einsum("bhlm,bhlmn->bhln", qf_c, S_c)
                den = torch.einsum("bhlm,bhlm->bhl", qf_c, Z_c).unsqueeze(-1).clamp_min(1e-8)
                u2[:, :, t0:t1, :] = num / den

                S_state = S_c[:, :, -1, :, :]
                Z_state = Z_c[:, :, -1, :]

            u2 = self.slotspace_dropout(u2)

            # Decode slot weights per token
            slot_w = self.slot_out(u2)  # [B,H,T,K]
            if self.slotspace_signed_weights:
                slot_w = torch.tanh(slot_w)
            else:
                slot_w = torch.softmax(slot_w, dim=-1)

            # Second streaming pass to decode slotspace contribution through slot states
            gate = self._slotspace_gate(dtype=state_dtype, device=x.device).to(state_dtype)

            denom_state = torch.zeros((B, H, K), device=x.device, dtype=state_dtype)
            numer_state = torch.zeros((B, H, K, d), device=x.device, dtype=state_dtype)
            m_state = torch.full((B, H, K), float("-inf"), device=x.device, dtype=state_dtype)

            delta_norm_sum = torch.zeros((), device=x.device, dtype=torch.float32)
            delta_norm_count = 0

            for t0 in range(0, T, WRITE_CHUNK):
                t1 = min(T, t0 + WRITE_CHUNK)
                L = t1 - t0

                wlog_c = write_logits[:, :, :, t0:t1]  # [B,H,K,L]
                m_c, _ = torch.cummax(wlog_c, dim=-1)
                m_new = torch.maximum(m_state.unsqueeze(-1), m_c)

                scale = torch.exp(m_state.unsqueeze(-1) - m_new)
                denom_c = denom_state.unsqueeze(-1) * scale
                numer_c = numer_state.unsqueeze(-2) * scale.unsqueeze(-1)

                w_new = self._safe_exp_sub_max(wlog_c, m_new)
                denom_c = denom_c + torch.cumsum(w_new, dim=-1)

                v_c = v_write[:, :, t0:t1, :].to(state_dtype)  # [B,H,L,d]
                add = torch.cumsum(w_new.unsqueeze(-1) * v_c.unsqueeze(2), dim=-2)
                numer_c = numer_c + add

                slot_state_c = numer_c / denom_c.clamp_min(1e-8).unsqueeze(-1)  # [B,H,K,L,d]
                slot_state_t = slot_state_c.permute(0, 1, 3, 2, 4).contiguous() # [B,H,L,K,d]

                slot_w_c = slot_w[:, :, t0:t1, :].to(state_dtype)  # [B,H,L,K]
                delta_c = torch.einsum("bhlk,bhlkd->bhld", slot_w_c, slot_state_t.to(state_dtype))  # [B,H,L,d]

                out_h[:, :, t0:t1, :] = out_h[:, :, t0:t1, :] + gate * delta_c

                delta_norm_sum = delta_norm_sum + delta_c.detach().to(torch.float32).norm(dim=-1).sum()
                delta_norm_count += (B * H * L)

                m_state = m_new[:, :, :, -1]
                denom_state = denom_c[:, :, :, -1]
                numer_state = numer_c[:, :, :, -1, :]

            slotspace_delta_norm_mean = (delta_norm_sum / max(1, delta_norm_count)).detach().cpu()

        # Finish
        out = out_h.transpose(1, 2).contiguous().view(B, T, C)
        out = self.out_proj(out)
        out = self.dropout(out)

        info = None
        if return_info:
            info = {
                "write_logits_raw": write_logits_raw.detach(),
                "write_logits": write_logits.detach().to(torch.float32),
                "read_weights": read_weights.detach(),
                # [B,H,K,T]
                "slot_state_norm": slot_state_norm_t.detach().permute(0, 1, 3, 2).contiguous() if slot_state_norm_t is not None else None,
                "content_read_gamma": content_read_gamma.detach().to(torch.float32).cpu(),
            }
            if alibi_bias_applied is not None:
                info["alibi_bias_applied"] = alibi_bias_applied.detach().to(torch.float32)
            if self.use_alibi_write and self.learn_alibi_strength:
                info["alibi_strength"] = self._alibi_strength(dtype=torch.float32, device=x.device).detach().cpu()
            if self.use_slotspace_refine:
                info["slotspace_gate"] = self._slotspace_gate(dtype=torch.float32, device=x.device).detach().cpu()
                info["use_rope_slotspace"] = torch.tensor(bool(self.use_rope_slotspace))
                if slotspace_delta_norm_mean is not None:
                    info["slotspace_delta_norm"] = slotspace_delta_norm_mean

            info["read_logits"] = read_logits_full.detach().to(torch.float32)
            info["read_logits_key"] = read_logits_key_full.detach().to(torch.float32)
            info["read_logits_content"] = (
                read_logits_content_full.detach().to(torch.float32) if read_logits_content_full is not None else None
            )

            info["routing_mode"] = routing_mode

        return out, info

In [ ]:

#@title Addressed State Attention (DROP-IN with slot_mask support)

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Dict, Tuple

# -------------------------
# RoPE helper (rotate-half)
# -------------------------
def _rotate_half(x: torch.Tensor) -> torch.Tensor:
    x1 = x[..., ::2]
    x2 = x[..., 1::2]
    return torch.stack((-x2, x1), dim=-1).flatten(-2)

class RotaryEmbedding(nn.Module):
    def __init__(self, dim: int, base: float = 10000.0):
        super().__init__()
        assert dim % 2 == 0, "RoPE requires even dim"
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)
        self._cos_cached = None
        self._sin_cached = None
        self._t_cached = None
        self._device_cached = None

    def get_cos_sin(self, T: int, device, dtype):
        if (
            self._t_cached == T
            and self._cos_cached is not None
            and self._device_cached == device
        ):
            return self._cos_cached.to(dtype=dtype), self._sin_cached.to(dtype=dtype)

        t = torch.arange(T, device=device, dtype=self.inv_freq.dtype)
        freqs = torch.einsum("t,f->tf", t, self.inv_freq)  # [T, d/2]
        emb = torch.cat([freqs, freqs], dim=-1)            # [T, d]
        cos = emb.cos()[None, None, :, :]                  # [1,1,T,d]
        sin = emb.sin()[None, None, :, :]                  # [1,1,T,d]

        self._t_cached = T
        self._device_cached = device
        self._cos_cached = cos
        self._sin_cached = sin
        return cos.to(dtype=dtype), sin.to(dtype=dtype)

def apply_rope(x: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor) -> torch.Tensor:
    return (x * cos) + (_rotate_half(x) * sin)

# -------------------------
# ALiBi slopes helper
# -------------------------
def alibi_slopes(num_heads: int, device=None, dtype=torch.float32) -> torch.Tensor:
    def get_slopes(n):
        def power_of_2_slopes(n):
            start = 2.0 ** (-(2.0 ** -(math.log2(n) - 3)))
            ratio = start
            return [start * (ratio ** i) for i in range(n)]
        if math.log2(n).is_integer():
            return power_of_2_slopes(n)
        closest = 2 ** math.floor(math.log2(n))
        return power_of_2_slopes(closest) + get_slopes(2 * closest)[0::2][: n - closest]
    return torch.tensor(get_slopes(num_heads), device=device, dtype=dtype)  # [H]

# -------------------------
# softplus init helpers
# -------------------------
def _inv_softplus(y: torch.Tensor) -> torch.Tensor:
    return torch.log(torch.expm1(y))

# -------------------------
# Linear attention feature map (Performer-style)
# -------------------------
def phi(x: torch.Tensor) -> torch.Tensor:
    return F.elu(x) + 1.0


class AddressedStateAttention(nn.Module):
    """
    Addressed State Attention (ASA):
      - prefix-softmax WRITE into slots (O(T))
      - READ routing from tokens -> slots (softmax over slots)
      - optional content-conditioned READ term (gamma)
      - RoPE on write keys (geometry)
      - ALiBi bias on write logits (prefix-friendly)

    Optional slot-space refinement:
      - causal linear attention in low-dim slot-address coordinate space
      - gated by learnable slotspace_gate (softplus)

    PERF:
      - Streaming prefix write states in chunks
      - Slot-space prefix scan chunked (exact)
    """

    def __init__(
        self,
        embed_dim: int,
        num_heads: int = 8,
        num_slots: int = 8,
        dropout: float = 0.1,

        read_temperature: float = 1.0,
        write_temperature: float = 1.0,
        state_fp32: bool = True,
        slot_dropout: float = 0.0,
        normalize_k: bool = False,

        use_rope_keys: bool = True,
        rope_base: float = 10000.0,

        use_alibi_write: bool = True,
        alibi_strength_init: float = 0.1,
        learn_alibi_strength: bool = True,
        min_strength: float = 0.0,

        use_content_read: bool = True,
        content_read_init: float = -4.0,
        content_read_max_gamma: float = 3.0,

        use_slotspace_refine: bool = True,
        slotspace_dim: int = 32,
        slotspace_gate_init: float = -4.0,
        slotspace_dropout: float = 0.05,
        slotspace_signed_weights: bool = True,

        use_rope_slotspace: bool = True,
        rope_base_slotspace: float = 100000.0,

        write_chunk_size: int = 128,
        slotspace_chunk_size: int = 128,
    ):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_slots = num_slots
        self.head_dim = embed_dim // num_heads

        self.dropout = nn.Dropout(dropout)

        self.read_temperature = float(read_temperature)
        self.write_temperature = float(write_temperature)
        self.state_fp32 = bool(state_fp32)
        self.slot_dropout = float(slot_dropout)
        self.normalize_k = bool(normalize_k)
        self.routing_override = None

        self.use_rope_keys = bool(use_rope_keys)
        self.use_alibi_write = bool(use_alibi_write)
        self.learn_alibi_strength = bool(learn_alibi_strength)
        self.min_strength = float(min_strength)

        self.use_content_read = bool(use_content_read)
        self.content_read_max_gamma = float(content_read_max_gamma)

        self.use_slotspace_refine = bool(use_slotspace_refine)
        self.slotspace_dim = int(slotspace_dim)
        self.slotspace_dropout = nn.Dropout(float(slotspace_dropout))
        self.slotspace_signed_weights = bool(slotspace_signed_weights)

        self.write_chunk_size = int(write_chunk_size)
        self.slotspace_chunk_size = int(slotspace_chunk_size)

        self.slot_keys = nn.Parameter(
            torch.randn(num_heads, num_slots, self.head_dim) / math.sqrt(self.head_dim)
        )

        self.Wk_write = nn.Linear(embed_dim, embed_dim, bias=False)
        self.Wv_write = nn.Linear(embed_dim, embed_dim, bias=False)
        self.Wq_read  = nn.Linear(embed_dim, embed_dim, bias=False)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=False)

        self.rope = RotaryEmbedding(self.head_dim, base=rope_base) if self.use_rope_keys else None

        if self.use_alibi_write:
            self.register_buffer("_alibi_slopes", alibi_slopes(num_heads), persistent=False)
        else:
            self.register_buffer("_alibi_slopes", torch.zeros(num_heads), persistent=False)

        if self.use_alibi_write and self.learn_alibi_strength:
            init = torch.tensor(float(alibi_strength_init) - self.min_strength).clamp_min(1e-8)
            self._alibi_strength_param = nn.Parameter(_inv_softplus(init))
        else:
            self._alibi_strength_param = None
            self.alibi_strength = float(alibi_strength_init)

        if self.use_content_read:
            self._content_read_gamma_raw = nn.Parameter(torch.tensor(float(content_read_init)))
        else:
            self._content_read_gamma_raw = None

        self.use_rope_slotspace = bool(use_rope_slotspace) and bool(self.use_slotspace_refine)
        if self.use_slotspace_refine:
            self.slot_in  = nn.Linear(num_slots, self.slotspace_dim, bias=False)
            self.slot_q   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_k   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_v   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_out = nn.Linear(self.slotspace_dim, num_slots, bias=False)
            self._slotspace_gate_raw = nn.Parameter(torch.tensor(float(slotspace_gate_init)))

            if self.use_rope_slotspace:
                assert (self.slotspace_dim % 2) == 0
                self.rope_slotspace = RotaryEmbedding(self.slotspace_dim, base=float(rope_base_slotspace))
            else:
                self.rope_slotspace = None
        else:
            self.slot_in = None
            self.slot_q = self.slot_k = self.slot_v = None
            self.slot_out = None
            self._slotspace_gate_raw = None
            self.rope_slotspace = None

    def _alibi_strength(self, dtype, device) -> torch.Tensor:
        if not (self.use_alibi_write and self.learn_alibi_strength):
            return torch.tensor(self.alibi_strength, dtype=dtype, device=device)
        return (F.softplus(self._alibi_strength_param) + self.min_strength).to(dtype=dtype, device=device)

    def _content_read_gamma(self, dtype, device) -> torch.Tensor:
        if not self.use_content_read:
            return torch.tensor(0.0, dtype=dtype, device=device)
        g = F.softplus(self._content_read_gamma_raw)
        if self.content_read_max_gamma is not None and self.content_read_max_gamma > 0:
            g = g.clamp(max=self.content_read_max_gamma)
        return g.to(dtype=dtype, device=device)

    def _slotspace_gate(self, dtype, device) -> torch.Tensor:
        if not self.use_slotspace_refine:
            return torch.tensor(0.0, dtype=dtype, device=device)
        return F.softplus(self._slotspace_gate_raw).to(dtype=dtype, device=device)

    @staticmethod
    def _safe_exp_sub_max(s: torch.Tensor, m: torch.Tensor) -> torch.Tensor:
        diff = s - m
        diff = diff.masked_fill(~torch.isfinite(m), float("-inf"))
        return torch.exp(diff)

    @staticmethod
    def _canonicalize_slot_mask(slot_mask: torch.Tensor, B: int, H: int, L: int, K: int, device, dtype):
        """
        Accepts [K] or any broadcastable shape to [B,H,L,K].
        Returns:
          m_bool: bool mask where True means "KEEP"
          m_flt : float mask (0/1) broadcasted
        """
        m = slot_mask
        if m.dim() == 1:
            m = m.view(1, 1, 1, K)
        m = m.to(device=device, dtype=dtype)
        # consider >0 as keep
        m_bool = (m > 0)
        # broadcast check happens implicitly in masked_fill
        return m_bool, m

    def forward(
        self,
        x: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        return_info: bool = False,
        routing_mode: str = "softmax",           # "softmax" | "top1" | "topk" | "external"
        routing_topk: int = 2,
        read_weights_override: Optional[torch.Tensor] = None,  # [B,H,T,K] or [B,H,L,K]
        routing_noise: Optional[str] = None,     # None | "gumbel" | "gaussian"
        routing_noise_scale: float = 1.0,
        slot_mask: Optional[torch.Tensor] = None,  # NEW: [K] or broadcastable to [B,H,L,K]
    ) -> Tuple[torch.Tensor, Optional[Dict[str, torch.Tensor]]]:
        B, T, C = x.shape
        H, K, d = self.num_heads, self.num_slots, self.head_dim

        k_write = self.Wk_write(x).view(B, T, H, d).transpose(1, 2)  # [B,H,T,d]
        v_write = self.Wv_write(x).view(B, T, H, d).transpose(1, 2)
        q_read  = self.Wq_read(x).view(B, T, H, d).transpose(1, 2)

        if self.normalize_k:
            k_write = F.normalize(k_write, dim=-1, eps=1e-8)

        if self.use_rope_keys:
            cos, sin = self.rope.get_cos_sin(T, device=x.device, dtype=k_write.dtype)
            k_write = apply_rope(k_write, cos, sin)

        slot_keys = self.slot_keys
        if self.training and self.slot_dropout > 0.0:
            drop = (torch.rand((H, K), device=x.device) < self.slot_dropout)
            slot_keys = slot_keys * (~drop).to(slot_keys.dtype).unsqueeze(-1)

        write_logits_raw = torch.einsum("hkd,bhtd->bhkt", slot_keys, k_write) / math.sqrt(d)

        state_dtype = torch.float32 if (self.state_fp32 and x.dtype != torch.float32) else x.dtype
        write_logits = write_logits_raw.to(state_dtype)

        wtemp = max(1e-6, self.write_temperature)
        write_logits = write_logits / wtemp

        alibi_bias_applied = None
        if self.use_alibi_write:
            strength = self._alibi_strength(dtype=state_dtype, device=x.device)
            slopes = self._alibi_slopes.to(device=x.device, dtype=state_dtype) * strength
            pos_i = torch.arange(T, device=x.device, dtype=state_dtype)
            alibi_bias = slopes.view(1, H, 1, 1) * pos_i.view(1, 1, 1, T)
            write_logits = write_logits + alibi_bias
            alibi_bias_applied = alibi_bias

        if attention_mask is not None:
            valid = attention_mask.to(dtype=torch.bool)
            write_logits = write_logits.masked_fill(~valid.view(B, 1, 1, T), float("-inf"))
        else:
            valid = None

        content_read_gamma = self._content_read_gamma(dtype=q_read.dtype, device=x.device)
        rtemp = max(1e-6, self.read_temperature)

        out_h = torch.empty((B, H, T, d), device=x.device, dtype=state_dtype)
        read_weights = torch.empty((B, H, T, K), device=x.device, dtype=q_read.dtype)
        slot_state_norm_t = torch.empty((B, H, T, K), device=x.device, dtype=torch.float32) if return_info else None

        denom_state = torch.zeros((B, H, K), device=x.device, dtype=state_dtype)
        numer_state = torch.zeros((B, H, K, d), device=x.device, dtype=state_dtype)
        m_state = torch.full((B, H, K), float("-inf"), device=x.device, dtype=state_dtype)

        if return_info:
            read_logits_full = torch.empty((B, H, T, K), device=x.device, dtype=state_dtype)
            read_logits_key_full = torch.empty((B, H, T, K), device=x.device, dtype=state_dtype)
            read_logits_content_full = (
                torch.empty((B, H, T, K), device=x.device, dtype=state_dtype) if self.use_content_read else None
            )
        else:
            read_logits_full = None
            read_logits_key_full = None
            read_logits_content_full = None

        WRITE_CHUNK = self.write_chunk_size

        for t0 in range(0, T, WRITE_CHUNK):
            t1 = min(T, t0 + WRITE_CHUNK)
            L = t1 - t0

            wlog_c = write_logits[:, :, :, t0:t1]  # [B,H,K,L]

            m_c, _ = torch.cummax(wlog_c, dim=-1)
            m_new = torch.maximum(m_state.unsqueeze(-1), m_c)

            scale = torch.exp(m_state.unsqueeze(-1) - m_new)

            denom_c = denom_state.unsqueeze(-1) * scale
            numer_c = numer_state.unsqueeze(-2) * scale.unsqueeze(-1)

            w_new = self._safe_exp_sub_max(wlog_c, m_new)

            denom_c = denom_c + torch.cumsum(w_new, dim=-1)
            v_c = v_write[:, :, t0:t1, :].to(state_dtype)
            add = torch.cumsum(w_new.unsqueeze(-1) * v_c.unsqueeze(2), dim=-2)
            numer_c = numer_c + add

            slot_state_c = numer_c / denom_c.clamp_min(1e-8).unsqueeze(-1)
            slot_state_t = slot_state_c.permute(0, 1, 3, 2, 4).contiguous()  # [B,H,L,K,d]

            q_read_c = q_read[:, :, t0:t1, :]  # [B,H,L,d]

            read_logits_key = torch.einsum("bhld,hkd->bhlk", q_read_c, slot_keys) / math.sqrt(d)

            read_logits_content = None
            read_logits = read_logits_key
            if self.use_content_read:
                read_logits_content = torch.einsum(
                    "bhld,bhlkd->bhlk",
                    q_read_c,
                    slot_state_t.to(q_read_c.dtype)
                ) / math.sqrt(d)
                read_logits = read_logits + content_read_gamma.to(read_logits.dtype) * read_logits_content

            # ---------- APPLY SLOT MASK (causal intervention) ----------
            if slot_mask is not None:
                keep_bool, _m = self._canonicalize_slot_mask(
                    slot_mask, B=B, H=H, L=L, K=K, device=read_logits.device, dtype=read_logits.dtype
                )
                read_logits = read_logits.masked_fill(~keep_bool, float("-inf"))
                read_logits_key = read_logits_key.masked_fill(~keep_bool, float("-inf"))
                if read_logits_content is not None:
                    read_logits_content = read_logits_content.masked_fill(~keep_bool, float("-inf"))

            if return_info:
                read_logits_full[:, :, t0:t1, :] = read_logits.to(state_dtype)
                read_logits_key_full[:, :, t0:t1, :] = read_logits_key.to(state_dtype)
                if self.use_content_read:
                    read_logits_content_full[:, :, t0:t1, :] = (
                        read_logits_content.to(state_dtype) if read_logits_content is not None else 0.0
                    )

            if routing_noise is not None:
                if routing_noise == "gumbel":
                    u = torch.rand_like(read_logits)
                    g = -torch.log(-torch.log(u.clamp_min(1e-8)).clamp_min(1e-8))
                    read_logits = read_logits + routing_noise_scale * g
                elif routing_noise == "gaussian":
                    read_logits = read_logits + routing_noise_scale * torch.randn_like(read_logits)
                else:
                    raise ValueError(f"Unknown routing_noise={routing_noise}")

            if self.routing_override is not None:
                if callable(self.routing_override):
                    ctx = {
                        "t0": t0, "t1": t1,
                        "B": B, "H": H, "T": T, "K": K, "d": d,
                        "rtemp": rtemp,
                        "state_dtype": state_dtype,
                        "q_read_c": q_read_c,
                        "slot_keys": slot_keys,
                        "slot_state_t": slot_state_t,
                        "valid": valid,
                        "slot_mask": slot_mask,  # handy
                    }
                    read_w_c = self.routing_override(
                        t0, t1, read_logits, read_logits_key, read_logits_content, ctx
                    )
                else:
                    read_w_c = self.routing_override[:, :, t0:t1, :].to(read_logits.dtype)

                read_w_c = torch.nan_to_num(read_w_c, nan=0.0, posinf=0.0, neginf=0.0)
                read_w_c = read_w_c.clamp_min(0.0)

                # if slot_mask provided, enforce it here too (in case override ignores it)
                if slot_mask is not None:
                    keep_bool, _ = self._canonicalize_slot_mask(
                        slot_mask, B=B, H=H, L=L, K=K, device=read_w_c.device, dtype=read_w_c.dtype
                    )
                    read_w_c = read_w_c * keep_bool.to(read_w_c.dtype)

                s = read_w_c.sum(dim=-1, keepdim=True)
                # avoid all-zero rows
                read_w_c = torch.where(
                    s > 0,
                    read_w_c / s.clamp_min(1e-8),
                    torch.full_like(read_w_c, 1.0 / K),
                )

            else:
                if routing_mode == "softmax":
                    read_w_c = torch.softmax(read_logits / rtemp, dim=-1)

                elif routing_mode == "top1":
                    top = read_logits.argmax(dim=-1)
                    read_w_c = F.one_hot(top, num_classes=K).to(read_logits.dtype)
                    # enforce mask (top1 can pick masked if all -inf -> argmax=0)
                    if slot_mask is not None:
                        keep_bool, _ = self._canonicalize_slot_mask(
                            slot_mask, B=B, H=H, L=L, K=K, device=read_w_c.device, dtype=read_w_c.dtype
                        )
                        read_w_c = read_w_c * keep_bool.to(read_w_c.dtype)
                        s = read_w_c.sum(dim=-1, keepdim=True)
                        read_w_c = torch.where(
                            s > 0,
                            read_w_c / s.clamp_min(1e-8),
                            torch.full_like(read_w_c, 1.0 / K),
                        )

                elif routing_mode == "topk":
                    kk = int(routing_topk)
                    kk = max(1, min(K, kk))
                    vals, idx = torch.topk(read_logits, k=kk, dim=-1)
                    masked = torch.full_like(read_logits, float("-inf"))
                    masked.scatter_(-1, idx, vals)
                    read_w_c = torch.softmax(masked / rtemp, dim=-1)

                elif routing_mode == "external":
                    if read_weights_override is None:
                        raise ValueError("routing_mode='external' requires read_weights_override")
                    if read_weights_override.shape[-2] == T:
                        read_w_c = read_weights_override[:, :, t0:t1, :]
                    else:
                        read_w_c = read_weights_override
                    read_w_c = read_w_c / read_w_c.sum(dim=-1, keepdim=True).clamp_min(1e-8)

                    if slot_mask is not None:
                        keep_bool, _ = self._canonicalize_slot_mask(
                            slot_mask, B=B, H=H, L=L, K=K, device=read_w_c.device, dtype=read_w_c.dtype
                        )
                        read_w_c = read_w_c * keep_bool.to(read_w_c.dtype)
                        s = read_w_c.sum(dim=-1, keepdim=True)
                        read_w_c = torch.where(
                            s > 0,
                            read_w_c / s.clamp_min(1e-8),
                            torch.full_like(read_w_c, 1.0 / K),
                        )

                else:
                    raise ValueError(f"Unknown routing_mode={routing_mode}")

            read_weights[:, :, t0:t1, :] = read_w_c

            out_h[:, :, t0:t1, :] = torch.einsum(
                "bhlk,bhlkd->bhld",
                read_w_c.to(state_dtype),
                slot_state_t.to(state_dtype),
            )

            if return_info:
                slot_state_norm_t[:, :, t0:t1, :] = slot_state_t.to(torch.float32).norm(dim=-1)

            m_state = m_new[:, :, :, -1]
            denom_state = denom_c[:, :, :, -1]
            numer_state = numer_c[:, :, :, -1, :]

        # =====================================================
        # Optional slot-space refinement
        # =====================================================
        slotspace_delta_norm_mean = None
        if self.use_slotspace_refine:
            slotspace_dtype = state_dtype
            M = self.slotspace_dim

            u = self.slot_in(read_weights.to(slotspace_dtype))
            q_s  = self.slot_q(u)
            k_s  = self.slot_k(u)
            v_s  = self.slot_v(u)

            if self.use_rope_slotspace:
                cos_s, sin_s = self.rope_slotspace.get_cos_sin(T, device=x.device, dtype=q_s.dtype)
                q_s = apply_rope(q_s, cos_s, sin_s)
                k_s = apply_rope(k_s, cos_s, sin_s)

            qf = phi(q_s)
            kf = phi(k_s)

            if valid is not None:
                mask = valid.view(B, 1, T, 1).to(slotspace_dtype)
                qf = qf * mask
                kf = kf * mask
                v_s = v_s * mask

            u2 = torch.empty((B, H, T, M), device=x.device, dtype=slotspace_dtype)

            S_state = torch.zeros((B, H, M, M), device=x.device, dtype=slotspace_dtype)
            Z_state = torch.zeros((B, H, M), device=x.device, dtype=slotspace_dtype)

            SS_CHUNK = self.slotspace_chunk_size

            for t0 in range(0, T, SS_CHUNK):
                t1 = min(T, t0 + SS_CHUNK)
                L = t1 - t0

                qf_c = qf[:, :, t0:t1, :]
                kf_c = kf[:, :, t0:t1, :]
                v_c  = v_s[:, :, t0:t1, :]

                kv = torch.einsum("bhlm,bhln->bhlmn", kf_c, v_c)
                S_c = torch.cumsum(kv, dim=2)
                Z_c = torch.cumsum(kf_c, dim=2)

                S_c = S_c + S_state.unsqueeze(2)
                Z_c = (Z_c + Z_state.unsqueeze(2)).clamp_min(1e-8)

                num = torch.einsum("bhlm,bhlmn->bhln", qf_c, S_c)
                den = torch.einsum("bhlm,bhlm->bhl", qf_c, Z_c).unsqueeze(-1).clamp_min(1e-8)
                u2[:, :, t0:t1, :] = num / den

                S_state = S_c[:, :, -1, :, :]
                Z_state = Z_c[:, :, -1, :]

            u2 = self.slotspace_dropout(u2)

            slot_w = self.slot_out(u2)
            if self.slotspace_signed_weights:
                slot_w = torch.tanh(slot_w)
            else:
                slot_w = torch.softmax(slot_w, dim=-1)

            gate = self._slotspace_gate(dtype=state_dtype, device=x.device).to(state_dtype)

            denom_state = torch.zeros((B, H, K), device=x.device, dtype=state_dtype)
            numer_state = torch.zeros((B, H, K, d), device=x.device, dtype=state_dtype)
            m_state = torch.full((B, H, K), float("-inf"), device=x.device, dtype=state_dtype)

            delta_norm_sum = torch.zeros((), device=x.device, dtype=torch.float32)
            delta_norm_count = 0

            for t0 in range(0, T, WRITE_CHUNK):
                t1 = min(T, t0 + WRITE_CHUNK)
                L = t1 - t0

                wlog_c = write_logits[:, :, :, t0:t1]
                m_c, _ = torch.cummax(wlog_c, dim=-1)
                m_new = torch.maximum(m_state.unsqueeze(-1), m_c)

                scale = torch.exp(m_state.unsqueeze(-1) - m_new)
                denom_c = denom_state.unsqueeze(-1) * scale
                numer_c = numer_state.unsqueeze(-2) * scale.unsqueeze(-1)

                w_new = self._safe_exp_sub_max(wlog_c, m_new)
                denom_c = denom_c + torch.cumsum(w_new, dim=-1)

                v_c = v_write[:, :, t0:t1, :].to(state_dtype)
                add = torch.cumsum(w_new.unsqueeze(-1) * v_c.unsqueeze(2), dim=-2)
                numer_c = numer_c + add

                slot_state_c = numer_c / denom_c.clamp_min(1e-8).unsqueeze(-1)
                slot_state_t = slot_state_c.permute(0, 1, 3, 2, 4).contiguous()

                slot_w_c = slot_w[:, :, t0:t1, :].to(state_dtype)

                # If slot_mask is active, suppress slotspace weights too (consistent intervention)
                if slot_mask is not None:
                    keep_bool, _ = self._canonicalize_slot_mask(
                        slot_mask, B=B, H=H, L=L, K=K, device=slot_w_c.device, dtype=slot_w_c.dtype
                    )
                    slot_w_c = slot_w_c * keep_bool.to(slot_w_c.dtype)

                delta_c = torch.einsum("bhlk,bhlkd->bhld", slot_w_c, slot_state_t.to(state_dtype))
                out_h[:, :, t0:t1, :] = out_h[:, :, t0:t1, :] + gate * delta_c

                delta_norm_sum = delta_norm_sum + delta_c.detach().to(torch.float32).norm(dim=-1).sum()
                delta_norm_count += (B * H * L)

                m_state = m_new[:, :, :, -1]
                denom_state = denom_c[:, :, :, -1]
                numer_state = numer_c[:, :, :, -1, :]

            slotspace_delta_norm_mean = (delta_norm_sum / max(1, delta_norm_count)).detach().cpu()

        out = out_h.transpose(1, 2).contiguous().view(B, T, C)
        out = self.out_proj(out)
        out = self.dropout(out)

        info = None
        if return_info:
            info = {
                "write_logits_raw": write_logits_raw.detach(),
                "write_logits": write_logits.detach().to(torch.float32),
                "read_weights": read_weights.detach(),
                "slot_state_norm": slot_state_norm_t.detach().permute(0, 1, 3, 2).contiguous() if slot_state_norm_t is not None else None,
                "content_read_gamma": content_read_gamma.detach().to(torch.float32).cpu(),
            }
            if alibi_bias_applied is not None:
                info["alibi_bias_applied"] = alibi_bias_applied.detach().to(torch.float32)
            if self.use_alibi_write and self.learn_alibi_strength:
                info["alibi_strength"] = self._alibi_strength(dtype=torch.float32, device=x.device).detach().cpu()
            if self.use_slotspace_refine:
                info["slotspace_gate"] = self._slotspace_gate(dtype=torch.float32, device=x.device).detach().cpu()
                info["use_rope_slotspace"] = torch.tensor(bool(self.use_rope_slotspace))
                if slotspace_delta_norm_mean is not None:
                    info["slotspace_delta_norm"] = slotspace_delta_norm_mean

            info["read_logits"] = read_logits_full.detach().to(torch.float32) if read_logits_full is not None else None
            info["read_logits_key"] = read_logits_key_full.detach().to(torch.float32) if read_logits_key_full is not None else None
            info["read_logits_content"] = (
                read_logits_content_full.detach().to(torch.float32) if read_logits_content_full is not None else None
            )
            info["routing_mode"] = routing_mode
            info["slot_mask_present"] = torch.tensor(slot_mask is not None)

        return out, info

In [ ]:

#@title Addressed State Attention (DROP-IN REPLACEMENT + slot_mask controls)

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Dict, Tuple

# -------------------------
# RoPE helper (rotate-half)
# -------------------------
def _rotate_half(x: torch.Tensor) -> torch.Tensor:
    x1 = x[..., ::2]
    x2 = x[..., 1::2]
    return torch.stack((-x2, x1), dim=-1).flatten(-2)

class RotaryEmbedding(nn.Module):
    def __init__(self, dim: int, base: float = 10000.0):
        super().__init__()
        assert dim % 2 == 0, "RoPE requires even dim"
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)
        self._cos_cached = None
        self._sin_cached = None
        self._t_cached = None
        self._device_cached = None

    def get_cos_sin(self, T: int, device, dtype):
        if (
            self._t_cached == T
            and self._cos_cached is not None
            and self._device_cached == device
        ):
            return self._cos_cached.to(dtype=dtype), self._sin_cached.to(dtype=dtype)

        t = torch.arange(T, device=device, dtype=self.inv_freq.dtype)
        freqs = torch.einsum("t,f->tf", t, self.inv_freq)  # [T, d/2]
        emb = torch.cat([freqs, freqs], dim=-1)            # [T, d]
        cos = emb.cos()[None, None, :, :]                  # [1,1,T,d]
        sin = emb.sin()[None, None, :, :]                  # [1,1,T,d]

        self._t_cached = T
        self._device_cached = device
        self._cos_cached = cos
        self._sin_cached = sin
        return cos.to(dtype=dtype), sin.to(dtype=dtype)

def apply_rope(x: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor) -> torch.Tensor:
    return (x * cos) + (_rotate_half(x) * sin)

# -------------------------
# ALiBi slopes helper
# -------------------------
def alibi_slopes(num_heads: int, device=None, dtype=torch.float32) -> torch.Tensor:
    def get_slopes(n):
        def power_of_2_slopes(n):
            start = 2.0 ** (-(2.0 ** -(math.log2(n) - 3)))
            ratio = start
            return [start * (ratio ** i) for i in range(n)]
        if math.log2(n).is_integer():
            return power_of_2_slopes(n)
        closest = 2 ** math.floor(math.log2(n))
        return power_of_2_slopes(closest) + get_slopes(2 * closest)[0::2][: n - closest]
    return torch.tensor(get_slopes(num_heads), device=device, dtype=dtype)  # [H]

# -------------------------
# softplus init helpers
# -------------------------
def _inv_softplus(y: torch.Tensor) -> torch.Tensor:
    return torch.log(torch.expm1(y))

# -------------------------
# Linear attention feature map (Performer-style)
# -------------------------
def phi(x: torch.Tensor) -> torch.Tensor:
    return F.elu(x) + 1.0


class AddressedStateAttention(nn.Module):
    """
    Addressed State Attention (ASA):
      - prefix-softmax WRITE into slots (O(T))
      - READ routing from tokens -> slots (softmax over slots)
      - content-conditioned READ term (gamma)
      - RoPE on write keys (geometry)
      - ALiBi bias on write logits (prefix-friendly)

    slot-space refinement:
      - causal linear attention in a low-dim slot-address coordinate space
      - produces per-token signed weights over slots
      - decoded through the same streaming slot-state basis
      - gated by learnable slotspace_gate (softplus)

    PERF (behavior-preserving):
      - Streaming prefix write states in chunks (no [B,H,K,T,d] materialization)
      - Slot-space prefix scan is chunked (exact)

    ADDITION (drop-in):
      - slot_mask controls for causal interventions:
          slot_mask: [K] float/bool where 1=keep, 0=mask
          slot_mask_where: "read" | "content_read_only" | "slotspace_only"
          slot_mask_scope: "all" | "last_pos_only"
      - also supports attribute fallback: self.slot_mask if slot_mask kwarg is None
    """

    def __init__(
        self,
        embed_dim: int,
        num_heads: int = 8,
        num_slots: int = 8,
        dropout: float = 0.1,

        # temperatures / numerics
        read_temperature: float = 1.0,
        write_temperature: float = 1.0,
        state_fp32: bool = True,
        slot_dropout: float = 0.0,
        normalize_k: bool = False,

        # positions (write geometry)
        use_rope_keys: bool = True,
        rope_base: float = 10000.0,

        # write bias (ALiBi)
        use_alibi_write: bool = True,
        alibi_strength_init: float = 0.1,
        learn_alibi_strength: bool = True,
        min_strength: float = 0.0,

        # content-conditioned read term
        use_content_read: bool = True,
        content_read_init: float = -4.0,
        content_read_max_gamma: float = 3.0,

        # slot-space refinement
        use_slotspace_refine: bool = True,
        slotspace_dim: int = 32,
        slotspace_gate_init: float = -4.0,
        slotspace_dropout: float = 0.05,
        slotspace_signed_weights: bool = True,

        # RoPE in slot-space matcher (Q/K only)
        use_rope_slotspace: bool = True,
        rope_base_slotspace: float = 100000.0,

        # perf knobs (no behavior change)
        write_chunk_size: int = 128,
        slotspace_chunk_size: int = 128,
    ):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_slots = num_slots
        self.head_dim = embed_dim // num_heads

        self.dropout = nn.Dropout(dropout)

        self.read_temperature = float(read_temperature)
        self.write_temperature = float(write_temperature)
        self.state_fp32 = bool(state_fp32)
        self.slot_dropout = float(slot_dropout)
        self.normalize_k = bool(normalize_k)
        self.routing_override = None

        # mask attribute fallback (may be set externally)
        # expected shape: [K], where 1=keep, 0=mask
        # self.slot_mask = None

        self.use_rope_keys = bool(use_rope_keys)
        self.use_alibi_write = bool(use_alibi_write)
        self.learn_alibi_strength = bool(learn_alibi_strength)
        self.min_strength = float(min_strength)

        self.use_content_read = bool(use_content_read)
        self.content_read_max_gamma = float(content_read_max_gamma)

        self.use_slotspace_refine = bool(use_slotspace_refine)
        self.slotspace_dim = int(slotspace_dim)
        self.slotspace_dropout = nn.Dropout(float(slotspace_dropout))
        self.slotspace_signed_weights = bool(slotspace_signed_weights)

        self.write_chunk_size = int(write_chunk_size)
        self.slotspace_chunk_size = int(slotspace_chunk_size)

        # Learned slot keys per head: [H,K,d]
        self.slot_keys = nn.Parameter(
            torch.randn(num_heads, num_slots, self.head_dim) / math.sqrt(self.head_dim)
        )

        # Projections
        self.Wk_write = nn.Linear(embed_dim, embed_dim, bias=False)
        self.Wv_write = nn.Linear(embed_dim, embed_dim, bias=False)
        self.Wq_read  = nn.Linear(embed_dim, embed_dim, bias=False)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=False)

        # RoPE (write geometry)
        self.rope = RotaryEmbedding(self.head_dim, base=rope_base) if self.use_rope_keys else None

        # ALiBi slopes (buffer)
        if self.use_alibi_write:
            self.register_buffer("_alibi_slopes", alibi_slopes(num_heads), persistent=False)  # [H]
        else:
            self.register_buffer("_alibi_slopes", torch.zeros(num_heads), persistent=False)

        # Learnable ALiBi strength (positive via softplus)
        if self.use_alibi_write and self.learn_alibi_strength:
            init = torch.tensor(float(alibi_strength_init) - self.min_strength).clamp_min(1e-8)
            self._alibi_strength_param = nn.Parameter(_inv_softplus(init))
        else:
            self._alibi_strength_param = None
            self.alibi_strength = float(alibi_strength_init)

        # Content read gamma (>=0 via softplus)
        if self.use_content_read:
            self._content_read_gamma_raw = nn.Parameter(torch.tensor(float(content_read_init)))
        else:
            self._content_read_gamma_raw = None

        # -------------------------
        # slot-space refinement
        # -------------------------
        self.use_rope_slotspace = bool(use_rope_slotspace) and bool(self.use_slotspace_refine)
        if self.use_slotspace_refine:
            self.slot_in  = nn.Linear(num_slots, self.slotspace_dim, bias=False)
            self.slot_q   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_k   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_v   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_out = nn.Linear(self.slotspace_dim, num_slots, bias=False)
            self._slotspace_gate_raw = nn.Parameter(torch.tensor(float(slotspace_gate_init)))

            if self.use_rope_slotspace:
                assert (self.slotspace_dim % 2) == 0, "use_rope_slotspace requires even slotspace_dim"
                self.rope_slotspace = RotaryEmbedding(self.slotspace_dim, base=float(rope_base_slotspace))
            else:
                self.rope_slotspace = None
        else:
            self.slot_in = None
            self.slot_q = self.slot_k = self.slot_v = None
            self.slot_out = None
            self._slotspace_gate_raw = None
            self.rope_slotspace = None

    def _alibi_strength(self, dtype, device) -> torch.Tensor:
        if not (self.use_alibi_write and self.learn_alibi_strength):
            return torch.tensor(self.alibi_strength, dtype=dtype, device=device)
        return (F.softplus(self._alibi_strength_param) + self.min_strength).to(dtype=dtype, device=device)

    def _content_read_gamma(self, dtype, device) -> torch.Tensor:
        if not self.use_content_read:
            return torch.tensor(0.0, dtype=dtype, device=device)
        g = F.softplus(self._content_read_gamma_raw)  # >= 0
        if self.content_read_max_gamma is not None and self.content_read_max_gamma > 0:
            g = g.clamp(max=self.content_read_max_gamma)
        return g.to(dtype=dtype, device=device)

    def _slotspace_gate(self, dtype, device) -> torch.Tensor:
        if not self.use_slotspace_refine:
            return torch.tensor(0.0, dtype=dtype, device=device)
        return F.softplus(self._slotspace_gate_raw).to(dtype=dtype, device=device)

    @staticmethod
    def _safe_exp_sub_max(s: torch.Tensor, m: torch.Tensor) -> torch.Tensor:
        diff = s - m
        diff = diff.masked_fill(~torch.isfinite(m), float("-inf"))
        return torch.exp(diff)

    def _resolve_slot_mask(
        self,
        slot_mask: Optional[torch.Tensor],
        *,
        B: int,
        H: int,
        L: int,
        K: int,
        device,
        dtype,
        scope: str,
    ) -> Optional[torch.Tensor]:
        """
        Returns expanded mask of shape [B,H,L,K] or None.
        slot_mask is expected as [K] where 1=keep, 0=mask.
        """
        if slot_mask is None:
            slot_mask = getattr(self, "slot_mask", None)
        if slot_mask is None:
            return None

        sm = slot_mask.to(device=device, dtype=dtype)
        if sm.ndim != 1 or sm.numel() != K:
            raise ValueError(f"slot_mask must be shape [K]={K}, got {tuple(sm.shape)}")

        sm = sm.view(1, 1, 1, K)  # [1,1,1,K]

        if scope == "all":
            return sm.expand(B, H, L, K)
        if scope == "last_pos_only":
            out = torch.ones((B, H, L, K), device=device, dtype=dtype)
            out[:, :, -1:, :] = sm.expand(B, H, 1, K)
            return out

        raise ValueError(f"Unknown slot_mask_scope={scope!r}")

    #@torch.compile
    def forward(
        self,
        x: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        return_info: bool = False,
        routing_mode: str = "softmax",           # "softmax" | "top1" | "topk" | "external"
        routing_topk: int = 2,                   # used if routing_mode=="topk"
        read_weights_override: Optional[torch.Tensor] = None,  # [B,H,T,K] or [B,H,L,K]
        routing_noise: Optional[str] = None,     # None | "gumbel" | "gaussian"
        routing_noise_scale: float = 1.0,

        # --- NEW (drop-in; defaults preserve old behavior) ---
        slot_mask: Optional[torch.Tensor] = None,            # [K], 1=keep, 0=mask
        slot_mask_where: str = "read",                       # "read" | "content_read_only" | "slotspace_only"
        slot_mask_scope: str = "all",                        # "all" | "last_pos_only"
    ) -> Tuple[torch.Tensor, Optional[Dict[str, torch.Tensor]]]:
        B, T, C = x.shape
        H, K, d = self.num_heads, self.num_slots, self.head_dim

        # Project (write K/V, read Q)
        k_write = self.Wk_write(x).view(B, T, H, d).transpose(1, 2)  # [B,H,T,d]
        v_write = self.Wv_write(x).view(B, T, H, d).transpose(1, 2)  # [B,H,T,d]
        q_read  = self.Wq_read(x).view(B, T, H, d).transpose(1, 2)   # [B,H,T,d]

        if self.normalize_k:
            k_write = F.normalize(k_write, dim=-1, eps=1e-8)

        # RoPE on write keys
        if self.use_rope_keys:
            cos, sin = self.rope.get_cos_sin(T, device=x.device, dtype=k_write.dtype)
            k_write = apply_rope(k_write, cos, sin)

        # Slot dropout
        slot_keys = self.slot_keys
        if self.training and self.slot_dropout > 0.0:
            drop = (torch.rand((H, K), device=x.device) < self.slot_dropout)
            slot_keys = slot_keys * (~drop).to(slot_keys.dtype).unsqueeze(-1)

        # WRITE logits: [B,H,K,T]
        write_logits_raw = torch.einsum("hkd,bhtd->bhkt", slot_keys, k_write) / math.sqrt(d)

        # Stable dtype for prefix-softmax math
        state_dtype = torch.float32 if (self.state_fp32 and x.dtype != torch.float32) else x.dtype
        write_logits = write_logits_raw.to(state_dtype)

        # Write temperature
        wtemp = max(1e-6, self.write_temperature)
        write_logits = write_logits / wtemp

        # ALiBi distance bias (prefix-friendly)
        alibi_bias_applied = None
        if self.use_alibi_write:
            strength = self._alibi_strength(dtype=state_dtype, device=x.device)  # scalar
            slopes = self._alibi_slopes.to(device=x.device, dtype=state_dtype) * strength  # [H]
            pos_i = torch.arange(T, device=x.device, dtype=state_dtype)  # [T]
            alibi_bias = slopes.view(1, H, 1, 1) * pos_i.view(1, 1, 1, T) # [1,H,1,T]
            write_logits = write_logits + alibi_bias
            alibi_bias_applied = alibi_bias

        # Key padding mask (mask positions that are padding)
        if attention_mask is not None:
            valid = attention_mask.to(dtype=torch.bool)
            write_logits = write_logits.masked_fill(~valid.view(B, 1, 1, T), float("-inf"))
        else:
            valid = None

        # =====================================================
        # STREAMING WRITE + READ (no [B,H,K,T,d] slot states)
        # =====================================================
        content_read_gamma = self._content_read_gamma(dtype=q_read.dtype, device=x.device)
        rtemp = max(1e-6, self.read_temperature)

        out_h = torch.empty((B, H, T, d), device=x.device, dtype=state_dtype)
        read_weights = torch.empty((B, H, T, K), device=x.device, dtype=q_read.dtype)

        # Optional analytics: [B,H,T,K] (later permuted to [B,H,K,T])
        slot_state_norm_t = torch.empty((B, H, T, K), device=x.device, dtype=torch.float32) if return_info else None

        denom_state = torch.zeros((B, H, K), device=x.device, dtype=state_dtype)
        numer_state = torch.zeros((B, H, K, d), device=x.device, dtype=state_dtype)
        m_state = torch.full((B, H, K), float("-inf"), device=x.device, dtype=state_dtype)

        # Optional analytics: full read logits across T
        if return_info:
            read_logits_full = torch.empty((B, H, T, K), device=x.device, dtype=state_dtype)
            read_logits_key_full = torch.empty((B, H, T, K), device=x.device, dtype=state_dtype)
            read_logits_content_full = (
                torch.empty((B, H, T, K), device=x.device, dtype=state_dtype) if self.use_content_read else None
            )
        else:
            read_logits_full = None
            read_logits_key_full = None
            read_logits_content_full = None

        WRITE_CHUNK = self.write_chunk_size

        for t0 in range(0, T, WRITE_CHUNK):
            t1 = min(T, t0 + WRITE_CHUNK)
            L = t1 - t0

            wlog_c = write_logits[:, :, :, t0:t1]  # [B,H,K,L]

            # streaming cummax
            m_c, _ = torch.cummax(wlog_c, dim=-1)  # [B,H,K,L]
            m_new = torch.maximum(m_state.unsqueeze(-1), m_c)  # [B,H,K,L]

            # rescale old prefix state to new max reference
            scale = torch.exp(m_state.unsqueeze(-1) - m_new)  # [B,H,K,L] (exp(-inf)=0)

            denom_c = denom_state.unsqueeze(-1) * scale                  # [B,H,K,L]
            numer_c = numer_state.unsqueeze(-2) * scale.unsqueeze(-1)    # [B,H,K,L,d]

            # new weights
            w_new = self._safe_exp_sub_max(wlog_c, m_new)  # [B,H,K,L]

            # accumulate within chunk
            denom_c = denom_c + torch.cumsum(w_new, dim=-1)  # [B,H,K,L]
            v_c = v_write[:, :, t0:t1, :].to(state_dtype)    # [B,H,L,d]
            add = torch.cumsum(w_new.unsqueeze(-1) * v_c.unsqueeze(2), dim=-2)  # [B,H,K,L,d]
            numer_c = numer_c + add

            # per-token slot state for this chunk only
            slot_state_c = numer_c / denom_c.clamp_min(1e-8).unsqueeze(-1)  # [B,H,K,L,d]
            slot_state_t = slot_state_c.permute(0, 1, 3, 2, 4).contiguous() # [B,H,L,K,d]

            # READ routing logits
            q_read_c = q_read[:, :, t0:t1, :]  # [B,H,L,d]

            read_logits_key = torch.einsum("bhld,hkd->bhlk", q_read_c, slot_keys) / math.sqrt(d)

            read_logits_content = None
            if self.use_content_read:
                read_logits_content = torch.einsum(
                    "bhld,bhlkd->bhlk",
                    q_read_c,
                    slot_state_t.to(q_read_c.dtype)
                ) / math.sqrt(d)

            # ---- NEW: slot masking (logit-level or deferred) ----
            sm = self._resolve_slot_mask(
                slot_mask,
                B=B, H=H, L=L, K=K,
                device=x.device,
                dtype=read_logits_key.dtype,
                scope=slot_mask_scope,
            )

            if slot_mask_where == "read":
                if sm is not None:
                    read_logits_key = read_logits_key.masked_fill(sm <= 0.0, float("-inf"))
                    if self.use_content_read and read_logits_content is not None:
                        read_logits_content = read_logits_content.masked_fill(sm <= 0.0, float("-inf"))

            elif slot_mask_where == "content_read_only":
                if sm is not None and self.use_content_read and read_logits_content is not None:
                    # Remove ONLY the content contribution for masked slots
                    read_logits_content = read_logits_content.masked_fill(sm <= 0.0, 0.0)

            elif slot_mask_where == "slotspace_only":
                pass  # applied later on slot_w
            else:
                raise ValueError(f"Unknown slot_mask_where={slot_mask_where!r}")

            # combine logits
            read_logits = read_logits_key
            if self.use_content_read and read_logits_content is not None:
                read_logits = read_logits + content_read_gamma.to(read_logits.dtype) * read_logits_content

            if return_info:
                read_logits_full[:, :, t0:t1, :] = read_logits.to(state_dtype)
                read_logits_key_full[:, :, t0:t1, :] = read_logits_key.to(state_dtype)
                if self.use_content_read and read_logits_content_full is not None:
                    read_logits_content_full[:, :, t0:t1, :] = read_logits_content.to(state_dtype)

            # Optional routing noise
            if routing_noise is not None:
                if routing_noise == "gumbel":
                    u = torch.rand_like(read_logits)
                    g = -torch.log(-torch.log(u.clamp_min(1e-8)).clamp_min(1e-8))
                    read_logits = read_logits + routing_noise_scale * g
                elif routing_noise == "gaussian":
                    read_logits = read_logits + routing_noise_scale * torch.randn_like(read_logits)
                else:
                    raise ValueError(f"Unknown routing_noise={routing_noise}")

            # Routing override / mode
            if self.routing_override is not None:
                if callable(self.routing_override):
                    ctx = {
                        "t0": t0, "t1": t1,
                        "B": B, "H": H, "T": T, "K": K, "d": d,
                        "rtemp": rtemp,
                        "state_dtype": state_dtype,
                        "q_read_c": q_read_c,          # [B,H,L,d]
                        "slot_keys": slot_keys,        # [H,K,d]
                        "slot_state_t": slot_state_t,  # [B,H,L,K,d]
                        "valid": valid,
                        "slot_mask": slot_mask,
                        "slot_mask_where": slot_mask_where,
                        "slot_mask_scope": slot_mask_scope,
                    }
                    read_w_c = self.routing_override(
                        t0, t1,
                        read_logits,
                        read_logits_key,
                        read_logits_content,
                        ctx,
                    )
                else:
                    read_w_c = self.routing_override[:, :, t0:t1, :].to(read_logits.dtype)

                read_w_c = torch.nan_to_num(read_w_c, nan=0.0, posinf=0.0, neginf=0.0)
                read_w_c = read_w_c.clamp_min(0.0)
                read_w_c = read_w_c / read_w_c.sum(dim=-1, keepdim=True).clamp_min(1e-8)

            else:
                if routing_mode == "softmax":
                    read_w_c = torch.softmax(read_logits / rtemp, dim=-1)  # [B,H,L,K]
                elif routing_mode == "top1":
                    top = read_logits.argmax(dim=-1)  # [B,H,L]
                    read_w_c = F.one_hot(top, num_classes=K).to(read_logits.dtype)
                elif routing_mode == "topk":
                    kk = max(1, min(K, int(routing_topk)))
                    vals, idx = torch.topk(read_logits, k=kk, dim=-1)
                    masked = torch.full_like(read_logits, float("-inf"))
                    masked.scatter_(-1, idx, vals)
                    read_w_c = torch.softmax(masked / rtemp, dim=-1)
                elif routing_mode == "external":
                    if read_weights_override is None:
                        raise ValueError("routing_mode='external' requires read_weights_override")
                    if read_weights_override.shape[-2] == T:
                        read_w_c = read_weights_override[:, :, t0:t1, :]
                    else:
                        read_w_c = read_weights_override
                    read_w_c = read_w_c / read_w_c.sum(dim=-1, keepdim=True).clamp_min(1e-8)
                else:
                    raise ValueError(f"Unknown routing_mode={routing_mode}")

            # If slot_mask_where == "read" but override/mode produced nonzero on masked slots,
            # enforce hard zero + renorm (keeps your masked_mass sanity check honest).
            if slot_mask_where == "read" and sm is not None:
                read_w_c = read_w_c * (sm > 0.0).to(read_w_c.dtype)
                read_w_c = read_w_c / read_w_c.sum(dim=-1, keepdim=True).clamp_min(1e-8)

            read_weights[:, :, t0:t1, :] = read_w_c

            # token output
            out_h[:, :, t0:t1, :] = torch.einsum(
                "bhlk,bhlkd->bhld",
                read_w_c.to(state_dtype),
                slot_state_t.to(state_dtype),
            )

            if return_info:
                slot_state_norm_t[:, :, t0:t1, :] = slot_state_t.to(torch.float32).norm(dim=-1)

            # update running states
            m_state = m_new[:, :, :, -1]
            denom_state = denom_c[:, :, :, -1]
            numer_state = numer_c[:, :, :, -1, :]

            pre_slotspace_norm = None
            if return_info:
                pre_slotspace_norm = out_h.detach().to(torch.float32).norm(dim=-1).mean().cpu()


        # =====================================================
        # causal linear attention in slot-space (CHUNKED prefix scan)
        # =====================================================
        slotspace_delta_norm_mean = None
        if self.use_slotspace_refine:
            slotspace_dtype = state_dtype
            M = self.slotspace_dim

            # Encode read weights into slot-space coordinates
            u = self.slot_in(read_weights.to(slotspace_dtype))  # [B,H,T,M]
            q_s  = self.slot_q(u)
            k_s  = self.slot_k(u)
            v_s  = self.slot_v(u)

            # RoPE in slot-space matcher (Q/K only)
            if self.use_rope_slotspace:
                cos_s, sin_s = self.rope_slotspace.get_cos_sin(T, device=x.device, dtype=q_s.dtype)
                q_s = apply_rope(q_s, cos_s, sin_s)
                k_s = apply_rope(k_s, cos_s, sin_s)

            qf = phi(q_s)
            kf = phi(k_s)

            if valid is not None:
                mask = valid.view(B, 1, T, 1).to(slotspace_dtype)
                qf = qf * mask
                kf = kf * mask
                v_s = v_s * mask

            u2 = torch.empty((B, H, T, M), device=x.device, dtype=slotspace_dtype)

            S_state = torch.zeros((B, H, M, M), device=x.device, dtype=slotspace_dtype)
            Z_state = torch.zeros((B, H, M), device=x.device, dtype=slotspace_dtype)

            SS_CHUNK = self.slotspace_chunk_size

            for t0 in range(0, T, SS_CHUNK):
                t1 = min(T, t0 + SS_CHUNK)
                L = t1 - t0

                qf_c = qf[:, :, t0:t1, :]   # [B,H,L,M]
                kf_c = kf[:, :, t0:t1, :]   # [B,H,L,M]
                v_c  = v_s[:, :, t0:t1, :]  # [B,H,L,M]

                kv = torch.einsum("bhlm,bhln->bhlmn", kf_c, v_c)  # [B,H,L,M,M]
                S_c = torch.cumsum(kv, dim=2)
                Z_c = torch.cumsum(kf_c, dim=2)

                S_c = S_c + S_state.unsqueeze(2)
                Z_c = (Z_c + Z_state.unsqueeze(2)).clamp_min(1e-8)

                num = torch.einsum("bhlm,bhlmn->bhln", qf_c, S_c)
                den = torch.einsum("bhlm,bhlm->bhl", qf_c, Z_c).unsqueeze(-1).clamp_min(1e-8)
                u2[:, :, t0:t1, :] = num / den

                S_state = S_c[:, :, -1, :, :]
                Z_state = Z_c[:, :, -1, :]

            u2 = self.slotspace_dropout(u2)

            # Decode slot weights per token
            slot_w = self.slot_out(u2)  # [B,H,T,K]
            if self.slotspace_signed_weights:
                slot_w = torch.tanh(slot_w)
            else:
                slot_w = torch.softmax(slot_w, dim=-1)

            # ---- NEW: slotspace-only mask (hard zero) ----
            if slot_mask_where == "slotspace_only":
                sm_full = self._resolve_slot_mask(
                    slot_mask,
                    B=B, H=H, L=T, K=K,
                    device=x.device,
                    dtype=slot_w.dtype,
                    scope=slot_mask_scope,  # "all" masks everywhere; "last_pos_only" masks only last token
                )
                if sm_full is not None:
                    slot_w = slot_w * (sm_full > 0.0).to(slot_w.dtype)
                    if not self.slotspace_signed_weights:
                        slot_w = slot_w / slot_w.sum(dim=-1, keepdim=True).clamp_min(1e-8)

            gate = self._slotspace_gate(dtype=state_dtype, device=x.device).to(state_dtype)

            # second streaming pass (decode delta through slot states)
            denom_state = torch.zeros((B, H, K), device=x.device, dtype=state_dtype)
            numer_state = torch.zeros((B, H, K, d), device=x.device, dtype=state_dtype)
            m_state = torch.full((B, H, K), float("-inf"), device=x.device, dtype=state_dtype)

            delta_norm_sum = torch.zeros((), device=x.device, dtype=torch.float32)
            delta_norm_count = 0

            for t0 in range(0, T, WRITE_CHUNK):
                t1 = min(T, t0 + WRITE_CHUNK)
                L = t1 - t0

                wlog_c = write_logits[:, :, :, t0:t1]  # [B,H,K,L]
                m_c, _ = torch.cummax(wlog_c, dim=-1)
                m_new = torch.maximum(m_state.unsqueeze(-1), m_c)

                scale = torch.exp(m_state.unsqueeze(-1) - m_new)
                denom_c = denom_state.unsqueeze(-1) * scale
                numer_c = numer_state.unsqueeze(-2) * scale.unsqueeze(-1)

                w_new = self._safe_exp_sub_max(wlog_c, m_new)
                denom_c = denom_c + torch.cumsum(w_new, dim=-1)

                v_c = v_write[:, :, t0:t1, :].to(state_dtype)  # [B,H,L,d]
                add = torch.cumsum(w_new.unsqueeze(-1) * v_c.unsqueeze(2), dim=-2)
                numer_c = numer_c + add

                slot_state_c = numer_c / denom_c.clamp_min(1e-8).unsqueeze(-1)  # [B,H,K,L,d]
                slot_state_t = slot_state_c.permute(0, 1, 3, 2, 4).contiguous() # [B,H,L,K,d]

                slot_w_c = slot_w[:, :, t0:t1, :].to(state_dtype)  # [B,H,L,K]

                delta_c = torch.einsum("bhlk,bhlkd->bhld", slot_w_c, slot_state_t.to(state_dtype))  # [B,H,L,d]

                out_h[:, :, t0:t1, :] = out_h[:, :, t0:t1, :] + gate * delta_c

                delta_norm_sum = delta_norm_sum + delta_c.detach().to(torch.float32).norm(dim=-1).sum()
                delta_norm_count += (B * H * L)

                m_state = m_new[:, :, :, -1]
                denom_state = denom_c[:, :, :, -1]
                numer_state = numer_c[:, :, :, -1, :]

            slotspace_delta_norm_mean = (delta_norm_sum / max(1, delta_norm_count)).detach().cpu()


            post_slotspace_norm = None
            if return_info:
                post_slotspace_norm = out_h.detach().to(torch.float32).norm(dim=-1).mean().cpu()



        # Finish
        out = out_h.transpose(1, 2).contiguous().view(B, T, C)
        out = self.out_proj(out)
        out = self.dropout(out)

        info = None
        if return_info:

            info = {
                "write_logits_raw": write_logits_raw.detach(),
                "write_logits": write_logits.detach().to(torch.float32),
                "read_weights": read_weights.detach(),
                "slot_state_norm": slot_state_norm_t.detach().permute(0, 1, 3, 2).contiguous() if slot_state_norm_t is not None else None,
                "content_read_gamma": content_read_gamma.detach().to(torch.float32).cpu(),
                "slot_mask_where": slot_mask_where,
                "slot_mask_scope": slot_mask_scope,
            }
            info["out_h_norm_pre_slotspace"] = pre_slotspace_norm
            info["out_h_norm_post_slotspace"] = post_slotspace_norm

            if alibi_bias_applied is not None:
                info["alibi_bias_applied"] = alibi_bias_applied.detach().to(torch.float32)
            if self.use_alibi_write and self.learn_alibi_strength:
                info["alibi_strength"] = self._alibi_strength(dtype=torch.float32, device=x.device).detach().cpu()
            if self.use_slotspace_refine:
                info["slotspace_gate"] = self._slotspace_gate(dtype=torch.float32, device=x.device).detach().cpu()
                info["use_rope_slotspace"] = torch.tensor(bool(self.use_rope_slotspace))
                if slotspace_delta_norm_mean is not None:
                    info["slotspace_delta_norm"] = slotspace_delta_norm_mean

            info["read_logits"] = read_logits_full.detach().to(torch.float32) if read_logits_full is not None else None
            info["read_logits_key"] = read_logits_key_full.detach().to(torch.float32) if read_logits_key_full is not None else None
            info["read_logits_content"] = (
                read_logits_content_full.detach().to(torch.float32) if read_logits_content_full is not None else None
            )
            info["routing_mode"] = routing_mode

        return out, info

In [ ]:

#@title Addressed State Attention ( cleaned + safer; checkpoint-stable)
#
# Goals:
# - Preserve checkpoint compatibility: parameter/buffer names and shapes are unchanged.
# - Preserve public API: forward() signature unchanged (incl slot_mask controls).
# - Make control-flow easier to reason about: isolate masking, routing, and streaming scans.
# - Reduce footguns: no chunk-local “pre/post” metrics, no dangling hooks, fewer hidden branches.
#
# Notes on checkpoint stability:
# - Do NOT rename any of the following: slot_keys, Wk_write, Wv_write, Wq_read, out_proj,
#   _alibi_slopes, _alibi_strength_param, _content_read_gamma_raw, slot_in/slot_q/slot_k/slot_v/slot_out,
#   _slotspace_gate_raw, rope/rope_slotspace buffers, etc.
# - This class keeps those names intact.

import math
from typing import Optional, Dict, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F

# -------------------------
# RoPE helper (rotate-half)
# -------------------------
def _rotate_half(x: torch.Tensor) -> torch.Tensor:
    x1 = x[..., ::2]
    x2 = x[..., 1::2]
    return torch.stack((-x2, x1), dim=-1).flatten(-2)

class RotaryEmbedding(nn.Module):
    def __init__(self, dim: int, base: float = 10000.0):
        super().__init__()
        assert dim % 2 == 0, "RoPE requires even dim"
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)
        self._cos_cached = None
        self._sin_cached = None
        self._t_cached = None
        self._device_cached = None

    def get_cos_sin(self, T: int, device, dtype):
        if (
            self._t_cached == T
            and self._cos_cached is not None
            and self._device_cached == device
        ):
            return self._cos_cached.to(dtype=dtype), self._sin_cached.to(dtype=dtype)

        t = torch.arange(T, device=device, dtype=self.inv_freq.dtype)
        freqs = torch.einsum("t,f->tf", t, self.inv_freq)  # [T, d/2]
        emb = torch.cat([freqs, freqs], dim=-1)            # [T, d]
        cos = emb.cos()[None, None, :, :]                  # [1,1,T,d]
        sin = emb.sin()[None, None, :, :]                  # [1,1,T,d]

        self._t_cached = T
        self._device_cached = device
        self._cos_cached = cos
        self._sin_cached = sin
        return cos.to(dtype=dtype), sin.to(dtype=dtype)

def apply_rope(x: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor) -> torch.Tensor:
    return (x * cos) + (_rotate_half(x) * sin)

# -------------------------
# ALiBi slopes helper
# -------------------------
def alibi_slopes(num_heads: int, device=None, dtype=torch.float32) -> torch.Tensor:
    def get_slopes(n):
        def power_of_2_slopes(n):
            start = 2.0 ** (-(2.0 ** -(math.log2(n) - 3)))
            ratio = start
            return [start * (ratio ** i) for i in range(n)]
        if math.log2(n).is_integer():
            return power_of_2_slopes(n)
        closest = 2 ** math.floor(math.log2(n))
        return power_of_2_slopes(closest) + get_slopes(2 * closest)[0::2][: n - closest]
    return torch.tensor(get_slopes(num_heads), device=device, dtype=dtype)  # [H]

# -------------------------
# softplus init helpers
# -------------------------
def _inv_softplus(y: torch.Tensor) -> torch.Tensor:
    return torch.log(torch.expm1(y))

# -------------------------
# Linear attention feature map (Performer-style)
# -------------------------
def phi(x: torch.Tensor) -> torch.Tensor:
    return F.elu(x) + 1.0


class AddressedStateAttention(nn.Module):
    """
    Addressed State Attention (ASA):
      - prefix-softmax WRITE into slots (O(T))
      - READ routing from tokens -> slots (softmax over slots)
      - content-conditioned READ term (gamma)
      - RoPE on write keys (geometry)
      - ALiBi bias on write logits (prefix-friendly)

    slot-space refinement:
      - causal linear attention in a low-dim slot-address coordinate space
      - produces per-token signed weights over slots
      - decoded through the same streaming slot-state basis
      - gated by learnable slotspace_gate (softplus)

    ADDITION:
      - slot_mask controls for causal interventions:
          slot_mask: [K] float/bool where 1=keep, 0=mask
          slot_mask_where: "read" | "content_read_only" | "slotspace_only"
          slot_mask_scope: "all" | "last_pos_only"
      - also supports attribute fallback: self.slot_mask if slot_mask kwarg is None
    """

    def __init__(
        self,
        embed_dim: int,
        num_heads: int = 8,
        num_slots: int = 8,
        dropout: float = 0.1,

        # temperatures / numerics
        read_temperature: float = 1.0,
        write_temperature: float = 1.0,
        state_fp32: bool = True,
        slot_dropout: float = 0.0,
        normalize_k: bool = False,

        # positions (write geometry)
        use_rope_keys: bool = True,
        rope_base: float = 10000.0,

        # write bias (ALiBi)
        use_alibi_write: bool = True,
        alibi_strength_init: float = 0.1,
        learn_alibi_strength: bool = True,
        min_strength: float = 0.0,

        # content-conditioned read term
        use_content_read: bool = True,
        content_read_init: float = -4.0,
        content_read_max_gamma: float = 3.0,

        # slot-space refinement
        use_slotspace_refine: bool = True,
        slotspace_dim: int = 32,
        slotspace_gate_init: float = -4.0,
        slotspace_dropout: float = 0.05,
        slotspace_signed_weights: bool = True,

        # RoPE in slot-space matcher (Q/K only)
        use_rope_slotspace: bool = True,
        rope_base_slotspace: float = 100000.0,

        # perf knobs (behavior change only if you change them)
        write_chunk_size: int = 128,
        slotspace_chunk_size: int = 128,
    ):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_slots = num_slots
        self.head_dim = embed_dim // num_heads

        self.dropout = nn.Dropout(dropout)

        self.read_temperature = float(read_temperature)
        self.write_temperature = float(write_temperature)
        self.state_fp32 = bool(state_fp32)
        self.slot_dropout = float(slot_dropout)
        self.normalize_k = bool(normalize_k)
        self.routing_override = None  # external override hook

        # mask attribute fallback (may be set externally)
        # expected shape: [K], where 1=keep, 0=mask
        # self.slot_mask = None

        self.use_rope_keys = bool(use_rope_keys)
        self.use_alibi_write = bool(use_alibi_write)
        self.learn_alibi_strength = bool(learn_alibi_strength)
        self.min_strength = float(min_strength)

        self.use_content_read = bool(use_content_read)
        self.content_read_max_gamma = float(content_read_max_gamma)

        self.use_slotspace_refine = bool(use_slotspace_refine)
        self.slotspace_dim = int(slotspace_dim)
        self.slotspace_dropout = nn.Dropout(float(slotspace_dropout))
        self.slotspace_signed_weights = bool(slotspace_signed_weights)

        self.write_chunk_size = int(write_chunk_size)
        self.slotspace_chunk_size = int(slotspace_chunk_size)

        # Learned slot keys per head: [H,K,d]
        self.slot_keys = nn.Parameter(
            torch.randn(num_heads, num_slots, self.head_dim) / math.sqrt(self.head_dim)
        )

        # Projections
        self.Wk_write = nn.Linear(embed_dim, embed_dim, bias=False)
        self.Wv_write = nn.Linear(embed_dim, embed_dim, bias=False)
        self.Wq_read  = nn.Linear(embed_dim, embed_dim, bias=False)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=False)

        # RoPE (write geometry)
        self.rope = RotaryEmbedding(self.head_dim, base=rope_base) if self.use_rope_keys else None

        # ALiBi slopes (buffer)
        if self.use_alibi_write:
            self.register_buffer("_alibi_slopes", alibi_slopes(num_heads), persistent=False)  # [H]
        else:
            self.register_buffer("_alibi_slopes", torch.zeros(num_heads), persistent=False)

        # Learnable ALiBi strength (positive via softplus)
        if self.use_alibi_write and self.learn_alibi_strength:
            init = torch.tensor(float(alibi_strength_init) - self.min_strength).clamp_min(1e-8)
            self._alibi_strength_param = nn.Parameter(_inv_softplus(init))
        else:
            self._alibi_strength_param = None
            self.alibi_strength = float(alibi_strength_init)

        # Content read gamma (>=0 via softplus)
        if self.use_content_read:
            self._content_read_gamma_raw = nn.Parameter(torch.tensor(float(content_read_init)))
        else:
            self._content_read_gamma_raw = None

        # slot-space refinement
        self.use_rope_slotspace = bool(use_rope_slotspace) and bool(self.use_slotspace_refine)
        if self.use_slotspace_refine:
            self.slot_in  = nn.Linear(num_slots, self.slotspace_dim, bias=False)
            self.slot_q   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_k   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_v   = nn.Linear(self.slotspace_dim, self.slotspace_dim, bias=False)
            self.slot_out = nn.Linear(self.slotspace_dim, num_slots, bias=False)
            self._slotspace_gate_raw = nn.Parameter(torch.tensor(float(slotspace_gate_init)))

            if self.use_rope_slotspace:
                assert (self.slotspace_dim % 2) == 0, "use_rope_slotspace requires even slotspace_dim"
                self.rope_slotspace = RotaryEmbedding(self.slotspace_dim, base=float(rope_base_slotspace))
            else:
                self.rope_slotspace = None
        else:
            self.slot_in = None
            self.slot_q = self.slot_k = self.slot_v = None
            self.slot_out = None
            self._slotspace_gate_raw = None
            self.rope_slotspace = None

    # -------------------------
    # scalar params
    # -------------------------
    def _alibi_strength(self, dtype, device) -> torch.Tensor:
        if not (self.use_alibi_write and self.learn_alibi_strength):
            return torch.tensor(self.alibi_strength, dtype=dtype, device=device)
        return (F.softplus(self._alibi_strength_param) + self.min_strength).to(dtype=dtype, device=device)

    def _content_read_gamma(self, dtype, device) -> torch.Tensor:
        if not self.use_content_read:
            return torch.tensor(0.0, dtype=dtype, device=device)
        g = F.softplus(self._content_read_gamma_raw)  # >=0
        if self.content_read_max_gamma is not None and self.content_read_max_gamma > 0:
            g = g.clamp(max=self.content_read_max_gamma)
        return g.to(dtype=dtype, device=device)

    def _slotspace_gate(self, dtype, device) -> torch.Tensor:
        if not self.use_slotspace_refine:
            return torch.tensor(0.0, dtype=dtype, device=device)
        return F.softplus(self._slotspace_gate_raw).to(dtype=dtype, device=device)

    # -------------------------
    # numerics helpers
    # -------------------------
    @staticmethod
    def _safe_exp_sub_max(s: torch.Tensor, m: torch.Tensor) -> torch.Tensor:
        # s, m broadcastable
        diff = s - m
        diff = diff.masked_fill(~torch.isfinite(m), float("-inf"))
        return torch.exp(diff)

    def _resolve_slot_mask(
        self,
        slot_mask: Optional[torch.Tensor],
        *,
        B: int,
        H: int,
        L: int,
        K: int,
        device,
        dtype,
        scope: str,
    ) -> Optional[torch.Tensor]:
        """
        Returns expanded mask of shape [B,H,L,K] or None.
        slot_mask is expected as [K] where 1=keep, 0=mask.
        """
        if slot_mask is None:
            slot_mask = getattr(self, "slot_mask", None)
        if slot_mask is None:
            return None

        sm = slot_mask.to(device=device, dtype=dtype)
        if sm.ndim != 1 or sm.numel() != K:
            raise ValueError(f"slot_mask must be shape [K]={K}, got {tuple(sm.shape)}")

        sm = sm.view(1, 1, 1, K)  # [1,1,1,K]
        if scope == "all":
            return sm.expand(B, H, L, K)
        if scope == "last_pos_only":
            out = torch.ones((B, H, L, K), device=device, dtype=dtype)
            out[:, :, -1:, :] = sm.expand(B, H, 1, K)
            return out
        raise ValueError(f"Unknown slot_mask_scope={scope!r}")

    @staticmethod
    def _apply_hard_mask_and_renorm(w: torch.Tensor, keep: torch.Tensor) -> torch.Tensor:
        """
        w: [...,K]
        keep: broadcastable boolean/float mask where True/1 means keep.
        Ensures masked entries are exactly zero and rows renormalized.
        """
        w = w * keep.to(w.dtype)
        return w / w.sum(dim=-1, keepdim=True).clamp_min(1e-8)

    # -------------------------
    # routing helper
    # -------------------------
    def _compute_read_weights(
        self,
        *,
        read_logits: torch.Tensor,               # [B,H,L,K]
        read_logits_key: torch.Tensor,           # [B,H,L,K]
        read_logits_content: Optional[torch.Tensor],  # [B,H,L,K] or None
        routing_mode: str,
        routing_topk: int,
        read_weights_override: Optional[torch.Tensor],
        routing_noise: Optional[str],
        routing_noise_scale: float,
        rtemp: float,
        sm: Optional[torch.Tensor],              # [B,H,L,K] or None (expanded slot mask)
        slot_mask_where: str,
        B: int, H: int, L: int, K: int,
        T_total: int,
        t0: int, t1: int,
    ) -> torch.Tensor:
        # routing noise (applies to combined logits)
        if routing_noise is not None:
            if routing_noise == "gumbel":
                u = torch.rand_like(read_logits)
                g = -torch.log(-torch.log(u.clamp_min(1e-8)).clamp_min(1e-8))
                read_logits = read_logits + routing_noise_scale * g
            elif routing_noise == "gaussian":
                read_logits = read_logits + routing_noise_scale * torch.randn_like(read_logits)
            else:
                raise ValueError(f"Unknown routing_noise={routing_noise}")

        # routing override (external) OR standard modes
        if self.routing_override is not None:
            if callable(self.routing_override):
                ctx = {
                    "t0": t0, "t1": t1,
                    "B": B, "H": H, "T": T_total, "K": K, "d": self.head_dim,
                    "rtemp": rtemp,
                    "state_dtype": read_logits.dtype,
                    "q_read_c": None,       # intentionally omitted here; keep ctx minimal/stable
                    "slot_keys": self.slot_keys,
                    "slot_state_t": None,   # intentionally omitted here
                    "valid": None,          # omitted; caller can supply via closure if needed
                    "slot_mask": None,      # omitted; caller already has it
                    "slot_mask_where": slot_mask_where,
                }
                read_w = self.routing_override(
                    t0, t1,
                    read_logits,
                    read_logits_key,
                    read_logits_content,
                    ctx,
                )
            else:
                read_w = self.routing_override[:, :, t0:t1, :].to(read_logits.dtype)

            read_w = torch.nan_to_num(read_w, nan=0.0, posinf=0.0, neginf=0.0)
            read_w = read_w.clamp_min(0.0)
            read_w = read_w / read_w.sum(dim=-1, keepdim=True).clamp_min(1e-8)
        else:
            if routing_mode == "softmax":
                read_w = torch.softmax(read_logits / rtemp, dim=-1)
            elif routing_mode == "top1":
                top = read_logits.argmax(dim=-1)  # [B,H,L]
                read_w = F.one_hot(top, num_classes=K).to(read_logits.dtype)
            elif routing_mode == "topk":
                kk = max(1, min(K, int(routing_topk)))
                vals, idx = torch.topk(read_logits, k=kk, dim=-1)
                masked = torch.full_like(read_logits, float("-inf"))
                masked.scatter_(-1, idx, vals)
                read_w = torch.softmax(masked / rtemp, dim=-1)
            elif routing_mode == "external":
                if read_weights_override is None:
                    raise ValueError("routing_mode='external' requires read_weights_override")
                if read_weights_override.shape[-2] == T_total:
                    read_w = read_weights_override[:, :, t0:t1, :]
                else:
                    read_w = read_weights_override
                read_w = read_w / read_w.sum(dim=-1, keepdim=True).clamp_min(1e-8)
            else:
                raise ValueError(f"Unknown routing_mode={routing_mode}")

        # enforce mask if the user requested mask-at-read
        if slot_mask_where == "read" and sm is not None:
            read_w = self._apply_hard_mask_and_renorm(read_w, (sm > 0.0))
        return read_w

    # -------------------------
    # forward
    # -------------------------
    def forward(
        self,
        x: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        return_info: bool = False,
        routing_mode: str = "softmax",
        routing_topk: int = 2,
        read_weights_override: Optional[torch.Tensor] = None,
        routing_noise: Optional[str] = None,
        routing_noise_scale: float = 1.0,

        slot_mask: Optional[torch.Tensor] = None,     # [K], 1=keep, 0=mask
        slot_mask_where: str = "read",                # "read" | "content_read_only" | "slotspace_only"
        slot_mask_scope: str = "all",                 # "all" | "last_pos_only"
    ) -> Tuple[torch.Tensor, Optional[Dict[str, torch.Tensor]]]:

        B, T, C = x.shape
        H, K, d = self.num_heads, self.num_slots, self.head_dim

        # Project (write K/V, read Q)
        k_write = self.Wk_write(x).view(B, T, H, d).transpose(1, 2)  # [B,H,T,d]
        v_write = self.Wv_write(x).view(B, T, H, d).transpose(1, 2)  # [B,H,T,d]
        q_read  = self.Wq_read(x).view(B, T, H, d).transpose(1, 2)   # [B,H,T,d]

        if self.normalize_k:
            k_write = F.normalize(k_write, dim=-1, eps=1e-8)

        # RoPE on write keys
        if self.use_rope_keys:
            cos, sin = self.rope.get_cos_sin(T, device=x.device, dtype=k_write.dtype)
            k_write = apply_rope(k_write, cos, sin)

        # Slot dropout (only affects slot_keys in training)
        slot_keys = self.slot_keys
        if self.training and self.slot_dropout > 0.0:
            drop = (torch.rand((H, K), device=x.device) < self.slot_dropout)
            slot_keys = slot_keys * (~drop).to(slot_keys.dtype).unsqueeze(-1)

        # WRITE logits: [B,H,K,T]
        write_logits_raw = torch.einsum("hkd,bhtd->bhkt", slot_keys, k_write) / math.sqrt(d)

        # Stable dtype for prefix-softmax math
        state_dtype = torch.float32 if (self.state_fp32 and x.dtype != torch.float32) else x.dtype
        write_logits = write_logits_raw.to(state_dtype)

        # Write temperature
        wtemp = max(1e-6, self.write_temperature)
        write_logits = write_logits / wtemp

        # ALiBi distance bias (prefix-friendly)
        alibi_bias_applied = None
        if self.use_alibi_write:
            strength = self._alibi_strength(dtype=state_dtype, device=x.device)  # scalar
            slopes = self._alibi_slopes.to(device=x.device, dtype=state_dtype) * strength  # [H]
            pos_i = torch.arange(T, device=x.device, dtype=state_dtype)  # [T]
            alibi_bias = slopes.view(1, H, 1, 1) * pos_i.view(1, 1, 1, T) # [1,H,1,T]
            write_logits = write_logits + alibi_bias
            alibi_bias_applied = alibi_bias

        # Key padding mask
        if attention_mask is not None:
            valid = attention_mask.to(dtype=torch.bool)
            write_logits = write_logits.masked_fill(~valid.view(B, 1, 1, T), float("-inf"))
        else:
            valid = None

        # -------------------------
        # Streaming WRITE + READ
        # -------------------------
        content_read_gamma = self._content_read_gamma(dtype=q_read.dtype, device=x.device)
        rtemp = max(1e-6, self.read_temperature)

        out_h = torch.empty((B, H, T, d), device=x.device, dtype=state_dtype)
        read_weights = torch.empty((B, H, T, K), device=x.device, dtype=q_read.dtype)

        slot_state_norm_t = torch.empty((B, H, T, K), device=x.device, dtype=torch.float32) if return_info else None

        denom_state = torch.zeros((B, H, K), device=x.device, dtype=state_dtype)
        numer_state = torch.zeros((B, H, K, d), device=x.device, dtype=state_dtype)
        m_state = torch.full((B, H, K), float("-inf"), device=x.device, dtype=state_dtype)

        if return_info:
            read_logits_full = torch.empty((B, H, T, K), device=x.device, dtype=state_dtype)
            read_logits_key_full = torch.empty((B, H, T, K), device=x.device, dtype=state_dtype)
            read_logits_content_full = (
                torch.empty((B, H, T, K), device=x.device, dtype=state_dtype) if self.use_content_read else None
            )
        else:
            read_logits_full = None
            read_logits_key_full = None
            read_logits_content_full = None

        WRITE_CHUNK = self.write_chunk_size

        for t0 in range(0, T, WRITE_CHUNK):
            t1 = min(T, t0 + WRITE_CHUNK)
            L = t1 - t0

            wlog_c = write_logits[:, :, :, t0:t1]  # [B,H,K,L]

            # streaming cummax
            m_c, _ = torch.cummax(wlog_c, dim=-1)                 # [B,H,K,L]
            m_new = torch.maximum(m_state.unsqueeze(-1), m_c)     # [B,H,K,L]

            # rescale old prefix state to new max reference
            scale = torch.exp(m_state.unsqueeze(-1) - m_new)      # [B,H,K,L]

            denom_c = denom_state.unsqueeze(-1) * scale           # [B,H,K,L]
            numer_c = numer_state.unsqueeze(-2) * scale.unsqueeze(-1)  # [B,H,K,L,d]

            # new weights under m_new reference
            w_new = self._safe_exp_sub_max(wlog_c, m_new)         # [B,H,K,L]

            # accumulate within chunk
            denom_c = denom_c + torch.cumsum(w_new, dim=-1)       # [B,H,K,L]
            v_c = v_write[:, :, t0:t1, :].to(state_dtype)         # [B,H,L,d]
            add = torch.cumsum(w_new.unsqueeze(-1) * v_c.unsqueeze(2), dim=-2)  # [B,H,K,L,d]
            numer_c = numer_c + add

            # per-token slot state for this chunk: [B,H,L,K,d]
            slot_state_c = numer_c / denom_c.clamp_min(1e-8).unsqueeze(-1)      # [B,H,K,L,d]
            slot_state_t = slot_state_c.permute(0, 1, 3, 2, 4).contiguous()     # [B,H,L,K,d]

            # READ routing logits
            q_read_c = q_read[:, :, t0:t1, :]  # [B,H,L,d]
            read_logits_key = torch.einsum("bhld,hkd->bhlk", q_read_c, slot_keys) / math.sqrt(d)

            read_logits_content = None
            if self.use_content_read:
                read_logits_content = torch.einsum(
                    "bhld,bhlkd->bhlk",
                    q_read_c,
                    slot_state_t.to(q_read_c.dtype),
                ) / math.sqrt(d)

            # slot mask expanded for this chunk (if any)
            sm = self._resolve_slot_mask(
                slot_mask,
                B=B, H=H, L=L, K=K,
                device=x.device,
                dtype=read_logits_key.dtype,
                scope=slot_mask_scope,
            )

            # Apply mask according to "where"
            if slot_mask_where == "read":
                if sm is not None:
                    read_logits_key = read_logits_key.masked_fill(sm <= 0.0, float("-inf"))
                    if self.use_content_read and read_logits_content is not None:
                        read_logits_content = read_logits_content.masked_fill(sm <= 0.0, float("-inf"))

            elif slot_mask_where == "content_read_only":
                # only remove the content contribution for masked slots;
                # do NOT change key routing logits.
                if sm is not None and self.use_content_read and read_logits_content is not None:
                    read_logits_content = read_logits_content.masked_fill(sm <= 0.0, 0.0)

            elif slot_mask_where == "slotspace_only":
                pass  # applied later on slot_w
            else:
                raise ValueError(f"Unknown slot_mask_where={slot_mask_where!r}")

            # combine logits
            read_logits = read_logits_key
            if self.use_content_read and read_logits_content is not None:
                read_logits = read_logits + content_read_gamma.to(read_logits.dtype) * read_logits_content

            if return_info:
                read_logits_full[:, :, t0:t1, :] = read_logits.to(state_dtype)
                read_logits_key_full[:, :, t0:t1, :] = read_logits_key.to(state_dtype)
                if self.use_content_read and read_logits_content_full is not None:
                    read_logits_content_full[:, :, t0:t1, :] = read_logits_content.to(state_dtype)

            # read weights
            read_w_c = self._compute_read_weights(
                read_logits=read_logits,
                read_logits_key=read_logits_key,
                read_logits_content=read_logits_content,
                routing_mode=routing_mode,
                routing_topk=routing_topk,
                read_weights_override=read_weights_override,
                routing_noise=routing_noise,
                routing_noise_scale=routing_noise_scale,
                rtemp=rtemp,
                sm=sm,
                slot_mask_where=slot_mask_where,
                B=B, H=H, L=L, K=K,
                T_total=T,
                t0=t0, t1=t1,
            )

            read_weights[:, :, t0:t1, :] = read_w_c

            # token output (base path)
            out_h[:, :, t0:t1, :] = torch.einsum(
                "bhlk,bhlkd->bhld",
                read_w_c.to(state_dtype),
                slot_state_t.to(state_dtype),
            )

            if return_info:
                slot_state_norm_t[:, :, t0:t1, :] = slot_state_t.to(torch.float32).norm(dim=-1)

            # update running states (carry prefix to next chunk)
            m_state = m_new[:, :, :, -1]
            denom_state = denom_c[:, :, :, -1]
            numer_state = numer_c[:, :, :, -1, :]

        # -------------------------
        # slot-space refinement (additive delta)
        # -------------------------
        slotspace_delta_norm_mean = None
        if self.use_slotspace_refine:
            slotspace_dtype = state_dtype
            M = self.slotspace_dim

            # Encode read weights into slot-space coordinates
            u = self.slot_in(read_weights.to(slotspace_dtype))  # [B,H,T,M]
            q_s = self.slot_q(u)
            k_s = self.slot_k(u)
            v_s = self.slot_v(u)

            # RoPE in slot-space matcher (Q/K only)
            if self.use_rope_slotspace:
                cos_s, sin_s = self.rope_slotspace.get_cos_sin(T, device=x.device, dtype=q_s.dtype)
                q_s = apply_rope(q_s, cos_s, sin_s)
                k_s = apply_rope(k_s, cos_s, sin_s)

            qf = phi(q_s)
            kf = phi(k_s)

            if valid is not None:
                mask = valid.view(B, 1, T, 1).to(slotspace_dtype)
                qf = qf * mask
                kf = kf * mask
                v_s = v_s * mask

            # causal linear attention prefix scan in chunks
            u2 = torch.empty((B, H, T, M), device=x.device, dtype=slotspace_dtype)

            S_state = torch.zeros((B, H, M, M), device=x.device, dtype=slotspace_dtype)
            Z_state = torch.zeros((B, H, M), device=x.device, dtype=slotspace_dtype)

            SS_CHUNK = self.slotspace_chunk_size

            for t0 in range(0, T, SS_CHUNK):
                t1 = min(T, t0 + SS_CHUNK)
                L = t1 - t0

                qf_c = qf[:, :, t0:t1, :]   # [B,H,L,M]
                kf_c = kf[:, :, t0:t1, :]   # [B,H,L,M]
                v_c  = v_s[:, :, t0:t1, :]  # [B,H,L,M]

                kv = torch.einsum("bhlm,bhln->bhlmn", kf_c, v_c)  # [B,H,L,M,M]
                S_c = torch.cumsum(kv, dim=2) + S_state.unsqueeze(2)
                Z_c = (torch.cumsum(kf_c, dim=2) + Z_state.unsqueeze(2)).clamp_min(1e-8)

                num = torch.einsum("bhlm,bhlmn->bhln", qf_c, S_c)
                den = torch.einsum("bhlm,bhlm->bhl", qf_c, Z_c).unsqueeze(-1).clamp_min(1e-8)
                u2[:, :, t0:t1, :] = num / den

                S_state = S_c[:, :, -1, :, :]
                Z_state = Z_c[:, :, -1, :]

            u2 = self.slotspace_dropout(u2)

            # Decode slot weights per token: [B,H,T,K]
            slot_w = self.slot_out(u2)
            if self.slotspace_signed_weights:
                slot_w = torch.tanh(slot_w)
            else:
                slot_w = torch.softmax(slot_w, dim=-1)

            # optional: slotspace-only mask
            if slot_mask_where == "slotspace_only":
                sm_full = self._resolve_slot_mask(
                    slot_mask,
                    B=B, H=H, L=T, K=K,
                    device=x.device,
                    dtype=slot_w.dtype,
                    scope=slot_mask_scope,
                )
                if sm_full is not None:
                    slot_w = slot_w * (sm_full > 0.0).to(slot_w.dtype)
                    if not self.slotspace_signed_weights:
                        slot_w = slot_w / slot_w.sum(dim=-1, keepdim=True).clamp_min(1e-8)

            gate = self._slotspace_gate(dtype=state_dtype, device=x.device).to(state_dtype)

            # second streaming pass to decode delta through slot states
            denom_state = torch.zeros((B, H, K), device=x.device, dtype=state_dtype)
            numer_state = torch.zeros((B, H, K, d), device=x.device, dtype=state_dtype)
            m_state = torch.full((B, H, K), float("-inf"), device=x.device, dtype=state_dtype)

            delta_norm_sum = torch.zeros((), device=x.device, dtype=torch.float32)
            delta_norm_count = 0

            for t0 in range(0, T, WRITE_CHUNK):
                t1 = min(T, t0 + WRITE_CHUNK)
                L = t1 - t0

                wlog_c = write_logits[:, :, :, t0:t1]  # [B,H,K,L]

                m_c, _ = torch.cummax(wlog_c, dim=-1)
                m_new = torch.maximum(m_state.unsqueeze(-1), m_c)

                scale = torch.exp(m_state.unsqueeze(-1) - m_new)
                denom_c = denom_state.unsqueeze(-1) * scale
                numer_c = numer_state.unsqueeze(-2) * scale.unsqueeze(-1)

                w_new = self._safe_exp_sub_max(wlog_c, m_new)
                denom_c = denom_c + torch.cumsum(w_new, dim=-1)

                v_c = v_write[:, :, t0:t1, :].to(state_dtype)
                add = torch.cumsum(w_new.unsqueeze(-1) * v_c.unsqueeze(2), dim=-2)
                numer_c = numer_c + add

                slot_state_c = numer_c / denom_c.clamp_min(1e-8).unsqueeze(-1)  # [B,H,K,L,d]
                slot_state_t = slot_state_c.permute(0, 1, 3, 2, 4).contiguous() # [B,H,L,K,d]

                slot_w_c = slot_w[:, :, t0:t1, :].to(state_dtype)                # [B,H,L,K]
                delta_c = torch.einsum("bhlk,bhlkd->bhld", slot_w_c, slot_state_t.to(state_dtype))

                out_h[:, :, t0:t1, :] = out_h[:, :, t0:t1, :] + gate * delta_c

                delta_norm_sum = delta_norm_sum + delta_c.detach().to(torch.float32).norm(dim=-1).sum()
                delta_norm_count += (B * H * L)

                m_state = m_new[:, :, :, -1]
                denom_state = denom_c[:, :, :, -1]
                numer_state = numer_c[:, :, :, -1, :]

            slotspace_delta_norm_mean = (delta_norm_sum / max(1, delta_norm_count)).detach().cpu()

        # -------------------------
        # finish
        # -------------------------
        out = out_h.transpose(1, 2).contiguous().view(B, T, C)
        out = self.out_proj(out)
        out = self.dropout(out)

        info = None
        if return_info:
            info = {
                "write_logits_raw": write_logits_raw.detach(),
                "write_logits": write_logits.detach().to(torch.float32),
                "read_weights": read_weights.detach(),
                "slot_state_norm": slot_state_norm_t.detach().permute(0, 1, 3, 2).contiguous() if slot_state_norm_t is not None else None,
                "content_read_gamma": content_read_gamma.detach().to(torch.float32).cpu(),
                "slot_mask_where": slot_mask_where,
                "slot_mask_scope": slot_mask_scope,
            }
            if alibi_bias_applied is not None:
                info["alibi_bias_applied"] = alibi_bias_applied.detach().to(torch.float32)
            if self.use_alibi_write and self.learn_alibi_strength:
                info["alibi_strength"] = self._alibi_strength(dtype=torch.float32, device=x.device).detach().cpu()
            if self.use_slotspace_refine:
                info["slotspace_gate"] = self._slotspace_gate(dtype=torch.float32, device=x.device).detach().cpu()
                info["use_rope_slotspace"] = torch.tensor(bool(self.use_rope_slotspace))
                if slotspace_delta_norm_mean is not None:
                    info["slotspace_delta_norm"] = slotspace_delta_norm_mean

            info["read_logits"] = read_logits_full.detach().to(torch.float32) if read_logits_full is not None else None
            info["read_logits_key"] = read_logits_key_full.detach().to(torch.float32) if read_logits_key_full is not None else None
            info["read_logits_content"] = (
                read_logits_content_full.detach().to(torch.float32) if read_logits_content_full is not None else None
            )
            info["routing_mode"] = routing_mode

        return out, info

In [ ]:


#@title LM and Config defs
# ============================================================================
# Addressed State Models (ASM): Config + Block + LM
# - Naming aligned with paper: slots, read/write, slot-space refinement
# - No compatibility layer (fresh public tooling)
# - Assumes AddressedStateAttention is defined elsewhere (the primitive module)
# ============================================================================

import math
from dataclasses import dataclass
from typing import Tuple, Optional

import torch
import torch.nn as nn


# ============================================================================
# Config
# ============================================================================
@dataclass
class ASMTrainConfig:
    # Data
    dataset_name: str = "wikitext"
    dataset_config: str = "wikitext-103-raw-v1"
    tokenizer_name: str = "gpt2"

    max_seq_len: int = 256
    stride_frac_val: float = 0.50
    seed: int = 1337

    # Sample budgets
    train_samples_target: int = 100_000_000
    val_samples_target: int = 25_000

    # Training
    batch_size: int = 64
    learning_rate: float = 3e-4
    weight_decay: float = 0.01
    betas: Tuple[float, float] = (0.9, 0.95)
    grad_clip: float = 1.0
    warmup_steps: int = 1_000
    total_steps: int = 75_000
    eval_interval: int = 1_000
    log_interval: int = 100

    # Model
    vocab_size: int = 50257
    embed_dim: int = 384
    num_layers: int = 23
    num_heads: int = 8
    num_slots: int = 32
    mlp_ratio: float = 4.0
    dropout: float = 0.1
    tie_weights: bool = True

    # Addressed State Attention (ASA) / numerics
    read_temperature: float = 1.0
    write_temperature: float = 1.0
    slot_dropout: float = 0.05
    state_fp32: bool = True
    normalize_k: bool = False

    # Positions
    use_abs_pos: bool = False
    use_rope_keys: bool = True
    rope_base: float = 10000.0
    use_alibi_write: bool = True
    alibi_strength_init: float = 0.1
    learn_alibi_strength: bool = True
    min_strength: float = 0.0

    # Content-conditioned read term (gamma)
    use_content_read: bool = True
    content_read_init: float = -4.0
    content_read_max_gamma: float = 3.0

    # Optional slot-space refinement (formerly "k-space")
    use_slotspace_refine: bool = True
    slotspace_dim: int = 64
    slotspace_gate_init: float = -4.0
    slotspace_dropout: float = 0.05
    slotspace_signed_weights: bool = True

    # RoPE inside slot-space matcher (Q/K only)
    use_rope_slotspace: bool = True
    rope_base_slotspace: float = 100000.0

    # Perf knobs (behavior-identical)
    write_chunk_size: int = 128
    slotspace_chunk_size: int = 128
    enable_compiled: bool = False

    # Analytics
    eval_max_batches: int = 150
    analytics_last_k: int = 32

    # IO / caches
    output_dir: str = "./drive/MyDrive/asm_outputs"
    tag: str = "asm_wikitext"
    cache_dir: str = "./drive/MyDrive/asm_caches"
    val_windows_cache: str = "./drive/MyDrive/asm_val_cache_windows_1024.pkl"


# ============================================================================
# Block
# ============================================================================
class ASMBlock(nn.Module):
    def __init__(
        self,
        embed_dim: int,
        num_heads: int,
        num_slots: int,
        mlp_ratio: float = 4.0,
        dropout: float = 0.1,

        # temperatures / numerics
        read_temperature: float = 1.0,
        write_temperature: float = 1.0,
        state_fp32: bool = True,
        slot_dropout: float = 0.0,
        normalize_k: bool = False,

        # positions
        use_rope_keys: bool = True,
        rope_base: float = 10000.0,
        use_alibi_write: bool = True,

        # ALiBi params
        alibi_strength_init: float = 0.1,
        learn_alibi_strength: bool = True,
        min_strength: float = 0.0,

        # content-conditioned read (gamma)
        use_content_read: bool = True,
        content_read_init: float = -4.0,
        content_read_max_gamma: float = 3.0,

        # optional slot-space refinement
        use_slotspace_refine: bool = True,
        slotspace_dim: int = 32,
        slotspace_gate_init: float = -10.0,
        slotspace_dropout: float = 0.0,
        slotspace_signed_weights: bool = True,

        # RoPE inside slot-space matcher
        use_rope_slotspace: bool = True,
        rope_base_slotspace: float = 100000.0,

        # chunk sizes
        write_chunk_size: int = 128,
        slotspace_chunk_size: int = 128,
    ):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)

        self.asa = AddressedStateAttention(
            embed_dim=embed_dim,
            num_heads=num_heads,
            num_slots=num_slots,
            dropout=dropout,

            read_temperature=read_temperature,
            write_temperature=write_temperature,
            state_fp32=state_fp32,
            slot_dropout=slot_dropout,
            normalize_k=normalize_k,

            use_rope_keys=use_rope_keys,
            rope_base=rope_base,
            use_alibi_write=use_alibi_write,
            alibi_strength_init=alibi_strength_init,
            learn_alibi_strength=learn_alibi_strength,
            min_strength=min_strength,

            use_content_read=use_content_read,
            content_read_init=content_read_init,
            content_read_max_gamma=content_read_max_gamma,

            use_slotspace_refine=use_slotspace_refine,
            slotspace_dim=slotspace_dim,
            slotspace_gate_init=slotspace_gate_init,
            slotspace_dropout=slotspace_dropout,
            slotspace_signed_weights=slotspace_signed_weights,

            use_rope_slotspace=use_rope_slotspace,
            rope_base_slotspace=rope_base_slotspace,

            write_chunk_size=write_chunk_size,
            slotspace_chunk_size=slotspace_chunk_size,
        )

        self.norm2 = nn.LayerNorm(embed_dim)
        hidden = int(embed_dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, hidden, bias=False),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, embed_dim, bias=False),
            nn.Dropout(dropout),
        )

    def forward(
        self,
        x: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        return_info: bool = False,

        # passthrough (optional; keeps existing callers working)
        routing_mode: str = "softmax",
        routing_topk: int = 2,
        read_weights_override: Optional[torch.Tensor] = None,
        routing_noise: Optional[str] = None,
        routing_noise_scale: float = 1.0,

        slot_mask: Optional[torch.Tensor] = None,       # [K], 1=keep, 0=mask
        slot_mask_where: str = "read",                  # "read" | "content_read_only" | "slotspace_only"
        slot_mask_scope: str = "all",                   # "all" | "last_pos_only"
    ):
        a, info = self.asa(
            self.norm1(x),
            attention_mask=attention_mask,
            return_info=return_info,
            routing_mode=routing_mode,
            routing_topk=routing_topk,
            read_weights_override=read_weights_override,
            routing_noise=routing_noise,
            routing_noise_scale=routing_noise_scale,
            slot_mask=slot_mask,
            slot_mask_where=slot_mask_where,
            slot_mask_scope=slot_mask_scope,
        )
        x = x + a
        x = x + self.mlp(self.norm2(x))
        return x, info

# ============================================================================
# LM
# ============================================================================
class ASMLanguageModel(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        embed_dim: int = 384,
        num_layers: int = 6,
        num_heads: int = 8,
        num_slots: int = 8,
        max_seq_len: int = 1024,
        mlp_ratio: float = 4.0,
        dropout: float = 0.1,

        # temperatures / numerics
        read_temperature: float = 1.0,
        write_temperature: float = 1.0,
        state_fp32: bool = True,
        slot_dropout: float = 0.05,
        normalize_k: bool = False,

        tie_weights: bool = True,

        # LM-level abs pos
        use_abs_pos: bool = False,

        # positions
        use_rope_keys: bool = True,
        rope_base: float = 10000.0,
        use_alibi_write: bool = True,

        # ALiBi
        alibi_strength_init: float = 0.1,
        learn_alibi_strength: bool = True,
        min_strength: float = 0.0,

        # content-conditioned read (gamma)
        use_content_read: bool = True,
        content_read_init: float = -4.0,
        content_read_max_gamma: float = 3.0,

        # optional slot-space refinement
        use_slotspace_refine: bool = True,
        slotspace_dim: int = 32,
        slotspace_gate_init: float = -10.0,
        slotspace_dropout: float = 0.0,
        slotspace_signed_weights: bool = True,

        # RoPE inside slot-space matcher
        use_rope_slotspace: bool = True,
        rope_base_slotspace: float = 100000.0,

        # chunk sizes
        write_chunk_size: int = 128,
        slotspace_chunk_size: int = 128,
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.max_seq_len = max_seq_len
        self.use_abs_pos = bool(use_abs_pos)

        self.tok = nn.Embedding(vocab_size, embed_dim)
        self.pos = nn.Embedding(max_seq_len, embed_dim) if self.use_abs_pos else None
        self.drop = nn.Dropout(dropout)

        self.blocks = nn.ModuleList([
            ASMBlock(
                embed_dim=embed_dim,
                num_heads=num_heads,
                num_slots=num_slots,
                mlp_ratio=mlp_ratio,
                dropout=dropout,

                read_temperature=read_temperature,
                write_temperature=write_temperature,
                state_fp32=state_fp32,
                slot_dropout=slot_dropout,
                normalize_k=normalize_k,

                use_rope_keys=use_rope_keys,
                rope_base=rope_base,
                use_alibi_write=use_alibi_write,

                alibi_strength_init=alibi_strength_init,
                learn_alibi_strength=learn_alibi_strength,
                min_strength=min_strength,

                use_content_read=use_content_read,
                content_read_init=content_read_init,
                content_read_max_gamma=content_read_max_gamma,

                use_slotspace_refine=use_slotspace_refine,
                slotspace_dim=slotspace_dim,
                slotspace_gate_init=slotspace_gate_init,
                slotspace_dropout=slotspace_dropout,
                slotspace_signed_weights=slotspace_signed_weights,
                use_rope_slotspace=use_rope_slotspace,
                rope_base_slotspace=rope_base_slotspace,

                write_chunk_size=write_chunk_size,
                slotspace_chunk_size=slotspace_chunk_size,
            )
            for _ in range(num_layers)
        ])

        self.norm = nn.LayerNorm(embed_dim)
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
        if tie_weights:
            self.lm_head.weight = self.tok.weight

        self.apply(self._init)

    def _init(self, m):
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, std=0.02)
        elif isinstance(m, nn.Embedding):
            nn.init.normal_(m.weight, std=0.02)
        elif isinstance(m, nn.LayerNorm):
            nn.init.ones_(m.weight)
            nn.init.zeros_(m.bias)

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        return_info: bool = False,

        # passthrough (optional)
        routing_mode: str = "softmax",
        routing_topk: int = 2,
        read_weights_override: Optional[torch.Tensor] = None,
        routing_noise: Optional[str] = None,
        routing_noise_scale: float = 1.0,

        slot_mask: Optional[torch.Tensor] = None,       # [K], 1=keep, 0=mask
        slot_mask_where: str = "read",                  # "read" | "content_read_only" | "slotspace_only"
        slot_mask_scope: str = "all",                   # "all" | "last_pos_only"
        return_light_stats:bool = False,
    ):
        B, T = input_ids.shape
        assert T <= self.max_seq_len, f"T={T} exceeds max_seq_len={self.max_seq_len}"

        x = self.tok(input_ids)
        if self.use_abs_pos:
            pos = torch.arange(T, device=input_ids.device).unsqueeze(0).expand(B, -1)
            x = x + self.pos(pos)
        x = self.drop(x)

        infos = []
        for blk in self.blocks:
            x, info = blk(
                x,
                attention_mask=attention_mask,
                return_info=return_info,
                routing_mode=routing_mode,
                routing_topk=routing_topk,
                read_weights_override=read_weights_override,
                routing_noise=routing_noise,
                routing_noise_scale=routing_noise_scale,
                slot_mask=slot_mask,
                slot_mask_where=slot_mask_where,
                slot_mask_scope=slot_mask_scope,
            )
            if return_info:
                infos.append(info)

        x = self.norm(x)
        logits = self.lm_head(x)
        return (logits, infos) if return_info else logits


# ============================================================================
# Convenience: build model from config
# ============================================================================
def build_model_from_cfg(cfg: ASMTrainConfig) -> ASMLanguageModel:
    return ASMLanguageModel(
        vocab_size=cfg.vocab_size,
        embed_dim=cfg.embed_dim,
        num_layers=cfg.num_layers,
        num_heads=cfg.num_heads,
        num_slots=cfg.num_slots,
        max_seq_len=cfg.max_seq_len,
        mlp_ratio=cfg.mlp_ratio,
        dropout=cfg.dropout,

        read_temperature=cfg.read_temperature,
        write_temperature=cfg.write_temperature,
        state_fp32=cfg.state_fp32,
        slot_dropout=cfg.slot_dropout,
        normalize_k=cfg.normalize_k,

        tie_weights=cfg.tie_weights,

        use_abs_pos=cfg.use_abs_pos,
        use_rope_keys=cfg.use_rope_keys,
        rope_base=cfg.rope_base,
        use_alibi_write=cfg.use_alibi_write,

        alibi_strength_init=cfg.alibi_strength_init,
        learn_alibi_strength=cfg.learn_alibi_strength,
        min_strength=cfg.min_strength,

        use_content_read=cfg.use_content_read,
        content_read_init=cfg.content_read_init,
        content_read_max_gamma=cfg.content_read_max_gamma,

        use_slotspace_refine=cfg.use_slotspace_refine,
        slotspace_dim=cfg.slotspace_dim,
        slotspace_gate_init=cfg.slotspace_gate_init,
        slotspace_dropout=cfg.slotspace_dropout,
        slotspace_signed_weights=cfg.slotspace_signed_weights,
        use_rope_slotspace=cfg.use_rope_slotspace,
        rope_base_slotspace=cfg.rope_base_slotspace,

        write_chunk_size=cfg.write_chunk_size,
        slotspace_chunk_size=cfg.slotspace_chunk_size,
    )

In [ ]:

#@title Train, Eval Functions and Utilities

import os, math, random, pickle, time
from dataclasses import asdict
from typing import Tuple, Dict, List, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, IterableDataset, DataLoader
from tqdm import tqdm

from datasets import load_dataset
from transformers import AutoTokenizer


# =========================================================
# Data: cached token streams + val windows + random-window train
# =========================================================
class StableValidationDataset(Dataset):
    def __init__(self, samples: List[Tuple[torch.Tensor, torch.Tensor]]):
        self.samples = samples
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx]

def _ensure_dir(path: str):
    d = os.path.dirname(path)
    if d:
        os.makedirs(d, exist_ok=True)

def build_or_load_token_stream(
    *,
    cache_path: str,
    dataset_name: str,
    dataset_config: str,
    split: str,
    tokenizer_name: str,
    min_chars: int = 1,
    add_eos_between_rows: bool = True,
    max_rows: Optional[int] = None,
) -> List[int]:
    if os.path.exists(cache_path):
        print(f"Loading cached token stream: {cache_path}")
        with open(cache_path, "rb") as f:
            stream = pickle.load(f)
        print(f"Loaded token stream tokens={len(stream):,}")
        return stream

    print(f"Building token stream for {dataset_name}/{dataset_config} split={split} ...")
    _ensure_dir(cache_path)
    tok = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
    eos = tok.eos_token_id
    assert eos is not None, "Tokenizer must have eos_token_id"

    ds = load_dataset(dataset_name, dataset_config, split=split)

    stream: List[int] = []
    used = 0
    for row in tqdm(ds, desc=f"Tokenizing {split}"):
        if max_rows is not None and used >= max_rows:
            break
        text = (row.get("text") or "").strip()
        if len(text) < min_chars:
            continue
        ids = tok.encode(text, add_special_tokens=False)
        if not ids:
            continue
        stream.extend(ids)
        if add_eos_between_rows:
            stream.append(eos)
        used += 1

    print(f"Built stream: rows_used={used:,} tokens={len(stream):,}")
    with open(cache_path, "wb") as f:
        pickle.dump(stream, f)
    print(f"Cached token stream to {cache_path}")
    return stream

def build_or_load_validation_windows(
    *,
    cache_path: str,
    token_stream: List[int],
    max_seq_len: int,
    stride_frac: float,
    val_samples_target: int,
) -> StableValidationDataset:
    if os.path.exists(cache_path):
        print(f"Loading cached validation windows: {cache_path}")
        with open(cache_path, "rb") as f:
            samples = pickle.load(f)
        print(f"Loaded val windows: {len(samples)}")
        return StableValidationDataset(samples)

    print("Building validation windows (cached)...")
    _ensure_dir(cache_path)

    T = int(max_seq_len)
    stride = max(1, int(T * float(stride_frac)))
    need = int(val_samples_target)

    max_start = len(token_stream) - (T + 1)
    if max_start <= 0:
        raise ValueError("Validation token stream too small for max_seq_len+1")

    samples: List[Tuple[torch.Tensor, torch.Tensor]] = []
    for start in tqdm(range(0, max_start + 1, stride), desc="Chunking val stream"):
        chunk = token_stream[start:start + T + 1]
        if len(chunk) < T + 1:
            break
        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        samples.append((x, y))
        if len(samples) >= need:
            break

    print(f"Built val windows={len(samples)} (stride={stride}, stream_tokens={len(token_stream):,})")
    with open(cache_path, "wb") as f:
        pickle.dump(samples, f)
    print(f"Cached validation windows to {cache_path}")
    return StableValidationDataset(samples)

class WikiTextRandomWindowStream(IterableDataset):
    def __init__(self, token_stream: List[int], max_seq_len: int, train_samples_target: int, seed: int):
        super().__init__()
        self.stream = token_stream
        self.T = int(max_seq_len)
        self.target = int(train_samples_target)
        self.seed = int(seed)
        self.max_start = len(self.stream) - (self.T + 1)
        if self.max_start <= 0:
            raise ValueError("Train token stream too small for max_seq_len+1")

    def __iter__(self):
        info = torch.utils.data.get_worker_info()
        wid = info.id if info is not None else 0
        rng = random.Random(self.seed + 17 * wid)

        yielded = 0
        while yielded < self.target:
            start = rng.randint(0, self.max_start)
            chunk = self.stream[start:start + self.T + 1]
            x = torch.tensor(chunk[:-1], dtype=torch.long)
            y = torch.tensor(chunk[1:], dtype=torch.long)
            yield x, y
            yielded += 1


# =========================================================
# LR schedule
# =========================================================
class WarmupCosine:
    def __init__(self, opt, warmup_steps, total_steps, base_lr):
        self.opt = opt
        self.warmup = int(warmup_steps)
        self.total = int(total_steps)
        self.base = float(base_lr)
        self.step_num = 0

    def step(self):
        self.step_num += 1
        if self.step_num <= self.warmup:
            lr = self.base * self.step_num / max(1, self.warmup)
        else:
            prog = (self.step_num - self.warmup) / max(1, (self.total - self.warmup))
            lr = self.base * 0.5 * (1 + math.cos(math.pi * min(1.0, prog)))
        for g in self.opt.param_groups:
            g["lr"] = lr
        return lr


# =========================================================
# Light metrics helpers (fast + tolerant)
# =========================================================
@torch.no_grad()
def _entropy_mean(read_w: torch.Tensor) -> float:
    # read_w: [B,H,T,K]
    eps = 1e-8
    p = read_w.clamp_min(eps)
    ent = -(p * p.log()).sum(dim=-1)  # [B,H,T]
    return float(ent.mean().detach().cpu().item())

@torch.no_grad()
def _top1freq_mean(read_w: torch.Tensor) -> float:
    # fraction of tokens assigned to most common top1 slot (averaged over batch+heads)
    top1 = read_w.argmax(dim=-1)  # [B,H,T]
    flat = top1.reshape(-1).detach().cpu()
    K = read_w.shape[-1]
    hist = torch.bincount(flat, minlength=K).float()
    denom = hist.sum().clamp_min(1.0)
    return float((hist.max() / denom).item())

@torch.no_grad()
def _write_stats(write_logits: torch.Tensor, last_k: int) -> Tuple[float, float]:
    # write_logits: [B,H,K,T] (already biased/masked)
    w = torch.softmax(write_logits, dim=-1)
    T = w.shape[-1]
    pos = torch.arange(T, device=w.device, dtype=w.dtype).view(1, 1, 1, T)
    com = (w * pos).sum(dim=-1) / max(1.0, float(T - 1))   # [B,H,K] normalized 0..1
    lastk = min(max(1, int(last_k)), T)
    lastk_mass = w[..., -lastk:].sum(dim=-1)               # [B,H,K]
    return float(com.mean().detach().cpu().item()), float(lastk_mass.mean().detach().cpu().item())

@torch.no_grad()
def _get_scalar(info: Dict, key: str) -> Optional[float]:
    v = info.get(key, None)
    if v is None:
        return None
    if isinstance(v, torch.Tensor):
        if v.numel() == 1:
            return float(v.detach().cpu().item())
        return float(v.detach().float().mean().cpu().item())
    if isinstance(v, (int, float)):
        return float(v)
    return None

@torch.no_grad()
def _layer_param_summaries(model) -> Dict[str, float]:
    """
    Truthy parameter summaries:
      - content_read_gamma uses softplus(raw) then clamped to content_read_max_gamma if present
      - slotspace_gate uses softplus(raw)
      - alibi_strength uses module's _alibi_strength when available
    """
    gammas = []
    gates = []
    alibis = []

    for blk in getattr(model, "blocks", []):
        attn = getattr(blk, "asa", None)
        if attn is None:
            continue

        # content read gamma (best-effort)
        if hasattr(attn, "_content_read_gamma_raw"):
            g = F.softplus(attn._content_read_gamma_raw.detach().float())
            mx = getattr(attn, "content_read_max_gamma", None)
            if mx is not None and float(mx) > 0:
                g = g.clamp(max=float(mx))
            gammas.append(float(g.cpu().item()))

        # slot-space gate
        if hasattr(attn, "_slotspace_gate_raw"):
            kg = F.softplus(attn._slotspace_gate_raw.detach().float())
            gates.append(float(kg.cpu().item()))

        # alibi strength (actual used)
        if hasattr(attn, "_alibi_strength"):
            try:
                a = attn._alibi_strength(dtype=torch.float32, device=next(attn.parameters()).device).detach().cpu().item()
                alibis.append(float(a))
            except Exception:
                pass

    out: Dict[str, float] = {}
    if gammas:
        t = torch.tensor(gammas)
        out["content_read_gamma_mean"] = float(t.mean().item())
        out["content_read_gamma_min"]  = float(t.min().item())
        out["content_read_gamma_max"]  = float(t.max().item())
    if gates:
        t = torch.tensor(gates)
        out["slotspace_gate_mean"] = float(t.mean().item())
        out["slotspace_gate_min"]  = float(t.min().item())
        out["slotspace_gate_max"]  = float(t.max().item())
    if alibis:
        t = torch.tensor(alibis)
        out["alibi_strength_mean"] = float(t.mean().item())
        out["alibi_strength_min"]  = float(t.min().item())
        out["alibi_strength_max"]  = float(t.max().item())
    return out


# =========================================================
# Eval
# =========================================================
@torch.no_grad()
def evaluate(model, val_loader, max_batches=50, last_k=8):
    model.eval()

    losses = []
    ent_acc = 0.0
    top1_acc = 0.0
    com_acc = 0.0
    lastk_acc = 0.0

    delta_acc = 0.0
    delta_n = 0
    n_batches = 0

    for i, (xb, yb) in enumerate(val_loader):
        if i >= max_batches:
            break
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)

        with torch.autocast(device_type=device.type, dtype=amp_dtype, enabled=use_amp):
            logits, infos = model(xb, return_info=True)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), yb.view(-1))
        losses.append(float(loss.item()))

        # Light metrics: layer-avg per batch
        if infos and infos[0] is not None:
            eL, tL, cL, lL = [], [], [], []
            dL = []
            for info in infos:
                read_w = info.get("read_weights", None)
                wl     = info.get("write_logits", None)

                if read_w is not None:
                    eL.append(_entropy_mean(read_w))
                    tL.append(_top1freq_mean(read_w))
                if wl is not None:
                    com, lastm = _write_stats(wl.to(torch.float32), last_k=last_k)
                    cL.append(com)
                    lL.append(lastm)

                dd = _get_scalar(info, "slotspace_delta_norm")
                if dd is not None:
                    dL.append(dd)

            if eL:
                ent_acc += sum(eL) / len(eL)
                top1_acc += sum(tL) / len(tL)
            if cL:
                com_acc += sum(cL) / len(cL)
                lastk_acc += sum(lL) / len(lL)
            if dL:
                delta_acc += sum(dL) / len(dL)
                delta_n += 1

            n_batches += 1

    mean = sum(losses) / max(1, len(losses))
    ppl = float(math.exp(min(20.0, mean)))

    stats: Dict[str, float] = {}
    if n_batches > 0:
        stats["entropy_mean"] = ent_acc / n_batches
        stats["top1freq_mean"] = top1_acc / n_batches
        stats["write_com_mean"] = com_acc / n_batches
        stats["write_lastk_mass_mean"] = lastk_acc / n_batches
    if delta_n > 0:
        stats["slotspace_delta_norm"] = delta_acc / delta_n

    # Parameter summaries (truthy, cheap)
    try:
        stats.update(_layer_param_summaries(model))
    except Exception:
        pass

    model.train()
    return mean, ppl, stats


# =========================================================
# Checkpointing
# =========================================================
def save_ckpt(path, cfg, model, opt, step, best_val):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(
        {"cfg": asdict(cfg), "model": model.state_dict(), "opt": opt.state_dict(),
         "step": step, "best_val": best_val},
        path,
    )
    print(f"✓ Saved {path}")


# =========================================================
# Pretty stats formatting (not slow)
# =========================================================
def _fmt_stats(stats: Dict[str, float], last_k: int) -> str:
    keys = [
        "alibi_strength_mean",
        "entropy_mean",
        "top1freq_mean",
        "write_com_mean",
        "write_lastk_mass_mean",
        "content_read_gamma_mean",
        "content_read_gamma_max",
        "slotspace_gate_mean",
        "slotspace_gate_max",
        "slotspace_delta_norm",
    ]
    parts = []
    for k in keys:
        if k in stats:
            if k == "write_lastk_mass_mean":
                parts.append(f"{k}(last_k={last_k})={stats[k]:.4f}")
            else:
                parts.append(f"{k}={stats[k]:.4f}")
    return " | ".join(parts)


# =========================================================
# Train
# =========================================================
def train_asm(cfg: ASMTrainConfig):
    random.seed(cfg.seed)
    torch.manual_seed(cfg.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(cfg.seed)

    # ---------- Data prep (cached streams) ----------
    os.makedirs(cfg.cache_dir, exist_ok=True)
    train_stream_cache = os.path.join(cfg.cache_dir, f"{cfg.dataset_config}_train_stream.pkl")
    val_stream_cache   = os.path.join(cfg.cache_dir, f"{cfg.dataset_config}_val_stream.pkl")

    train_stream = build_or_load_token_stream(
        cache_path=train_stream_cache,
        dataset_name=cfg.dataset_name,
        dataset_config=cfg.dataset_config,
        split="train",
        tokenizer_name=cfg.tokenizer_name,
        min_chars=1,
        add_eos_between_rows=True,
    )
    val_stream = build_or_load_token_stream(
        cache_path=val_stream_cache,
        dataset_name=cfg.dataset_name,
        dataset_config=cfg.dataset_config,
        split="validation",
        tokenizer_name=cfg.tokenizer_name,
        min_chars=1,
        add_eos_between_rows=True,
    )

    val_dataset = build_or_load_validation_windows(
        cache_path=cfg.val_windows_cache,
        token_stream=val_stream,
        max_seq_len=cfg.max_seq_len,
        stride_frac=cfg.stride_frac_val,
        val_samples_target=cfg.val_samples_target,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=torch.cuda.is_available(),
    )

    train_ds = WikiTextRandomWindowStream(
        token_stream=train_stream,
        max_seq_len=cfg.max_seq_len,
        train_samples_target=cfg.train_samples_target,
        seed=cfg.seed,
    )
    train_loader = DataLoader(
        train_ds,
        batch_size=cfg.batch_size,
        num_workers=3,
        pin_memory=torch.cuda.is_available(),
    )

    # ---------- Model ----------
    model = ASMLanguageModel(
        vocab_size=cfg.vocab_size,
        embed_dim=cfg.embed_dim,
        num_layers=cfg.num_layers,
        num_heads=cfg.num_heads,
        num_slots=cfg.num_slots,
        max_seq_len=cfg.max_seq_len,
        mlp_ratio=cfg.mlp_ratio,
        dropout=cfg.dropout,

        read_temperature=cfg.read_temperature,
        write_temperature=cfg.write_temperature,
        state_fp32=cfg.state_fp32,
        slot_dropout=cfg.slot_dropout,
        normalize_k=cfg.normalize_k,

        tie_weights=cfg.tie_weights,

        use_abs_pos=cfg.use_abs_pos,

        use_rope_keys=cfg.use_rope_keys,
        rope_base=cfg.rope_base,
        use_alibi_write=cfg.use_alibi_write,
        alibi_strength_init=cfg.alibi_strength_init,
        learn_alibi_strength=cfg.learn_alibi_strength,
        min_strength=cfg.min_strength,

        use_content_read=cfg.use_content_read,
        content_read_init=cfg.content_read_init,
        content_read_max_gamma=cfg.content_read_max_gamma,

        use_slotspace_refine=cfg.use_slotspace_refine,
        slotspace_dim=cfg.slotspace_dim,
        slotspace_gate_init=cfg.slotspace_gate_init,
        slotspace_dropout=cfg.slotspace_dropout,
        slotspace_signed_weights=cfg.slotspace_signed_weights,

        use_rope_slotspace=cfg.use_rope_slotspace,
        rope_base_slotspace=cfg.rope_base_slotspace,

        write_chunk_size=cfg.write_chunk_size,
        slotspace_chunk_size=cfg.slotspace_chunk_size,
    ).to(device)

    out_dir = os.path.join(cfg.output_dir, cfg.tag)
    os.makedirs(out_dir, exist_ok=True)

    n_params = sum(p.numel() for p in model.parameters())
    print("=" * 108)
    print(f"Training [{cfg.tag}] on {cfg.dataset_name}/{cfg.dataset_config}")
    print(f"Params: {n_params:,}")
    print(f"Train tokens: {len(train_stream):,} | Val tokens: {len(val_stream):,} | Val windows: {len(val_dataset):,}")
    print(f"T={cfg.max_seq_len} | val_stride_frac={cfg.stride_frac_val} | last_k={cfg.analytics_last_k}")
    print(f"Chunks: write={cfg.write_chunk_size} | slotspace={cfg.slotspace_chunk_size} | amp={use_amp}({amp_dtype}) | state_fp32={cfg.state_fp32}")
    print(f"RoPE: keys={cfg.use_rope_keys}(base={cfg.rope_base:g}) | slotspace={cfg.use_rope_slotspace}(base={cfg.rope_base_slotspace:g})")
    print("=" * 108)

    opt = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate, weight_decay=cfg.weight_decay, betas=cfg.betas)
    sched = WarmupCosine(opt, cfg.warmup_steps, cfg.total_steps, cfg.learning_rate)

    # ---------- Initial eval ----------
    best_val = float("inf")
    vloss, vppl, vstats = evaluate(model, val_loader, max_batches=cfg.eval_max_batches, last_k=cfg.analytics_last_k)
    best_val = vloss
    save_ckpt(os.path.join(out_dir, "best.pt"), cfg, model, opt, 0, best_val)

    print(f"[VAL step 0] loss={vloss:.3f} ppl={vppl:.2f}")
    if vstats:
        print("  " + _fmt_stats(vstats, last_k=cfg.analytics_last_k))

    # ---------- Training loop ----------
    running = 0.0
    step = 0
    t_last = time.time()

    pbar = tqdm(total=cfg.total_steps, desc=f"[{cfg.tag}]")
    for xb, yb in train_loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        with torch.autocast(device_type=device.type, dtype=amp_dtype, enabled=use_amp):
            logits = model(xb)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), yb.view(-1))

        opt.zero_grad(set_to_none=True)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)
        opt.step()
        lr = sched.step()

        step += 1
        running += float(loss.item())
        pbar.update(1)

        if step % cfg.log_interval == 0:
            avg = running / cfg.log_interval
            running = 0.0

            ps = _layer_param_summaries(model)

            it_s = cfg.log_interval / max(1e-9, (time.time() - t_last))
            t_last = time.time()

            postfix = {
                "loss": f"{avg:.3f}",
                "ppl": f"{math.exp(min(20.0, avg)):.2f}",
                "lr": f"{lr:.2e}",
                "it/s": f"{it_s:.2f}",
            }
            if "content_read_gamma_mean" in ps: postfix["γμ"] = f"{ps['content_read_gamma_mean']:.3f}"
            if "slotspace_gate_mean" in ps: postfix["sgμ"] = f"{ps['slotspace_gate_mean']:.3f}"
            pbar.set_postfix(postfix)

            msg = f"[step {step}] train_loss={avg:.3f} ppl={math.exp(min(20.0, avg)):.2f} lr={lr:.2e} it/s={it_s:.2f}"
            if "content_read_gamma_mean" in ps: msg += f" | content_read_gamma_mean={ps['content_read_gamma_mean']:.4f}"
            if "slotspace_gate_mean" in ps: msg += f" | slotspace_gate_mean={ps['slotspace_gate_mean']:.4f}"
            if "alibi_strength_mean" in ps: msg += f" | alibi_strength_mean={ps['alibi_strength_mean']:.4f}"
            print(msg)

        if step % cfg.eval_interval == 0:
            vloss, vppl, vstats = evaluate(model, val_loader, max_batches=cfg.eval_max_batches, last_k=cfg.analytics_last_k)
            print(f"\n[VAL step {step}] loss={vloss:.3f} ppl={vppl:.2f}")
            if vstats:
                print("  " + _fmt_stats(vstats, last_k=cfg.analytics_last_k))

            if vloss < best_val:
                best_val = vloss
                save_ckpt(os.path.join(out_dir, "best.pt"), cfg, model, opt, step, best_val)

        if step >= cfg.total_steps:
            break

    save_ckpt(os.path.join(out_dir, "final.pt"), cfg, model, opt, step, best_val)
    print(f"[{cfg.tag}] Done. Best val loss: {best_val:.4f}")
    return model

In [ ]:

#@title Load Checkpointed Config, Model

import math
import torch
import torch.nn.functional as F
from typing import List, Dict, Tuple, Optional
import matplotlib.pyplot as plt

# ============================
# USER EDIT SECTION
# ============================

CKPT_PATH = "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_1024d_32h_64sd_64s_128cs_12l/best.pt"


# ppl ? on test, 56.71M
#CKPT_PATH = "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_32sd_16s_35l/best.pt"

# ppl ? on test, ?M
#CKPT_PATH = "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_256d_16a_11l_75ksteps/best.pt"

# ppl ? on test, 79.63M
#CKPT_PATH = "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_16a_15l/best.pt"

# ppl ? on test, ?M
#CKPT_PATH = "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_32a_5l/best.pt"

# ppl ? on test, ?M
#CKPT_PATH = "./drive/MyDrive/asm_outputs/asm_wikitext_3072t_384d_32sd_16s_7l/best.pt"

#ppl ? on test, 91.55M
#CKPT_PATH = "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_640d_64sd_24s_12l/best.pt"

# ppl ? on test, 91.94M
#CKPT_PATH = "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_16sd_8s_21l/best.pt"

# ppl ? on test, 159.48M
#CKPT_PATH = "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_768d_32sd_32cs_17l/best.pt"

# ppl ? on test, 230.59M
#CKPT_PATH = "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_640d_64sd_24s_17l/best.pt"



DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def build_model_from_cfg_dict(cfg: dict):
    cfg_obj = ASMTrainConfig(**cfg)
    model = build_model_from_cfg(cfg_obj)
    return model, cfg_obj

# 2) If your checkpoint stores model state under a different key, edit here
MODEL_STATE_KEY = "model"
CFG_KEY = "cfg"

# ============================
# END USER EDIT SECTION
# ============================

def count_params(model):
    n = sum(p.numel() for p in model.parameters())
    n_train = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return n, n_train

def cfg_summary(cfg):
    return {
        "layers": cfg.num_layers,
        "d_model": cfg.embed_dim,
        "heads": cfg.num_heads,
        "slots": cfg.num_slots,
        "T": cfg.max_seq_len,
        "slotspace": cfg.use_slotspace_refine,
        "slotspace_dim": cfg.slotspace_dim,
        "content_read": cfg.use_content_read,
        "alibi_write": cfg.use_alibi_write,
        "rope_keys": cfg.use_rope_keys,
    }

def print_cfg_summary(cfg, model):
    n, n_train = count_params(model)
    summ = cfg_summary(cfg)
    print("Config:", ", ".join([f"{k}={v}" for k,v in summ.items()]))
    print(f"Params: {n/1e6:.2f}M")


def load_model_and_cfg(ckpt_path: str):
    ckpt = torch.load(ckpt_path, map_location="cpu")
    cfg = ckpt.get(CFG_KEY, None)
    if cfg is None:
        raise KeyError(f"Checkpoint missing '{CFG_KEY}'. Keys: {list(ckpt.keys())}")

    model, cfg_obj = build_model_from_cfg_dict(cfg)
    sd = ckpt.get(MODEL_STATE_KEY, None)
    if sd is None:
        raise KeyError(f"Checkpoint missing '{MODEL_STATE_KEY}'. Keys: {list(ckpt.keys())}")

    missing, unexpected = model.load_state_dict(sd, strict=False)
    print("Loaded state_dict.")
    if missing: print("  Missing keys:", len(missing))
    if unexpected: print("  Unexpected keys:", len(unexpected))

    model = model.to(DEVICE).eval()
    return model, cfg_obj, cfg, ckpt

model, cfg, cfg_dict, ckpt = load_model_and_cfg(CKPT_PATH)
print("✓ Model ready on", DEVICE)

print("cfg keys:", list(cfg_dict.keys()), "...")
print(cfg)

#model, cfg, cfg_dict, ckpt = load_model_and_cfg(CKPT_PATH)
#print_cfg_summary(cfg, model)

In [ ]:

# ============================================================
#@title ASA / ASM: Trajectory Bundle + Energy Basin (B + D combined)
# Single comprehensive cell
# - Collect routing-logit trajectories (per-layer)
# - Fit PCA (2D) on read_logits (head-mean)
# - Plot trajectory bundle
# - Build an energy basin via causal intervention:
#     force routing distribution at each sample’s own last valid position
#     and measure E = -log P(target) averaged over prompts
# - Optional improvements:
#     * per-example t_target (last non-pad)
#     * choose layer for PCA and a (possibly different) layer for intervention
#     * optional multi-layer intervention
#     * optional intervention "where" (read / content_read_only / slotspace_only)
#     * optional smoothing / robust bounds
# ============================================================

import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from typing import List, Optional, Tuple, Dict, Union

# If transformers isn't available in your runtime, replace tokenizer bits with your own tokenizer.
from transformers import AutoTokenizer

# ----------------------------
# User edit section
# ----------------------------
PROMPTS: List[str] = [
    "The capital of France is",
    "In Europe, the city that serves as France's capital is",
    "France's capital city, known for the Eiffel Tower, is",
    "A major European capital located on the Seine is",
    "The French capital is",
]

TARGET_TEXT: str = " Paris"   # IMPORTANT: must be a single token for energy calculation
PCA_LAYER: int = 9 # None         # None => auto pick mid-late layer (min(10, n_layers-1))
INTERVENE_LAYERS: Union[int, List[int]] = [3,7,13] # None  # None => [PCA_LAYER]; int => single; list => multiple

# Grid + plotting
GRID_N: int = 60              # try 40-60 while iterating; 70+ is expensive
GRID_PAD: float = 0.35        # pad around trajectory extents
CONTOUR_LEVELS: int = 35

# Routing intervention settings
# Where to apply slot masks if provided (matches your API)
SLOT_MASK: Optional[torch.Tensor] = None         # shape [K], 1 keep / 0 mask
SLOT_MASK_WHERE: str = "read"                    # "read" | "content_read_only" | "slotspace_only"
SLOT_MASK_SCOPE: str = "all"                     # "all" | "last_pos_only"

# Control for the forced routing distribution mapping
FORCED_ROUTING_TEMP: float = 1.0   # temp used to convert PCA-inverse logits -> distribution; keep 1.0 unless you want sharper basins
FORCED_ROUTING_FLOOR: float = 0.0  # e.g. 1e-6 to avoid exact zeros; usually not needed

# Performance knobs
USE_HALF_FOR_INFOS: bool = False   # set True only if you know your info tensors are safe in fp16
DEVICE = DEVICE  # assumes you already set DEVICE earlier

# ----------------------------
# Sanity: model exists
# ----------------------------
assert "model" in globals(), "Expected `model` to exist (from your checkpoint load cell)."
assert "cfg" in globals(),   "Expected `cfg` to exist (your ASMTrainConfig)."

# ----------------------------
# Tokenizer
# ----------------------------
def get_tokenizer(cfg):
    tok = AutoTokenizer.from_pretrained(cfg.tokenizer_name, use_fast=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok

def encode_prompts(tokenizer, prompts: List[str], max_len: int, device):
    enc = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_len,
    )
    input_ids = enc["input_ids"].to(device)
    attn_mask = enc["attention_mask"].to(device)
    return input_ids, attn_mask

def token_id_single(tokenizer, text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) != 1:
        raise ValueError(
            f"TARGET_TEXT {text!r} maps to {len(ids)} tokens {ids}. "
            "Pick a single-token target (often include leading space for GPT2 tokenizers)."
        )
    return ids[0]

tokenizer = get_tokenizer(cfg)
target_id = token_id_single(tokenizer, TARGET_TEXT)

# ----------------------------
# Helpers: PCA
# ----------------------------
def fit_pca_2d(Z_btk: np.ndarray):
    # Z_btk: [B,T,K]
    X = Z_btk.reshape(-1, Z_btk.shape[-1])  # [B*T, K]
    mean = X.mean(axis=0, keepdims=True)    # [1,K]
    Xc = X - mean
    # SVD PCA
    _, _, Vt = np.linalg.svd(Xc, full_matrices=False)
    W = Vt[:2].T  # [K,2]
    return mean.squeeze(0), W

def pca_project(Z_btk: np.ndarray, mean_k: np.ndarray, W_k2: np.ndarray):
    X = Z_btk.reshape(-1, Z_btk.shape[-1])
    X2 = (X - mean_k) @ W_k2
    return X2.reshape(Z_btk.shape[0], Z_btk.shape[1], 2)  # [B,T,2]

def pca_inverse(xy_2: np.ndarray, mean_k: np.ndarray, W_k2: np.ndarray):
    # [2] -> [K]
    return mean_k + (W_k2 @ xy_2)

def robust_bounds(x: np.ndarray, qlo=0.01, qhi=0.99):
    lo = np.quantile(x, qlo)
    hi = np.quantile(x, qhi)
    return float(lo), float(hi)

# ----------------------------
# Routing override callable
# ----------------------------
class ForceRoutingAtLastValid:
    def __init__(self, t_targets_b: torch.Tensor, r_forced_bhk: torch.Tensor):
        """
        t_targets_b: [B] last valid position index per example (global time index)
        r_forced_bhk: [B,H,K] forced routing distribution
        """
        self.t_targets_b = t_targets_b
        self.r_forced_bhk = r_forced_bhk

    def __call__(self, t0, t1, read_logits, read_logits_key, read_logits_content, ctx):
        # Default softmax routing on combined logits
        rtemp = float(ctx.get("rtemp", 1.0))
        w = torch.softmax(read_logits / max(1e-6, rtemp), dim=-1)  # [B,H,L,K]

        # Override each sample's target position if it falls within this chunk
        B = w.shape[0]
        for b in range(B):
            tt = int(self.t_targets_b[b].item())
            if t0 <= tt < t1:
                local = tt - t0
                w[b, :, local, :] = self.r_forced_bhk[b].to(dtype=w.dtype, device=w.device)

        w = torch.nan_to_num(w, nan=0.0, posinf=0.0, neginf=0.0)
        w = w.clamp_min(0.0)
        w = w / w.sum(dim=-1, keepdim=True).clamp_min(1e-8)
        return w

# ----------------------------
# Run model with infos
# ----------------------------
@torch.no_grad()
def run_with_info(model, input_ids, attention_mask):
    logits, infos = model(
        input_ids,
        attention_mask=attention_mask,
        return_info=True,
        routing_mode="softmax",
        routing_topk=2,
        routing_noise=None,
        slot_mask=SLOT_MASK,
        slot_mask_where=SLOT_MASK_WHERE,
        slot_mask_scope=SLOT_MASK_SCOPE,
    )
    return logits, infos

# ----------------------------
# Energy computation (supports multi-layer intervention)
# ----------------------------
@torch.no_grad()
def energy_at_xy(
    model,
    *,
    input_ids: torch.Tensor,
    attention_mask: torch.Tensor,
    layers: List[int],
    t_targets_b: torch.Tensor,       # [B]
    xy_2: np.ndarray,                # [2]
    mean_k: np.ndarray,              # [K]
    W_k2: np.ndarray,                # [K,2]
    target_id: int,
):
    # Map xy -> K logits -> forced distribution
    z_k = pca_inverse(xy_2, mean_k, W_k2)  # [K]
    z = torch.tensor(z_k, device=DEVICE, dtype=torch.float32)

    # Build [B,H,K] per layer (H,K must match that layer)
    overrides = []
    old = []

    try:
        for li in layers:
            asa = model.blocks[li].asa
            H = asa.num_heads
            K = asa.num_slots
            assert z.numel() == K, f"PCA space K={z.numel()} but layer {li} has K={K}. Use same K layers/models."

            logits = z / max(1e-6, float(FORCED_ROUTING_TEMP))
            r = torch.softmax(logits, dim=-1)

            if FORCED_ROUTING_FLOOR and FORCED_ROUTING_FLOOR > 0:
                r = (r + float(FORCED_ROUTING_FLOOR))
                r = r / r.sum()

            r_bhk = r.view(1, 1, K).expand(input_ids.shape[0], H, K).contiguous()

            old.append(asa.routing_override)
            override = ForceRoutingAtLastValid(t_targets_b=t_targets_b, r_forced_bhk=r_bhk)
            asa.routing_override = override
            overrides.append(override)

        # Forward pass with overrides active
        logits_out = model(
            input_ids,
            attention_mask=attention_mask,
            return_info=False,
            routing_mode="softmax",
            routing_topk=2,
            routing_noise=None,
            slot_mask=SLOT_MASK,
            slot_mask_where=SLOT_MASK_WHERE,
            slot_mask_scope=SLOT_MASK_SCOPE,
        )

        # Compute per-example -log P(target) at each example’s own t_target, then average
        logp = F.log_softmax(logits_out, dim=-1)  # [B,T,V]
        B = logp.shape[0]
        idx = t_targets_b.view(B, 1, 1).expand(B, 1, 1)
        # gather logp[b, t_targets_b[b], target_id]
        gather_t = logp.gather(dim=1, index=idx.expand(B, 1, logp.shape[-1]))[:, 0, :]  # [B,V]
        tgt = gather_t[:, target_id]  # [B]
        E = (-tgt).mean().item()

    finally:
        # restore
        for li, prev in zip(layers, old):
            model.blocks[li].asa.routing_override = prev

    return E

# ----------------------------
# Plotting
# ----------------------------
def plot_trajectories(traj_xy: np.ndarray, prompts: List[str], title: str):
    plt.figure(figsize=(8.5, 6.5))
    for b in range(traj_xy.shape[0]):
        xy = traj_xy[b]
        plt.plot(xy[:, 0], xy[:, 1], linewidth=1.5)
        plt.scatter([xy[-1, 0]], [xy[-1, 1]], s=30)
    plt.title(title)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.tight_layout()

def plot_basin_with_trajectories(XX, YY, Egrid, traj_xy, title: str):
    plt.figure(figsize=(9.5, 7.5))
    cs = plt.contourf(XX, YY, Egrid, levels=int(CONTOUR_LEVELS))
    plt.colorbar(cs, label="-log P(target)")
    for b in range(traj_xy.shape[0]):
        xy = traj_xy[b]
        plt.plot(xy[:, 0], xy[:, 1], linewidth=1.5)
        plt.scatter([xy[-1, 0]], [xy[-1, 1]], s=30)
    plt.title(title)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.tight_layout()

# ----------------------------
# Execute pipeline
# ----------------------------
# 1) Encode
input_ids, attention_mask = encode_prompts(tokenizer, PROMPTS, max_len=cfg.max_seq_len, device=DEVICE)

# 2) Determine layer(s)
n_layers = len(model.blocks)
if PCA_LAYER is None:
    PCA_LAYER = min(10, n_layers - 1)
if INTERVENE_LAYERS is None:
    INTERVENE_LAYERS = [PCA_LAYER]
elif isinstance(INTERVENE_LAYERS, int):
    INTERVENE_LAYERS = [INTERVENE_LAYERS]
else:
    INTERVENE_LAYERS = list(INTERVENE_LAYERS)

print(f"[INFO] Using PCA_LAYER={PCA_LAYER}, INTERVENE_LAYERS={INTERVENE_LAYERS}")

# 3) Last valid position per example
t_targets_b = attention_mask.long().sum(dim=1) - 1  # [B]
t_targets_b = t_targets_b.clamp_min(0)

# 4) Baseline forward with infos
logits_base, infos_base = run_with_info(model, input_ids, attention_mask)

# 5) Extract read_logits at PCA_LAYER and build trajectory vectors
read_logits = infos_base[PCA_LAYER]["read_logits"]  # [B,H,T,K] float32 (as returned)
if USE_HALF_FOR_INFOS:
    read_logits = read_logits.half()

B, H, T, K = read_logits.shape
Z = read_logits.mean(dim=1)  # [B,T,K]
Z_np = Z.detach().cpu().numpy()

# 6) PCA fit + project
mean_k, W_k2 = fit_pca_2d(Z_np)
traj_xy = pca_project(Z_np, mean_k, W_k2)  # [B,T,2]

# 7) Trajectory plot
plot_trajectories(traj_xy, PROMPTS, title=f"Trajectory bundle in PCA(read_logits) (layer {PCA_LAYER})")
plt.show()

# 8) Grid bounds (robust, to avoid outliers dominating)
x_all = traj_xy[..., 0].reshape(-1)
y_all = traj_xy[..., 1].reshape(-1)
xmin, xmax = robust_bounds(x_all, 0.01, 0.99)
ymin, ymax = robust_bounds(y_all, 0.01, 0.99)
dx = (xmax - xmin) * float(GRID_PAD)
dy = (ymax - ymin) * float(GRID_PAD)

xs = np.linspace(xmin - dx, xmax + dx, int(GRID_N))
ys = np.linspace(ymin - dy, ymax + dy, int(GRID_N))
XX, YY = np.meshgrid(xs, ys)

print(f"[INFO] Grid: {GRID_N}x{GRID_N} over x=[{xs[0]:.3f},{xs[-1]:.3f}], y=[{ys[0]:.3f},{ys[-1]:.3f}]")

# 9) Energy basin computation
Egrid = np.zeros_like(XX, dtype=np.float32)

# Small speed hack: disable grad + ensure eval
model.eval()
torch.set_grad_enabled(False)

# Compute energy at each grid point
for i in range(YY.shape[0]):
    for j in range(XX.shape[1]):
        xy = np.array([XX[i, j], YY[i, j]], dtype=np.float32)
        Egrid[i, j] = energy_at_xy(
            model,
            input_ids=input_ids,
            attention_mask=attention_mask,
            layers=INTERVENE_LAYERS,
            t_targets_b=t_targets_b,
            xy_2=xy,
            mean_k=mean_k,
            W_k2=W_k2,
            target_id=target_id,
        )
    if (i + 1) % max(1, (YY.shape[0] // 10)) == 0:
        print(f"[INFO] Basin progress: {i+1}/{YY.shape[0]} rows")

# 10) Basin + trajectories overlay
plot_basin_with_trajectories(
    XX, YY, Egrid, traj_xy,
    title=(
        f"Energy basin (forced routing) + trajectories\n"
        f"target={TARGET_TEXT!r} | PCA layer={PCA_LAYER} | intervene layers={INTERVENE_LAYERS} | last-valid t per sample"
    )
)
plt.show()

# 11) Report baseline target logprob (for reference)
with torch.no_grad():
    logp_base = F.log_softmax(logits_base, dim=-1)  # [B,T,V]
    B0 = logp_base.shape[0]
    # gather per-example at last valid position
    idx = t_targets_b.view(B0, 1, 1).expand(B0, 1, 1)
    gather_t = logp_base.gather(dim=1, index=idx.expand(B0, 1, logp_base.shape[-1]))[:, 0, :]
    tgt = gather_t[:, target_id]
    print(f"[INFO] Baseline mean -logP(target) at last valid pos: {(-tgt).mean().item():.4f}")
    print(f"[INFO] Baseline mean P(target): {tgt.exp().mean().item():.6f}")

In [ ]:

# ============================
#@title Visualization A: Concept Constellations on a Simplex Sphere
#   - Extract ASA routing logits (pre-softmax) per token
#   - Reduce to 3D (PCA) and project to S² (unit sphere)
#   - Plot each prompt as an arc (trajectory) on a “globe”
#   - Optionally color by final predicted token or by semantic class
# ============================

import numpy as np
import torch
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# ----------------------------
# 0) Tokenizer (robust)
# ----------------------------
try:
    from transformers import GPT2TokenizerFast
    tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
except Exception as e:
    tokenizer = None
    print("WARNING: Could not load GPT2TokenizerFast. If you already have `tokenizer`, ignore this.")
    print("Error:", repr(e))

# ----------------------------
# 1) Prompt set (edit here)
# ----------------------------
# Provide either:
#  - prompts: list[str], and optional classes: list[str] same length
# Or keep defaults to validate plumbing.

prompts = [
    "The capital of France is",
    "France's capital city is",
    "Paris is the capital of",
    "In Europe, the capital of France is",
    "The Eiffel Tower is located in",
]

# Optional: semantic labels (same length as prompts). If None, we color by final token.
classes = [
    "capital",
    "capital",
    "capital_reverse",
    "capital_context",
    "location_landmark",
]

assert len(prompts) == len(classes), "If you set `classes`, it must match `prompts` length."

# ----------------------------
# 2) Which layer / aggregation settings
# ----------------------------
LAYER_IDX = 9           # choose which ASA layer to inspect
AGG_HEADS = "mean"      # "mean" or "none"
TOKEN_RANGE = "all"     # "all" | "last_k"
LAST_K = 64             # used if TOKEN_RANGE == "last_k"

# ----------------------------
# 3) Helpers
# ----------------------------
def _safe_decode(tok_ids):
    if tokenizer is None:
        return str(tok_ids)
    return tokenizer.decode(tok_ids)

def get_last_valid_index(attn_mask_1d: torch.Tensor) -> int:
    # attn_mask_1d: [T] {0,1}
    idx = int(attn_mask_1d.long().sum().item()) - 1
    return max(idx, 0)

@torch.no_grad()
def run_and_extract_read_logits(model, text: str, layer_idx: int):
    """
    Returns:
      read_logits_seq: [T, K] float32 (pre-softmax combined logits, head-aggregated if AGG_HEADS='mean')
      attn_mask: [T] bool
      final_pred_token: str (decoded argmax at last valid position)
      input_ids: [T] long
    """
    assert tokenizer is not None, "Tokenizer is required here. Please define `tokenizer`."

    enc = tokenizer(text, return_tensors="pt")
    input_ids = enc["input_ids"].to(DEVICE)
    attn_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

    # model forward
    logits, infos = model(
        input_ids=input_ids,
        attention_mask=attn_mask,
        return_info=True,
        routing_mode="softmax",   # keep default routing
    )

    # select layer
    info = infos[layer_idx]
    rl = info["read_logits"]  # [B,H,T,K] float32 (per your module)
    # sanity
    if rl is None:
        raise RuntimeError("info['read_logits'] is None. Ensure return_info=True and ASA returns read_logits.")

    # squeeze B=1
    rl = rl[0]  # [H,T,K]
    if AGG_HEADS == "mean":
        rl = rl.mean(dim=0)   # [T,K]
    elif AGG_HEADS == "none":
        # keep heads: [H,T,K] -> we will flatten H into feature dim later
        pass
    else:
        raise ValueError(f"Unknown AGG_HEADS={AGG_HEADS}")

    # final predicted token at last valid position
    last_t = get_last_valid_index(attn_mask[0])
    pred_id = int(torch.argmax(logits[0, last_t]).item())
    pred_tok = _safe_decode([pred_id])

    # token trimming choice
    T = input_ids.shape[1]
    if TOKEN_RANGE == "last_k":
        t0 = max(0, T - LAST_K)
        if AGG_HEADS == "mean":
            rl = rl[t0:T, :]                  # [L,K]
        else:
            rl = rl[:, t0:T, :]               # [H,L,K]
        input_ids = input_ids[:, t0:T]
        attn_mask = attn_mask[:, t0:T]

    return rl.detach().cpu().float(), attn_mask[0].detach().cpu().bool(), pred_tok, input_ids[0].detach().cpu()

def stack_features(read_logits, agg_heads: str):
    """
    Convert per-prompt read_logits into a 2D array of points for PCA.
      if agg_heads == "mean":  read_logits is [T,K] -> points [T,K]
      if agg_heads == "none":  read_logits is [H,T,K] -> points [T,H*K]
    """
    if agg_heads == "mean":
        return read_logits.numpy()  # [T,K]
    else:
        H, T, K = read_logits.shape
        return read_logits.permute(1,0,2).reshape(T, H*K).numpy()  # [T,H*K]

def sphere_project(x3: np.ndarray, eps: float = 1e-9) -> np.ndarray:
    """
    x3: [N,3]
    return: [N,3] normalized to unit sphere
    """
    n = np.linalg.norm(x3, axis=1, keepdims=True)
    return x3 / np.maximum(n, eps)

# ----------------------------
# 4) Extract trajectories
# ----------------------------
trajectories = []
all_points = []
meta = []

for i, (p, cls) in enumerate(zip(prompts, classes)):
    rl, am, pred_tok, inp_ids = run_and_extract_read_logits(model, p, LAYER_IDX)

    # filter valid tokens only (mask)
    # (for GPT2, attention mask is typically all-ones, but we keep it correct)
    valid_idx = np.where(am.numpy())[0]
    rl_valid = rl if AGG_HEADS == "mean" else rl  # same object; we slice below

    if AGG_HEADS == "mean":
        rl_valid = rl_valid[valid_idx, :]                  # [Tv,K]
    else:
        rl_valid = rl_valid[:, valid_idx, :]               # [H,Tv,K]

    pts = stack_features(rl_valid, AGG_HEADS)              # [Tv,D]
    trajectories.append(pts)
    all_points.append(pts)

    # token strings for hover-like inspection (printed later)
    toks = [_safe_decode([int(t)]) for t in inp_ids.numpy().tolist()]
    toks_valid = [toks[j] for j in valid_idx.tolist()]

    meta.append({
        "prompt": p,
        "class": cls,
        "pred_tok": pred_tok,
        "tokens": toks_valid,
        "n": pts.shape[0],
    })

X = np.concatenate(all_points, axis=0)  # [N_total, D]

print(f"[OK] Collected {len(trajectories)} trajectories | total points: {X.shape[0]} | feature dim: {X.shape[1]}")
print("Layer:", LAYER_IDX, "| Heads aggregation:", AGG_HEADS, "| Token range:", TOKEN_RANGE)

# ----------------------------
# 5) PCA -> 3D -> Sphere
# ----------------------------
pca = PCA(n_components=3, random_state=0)
X3 = pca.fit_transform(X)               # [N,3]
X3s = sphere_project(X3)                # [N,3]

# Split back per-trajectory
splits = np.cumsum([tr.shape[0] for tr in trajectories])[:-1]
traj3s = np.split(X3s, splits, axis=0)

print("Explained variance (PC1..3):", np.round(pca.explained_variance_ratio_, 4))

# ----------------------------
# 6) Plot: Simplex Sphere “globe” + arcs
# ----------------------------
def plot_globe_with_trajectories(traj3_list, meta, color_by="pred_tok"):
    """
    color_by:
      - "pred_tok" (final predicted token string)
      - "class"    (semantic class label you provided)
      - "index"    (each trajectory unique color)
    """
    fig = plt.figure(figsize=(10, 9))
    ax = fig.add_subplot(111, projection="3d")
    ax.set_title(f"Concept Constellations on a Simplex Sphere\n(read_logits PCA→S²) | layer={LAYER_IDX} | color_by={color_by}")

    # draw a faint sphere wireframe
    u = np.linspace(0, 2*np.pi, 40)
    v = np.linspace(0, np.pi, 20)
    xs = np.outer(np.cos(u), np.sin(v))
    ys = np.outer(np.sin(u), np.sin(v))
    zs = np.outer(np.ones_like(u), np.cos(v))
    ax.plot_wireframe(xs, ys, zs, rstride=2, cstride=2, linewidth=0.3, alpha=0.15)

    # grouping key
    if color_by == "pred_tok":
        keys = [m["pred_tok"] for m in meta]
    elif color_by == "class":
        keys = [m["class"] for m in meta]
    elif color_by == "index":
        keys = [f"traj_{i}" for i in range(len(meta))]
    else:
        raise ValueError("color_by must be one of: pred_tok, class, index")

    uniq = list(dict.fromkeys(keys))  # preserve order
    key_to_id = {k:i for i,k in enumerate(uniq)}

    for i, (pts, m) in enumerate(zip(traj3_list, meta)):
        k = keys[i]
        cid = key_to_id[k]

        # trajectory curve
        ax.plot(pts[:,0], pts[:,1], pts[:,2], linewidth=2.0, alpha=0.9)

        # start/end markers
        ax.scatter([pts[0,0]], [pts[0,1]], [pts[0,2]], s=40, marker="o", alpha=0.9)
        ax.scatter([pts[-1,0]], [pts[-1,1]], [pts[-1,2]], s=60, marker="X", alpha=0.95)

        # label at end
        label = f"{i}: {k}"
        ax.text(pts[-1,0], pts[-1,1], pts[-1,2], label, fontsize=9)

    ax.set_xlabel("PC1 (sphere)")
    ax.set_ylabel("PC2 (sphere)")
    ax.set_zlabel("PC3 (sphere)")
    ax.set_box_aspect([1,1,1])
    plt.tight_layout()
    plt.show()

# Choose coloring mode:
# - If you want the “many routes lead to the same city” punchline: use pred_tok.
# - If you want semantic grouping: use class.
plot_globe_with_trajectories(traj3s, meta, color_by="pred_tok")

# ----------------------------
# 7) Optional: quick textual inspection
# ----------------------------
for i, m in enumerate(meta):
    print(f"\n--- Trajectory {i} ---")
    print("class:", m["class"], "| final_pred_tok:", repr(m["pred_tok"]))
    print("prompt:", m["prompt"])
    print("tokens (last 12):", m["tokens"][-12:])

# ----------------------------
# 8) Optional: 2D fallback (PC1/PC2) WITHOUT sphere normalization
#     (Useful if you want a “flat map” of the constellation)
# ----------------------------
def plot_flat_constellation(traj3_list, meta, use_sphere=True, color_by="pred_tok"):
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111)
    title = "Flat Constellation (PCA)" + (" on S² (PC normalized)" if use_sphere else " (raw PC coords)")
    ax.set_title(f"{title} | layer={LAYER_IDX} | color_by={color_by}")

    if color_by == "pred_tok":
        keys = [m["pred_tok"] for m in meta]
    elif color_by == "class":
        keys = [m["class"] for m in meta]
    else:
        keys = [f"traj_{i}" for i in range(len(meta))]

    for i, (pts, m) in enumerate(zip(traj3_list, meta)):
        ax.plot(pts[:,0], pts[:,1], linewidth=2.0, alpha=0.9)
        ax.scatter([pts[0,0]], [pts[0,1]], s=35, marker="o", alpha=0.9)
        ax.scatter([pts[-1,0]], [pts[-1,1]], s=55, marker="X", alpha=0.95)
        ax.text(pts[-1,0], pts[-1,1], f"{i}:{keys[i]}", fontsize=9)

    ax.set_xlabel("PC1")
    ax.set_ylabel("PC2")
    ax.grid(True, alpha=0.2)
    plt.tight_layout()
    plt.show()

# Uncomment if you want the flat view too:
# plot_flat_constellation(traj3s, meta, use_sphere=True, color_by="pred_tok")

In [ ]:

# ============================================================
#@title Visualization C: Slot Simplex Barycentric Animation (faithful)
#   - Cluster slots into M meta-slots (M=3 or 4)
#   - Aggregate routing mass per meta-slot per token
#   - Plot barycentric point on simplex (triangle / tetrahedron)
#   - Animate token-by-token with trails
#   - Vertex colors reflect “affinity” to the last token via
#     content-read logits at the last valid position (proxy for
#     slot_state similarity; mechanically available in info)
# ============================================================

import math
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# If you want automatic meta-slot grouping:
from sklearn.cluster import KMeans

# ----------------------------
# User knobs
# ----------------------------
LAYER_IDX = 7                  # ASA layer to visualize
META_SLOTS = 3                  # 3 -> triangle, 4 -> tetrahedron
PROMPTS = [
    "The capital of France is",
    "France's capital city is",
    "Paris is the capital of",
    "In Europe, the capital of France is",
    "The Eiffel Tower is located in",
]
# Optional names for the META_SLOTS vertices (purely cosmetic)
META_NAMES = None               # e.g. ["geo","hist","narr"] or None

# How to aggregate heads:
HEAD_AGG = "mean"               # "mean" (recommended) or "none" (not supported here)

# Slot grouping method:
GROUPING = "kmeans_slotkeys"    # "kmeans_slotkeys" or "manual"
# If manual, define groups as list[list[int]] of length META_SLOTS
MANUAL_GROUPS = None            # e.g. [[0,1,2,3],[4,5,6,7],[8,9,10,11]] etc.

# Animation options
TAIL = 40                       # number of past points shown in trail
INTERVAL_MS = 1000                # frame interval
SHOW_START_END = True
USE_LAST_VALID_T = True         # if False, uses final token index T-1
LIMIT_TOKENS = None             # None or int (e.g., 128) to cap length for speed

# Vertex coloring proxy:
# We use mean content-read logits at last valid position per meta-slot.
# If content-read logits absent, fallback to combined read_logits.
VERTEX_SCORE_SOURCE = "content" # "content" or "combined"

# ------------------------------------------------------------
# Helpers
# ------------------------------------------------------------
def _get_last_valid_index(attn_mask_1d: torch.Tensor) -> int:
    idx = int(attn_mask_1d.long().sum().item()) - 1
    return max(idx, 0)

def _barycentric_to_xy(w3: np.ndarray) -> np.ndarray:
    """w3: [T,3] -> xy: [T,2] on an equilateral triangle"""
    # vertices
    v0 = np.array([0.0, 0.0])
    v1 = np.array([1.0, 0.0])
    v2 = np.array([0.5, math.sqrt(3)/2.0])
    V = np.stack([v0, v1, v2], axis=0)  # [3,2]
    return w3 @ V                       # [T,2]

def _barycentric_to_xyz(w4: np.ndarray) -> np.ndarray:
    """w4: [T,4] -> xyz: [T,3] on a regular tetrahedron"""
    v0 = np.array([0.0, 0.0, 0.0])
    v1 = np.array([1.0, 0.0, 0.0])
    v2 = np.array([0.5, math.sqrt(3)/2.0, 0.0])
    v3 = np.array([0.5, math.sqrt(3)/6.0, math.sqrt(2.0/3.0)])
    V = np.stack([v0, v1, v2, v3], axis=0)  # [4,3]
    return w4 @ V                            # [T,3]

def _normalize01(x: np.ndarray, eps: float = 1e-9) -> np.ndarray:
    lo, hi = np.min(x), np.max(x)
    if hi - lo < eps:
        return np.zeros_like(x)
    return (x - lo) / (hi - lo)

@torch.no_grad()
def extract_routing_and_scores(model, tokenizer, prompt: str, layer_idx: int):
    """
    Returns:
      mass: [T, K] float32  (mean over heads)
      score: [T, K] float32 (mean over heads)  content-read logits or combined read_logits
      tokens: list[str] length T (decoded per token)
      last_t: int last valid position
    """
    enc = tokenizer(prompt, return_tensors="pt")
    input_ids = enc["input_ids"].to(DEVICE)
    attn_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

    logits, infos = model(
        input_ids=input_ids,
        attention_mask=attn_mask,
        return_info=True,
        routing_mode="softmax",
    )
    info = infos[layer_idx]

    rw = info["read_weights"]          # [B,H,T,K]
    rl = info["read_logits"]           # [B,H,T,K] combined
    rlc = info.get("read_logits_content", None)  # [B,H,T,K] or None

    # mean over heads
    rw = rw[0].mean(dim=0).float().cpu()     # [T,K]
    rl = rl[0].mean(dim=0).float().cpu()     # [T,K]
    if rlc is not None:
        rlc = rlc[0].mean(dim=0).float().cpu()  # [T,K]

    # token strings
    toks = [tokenizer.decode([int(t)]) for t in input_ids[0].detach().cpu().tolist()]

    last_t = _get_last_valid_index(attn_mask[0].detach().cpu()) if USE_LAST_VALID_T else (rw.shape[0] - 1)

    # optional cap for speed
    if LIMIT_TOKENS is not None:
        Tcap = int(LIMIT_TOKENS)
        rw = rw[:Tcap]
        rl = rl[:Tcap]
        if rlc is not None:
            rlc = rlc[:Tcap]
        toks = toks[:Tcap]
        last_t = min(last_t, Tcap - 1)

    if VERTEX_SCORE_SOURCE == "content" and rlc is not None:
        score = rlc
    else:
        score = rl

    return rw.numpy(), score.numpy(), toks, last_t

def build_groups_from_slotkeys_kmeans(model, layer_idx: int, meta_slots: int):
    """
    Cluster slots into meta slots using mean slot_keys over heads
    from the ASA module at the chosen layer.
    """
    asa = model.blocks[layer_idx].asa
    with torch.no_grad():
        # slot_keys: [H,K,d] -> mean over heads -> [K,d]
        sk = asa.slot_keys.detach().float().mean(dim=0).cpu().numpy()

    km = KMeans(n_clusters=meta_slots, random_state=0, n_init="auto")
    labels = km.fit_predict(sk)  # [K]

    groups = []
    for m in range(meta_slots):
        idxs = np.where(labels == m)[0].tolist()
        groups.append(idxs)

    # stable ordering: largest group first (optional; makes plots more stable)
    groups = sorted(groups, key=lambda g: (-len(g), g[0] if len(g) else 10**9))
    return groups

def aggregate_to_meta(rw_TK: np.ndarray, score_TK: np.ndarray, groups):
    """
    rw_TK: [T,K] routing masses
    score_TK: [T,K] logits proxy for affinity
    groups: list[list[int]] length M
    Returns:
      meta_mass: [T,M]
      meta_vertex_score: [M] score at last_t aggregated over group
      meta_per_token_score: [T,M] (sometimes useful)
    """
    T, K = rw_TK.shape
    M = len(groups)
    meta_mass = np.zeros((T, M), dtype=np.float32)
    meta_score = np.zeros((T, M), dtype=np.float32)

    for m, idxs in enumerate(groups):
        if len(idxs) == 0:
            continue
        meta_mass[:, m] = rw_TK[:, idxs].sum(axis=1)
        meta_score[:, m] = score_TK[:, idxs].mean(axis=1)

    # renormalize mass (should already sum to 1, but grouping can introduce small drift)
    meta_mass = meta_mass / np.clip(meta_mass.sum(axis=1, keepdims=True), 1e-8, None)
    return meta_mass, meta_score

# ------------------------------------------------------------
# 1) Build / validate groups
# ------------------------------------------------------------
assert META_SLOTS in (3, 4), "META_SLOTS must be 3 or 4 for triangle/tetrahedron."

if GROUPING == "manual":
    assert MANUAL_GROUPS is not None and len(MANUAL_GROUPS) == META_SLOTS, \
        "Provide MANUAL_GROUPS as list of length META_SLOTS."
    groups = MANUAL_GROUPS
elif GROUPING == "kmeans_slotkeys":
    groups = build_groups_from_slotkeys_kmeans(model, LAYER_IDX, META_SLOTS)
else:
    raise ValueError(f"Unknown GROUPING={GROUPING}")

print(f"[OK] META_SLOTS={META_SLOTS} grouping={GROUPING}")
for i, g in enumerate(groups):
    print(f"  meta {i}: n={len(g)} slots -> {g[:10]}{'...' if len(g)>10 else ''}")

if META_NAMES is None:
    META_NAMES = [f"meta_{i}" for i in range(META_SLOTS)]
else:
    assert len(META_NAMES) == META_SLOTS

# ------------------------------------------------------------
# 2) Extract per-prompt barycentric trajectories
# ------------------------------------------------------------
all_trajs = []
all_tokens = []
all_last_t = []
all_vertex_scores = []  # per prompt: [M]

for p in PROMPTS:
    rw, score, toks, last_t = extract_routing_and_scores(model, tokenizer, p, LAYER_IDX)
    meta_mass, meta_score = aggregate_to_meta(rw, score, groups)

    # Vertex score: mean meta_score at last_t (then normalized later)
    vscore = meta_score[last_t].copy()

    all_trajs.append(meta_mass)     # [T,M]
    all_tokens.append(toks)
    all_last_t.append(last_t)
    all_vertex_scores.append(vscore)

# Normalize vertex scores across vertices per prompt for colormap scaling
all_vertex_scores = np.stack(all_vertex_scores, axis=0)  # [P,M]
# global scaling across all prompts for comparability:
vscore_norm = _normalize01(all_vertex_scores.reshape(-1)).reshape(all_vertex_scores.shape)

print("[OK] Extracted trajectories.")
for i, p in enumerate(PROMPTS):
    print(f"  {i}: T={all_trajs[i].shape[0]} last_t={all_last_t[i]} | prompt={p}")

# ------------------------------------------------------------
# 3) Plot + animate
# ------------------------------------------------------------
P = len(PROMPTS)
Tmax = max(tr.shape[0] for tr in all_trajs)

if META_SLOTS == 3:
    # precompute XY
    XY = [_barycentric_to_xy(tr) for tr in all_trajs]  # list of [T,2]

    fig, ax = plt.subplots(figsize=(9, 8))
    ax.set_title(f"Slot Simplex Barycentric Animation (Triangle)\nlayer={LAYER_IDX} | groups={GROUPING} | score={VERTEX_SCORE_SOURCE}")
    ax.set_aspect("equal")
    ax.set_xlabel("simplex-x")
    ax.set_ylabel("simplex-y")

    # draw triangle
    tri = np.array([[0,0],[1,0],[0.5,math.sqrt(3)/2],[0,0]])
    ax.plot(tri[:,0], tri[:,1], linewidth=1.5)
    verts = tri[:3]

    # vertex colors from vscore_norm averaged over prompts (or you can pick a prompt)
    vcol = vscore_norm.mean(axis=0)  # [3]
    scat_v = ax.scatter(verts[:,0], verts[:,1], s=140, c=vcol, cmap="viridis")
    for i, name in enumerate(META_NAMES):
        ax.text(verts[i,0], verts[i,1], f"  {name}", fontsize=10, va="center")

    # one trail + point per prompt
    trails = []
    points = []
    labels = []
    for i in range(P):
        (ln,) = ax.plot([], [], linewidth=2.0, alpha=0.9)
        pt = ax.scatter([], [], s=80, marker="o")
        trails.append(ln)
        points.append(pt)
        labels.append(ax.text(0.02, 0.98 - 0.04*i, f"{i}: {PROMPTS[i][:40]}",
                              transform=ax.transAxes, fontsize=9, va="top"))

    token_text = ax.text(0.02, 0.02, "", transform=ax.transAxes, fontsize=11, va="bottom")

    def init():
        for ln in trails:
            ln.set_data([], [])
        for pt in points:
            pt.set_offsets(np.array([[np.nan, np.nan]]))
        token_text.set_text("")
        return trails + points + [token_text]

    def update(frame):
        # frame is global token index; each prompt uses min(frame, T-1)
        frame = int(frame)
        txt_parts = []
        for i in range(P):
            xy = XY[i]
            t = min(frame, xy.shape[0]-1)
            t0 = max(0, t - TAIL)
            seg = xy[t0:t+1]

            trails[i].set_data(seg[:,0], seg[:,1])
            points[i].set_offsets(seg[-1:, :])

            tok = all_tokens[i][t] if t < len(all_tokens[i]) else ""
            txt_parts.append(f"{i}:{repr(tok)}")

        token_text.set_text(" | ".join(txt_parts[:3]) + (" | ..." if len(txt_parts) > 3 else ""))
        return trails + points + [token_text]

    ani = FuncAnimation(fig, update, frames=Tmax, init_func=init, interval=INTERVAL_MS, blit=False)
    plt.show()

else:
    # META_SLOTS == 4 -> tetrahedron in 3D
    from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

    XYZ = [_barycentric_to_xyz(tr) for tr in all_trajs]  # list of [T,3]

    fig = plt.figure(figsize=(10, 9))
    ax = fig.add_subplot(111, projection="3d")
    ax.set_title(f"Slot Simplex Barycentric Animation (Tetrahedron)\nlayer={LAYER_IDX} | groups={GROUPING} | score={VERTEX_SCORE_SOURCE}")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")

    # tetrahedron vertices
    V = np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.5, math.sqrt(3)/2.0, 0.0],
        [0.5, math.sqrt(3)/6.0, math.sqrt(2.0/3.0)],
    ])

    # edges
    edges = [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]
    for a,b in edges:
        ax.plot([V[a,0],V[b,0]],[V[a,1],V[b,1]],[V[a,2],V[b,2]], linewidth=1.3, alpha=0.7)

    vcol = vscore_norm.mean(axis=0)  # [4]
    ax.scatter(V[:,0], V[:,1], V[:,2], s=140, c=vcol, cmap="viridis")
    for i, name in enumerate(META_NAMES):
        ax.text(V[i,0], V[i,1], V[i,2], f"  {name}", fontsize=10)

    trails = []
    points = []
    for i in range(P):
        (ln,) = ax.plot([], [], [], linewidth=2.0, alpha=0.9)
        pt = ax.scatter([], [], [], s=80, marker="o")
        trails.append(ln)
        points.append(pt)

    def init():
        for ln in trails:
            ln.set_data([], [])
            ln.set_3d_properties([])
        for pt in points:
            pt._offsets3d = (np.array([np.nan]), np.array([np.nan]), np.array([np.nan]))
        return trails + points

    def update(frame):
        frame = int(frame)
        for i in range(P):
            xyz = XYZ[i]
            t = min(frame, xyz.shape[0]-1)
            t0 = max(0, t - TAIL)
            seg = xyz[t0:t+1]

            trails[i].set_data(seg[:,0], seg[:,1])
            trails[i].set_3d_properties(seg[:,2])

            points[i]._offsets3d = (seg[-1:,0], seg[-1:,1], seg[-1:,2])

        return trails + points

    ani = FuncAnimation(fig, update, frames=Tmax, init_func=init, interval=INTERVAL_MS, blit=False)
    plt.show()

# ------------------------------------------------------------
# Optional: Save the animation (uncomment one)
# ------------------------------------------------------------
# ani.save("slot_simplex_barycentric.mp4", dpi=150)  # requires ffmpeg in environment
ani.save("slot_simplex_barycentric.gif", dpi=120)  # may require imagemagick/pillow writer

In [ ]:

# ================================================================
#@title Next Cell (Robust): “Paris Basin on a Simplex Sphere” (Viz A++) + Health Gate
# Fixes:
#  - pad_token
#  - NaN/Inf detection with layer/token localization
#  - Sanitization for visualization (without hiding diagnostics)
#  - Stable PCA on CPU
#  - Safe sphere normalization (no division by zero)
#  - KMeans clustering with NaN filtering
# ================================================================
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# ---------------------------
# SETTINGS
# ---------------------------
LAYER = 10
HEAD_AGG = "mean"          # "mean" or "none"
MAX_PROMPT_LEN = cfg.max_seq_len

PLOT_STRIDE = 1
SHOW_START_MARKERS = True
SHOW_END_MARKERS = True
LABEL_ENDS = True
LABEL_MAX = 12

SPHERE_ALPHA = 0.06
SPHERE_WIREFRAME = True

ENDPOINT_COLOR_MODE = "paris_margin"  # "paris_logp" | "paris_margin"
RIVAL_TEXT = " the"

COLOR_TRAJ_BY_ENTROPY = False
ENTROPY_TEMP = 1.0

DO_ENDPOINT_CLUSTERING = True
N_CLUSTERS = 3

PROMPTS = [
    "The capital of France is",
    "France's capital city is",
    "In France, the capital is",
    "The seat of government of France is",
    "The city that is France's capital is",
    "The Eiffel Tower is located in",
    "The Louvre is in",
    "The Seine river flows through",
    "During the French Revolution, the city of",
    "Napoleon crowned himself in",
    "In World War II, the liberation of",
    "The largest city in France is",
    "A major European city in France is",
    "A famous city on the Seine is",
    "The capital of Germany is",
    "The capital of England is",
    "The capital of Italy is",
]

# Health gate knobs
DEBUG_PRINT_MAX = 6
SANITIZE_FOR_VIZ = True   # keep True; diagnostics still report non-finites
FORCE_CPU_ONE_PASS = False  # set True if you suspect GPU numeric issues

# ---------------------------
# Tokenizer
# ---------------------------
def _get_tokenizer_from_context():
    if "tokenizer" in globals():
        tok = globals()["tokenizer"]
    else:
        from transformers import AutoTokenizer
        tok = AutoTokenizer.from_pretrained(getattr(cfg, "tokenizer_name", "gpt2"))

    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "left"
    return tok

tokenizer = _get_tokenizer_from_context()

def ensure_single_token(text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) != 1:
        raise ValueError(f"{text!r} must be exactly 1 token. Got ids={ids}")
    return ids[0]

TARGET_ID = ensure_single_token(" Paris")
RIVAL_ID = ensure_single_token(RIVAL_TEXT)

# ---------------------------
# Helpers: finiteness diagnostics
# ---------------------------
def _finite_stats(name, x: torch.Tensor):
    if x is None:
        return f"{name}: None"
    x = x.detach()
    fin = torch.isfinite(x)
    n = x.numel()
    nf = fin.sum().item()
    msg = f"{name}: finite {nf}/{n} ({100.0*nf/max(1,n):.2f}%)"
    if nf < n:
        # report some indices
        bad = (~fin).nonzero(as_tuple=False)
        msg += f" | first_bad_idx={bad[0].tolist() if bad.numel() else None}"
    # value stats on finite subset only
    if nf > 0:
        xf = x[fin]
        msg += f" | min={xf.min().item():.4g} max={xf.max().item():.4g} mean={xf.mean().item():.4g}"
    return msg

def _sanitize(x: torch.Tensor, name="x"):
    # convert NaN/Inf -> 0; leave finite untouched
    if x is None:
        return None
    x2 = torch.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)
    return x2

def _row_has_all_neg_inf(logits_row: torch.Tensor):
    # logits_row: [V]
    return torch.isneginf(logits_row).all().item()

# ---------------------------
# Forward
# ---------------------------
@torch.no_grad()
def forward_with_info(ids, mask):
    if FORCE_CPU_ONE_PASS:
        m_cpu = model.to("cpu").eval()
        ids2 = ids.to("cpu")
        mask2 = mask.to("cpu")
        logits, infos = m_cpu(ids2, attention_mask=mask2, return_info=True, routing_mode="softmax")
        # bring back to DEVICE for plotting convenience
        model.to(DEVICE).eval()
        return logits.to(DEVICE), infos
    else:
        logits, infos = model(ids, attention_mask=mask, return_info=True, routing_mode="softmax")
        return logits, infos

enc = tokenizer(
    PROMPTS,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=MAX_PROMPT_LEN,
)
input_ids = enc["input_ids"].to(DEVICE)
attention_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

B_prompts, T = input_ids.shape
lengths = attention_mask.sum(dim=1).long()
t_last = (lengths - 1).clamp(min=0)

logits, infos = forward_with_info(input_ids, attention_mask)

# ---------------------------
# Health gate: logits
# ---------------------------
print(_finite_stats("logits", logits))
if not torch.isfinite(logits).all():
    # localize offending batch/time
    bad = (~torch.isfinite(logits)).nonzero(as_tuple=False)
    # bad indices are [n_bad, 3] = (b,t,v)
    print(f"[health] logits non-finite count={bad.shape[0]}")
    for i in range(min(DEBUG_PRINT_MAX, bad.shape[0])):
        b, t, v = bad[i].tolist()
        print(f"  bad@ b={b} t={t} vocab={v} | prompt={PROMPTS[b][:60]!r}")
        # check if whole row is -inf
        row = logits[b, t]
        if _row_has_all_neg_inf(row):
            print("    row is all -inf (likely masking issue).")

info = infos[LAYER]
read_logits = info.get("read_logits", None)
if read_logits is None:
    raise KeyError(f"infos[{LAYER}] missing 'read_logits'. Keys: {list(info.keys())}")

print(_finite_stats("read_logits(layer)", read_logits))

# If read_logits are busted, identify where
if not torch.isfinite(read_logits).all():
    bad = (~torch.isfinite(read_logits)).nonzero(as_tuple=False)
    # indices are (b,h,t,k)
    print(f"[health] read_logits non-finite count={bad.shape[0]}")
    for i in range(min(DEBUG_PRINT_MAX, bad.shape[0])):
        b, h, t, k = bad[i].tolist()
        print(f"  bad@ b={b} h={h} t={t} k={k} | prompt={PROMPTS[b][:60]!r}")

# Optional sanitize for visualization (still keep diagnostics above)
if SANITIZE_FOR_VIZ:
    logits_viz = _sanitize(logits, "logits")
    read_logits_viz = _sanitize(read_logits, "read_logits")
else:
    logits_viz = logits
    read_logits_viz = read_logits

# ---------------------------
# Head aggregation
# ---------------------------
B0, H0, T0, K = read_logits_viz.shape
assert B0 == B_prompts and T0 == T, (read_logits_viz.shape, (B_prompts, T))

if HEAD_AGG == "mean":
    Z = read_logits_viz.mean(dim=1).float()  # [B,T,K]
    attn_mask_Z = attention_mask
    t_last_Z = t_last
    B = B_prompts
elif HEAD_AGG == "none":
    Z = read_logits_viz.reshape(B_prompts * H0, T, K).float()
    attn_mask_Z = attention_mask.repeat_interleave(H0, dim=0)
    t_last_Z = t_last.repeat_interleave(H0, dim=0)
    B = B_prompts * H0
else:
    raise ValueError("HEAD_AGG must be 'mean' or 'none'")

# ---------------------------
# Stable PCA on CPU (covariance + eigh)
# ---------------------------
def robust_pca_3d(Z, mask, ridge=1e-6):
    # Z: [B,T,K]
    BtK = Z.reshape(-1, Z.shape[-1])
    m = mask.reshape(-1).bool()
    X = BtK[m].to(torch.float32)
    # If X is degenerate (all zeros), bail early
    X = torch.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    mu = X.mean(dim=0, keepdim=True)
    Xc = (X - mu).detach().cpu()
    N, Kc = Xc.shape
    C = (Xc.T @ Xc) / float(max(1, N - 1))
    C = C + ridge * torch.eye(Kc)
    evals, evecs = torch.linalg.eigh(C)
    idx = torch.argsort(evals, descending=True)
    evals3 = evals[idx][:3].contiguous()
    W3 = evecs[:, idx][:, :3].contiguous()  # [K,3]
    return mu.to(Z.device), W3.to(Z.device), evals3.to(Z.device)

mu, W3, evals3 = robust_pca_3d(Z, attn_mask_Z, ridge=1e-6)
evr = (evals3 / (evals3.sum() + 1e-12)).detach().cpu().numpy()
print(f"[PCA] evals={evals3.detach().cpu().numpy()} | EVR≈{np.round(evr,4)}")

XYZ = (Z - mu.view(1,1,K)) @ W3  # [B,T,3]
XYZ_np = XYZ.detach().cpu().numpy()

# Safe sphere normalization
norm = np.linalg.norm(XYZ_np, axis=-1, keepdims=True)
bad_norm = ~np.isfinite(norm) | (norm < 1e-9)
if bad_norm.any():
    # set those points to origin; they will be ignored by plotting constraints
    norm = np.where(bad_norm, 1.0, norm)
XYZ_s = XYZ_np / norm
XYZ_s = np.nan_to_num(XYZ_s, nan=0.0, posinf=0.0, neginf=0.0)

# ---------------------------
# Endpoint scoring (Paris logp/margin) with NaN-safe softmax
# ---------------------------
logp = F.log_softmax(logits_viz, dim=-1)  # [B_prompts,T,V]
lp_paris = logp[torch.arange(B_prompts, device=DEVICE), t_last, TARGET_ID].detach().cpu().numpy()
lp_rival = logp[torch.arange(B_prompts, device=DEVICE), t_last, RIVAL_ID].detach().cpu().numpy()
lp_paris = np.nan_to_num(lp_paris, nan=-1e9, posinf=1e9, neginf=-1e9)
lp_rival = np.nan_to_num(lp_rival, nan=-1e9, posinf=1e9, neginf=-1e9)

if ENDPOINT_COLOR_MODE == "paris_logp":
    end_score = lp_paris
    end_label = "logP(' Paris')"
else:
    end_score = lp_paris - lp_rival
    end_label = f"logP(' Paris') - logP({RIVAL_TEXT!r})"

if HEAD_AGG == "none":
    end_score_plot = np.repeat(end_score, H0)
else:
    end_score_plot = end_score

print(f"[Paris-score] {end_label}: mean={np.mean(end_score):.4g} med={np.median(end_score):.4g} "
      f"min={np.min(end_score):.4g} max={np.max(end_score):.4g}")

# ---------------------------
# Optional entropy coloring
# ---------------------------
entropy = None
if COLOR_TRAJ_BY_ENTROPY:
    w = torch.softmax(Z / max(1e-6, float(ENTROPY_TEMP)), dim=-1)
    ent = -(w * (w.clamp_min(1e-9).log())).sum(dim=-1)
    entropy = ent.detach().cpu().numpy()
    entropy = np.nan_to_num(entropy, nan=0.0, posinf=0.0, neginf=0.0)

# ---------------------------
# Endpoint clustering (NaN-safe)
# ---------------------------
cluster_id = None
cluster_centers = None
if DO_ENDPOINT_CLUSTERING and HEAD_AGG == "mean":
    try:
        from sklearn.cluster import KMeans
        ends = []
        keep = []
        for b in range(B_prompts):
            p = XYZ_s[b, int(t_last[b].item()), :]
            if np.isfinite(p).all() and (np.linalg.norm(p) > 1e-9):
                ends.append(p)
                keep.append(b)
        ends = np.stack(ends, axis=0) if len(ends) else None

        if ends is not None and ends.shape[0] >= N_CLUSTERS:
            km = KMeans(n_clusters=N_CLUSTERS, n_init=10, random_state=0).fit(ends)
            cluster_centers = km.cluster_centers_
            cluster_id = -np.ones((B_prompts,), dtype=np.int64)
            cluster_id[np.array(keep, dtype=np.int64)] = km.labels_
        else:
            print("[warn] clustering skipped (not enough finite endpoints).")
            DO_ENDPOINT_CLUSTERING = False
    except Exception as e:
        print("[warn] clustering unavailable or failed:", repr(e))
        DO_ENDPOINT_CLUSTERING = False

# ---------------------------
# Plot: sphere + trajectories
# ---------------------------
fig = plt.figure(figsize=(10.5, 8.5))
ax = fig.add_subplot(111, projection="3d")

u = np.linspace(0, 2 * np.pi, 48)
v = np.linspace(0, np.pi, 24)
xs = np.outer(np.cos(u), np.sin(v))
ys = np.outer(np.sin(u), np.sin(v))
zs = np.outer(np.ones_like(u), np.cos(v))
if SPHERE_WIREFRAME:
    ax.plot_wireframe(xs, ys, zs, rstride=3, cstride=3, linewidth=0.5, alpha=SPHERE_ALPHA)
else:
    ax.plot_surface(xs, ys, zs, alpha=SPHERE_ALPHA, linewidth=0)

for b in range(B):
    t_end = int(t_last_Z[b].item())
    pts = XYZ_s[b, :t_end + 1:PLOT_STRIDE, :]
    if pts.shape[0] < 2:
        continue

    # If points are all-zero (degenerate), skip
    if np.linalg.norm(pts, axis=1).max() < 1e-9:
        continue

    if COLOR_TRAJ_BY_ENTROPY and entropy is not None:
        ent_b = entropy[b, :t_end + 1:PLOT_STRIDE]
        ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2], c=ent_b, s=12, alpha=0.9)
        ax.plot(pts[:, 0], pts[:, 1], pts[:, 2], alpha=0.35)
    else:
        ax.plot(pts[:, 0], pts[:, 1], pts[:, 2], linewidth=2.0, alpha=0.85)

    if SHOW_START_MARKERS:
        ax.scatter([pts[0, 0]], [pts[0, 1]], [pts[0, 2]], s=40, marker="o", alpha=0.9)
    if SHOW_END_MARKERS:
        ax.scatter([pts[-1, 0]], [pts[-1, 1]], [pts[-1, 2]],
                   c=[end_score_plot[b]], s=70, marker="o", alpha=0.95)

import matplotlib as mpl
normc = mpl.colors.Normalize(vmin=float(np.min(end_score_plot)), vmax=float(np.max(end_score_plot)))
sm = mpl.cm.ScalarMappable(cmap=plt.cm.viridis, norm=normc)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.08)
cbar.set_label(end_label)

if LABEL_ENDS and HEAD_AGG == "mean":
    order = np.argsort(-end_score)
    for i in order[:min(LABEL_MAX, B_prompts)]:
        p = XYZ_s[i, int(t_last[i].item()), :]
        if np.isfinite(p).all() and np.linalg.norm(p) > 1e-9:
            ax.text(p[0], p[1], p[2], f" {i}:{PROMPTS[i][:40]}", fontsize=8)

if DO_ENDPOINT_CLUSTERING and cluster_centers is not None:
    cc = np.nan_to_num(cluster_centers, nan=0.0, posinf=0.0, neginf=0.0)
    ax.scatter(cc[:, 0], cc[:, 1], cc[:, 2], s=180, marker="X", alpha=0.95)

ax.set_title(
    f"Paris Basin on a Simplex Sphere (read_logits PCA→S²)\n"
    f"layer={LAYER} | head_agg={HEAD_AGG} | endpoints colored by {end_label}\n"
    f"PCA EVR (PC1..3) ≈ {np.round(evr, 4)}"
)
ax.set_xlabel("PC1 (sphere)")
ax.set_ylabel("PC2 (sphere)")
ax.set_zlabel("PC3 (sphere)")
plt.tight_layout()
plt.show()

# ---------------------------
# 2D endpoint view (raw PC1,PC2 before sphere)
# ---------------------------
ends2 = []
for b in range(B):
    ends2.append(XYZ_np[b, int(t_last_Z[b].item()), :2])
ends2 = np.stack(ends2, axis=0)
ends2 = np.nan_to_num(ends2, nan=0.0, posinf=0.0, neginf=0.0)

plt.figure(figsize=(7.5, 6.2))
plt.scatter(ends2[:, 0], ends2[:, 1], c=end_score_plot, s=60)
plt.colorbar(label=end_label)
for i in range(min(B_prompts if HEAD_AGG=="mean" else B, LABEL_MAX)):
    plt.text(ends2[i, 0], ends2[i, 1], f" {i}", fontsize=9)
plt.title(f"Endpoints in PCA(PC1,PC2) space (layer={LAYER})")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.tight_layout()
plt.show()

# ---------------------------
# Cluster composition summary
# ---------------------------
if DO_ENDPOINT_CLUSTERING and cluster_id is not None:
    for c in range(N_CLUSTERS):
        idxs = np.where(cluster_id == c)[0].tolist()
        idxs_sorted = sorted(idxs, key=lambda i: float(end_score[i]), reverse=True)
        show = idxs_sorted[:3]
        print(f"[cluster {c}] n={len(idxs)} | top prompts:", [(i, PROMPTS[i][:30]) for i in show])

In [ ]:

# ================================================================
#@title COMPLETE REPLACEMENT: “Paris Basin on a Simplex Sphere (A++)”
# Robust to: missing pad_token, NaNs/Infs in logits/read_logits, GPU SVD failures,
# device mismatches, and partial non-finite activations.
#
# Outputs:
#  1) 3D “simplex sphere” trajectories (read_logits PCA→S²) with endpoint color = Paris margin
#  2) 2D PCA endpoint scatter (PC1, PC2)
#  3) Health report + optional clustering that auto-drops NaN endpoints
#
# Notes:
#  - This does NOT require attention_mask for causality; it uses it only for padding hygiene.
#  - PCA is fit on CPU from finite points only (stable).
# ================================================================
import math
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# ---------------------------
# (Optional) progress bar
# ---------------------------
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kwargs): return x

# ---------------------------
# SETTINGS
# ---------------------------
LAYER = 10                 # router layer (infos[LAYER]["read_logits"])
HEAD_AGG = "mean"          # "mean" or "none" (none flattens heads into batch; heavier)
MAX_PROMPT_LEN = cfg.max_seq_len

# Plotting
PLOT_STRIDE = 1
SHOW_START_MARKERS = True
SHOW_END_MARKERS = True
LABEL_ENDS = True
LABEL_MAX = 12

# Sphere aesthetics
SPHERE_ALPHA = 0.06
SPHERE_WIREFRAME = True

# Endpoint coloring
ENDPOINT_COLOR_MODE = "paris_margin"     # "paris_logp" or "paris_margin"
RIVAL_TEXT = " the"                      # single-token rival for margin
TARGET_TEXT = " Paris"                   # single-token target (GPT2-style leading space)

# PCA / sanitation
SKIP_T0_IN_PCA = True                    # t=0 is often pathological when padding or NaNs occur
SANITIZE_MODE = "drop"                   # "drop" (recommended) or "nan_to_num"
MIN_PCA_POINTS = 64
EPS = 1e-9

# Clustering
DO_ENDPOINT_CLUSTERING = True
N_CLUSTERS = 3

# Prompts
PROMPTS = [
    # direct capital route
    "The capital of France is",
    "France's capital city is",
    "In France, the capital is",
    "The seat of government of France is",
    "The city that is France's capital is",

    # landmark route
    "The Eiffel Tower is located in",
    "The Louvre is in",
    "The Seine river flows through",

    # history route
    "During the French Revolution, the city of",
    "Napoleon crowned himself in",
    "In World War II, the liberation of",

    # geography route
    "The largest city in France is",
    "A major European city in France is",
    "A famous city on the Seine is",

    # distractors / contrast
    "The capital of Germany is",
    "The capital of England is",
    "The capital of Italy is",
]

# ---------------------------
# Tokenizer helpers (pad_token fix included)
# ---------------------------
def _get_tokenizer_from_context():
    if "tokeizer" in globals():
        tok = globals()["tokenizer"]
    else:
        from transformers import AutoTokenizer
        tok = AutoTokenizer.from_pretrained(getattr(cfg, "tokenizer_name", "gpt2"))

    tok.padding_side = "right"
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok

tokenizer = _get_tokenizer_from_context()

def ensure_single_token(text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) != 1:
        raise ValueError(f"{text!r} must be exactly 1 token under tokenizer. Got ids={ids}")
    return ids[0]

TARGET_ID = ensure_single_token(TARGET_TEXT)
RIVAL_ID  = ensure_single_token(RIVAL_TEXT)

# ---------------------------
# Forward helper
# ---------------------------
@torch.no_grad()
def forward_with_info(ids, mask=None, routing_mode="softmax"):
    # If you suspect attention_mask causes issues, you can pass mask=None.
    # We keep it for padding hygiene, but it should not be required for causality.
    if mask is None:
        logits, infos = model(ids, return_info=True, routing_mode=routing_mode)
    else:
        logits, infos = model(ids, attention_mask=mask, return_info=True, routing_mode=routing_mode)
    return logits, infos

# ---------------------------
# Tokenize
# ---------------------------
enc = tokenizer(
    PROMPTS,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=MAX_PROMPT_LEN,
)
input_ids = enc["input_ids"].to(DEVICE)
attention_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

B_prompts, T = input_ids.shape
lengths = attention_mask.sum(dim=1).long()
t_last = (lengths - 1).clamp(min=0)  # last valid index per prompt

# ---------------------------
# Run model
# ---------------------------
logits, infos = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
info = infos[LAYER]
read_logits = info.get("read_logits", None)
if read_logits is None:
    raise KeyError(f"infos[{LAYER}] missing 'read_logits'. Keys: {list(info.keys())}")

# read_logits: [B,H,T,K]
B0, H0, T0, K = read_logits.shape
assert B0 == B_prompts and T0 == T, (read_logits.shape, (B_prompts, T))

# ---------------------------
# Health checks (logits + read_logits)
# ---------------------------
def _health(name, x, prompt_list=None):
    x_det = x.detach()
    fin = torch.isfinite(x_det)
    n_fin = int(fin.sum().item())
    n_tot = x_det.numel()
    print(f"[health] {name}: finite {n_fin}/{n_tot} ({100*n_fin/max(1,n_tot):.2f}%)", end="")
    if n_fin > 0:
        x_ok = x_det[fin]
        print(f" | min={x_ok.min().item():.4g} max={x_ok.max().item():.4g} mean={x_ok.mean().item():.4g}")
    else:
        print(" | ALL NON-FINITE")
    if n_fin < n_tot and prompt_list is not None:
        # print first few bad indices in a friendly way
        bad = (~fin).nonzero(as_tuple=False)
        show = bad[:6]
        for idx in show:
            # best-effort mapping for common shapes
            if x_det.ndim == 3:   # [B,T,V]
                b, t, v = idx.tolist()
                print(f"   bad@ b={b} t={t} vocab={v} | prompt={prompt_list[b]!r}")
            elif x_det.ndim == 4: # [B,H,T,K]
                b, h, t, k = idx.tolist()
                print(f"   bad@ b={b} h={h} t={t} k={k} | prompt={prompt_list[b]!r}")

_health("logits", logits, PROMPTS)
_health("read_logits(layer)", read_logits, PROMPTS)

# ---------------------------
# Head aggregation
# ---------------------------
if HEAD_AGG == "mean":
    Z = read_logits.mean(dim=1).float()   # [B,T,K]
    B = B_prompts
    t_last_plot = t_last
    mask_plot = attention_mask
elif HEAD_AGG == "none":
    # Flatten heads into batch dimension
    Z = read_logits.reshape(B_prompts * H0, T, K).float()
    B = B_prompts * H0
    t_last_plot = t_last.repeat_interleave(H0, dim=0)
    mask_plot = attention_mask.repeat_interleave(H0, dim=0)
else:
    raise ValueError("HEAD_AGG must be 'mean' or 'none'")

# ---------------------------
# PCA fit + projection (CPU end-to-end, finite-only)
# ---------------------------
Z_cpu = Z.detach().cpu()                          # [B,T,K] CPU
Z_flat_cpu = Z_cpu.reshape(B*T, K)                # [BT,K] CPU

finite_mask = torch.isfinite(Z_flat_cpu).all(dim=-1)
if SKIP_T0_IN_PCA:
    tt = torch.arange(T).repeat(B)                # [BT]
    finite_mask = finite_mask & (tt != 0)

n_total = Z_flat_cpu.shape[0]
n_good = int(finite_mask.sum().item())
print(f"[health] PCA fit points: good={n_good}/{n_total} ({100*n_good/max(1,n_total):.2f}%) "
      f"| SKIP_T0_IN_PCA={SKIP_T0_IN_PCA} | SANITIZE_MODE={SANITIZE_MODE}")

if n_good < MIN_PCA_POINTS:
    raise RuntimeError(
        f"Too few finite points to fit PCA (good={n_good}). "
        "Fix numerical stability first or lower MIN_PCA_POINTS."
    )

X_good = Z_flat_cpu[finite_mask]                  # [Ng,K] CPU
mu = X_good.mean(dim=0, keepdim=True)             # [1,K] CPU
Xc = X_good - mu                                  # [Ng,K] CPU

# SVD on CPU (stable). If this still fails, fallback to PCA-lowrank.
try:
    U, S, Vh = torch.linalg.svd(Xc, full_matrices=False)
    W2 = Vh[:2].T.contiguous()                    # [K,2]
    W3 = Vh[:3].T.contiguous()                    # [K,3]
    evals = (S[:3]**2).numpy()
    evr = evals / (S**2).sum().numpy()
except Exception as e:
    print("[warn] CPU SVD failed; falling back to torch.pca_lowrank:", repr(e))
    # q must be >= 3
    q = 8
    U2, S2, V2 = torch.pca_lowrank(Xc, q=q, center=False)
    W3 = V2[:, :3].contiguous()
    W2 = V2[:, :2].contiguous()
    # approximate EVR from singular values
    evals = (S2[:3]**2).numpy()
    evr = evals / (S2**2).sum().numpy()

print(f"[PCA] EVR(PC1..3) ≈ {np.round(evr, 4)}")

# Project ALL points (CPU)
Z2 = (Z_cpu - mu) @ W2                            # [B,T,2]
Z3 = (Z_cpu - mu) @ W3                            # [B,T,3]

Z2_np = Z2.numpy()
Z3_np = Z3.numpy()

# Sanitize projections if needed
if SANITIZE_MODE == "nan_to_num":
    Z2_np = np.nan_to_num(Z2_np, nan=0.0, posinf=0.0, neginf=0.0)
    Z3_np = np.nan_to_num(Z3_np, nan=0.0, posinf=0.0, neginf=0.0)

# Sphere projection
norm = np.linalg.norm(Z3_np, axis=-1, keepdims=True)
Z3_sphere = Z3_np / np.clip(norm, EPS, None)

# ---------------------------
# Endpoint Paris score (compute on original prompts only)
# ---------------------------
logp = F.log_softmax(logits, dim=-1)              # [B_prompts,T,V] on DEVICE
idx = torch.arange(B_prompts, device=DEVICE)

lp_paris = logp[idx, t_last, TARGET_ID].detach().cpu().numpy()
lp_rival = logp[idx, t_last, RIVAL_ID].detach().cpu().numpy()

if ENDPOINT_COLOR_MODE == "paris_logp":
    end_score = lp_paris
    end_label = f"logP({TARGET_TEXT!r})"
elif ENDPOINT_COLOR_MODE == "paris_margin":
    end_score = (lp_paris - lp_rival)
    end_label = f"logP({TARGET_TEXT!r}) - logP({RIVAL_TEXT!r})"
else:
    raise ValueError("ENDPOINT_COLOR_MODE must be 'paris_logp' or 'paris_margin'")

# If HEAD_AGG=="none", replicate scores per head
if HEAD_AGG == "none":
    end_score_plot = np.repeat(end_score, H0)
else:
    end_score_plot = end_score

# Replace non-finite scores with NaN for plotting/clustering decisions
end_score_plot = np.array(end_score_plot, dtype=np.float64)
end_score_plot[~np.isfinite(end_score_plot)] = np.nan

print(f"[Paris-score] {end_label}: mean={np.nanmean(end_score_plot):.4f} "
      f"med={np.nanmedian(end_score_plot):.4f} "
      f"min={np.nanmin(end_score_plot):.4f} max={np.nanmax(end_score_plot):.4f}")

# ---------------------------
# Compute endpoints on sphere + 2D (for clustering & endpoint plot)
# ---------------------------
ends_sphere = []
ends2 = []
for b in range(B):
    te = int(t_last_plot[b].item())
    ends_sphere.append(Z3_sphere[b, te, :])
    ends2.append(Z2_np[b, te, :])
ends_sphere = np.stack(ends_sphere, axis=0)       # [B,3]
ends2 = np.stack(ends2, axis=0)                   # [B,2]

# Drop non-finite endpoints for clustering
valid_end = np.isfinite(ends_sphere).all(axis=1) & np.isfinite(end_score_plot)
n_valid_end = int(valid_end.sum())
if DO_ENDPOINT_CLUSTERING:
    if n_valid_end < max(N_CLUSTERS, 3):
        print(f"[warn] clustering disabled (valid endpoints={n_valid_end} < clusters={N_CLUSTERS}).")
        DO_ENDPOINT_CLUSTERING = False

cluster_id = None
cluster_centers = None
if DO_ENDPOINT_CLUSTERING:
    try:
        from sklearn.cluster import KMeans
        Xc_end = ends_sphere[valid_end]
        km = KMeans(n_clusters=N_CLUSTERS, n_init=10, random_state=0).fit(Xc_end)
        cluster_centers = km.cluster_centers_
        # assign labels back into full array with -1 for invalid
        cluster_id = -np.ones((B,), dtype=np.int64)
        cluster_id[valid_end] = km.labels_
    except Exception as e:
        print("[warn] clustering unavailable or failed:", repr(e))
        DO_ENDPOINT_CLUSTERING = False

# ---------------------------
# Plot 1: Sphere + trajectories
# ---------------------------
fig = plt.figure(figsize=(11.0, 8.7))
ax = fig.add_subplot(111, projection="3d")

# Sphere wireframe
u = np.linspace(0, 2 * np.pi, 48)
v = np.linspace(0, np.pi, 24)
xs = np.outer(np.cos(u), np.sin(v))
ys = np.outer(np.sin(u), np.sin(v))
zs = np.outer(np.ones_like(u), np.cos(v))
if SPHERE_WIREFRAME:
    ax.plot_wireframe(xs, ys, zs, rstride=3, cstride=3, linewidth=0.5, alpha=SPHERE_ALPHA)
else:
    ax.plot_surface(xs, ys, zs, alpha=SPHERE_ALPHA, linewidth=0)

# Plot trajectories (skip any non-finite segments if SANITIZE_MODE=="drop")
for b in range(B):
    te = int(t_last_plot[b].item())
    pts = Z3_sphere[b, :te+1:PLOT_STRIDE, :]     # [L,3]

    if SANITIZE_MODE == "drop":
        ok = np.isfinite(pts).all(axis=1)
        pts = pts[ok]
    if pts.shape[0] < 2:
        continue

    ax.plot(pts[:,0], pts[:,1], pts[:,2], linewidth=2.0, alpha=0.85)

    if SHOW_START_MARKERS:
        ax.scatter([pts[0,0]], [pts[0,1]], [pts[0,2]], s=40, marker="o", alpha=0.9)

    if SHOW_END_MARKERS:
        cval = end_score_plot[b]
        # if NaN score, plot in gray
        if np.isfinite(cval):
            ax.scatter([pts[-1,0]], [pts[-1,1]], [pts[-1,2]], c=[cval], s=70, marker="o", alpha=0.95)
        else:
            ax.scatter([pts[-1,0]], [pts[-1,1]], [pts[-1,2]], c="gray", s=70, marker="o", alpha=0.8)

# Colorbar for endpoint scores
import matplotlib as mpl
finite_scores = end_score_plot[np.isfinite(end_score_plot)]
if finite_scores.size > 0:
    normc = mpl.colors.Normalize(vmin=float(finite_scores.min()), vmax=float(finite_scores.max()))
else:
    normc = mpl.colors.Normalize(vmin=-1.0, vmax=1.0)
sm = mpl.cm.ScalarMappable(cmap=plt.cm.viridis, norm=normc)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.08)
cbar.set_label(end_label)

# Label endpoints (best score first) only if head-agg is prompt-level
if LABEL_ENDS and HEAD_AGG == "mean":
    order = np.argsort(-end_score)  # descending; end_score is per prompt
    for i in order[: min(LABEL_MAX, B_prompts)]:
        te = int(t_last[i].item())
        p = Z3_sphere[i, te, :]
        snippet = PROMPTS[i][:45].replace("\n"," ")
        ax.text(p[0], p[1], p[2], f" {i}:{snippet}", fontsize=8)

# Cluster centers
if DO_ENDPOINT_CLUSTERING and cluster_centers is not None:
    cc = cluster_centers
    ax.scatter(cc[:,0], cc[:,1], cc[:,2], s=180, marker="X", alpha=0.95)

ax.set_title(
    f"Paris Basin on a Simplex Sphere (read_logits PCA→S²)\n"
    f"layer={LAYER} | head_agg={HEAD_AGG} | endpoints colored by {end_label}\n"
    f"PCA EVR (PC1..3) ≈ {np.round(evr, 4)}"
)
ax.set_xlabel("PC1 (sphere)")
ax.set_ylabel("PC2 (sphere)")
ax.set_zlabel("PC3 (sphere)")
plt.tight_layout()
plt.show()

# ---------------------------
# Plot 2: Endpoints in PCA(PC1,PC2) space
# ---------------------------
plt.figure(figsize=(7.8, 6.4))
c = end_score_plot
plt.scatter(ends2[:,0], ends2[:,1], c=c, s=70)
plt.colorbar(label=end_label)
if HEAD_AGG == "mean":
    for i in range(min(B_prompts, LABEL_MAX)):
        plt.text(ends2[i,0], ends2[i,1], f" {i}", fontsize=9)
plt.title(f"Endpoints in PCA(PC1,PC2) space (layer={LAYER})")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.tight_layout()
plt.show()

# ---------------------------
# Cluster report
# ---------------------------
print(f"[OK] prompts={B_prompts} | seq_len(T)={T} | slots(K)={K} | layer={LAYER} | head_agg={HEAD_AGG}")
if DO_ENDPOINT_CLUSTERING and cluster_id is not None and HEAD_AGG == "mean":
    for c_id in range(N_CLUSTERS):
        idxs = np.where(cluster_id[:B_prompts] == c_id)[0].tolist()
        idxs_sorted = sorted(idxs, key=lambda i: float(end_score[i]), reverse=True)
        top3 = [(i, PROMPTS[i][:35]) for i in idxs_sorted[:3]]
        print(f"[cluster {c_id}] n={len(idxs)} | top prompts:", top3)
else:
    if HEAD_AGG == "mean":
        print("[cluster] disabled or unavailable (either NaNs, sklearn missing, or insufficient points).")

In [ ]:

# ================================================================
#@title Next Cell: Late-Phase Trajectory Bundle + Commitment + Paris Attractor
# Uses right-padding to avoid NaNs. Highlights the phenomenon:
#   - show only last W tokens (late-stage convergence)
#   - color endpoints by Paris margin
#   - show routing entropy vs time (gradual commitment)
#   - compute and plot a Paris-attractor centroid (weighted by margin)
#
# Assumptions:
#   - model, cfg, DEVICE exist
#   - model(..., return_info=True) -> (logits, infos)
#   - infos[layer]["read_logits"] is [B,H,T,K] float32
# ================================================================
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# ---------------------------
# SETTINGS
# ---------------------------
LAYER = 7
HEAD_AGG = "mean"          # "mean" only here (keeps plots readable)
MAX_LEN = cfg.max_seq_len

# Focus window: last W tokens (where alignment should appear)
W_LATE = 5                 # try 4-8
PLOT_STRIDE = 1

# Paris scoring
TARGET_TEXT = " Paris"
RIVAL_TEXT  = " the"        # margin vs function-word continuation

# Commitment / entropy
ENTROPY_TEMP = 1.0

# Visual declutter
LABEL_MAX = 10              # label only top Paris-margin prompts
SPHERE_ALPHA = 0.06
SPHERE_WIREFRAME = True

# Prompts (use your current 17 set)
PROMPTS = [
    "The capital of France is",
    "France's capital city is",
    "In France, the capital is",
    "The seat of government of France is",
    "The city that is France's capital is",
    "The Eiffel Tower is located in",
    "The Louvre is in",
    "The Seine river flows through",
    "During the French Revolution, the city of",
    "Napoleon crowned himself in",
    "In World War II, the liberation of",
    "The largest city in France is",
    "A major European city in France is",
    "A famous city on the Seine is",
    "The capital of Germany is",
    "The capital of England is",
    "The capital of Italy is",
]

# ---------------------------
# Tokenizer (robust) + RIGHT PADDING
# ---------------------------
def _get_tokenizer():
    if "tokenizer" in globals():
        tok = globals()["tokenizer"]
    else:
        from transformers import AutoTokenizer
        tok = AutoTokenizer.from_pretrained(getattr(cfg, "tokenizer_name", "gpt2"))

    tok.padding_side = "right"                     # IMPORTANT: right padding
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok

tokenizer = _get_tokenizer()

def ensure_single_token(text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) != 1:
        raise ValueError(f"{text!r} must be exactly 1 token. Got ids={ids}")
    return ids[0]

TARGET_ID = ensure_single_token(TARGET_TEXT)
RIVAL_ID  = ensure_single_token(RIVAL_TEXT)

# ---------------------------
# Forward
# ---------------------------
@torch.no_grad()
def forward_with_info(ids, mask):
    logits, infos = model(ids, attention_mask=mask, return_info=True, routing_mode="softmax")
    return logits, infos

enc = tokenizer(
    PROMPTS,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=MAX_LEN,
)
input_ids = enc["input_ids"].to(DEVICE)
attention_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

B, T = input_ids.shape
lengths = attention_mask.sum(dim=1).long()
t_last = (lengths - 1).clamp(min=0)

logits, infos = forward_with_info(input_ids, attention_mask)
info = infos[LAYER]
read_logits = info.get("read_logits", None)
if read_logits is None:
    raise KeyError(f"infos[{LAYER}] missing 'read_logits'. Keys={list(info.keys())}")

# read_logits: [B,H,T,K]
B0, H0, T0, K = read_logits.shape
assert (B0, T0) == (B, T), (read_logits.shape, (B, T))

# Aggregate heads
Z = read_logits.mean(dim=1).float()  # [B,T,K]

# ---------------------------
# Health checks (finite)
# ---------------------------
def _health(name, x):
    x = x.detach()
    finite = torch.isfinite(x)
    frac = finite.float().mean().item()
    if frac < 1.0:
        idx = torch.nonzero(~finite, as_tuple=False)[0].tolist()
        mn = torch.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0).min().item()
        mx = torch.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0).max().item()
        print(f"[health] {name}: finite={frac*100:.2f}% | first_bad_idx={idx} | min~={mn:.3f} max~={mx:.3f}")
    else:
        print(f"[health] {name}: finite=100% | min={x.min().item():.3f} max={x.max().item():.3f} mean={x.mean().item():.3f}")

_health("logits", logits)
_health("read_logits(layer)", Z)

# ---------------------------
# Paris margin score at last valid
# ---------------------------
logp = F.log_softmax(logits, dim=-1)  # [B,T,V]
idx = torch.arange(B, device=DEVICE)
lp_paris = logp[idx, t_last, TARGET_ID]
lp_rival = logp[idx, t_last, RIVAL_ID]
margin = (lp_paris - lp_rival).detach().cpu().numpy()

print(f"[Paris-score] mean={margin.mean():.4f} med={np.median(margin):.4f} min={margin.min():.4f} max={margin.max():.4f}")

# ---------------------------
# Routing entropy vs time (commitment curve)
# ---------------------------
w = torch.softmax(Z / max(1e-6, float(ENTROPY_TEMP)), dim=-1)  # [B,T,K]
entropy = -(w * (w.clamp_min(1e-9).log())).sum(dim=-1)         # [B,T]
entropy_np = entropy.detach().cpu().numpy()

# ---------------------------
# PCA (robust): fit on LAST-W window only (makes convergence structure pop)
# We fit PCA in torch on GPU then move results to CPU for plotting.
# ---------------------------
# Build late-window indices per sequence
late_mask = torch.zeros((B, T), device=DEVICE, dtype=torch.bool)
for b in range(B):
    t_end = int(t_last[b].item())
    t0 = max(0, t_end - (W_LATE - 1))
    late_mask[b, t0:t_end+1] = True

# collect points [N,K] from late window
pts = Z[late_mask]  # [N,K]
# center
mu = pts.mean(dim=0, keepdim=True)
X = pts - mu

# PCA via covariance eig (more stable than SVD when nearly rank-deficient)
# C = X^T X / (N-1)
N = X.shape[0]
C = (X.T @ X) / max(1, (N - 1))
# eigendecomp (symmetric)
evals, evecs = torch.linalg.eigh(C)  # ascending
# take top-3
W3 = evecs[:, -3:]                   # [K,3]
# explained variance ratio
evr = (evals[-3:] / evals.clamp_min(1e-12).sum()).detach().cpu().numpy()[::-1]
print(f"[PCA] late-window points={N} | EVR(PC1..3)≈ {np.round(evr,4)}")

# Project ALL timepoints into 3D using the same basis, then normalize to sphere
XYZ = (Z - mu.view(1,1,K)) @ W3       # [B,T,3]
XYZ_np = XYZ.detach().cpu().numpy()
norm = np.linalg.norm(XYZ_np, axis=-1, keepdims=True)
XYZ_s = XYZ_np / np.clip(norm, 1e-9, None)

# Endpoints (sphere + raw PC1/PC2)
end_s = np.stack([XYZ_s[b, int(t_last[b].item())] for b in range(B)], axis=0)   # [B,3]
end_2 = np.stack([XYZ_np[b, int(t_last[b].item()), :2] for b in range(B)], axis=0)

# Paris-attractor centroid in endpoint space (weighted by margin)
# weights: sigmoid(margin) so negative margins still contribute slightly
w_cent = 1.0 / (1.0 + np.exp(-margin))
w_cent = w_cent / (w_cent.sum() + 1e-9)
cent_s = (end_s * w_cent[:,None]).sum(axis=0)
cent_s = cent_s / (np.linalg.norm(cent_s) + 1e-9)
cent_2 = (end_2 * w_cent[:,None]).sum(axis=0)

# ---------------------------
# Plot 1: Late-phase bundle on sphere
# ---------------------------
fig = plt.figure(figsize=(10.5, 8.5))
ax = fig.add_subplot(111, projection="3d")

# faint sphere
u = np.linspace(0, 2*np.pi, 48)
v = np.linspace(0, np.pi, 24)
xs = np.outer(np.cos(u), np.sin(v))
ys = np.outer(np.sin(u), np.sin(v))
zs = np.outer(np.ones_like(u), np.cos(v))
if SPHERE_WIREFRAME:
    ax.plot_wireframe(xs, ys, zs, rstride=3, cstride=3, linewidth=0.5, alpha=SPHERE_ALPHA)
else:
    ax.plot_surface(xs, ys, zs, alpha=SPHERE_ALPHA, linewidth=0)

# trajectories: plot only late window for each prompt
for b in range(B):
    t_end = int(t_last[b].item())
    t0 = max(0, t_end - (W_LATE - 1))
    pts_b = XYZ_s[b, t0:t_end+1:PLOT_STRIDE, :]
    if pts_b.shape[0] >= 2:
        ax.plot(pts_b[:,0], pts_b[:,1], pts_b[:,2], linewidth=2.2, alpha=0.9)

    # endpoint marker colored by margin
    ax.scatter([end_s[b,0]],[end_s[b,1]],[end_s[b,2]], c=[margin[b]], s=70, alpha=0.95)

# attractor centroid
ax.scatter([cent_s[0]],[cent_s[1]],[cent_s[2]], s=220, marker="X", alpha=0.95)

# colorbar (margin)
import matplotlib as mpl
normc = mpl.colors.Normalize(vmin=float(np.min(margin)), vmax=float(np.max(margin)))
sm = mpl.cm.ScalarMappable(cmap=plt.cm.viridis, norm=normc)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.08)
cbar.set_label(f"logP({TARGET_TEXT!r}) - logP({RIVAL_TEXT!r})")

# label top prompts by margin
order = np.argsort(-margin)
for i in order[:min(LABEL_MAX, B)]:
    p = end_s[i]
    ax.text(p[0], p[1], p[2], f" {i}", fontsize=9)

ax.set_title(
    f"Late-Phase Bundle on Simplex Sphere (read_logits PCA→S²)\n"
    f"layer={LAYER} | window=last {W_LATE} tokens | EVR≈{np.round(evr,4)} | X = Paris-attractor centroid"
)
ax.set_xlabel("PC1 (sphere)")
ax.set_ylabel("PC2 (sphere)")
ax.set_zlabel("PC3 (sphere)")
plt.tight_layout()
plt.show()

# ---------------------------
# Plot 2: 2D endpoints + centroid
# ---------------------------
plt.figure(figsize=(7.8, 6.3))
plt.scatter(end_2[:,0], end_2[:,1], c=margin, s=70)
plt.scatter([cent_2[0]],[cent_2[1]], s=220, marker="X")
plt.colorbar(label=f"logP({TARGET_TEXT!r}) - logP({RIVAL_TEXT!r})")
for i in range(B):
    plt.text(end_2[i,0], end_2[i,1], f" {i}", fontsize=9)
plt.title(f"Endpoints in PCA(PC1,PC2) space (layer={LAYER}) | X = Paris-attractor centroid")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.tight_layout()
plt.show()

# ---------------------------
# Plot 3: Commitment curves (entropy vs time)
# ---------------------------
plt.figure(figsize=(9.2, 5.6))
for b in range(B):
    t_end = int(t_last[b].item())
    ts = np.arange(t_end+1)
    plt.plot(ts, entropy_np[b, :t_end+1], alpha=0.55)
plt.title(f"Routing Entropy vs Time (commitment) | layer={LAYER} | lower = more committed")
plt.xlabel("token index")
plt.ylabel("entropy(softmax(read_logits))")
plt.tight_layout()
plt.show()

# ---------------------------
# Print: rank prompts by margin
# ---------------------------
rank = np.argsort(-margin)
print("\n=== Prompts ranked by Paris-margin (highest first) ===")
for r, i in enumerate(rank):
    print(f"{r:2d}  idx={i:2d}  margin={margin[i]: .3f}   prompt={PROMPTS[i]}")

In [ ]:

# ================================================================
#@title Layer Sweep: Where does the Paris-attractor geometry emerge?
# Computes per-layer summary metrics using the same probe:
#   - router read_logits (mean over heads)
#   - late-window PCA fit (last W tokens per prompt)
#   - Paris margin (logP(' Paris') - logP(' the')) at last valid
#   - attractor sharpness: corr( -dist_to_centroid , margin )
#   - EVR(PC1) of late-window PCA
#   - mean routing entropy in late window
#
# Assumptions: model, cfg, DEVICE, tokenizer available (or cfg.tokenizer_name).
# ================================================================
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# ---------------------------
# SETTINGS
# ---------------------------
W_LATE = 5
ENTROPY_TEMP = 1.0
TARGET_TEXT = " Paris"
RIVAL_TEXT  = " the"

# Auto sweep control
USE_EVERY_N = 1          # set 2 or 3 for quicker coarse scan
ALSO_INCLUDE_LAST = True # ensure last layer included even if stepping skips it

# Prompts (same set you used)
PROMPTS = [
    "The capital of France is",
    "France's capital city is",
    "In France, the capital is",
    "The seat of government of France is",
    "The city that is France's capital is",
    "The Eiffel Tower is located in",
    "The Louvre is in",
    "The Seine river flows through",
    "During the French Revolution, the city of",
    "Napoleon crowned himself in",
    "In World War II, the liberation of",
    "The largest city in France is",
    "A major European city in France is",
    "A famous city on the Seine is",
    "The capital of Germany is",
    "The capital of England is",
    "The capital of Italy is",
]

# ---------------------------
# Tokenizer (RIGHT padding)
# ---------------------------
def _get_tokenizer():
    if "tokenizer" in globals():
        tok = globals()["tokenizer"]
    else:
        from transformers import AutoTokenizer
        tok = AutoTokenizer.from_pretrained(getattr(cfg, "tokenizer_name", "gpt2"))
    tok.padding_side = "right"
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok

tokenizer = _get_tokenizer()

def ensure_single_token(text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) != 1:
        raise ValueError(f"{text!r} must be exactly 1 token. Got ids={ids}")
    return ids[0]

TARGET_ID = ensure_single_token(TARGET_TEXT)
RIVAL_ID  = ensure_single_token(RIVAL_TEXT)

# ---------------------------
# Prepare batch
# ---------------------------
enc = tokenizer(PROMPTS, return_tensors="pt", padding=True, truncation=True, max_length=cfg.max_seq_len)
input_ids = enc["input_ids"].to(DEVICE)
attention_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

B, T = input_ids.shape
lengths = attention_mask.sum(dim=1).long()
t_last = (lengths - 1).clamp(min=0)

# ---------------------------
# Forward once to get logits + infos for all layers
# ---------------------------
@torch.no_grad()
def forward_with_info(ids, mask):
    logits, infos = model(ids, attention_mask=mask, return_info=True, routing_mode="softmax")
    return logits, infos

logits, infos = forward_with_info(input_ids, attention_mask)

n_layers = len(infos)
print(f"[info] detected layers = {n_layers}")

# Paris margin (fixed across layer; uses final logits only)
logp = F.log_softmax(logits, dim=-1)
idx = torch.arange(B, device=DEVICE)
margin = (logp[idx, t_last, TARGET_ID] - logp[idx, t_last, RIVAL_ID]).detach().cpu().numpy()

# Layer list
layers = list(range(0, n_layers, max(1, int(USE_EVERY_N))))
if ALSO_INCLUDE_LAST and (n_layers - 1) not in layers:
    layers.append(n_layers - 1)
layers = sorted(set(layers))

# ---------------------------
# Metrics per layer
# ---------------------------
rows = []

def late_mask_from_tlast(t_last_tensor, T, W):
    lm = torch.zeros((t_last_tensor.shape[0], T), device=t_last_tensor.device, dtype=torch.bool)
    for b in range(t_last_tensor.shape[0]):
        t_end = int(t_last_tensor[b].item())
        t0 = max(0, t_end - (W - 1))
        lm[b, t0:t_end+1] = True
    return lm

late_mask = late_mask_from_tlast(t_last, T, W_LATE)

for L in layers:
    rl = infos[L].get("read_logits", None)
    if rl is None:
        print(f"[warn] layer {L} missing read_logits, skipping")
        continue

    # [B,H,T,K] -> [B,T,K]
    Z = rl.mean(dim=1).float()

    # entropy in late window
    w = torch.softmax(Z / max(1e-6, float(ENTROPY_TEMP)), dim=-1)
    ent = -(w * w.clamp_min(1e-9).log()).sum(dim=-1)  # [B,T]
    ent_late = ent[late_mask].mean().item()

    # late-window PCA in slot-logit space (cov eig; stable)
    pts = Z[late_mask]            # [N,K]
    mu = pts.mean(dim=0, keepdim=True)
    X = pts - mu
    N = X.shape[0]
    C = (X.T @ X) / max(1, (N - 1))
    evals, evecs = torch.linalg.eigh(C)
    top = evals[-3:].clamp_min(1e-12)
    evr3 = (top / evals.clamp_min(1e-12).sum()).detach().cpu().numpy()[::-1]  # PC1..3
    W2 = evecs[:, -2:]  # [K,2]

    # project endpoints into 2D PCA space
    Z_end = Z[torch.arange(B, device=DEVICE), t_last, :]          # [B,K]
    end2 = ((Z_end - mu) @ W2).detach().cpu().numpy()             # [B,2]

    # centroid weighted by sigmoid(margin)
    w_cent = 1.0 / (1.0 + np.exp(-margin))
    w_cent = w_cent / (w_cent.sum() + 1e-9)
    cent2 = (end2 * w_cent[:, None]).sum(axis=0)

    # attractor sharpness: corr(-dist, margin)
    dist = np.linalg.norm(end2 - cent2[None, :], axis=1)
    x = -dist
    y = margin
    # safe corr
    if np.std(x) < 1e-9 or np.std(y) < 1e-9:
        sharp = np.nan
    else:
        sharp = float(np.corrcoef(x, y)[0, 1])

    rows.append({
        "layer": L,
        "evr_pc1": float(evr3[0]),
        "evr_pc2": float(evr3[1]),
        "evr_pc3": float(evr3[2]),
        "entropy_late": float(ent_late),
        "margin_mean": float(np.mean(margin)),
        "margin_median": float(np.median(margin)),
        "margin_max": float(np.max(margin)),
        "margin_min": float(np.min(margin)),
        "attractor_sharp": sharp,
    })

# sort by layer
rows = sorted(rows, key=lambda r: r["layer"])

# ---------------------------
# Print summary + best layers
# ---------------------------
def best_by(key, maximize=True):
    vals = [(r[key], r["layer"]) for r in rows if np.isfinite(r[key])]
    if not vals:
        return None
    return max(vals) if maximize else min(vals)

best_sharp = best_by("attractor_sharp", maximize=True)
best_evr   = best_by("evr_pc1", maximize=True)
best_ent   = best_by("entropy_late", maximize=False)

print("\n=== Best layers (by metric) ===")
print("best attractor_sharp (corr(-dist,margin)):", best_sharp)
print("best evr_pc1 (most 1D late space):", best_evr)
print("best entropy_late (most committed):", best_ent)

print("\n=== Per-layer table (compact) ===")
for r in rows:
    print(f"L{r['layer']:2d} | sharp={r['attractor_sharp']:+.3f} | EVR1={r['evr_pc1']:.3f} | ent={r['entropy_late']:.3f} | margin_med={r['margin_median']:+.3f}")

# ---------------------------
# Plots
# ---------------------------
Lx = [r["layer"] for r in rows]
sharp = [r["attractor_sharp"] for r in rows]
evr1  = [r["evr_pc1"] for r in rows]
entl  = [r["entropy_late"] for r in rows]

plt.figure(figsize=(9.5, 4.8))
plt.plot(Lx, sharp, marker="o")
plt.axhline(0.0, linewidth=1)
plt.title("Attractor sharpness vs layer  (corr(-dist_to_centroid, Paris-margin))")
plt.xlabel("layer")
plt.ylabel("corr")
plt.tight_layout()
plt.show()

plt.figure(figsize=(9.5, 4.8))
plt.plot(Lx, evr1, marker="o")
plt.title("Late-window PCA EVR(PC1) vs layer  (how 1D is router commitment space?)")
plt.xlabel("layer")
plt.ylabel("EVR(PC1)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(9.5, 4.8))
plt.plot(Lx, entl, marker="o")
plt.title(f"Mean routing entropy in late window (W={W_LATE}) vs layer  (lower=more committed)")
plt.xlabel("layer")
plt.ylabel("entropy")
plt.tight_layout()
plt.show()

In [ ]:

# ================================================================
#@title Experiment 1: Slot Simplex Barycentric Animation (Surgical)
# Layers: [0,5,13] (auto-clipped)
# Clustering: late-window covariance of routing mass (NOT magnitude)
# Reduce: K -> meta_K in {3,4}
# Animate: barycentric point over time on simplex (triangle/tetra)
#
# Assumptions:
#   - model, cfg, DEVICE exist
#   - model(input_ids, attention_mask=..., return_info=True) -> (logits, infos)
#   - infos[layer]["read_logits"] exists: [B,H,T,K] float32
#
# Output:
#   - For each chosen layer: clustering summary + animation(s)
# ================================================================
import os, math, numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from matplotlib import animation
from IPython.display import HTML, display

# --------------------------
# USER SETTINGS
# --------------------------
LAYERS_WANTED = [ 0, 5, 13 ]
META_K = 3               # 3=triangle (recommended first), 4=tetra (also supported)
LATE_W = 2               # late window length (tokens from end, per-sample)
ENTROPY_TEMP = 1.0       # softmax temp for entropy readout (not changing routing)

PADDING_SIDE = "right"   # "right" fixed your NaNs earlier. keep it.
USE_ATTENTION_MASK = True  # you can set False to ignore mask if you want

PROMPTS = [
    "The capital of France is",
    "France's capital city is",
    "In France, the capital is",
    "The seat of government of France is",
    "The city that is France's capital is",
    "The Eiffel Tower is located in",
    "The Louvre is in",
    "The Seine river flows through",
    "During the French Revolution, the city of",
    "Napoleon crowned himself in",
    "In World War II, the liberation of",
    "The largest city in France is",
    "A major European city in France is",
    "A famous city on the Seine is",
    "The capital of Germany is",
    "The capital of England is",
    "The capital of Italy is",
]

TARGET_TEXT = " Paris"     # must be single token
RIVAL_TEXT  = " the"       # must be single token

FPS = 1
SAVE_GIF = True
GIF_DIR = "./asm_viz_gifs"
os.makedirs(GIF_DIR, exist_ok=True)

# --------------------------
# Tokenizer (robust)
# --------------------------
def _get_tokenizer():
    if "tokenizer" in globals():
        tok = globals()["tokenizer"]
    else:
        from transformers import AutoTokenizer
        tok = AutoTokenizer.from_pretrained(getattr(cfg, "tokenizer_name", "gpt2"))

    # padding token & side
    tok.padding_side = PADDING_SIDE
    if tok.pad_token is None:
        # GPT2-style: use eos as pad
        tok.pad_token = tok.eos_token
    return tok

tokenizer = _get_tokenizer()

def ensure_single_token(text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) != 1:
        raise ValueError(f"{text!r} must be exactly 1 token. Got ids={ids} (len={len(ids)})")
    return ids[0]

TARGET_ID = ensure_single_token(TARGET_TEXT)
RIVAL_ID  = ensure_single_token(RIVAL_TEXT)

# --------------------------
# Forward helper
# --------------------------
@torch.no_grad()
def forward_with_info(input_ids, attention_mask):
    out = model(input_ids, attention_mask=attention_mask, return_info=True, routing_mode="softmax")
    logits, infos = out
    return logits, infos

# --------------------------
# Encode prompts
# --------------------------
enc = tokenizer(
    PROMPTS,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=getattr(cfg, "max_seq_len", 1024),
)
input_ids = enc["input_ids"].to(DEVICE)
attention_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

B, T = input_ids.shape
lengths = attention_mask.sum(dim=1).long()
t_last = (lengths - 1).clamp(min=0)

# late window indices per sample (clip)
def late_indices_for_sample(t_end, W, T_total):
    t1 = int(t_end)
    t0 = max(0, t1 - (W - 1))
    return list(range(t0, t1 + 1))

# --------------------------
# Determine available layers
# --------------------------
n_layers = len(getattr(model, "blocks", []))
if n_layers == 0:
    raise RuntimeError("Could not detect model.blocks; expected ASMLanguageModel style.")
layers = [L for L in LAYERS_WANTED if 0 <= L < n_layers]
if len(layers) == 0:
    raise RuntimeError(f"No requested layers are valid. model has n_layers={n_layers}")

print(f"[info] model layers={n_layers} | using layers={layers} | META_K={META_K} | LATE_W={LATE_W} | padding_side={PADDING_SIDE}")

# --------------------------
# Sanitize utility
# --------------------------
def sanitize(x: torch.Tensor, name: str):
    # Replace non-finite with zeros; also report rate.
    good = torch.isfinite(x)
    frac = good.float().mean().item()
    if frac < 0.999:
        print(f"[health] {name}: finite={100*frac:.2f}%  (sanitizing non-finite to 0)")
    return torch.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)

# --------------------------
# Extract routing mass for a layer
# --------------------------
@torch.no_grad()
def get_routing_mass_for_layer(infos, layer_idx):
    info = infos[layer_idx]
    read_logits = info.get("read_logits", None)  # [B,H,T,K]
    if read_logits is None:
        raise KeyError(f"infos[{layer_idx}] missing 'read_logits'. keys={list(info.keys())}")

    rl = sanitize(read_logits.float(), f"read_logits(L{layer_idx})")
    # head-mean (router mass is per-head in module; here we use mean-head lens)
    Z = rl.mean(dim=1)  # [B,T,K]
    # routing mass
    W = torch.softmax(Z, dim=-1)  # [B,T,K]
    return Z, W  # logits, mass

# --------------------------
# Late-window covariance clustering (NOT magnitude)
# --------------------------
def cluster_slots_by_late_cov(W_btk: torch.Tensor, t_last: torch.Tensor, meta_k: int, late_w: int):
    """
    W_btk: [B,T,K] routing mass
    Clustering features: covariance structure across (b,t in late window)
    Approach:
      1) collect late-window W rows -> X [N,K]
      2) compute covariance C [K,K]
      3) cluster slots based on rows of C (covariance fingerprint)
    """
    B, T, K = W_btk.shape
    rows = []
    for b in range(B):
        for t in late_indices_for_sample(t_last[b].item(), late_w, T):
            rows.append(W_btk[b, t].detach().cpu())
    X = torch.stack(rows, dim=0).float()  # [N,K]
    # center
    X = X - X.mean(dim=0, keepdim=True)
    # covariance
    C = (X.T @ X) / max(1, X.shape[0] - 1)  # [K,K]
    C = C.numpy()

    # cluster on covariance rows (each slot i represented by C[i,:])
    feats = C.copy()

    # Try sklearn KMeans, fallback to a simple spectral-ish heuristic if unavailable
    labels = None
    try:
        from sklearn.cluster import KMeans
        km = KMeans(n_clusters=meta_k, n_init=20, random_state=0)
        labels = km.fit_predict(feats)
    except Exception as e:
        print("[warn] sklearn KMeans unavailable/failed, using fallback clustering:", repr(e))
        # fallback: farthest-point seeding + assignment by cosine sim
        feats_t = feats / (np.linalg.norm(feats, axis=1, keepdims=True) + 1e-9)
        # seed 0 = max variance row
        seeds = [int(np.argmax(np.linalg.norm(feats_t, axis=1)))]
        for _ in range(1, meta_k):
            sims = feats_t @ feats_t[seeds].T  # [K, len(seeds)]
            dist = 1.0 - np.max(sims, axis=1)
            seeds.append(int(np.argmax(dist)))
        seed_vecs = feats_t[seeds]  # [meta_k,K]
        sims = feats_t @ seed_vecs.T  # [K,meta_k]
        labels = np.argmax(sims, axis=1)

    # build groups
    groups = [[] for _ in range(meta_k)]
    for i, g in enumerate(labels.tolist()):
        groups[g].append(i)

    # ensure no empty groups (rare but possible): fix by moving one slot from largest group
    empties = [i for i,g in enumerate(groups) if len(g)==0]
    if len(empties) > 0:
        sizes = [len(g) for g in groups]
        for eg in empties:
            lg = int(np.argmax(sizes))
            move = groups[lg].pop()  # move one
            groups[eg].append(move)
            sizes[lg] -= 1
            sizes[eg] += 1

    return groups, C

# --------------------------
# Aggregate routing mass to meta-slots
# --------------------------
def aggregate_to_meta(W_btk: torch.Tensor, groups):
    """
    returns W_meta: [B,T,M] where M=len(groups)
    """
    B,T,K = W_btk.shape
    M = len(groups)
    out = torch.zeros((B,T,M), device=W_btk.device, dtype=W_btk.dtype)
    for m, idxs in enumerate(groups):
        out[:,:,m] = W_btk[:,:,idxs].sum(dim=-1)
    # renorm safety
    out = out / out.sum(dim=-1, keepdim=True).clamp_min(1e-9)
    return out

# --------------------------
# Paris margin per prompt
# --------------------------
@torch.no_grad()
def paris_margin(logits, t_last):
    logp = F.log_softmax(sanitize(logits.float(), "logits"), dim=-1)
    idx = torch.arange(logp.shape[0], device=logp.device)
    lp_t = logp[idx, t_last, TARGET_ID]
    lp_r = logp[idx, t_last, RIVAL_ID]
    return (lp_t - lp_r).detach().cpu().numpy()

# --------------------------
# Barycentric plotting helpers
# --------------------------
def simplex_vertices(M):
    if M == 3:
        # 2D equilateral triangle
        V = np.array([
            [0.0, 0.0],
            [1.0, 0.0],
            [0.5, math.sqrt(3)/2],
        ], dtype=np.float32)
        return V
    if M == 4:
        # 3D tetrahedron
        V = np.array([
            [0.0, 0.0, 0.0],
            [1.0, 0.0, 0.0],
            [0.5, math.sqrt(3)/2, 0.0],
            [0.5, math.sqrt(3)/6, math.sqrt(2/3)],
        ], dtype=np.float32)
        return V
    raise ValueError("M must be 3 or 4")

def bary_to_cart(w_m, V):
    # w_m: [M], V: [M,D] -> [D]
    return (w_m.reshape(-1,1) * V).sum(axis=0)

# --------------------------
# Make animation for a single layer
# --------------------------
def animate_layer(layer_idx, Z_btk, W_btk, groups, margins_np, meta_k, late_w):
    """
    For each prompt b, animate barycentric point over t=0..t_last[b].
    Color endpoints by Paris margin.
    Also show trails (last N points).
    """
    B, T, K = W_btk.shape
    M = meta_k
    V = simplex_vertices(M)

    # aggregate
    Wm = aggregate_to_meta(W_btk, groups)  # [B,T,M]
    Wm_cpu = Wm.detach().cpu().numpy()

    # compute per-token entropy (commitment proxy)
    w_ent = -(W_btk * (W_btk.clamp_min(1e-9).log())).sum(dim=-1)  # [B,T]
    ent_cpu = w_ent.detach().cpu().numpy()

    # time axis alignment: choose max t_end across prompts, animate in global time
    t_ends = t_last.detach().cpu().numpy().astype(int)
    T_anim = int(t_ends.max()) + 1

    # Precompute paths in cart coords
    if M == 3:
        paths = np.zeros((B, T_anim, 2), dtype=np.float32)
    else:
        paths = np.zeros((B, T_anim, 3), dtype=np.float32)

    for b in range(B):
        for t in range(T_anim):
            tt = min(t, t_ends[b])
            paths[b, t] = bary_to_cart(Wm_cpu[b, tt], V)

    # figure
    if M == 3:
        fig, ax = plt.subplots(figsize=(8.2, 7.2))
        ax.set_title(f"Layer {layer_idx} | META_K=3 barycentric routing (late cov clustering)\n"
                     f"endpoint color = Paris-margin | trails show motion")
        # draw triangle
        tri = np.vstack([V, V[0]])
        ax.plot(tri[:,0], tri[:,1], linewidth=2.0, alpha=0.7)
        for i in range(3):
            ax.text(V[i,0], V[i,1], f"  G{i} (|S|={len(groups[i])})", fontsize=10)
        ax.set_aspect("equal", adjustable="box")
        ax.set_xlim(-0.1, 1.1)
        ax.set_ylim(-0.1, math.sqrt(3)/2 + 0.1)
        ax.grid(True, alpha=0.2)

        # artists
        scat = ax.scatter([], [], s=50)
        trails = [ax.plot([], [], alpha=0.35, linewidth=1.5)[0] for _ in range(B)]

        # color normalize by margins
        vmin, vmax = float(np.min(margins_np)), float(np.max(margins_np))
        def col(b):
            # use viridis mapping
            import matplotlib as mpl
            cmap = plt.cm.viridis
            z = (margins_np[b] - vmin) / (vmax - vmin + 1e-9)
            return cmap(np.clip(z, 0, 1))

        def init():
            scat.set_offsets(np.zeros((0,2)))
            for tr in trails:
                tr.set_data([], [])
            return [scat] + trails

        TRAIL_LEN = max(3, min(10, T_anim))

        def update(t):
            pts = np.zeros((B,2), dtype=np.float32)
            colors = []
            for b in range(B):
                pts[b] = paths[b, t]
                colors.append(col(b))
                t0 = max(0, t - TRAIL_LEN + 1)
                seg = paths[b, t0:t+1]
                trails[b].set_data(seg[:,0], seg[:,1])
            scat.set_offsets(pts)
            scat.set_color(colors)
            ax.set_xlabel(f"t={t} (global), entropy mean={ent_cpu[:, min(t, ent_cpu.shape[1]-1)].mean():.3f}")
            return [scat] + trails

        anim = animation.FuncAnimation(fig, update, init_func=init, frames=T_anim, interval=1000//FPS, blit=True)
        plt.close(fig)
        return anim

    else:
        # META_K=4 tetra animation (3D)
        from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
        fig = plt.figure(figsize=(9.0, 7.5))
        ax = fig.add_subplot(111, projection="3d")
        ax.set_title(f"Layer {layer_idx} | META_K=4 barycentric routing (late cov clustering)\n"
                     f"endpoint color = Paris-margin | trails show motion")

        # draw tetra edges
        edges = [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]
        for (i,j) in edges:
            ax.plot([V[i,0], V[j,0]], [V[i,1], V[j,1]], [V[i,2], V[j,2]], alpha=0.5)
        for i in range(4):
            ax.text(V[i,0], V[i,1], V[i,2], f"G{i}(|S|={len(groups[i])})", fontsize=9)

        ax.set_xlim(-0.1, 1.1)
        ax.set_ylim(-0.1, math.sqrt(3)/2 + 0.1)
        ax.set_zlim(-0.1, math.sqrt(2/3) + 0.1)

        # artists
        scat = ax.scatter([], [], [], s=50)
        trails = [ax.plot([], [], [], alpha=0.35, linewidth=1.5)[0] for _ in range(B)]

        vmin, vmax = float(np.min(margins_np)), float(np.max(margins_np))
        def col(b):
            import matplotlib as mpl
            cmap = plt.cm.viridis
            z = (margins_np[b] - vmin) / (vmax - vmin + 1e-9)
            return cmap(np.clip(z, 0, 1))

        def init():
            scat._offsets3d = ([], [], [])
            for tr in trails:
                tr.set_data([], [])
                tr.set_3d_properties([])
            return [scat] + trails

        TRAIL_LEN = max(3, min(10, T_anim))

        def update(t):
            xs, ys, zs, colors = [], [], [], []
            for b in range(B):
                p = paths[b, t]
                xs.append(p[0]); ys.append(p[1]); zs.append(p[2])
                colors.append(col(b))
                t0 = max(0, t - TRAIL_LEN + 1)
                seg = paths[b, t0:t+1]
                trails[b].set_data(seg[:,0], seg[:,1])
                trails[b].set_3d_properties(seg[:,2])
            scat._offsets3d = (xs, ys, zs)
            scat.set_color(colors)
            ax.set_xlabel(f"t={t} | entropy mean={ent_cpu[:, min(t, ent_cpu.shape[1]-1)].mean():.3f}")
            return [scat] + trails

        anim = animation.FuncAnimation(fig, update, init_func=init, frames=T_anim, interval=1000//FPS, blit=False)
        plt.close(fig)
        return anim

# --------------------------
# Run: forward once, then per-layer analysis+animation
# --------------------------
with torch.no_grad():
    logits, infos = forward_with_info(
        input_ids,
        attention_mask if USE_ATTENTION_MASK else None
    )

margins_np = paris_margin(logits, t_last)

# top-level progress
pbar = tqdm(layers, desc="Experiment 1: barycentric animations (per layer)")
for L in pbar:
    pbar.set_postfix(layer=L)

    # extract routing mass
    Z, W = get_routing_mass_for_layer(infos, L)  # [B,T,K] logits & mass

    # cluster slots by late-window covariance
    groups, C = cluster_slots_by_late_cov(W, t_last, META_K, LATE_W)

    # print cluster summary
    sizes = [len(g) for g in groups]
    print(f"\n=== Layer {L} | META_K={META_K} late-cov clustering ===")
    print("group sizes:", sizes)
    # show a few slot ids per group
    for gi, g in enumerate(groups):
        print(f"  G{gi}: {g[:12]}{' ...' if len(g) > 12 else ''}")

    # animate
    anim = animate_layer(L, Z, W, groups, margins_np, META_K, LATE_W)

    # render HTML
    display(HTML(anim.to_jshtml()))

    # optional save
    if SAVE_GIF:
        out_path = os.path.join(GIF_DIR, f"bary_L{L}_M{META_K}.gif")
        try:
            anim.save(out_path, writer=animation.PillowWriter(fps=FPS))
            print("[saved]", out_path)
        except Exception as e:
            print("[warn] gif save failed:", repr(e))

print("\n[done] Experiment 1 complete.")

In [ ]:

# ================================================================
#@title Experiment 2 (robust): Prompt-conditional Jacobian alignment via
# FINITE DIFFERENCES in PCA space (works even if autograd is detached).
#
# For each layer:
#  1) Collect late-window read_logits [B,T,K] (head-mean) and fit PCA (PC1..3)
#  2) For each PC direction i=1..3:
#       - Inject read_logits += +eps * PCi at late-window tokens (per-sample)
#       - Inject read_logits += -eps * PCi
#       - Slope g_i ≈ (margin_plus - margin_minus) / (2*eps)
#     This gives gradient coordinates directly in PCA basis.
#
# Outputs:
#  - frac_pc1 = g1^2 / (g1^2+g2^2+g3^2)  (PC1 dominance)
#  - frac_pc2, frac_pc3 similarly (rotation/spread)
#  - angle_pc12 = atan2(g2, g1)
#
# Assumptions:
#  - model, cfg exist
#  - model(..., return_info=True, routing_mode="softmax") -> (logits, infos)
#  - infos[layer]["read_logits"] exists as [B,H,T,K]
#  - model.blocks[li].asa.routing_override exists and is used like earlier cells
# ================================================================

import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

DEVICE = next(model.parameters()).device

# ---------------------------
# SETTINGS
# ---------------------------
PROMPTS = [
    "The capital of France is",
    "France's capital city is",
    "In France, the capital is",
    "The seat of government of France is",
    "The city that is France's capital is",
    "The Eiffel Tower is located in",
    "The Louvre is in",
    "The Seine river flows through",
    "During the French Revolution, the city of",
    "Napoleon crowned himself in",
    "In World War II, the liberation of",
    "The largest city in France is",
    "A major European city in France is",
    "A famous city on the Seine is",
    "The capital of Germany is",
    "The capital of England is",
    "The capital of Italy is",
]

TARGET_TEXT = " Paris"   # must be single token
RIVAL_TEXT  = " the"     # must be single token

MAX_LEN = cfg.max_seq_len
LATE_W = 5               # late window length
PCA_D = 3
SKIP_T0 = True

# finite difference magnitude (start here; if noisy, try 2e-2; if saturating, try 5e-3)
FD_EPS = 1e-2

# how many layers to scan (auto-detect)
# (we’ll scan all that expose read_logits)
PLOT = True

EPS = 1e-9
MIN_PCA_POINTS = 32

# ---------------------------
# Tokenizer (robust + RIGHT padding)
# ---------------------------
def _get_tokenizer():
    if "tokenizer" in globals():
        tok = globals()["tokenizer"]
    else:
        from transformers import AutoTokenizer
        tok = AutoTokenizer.from_pretrained(getattr(cfg, "tokenizer_name", "gpt2"))
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "right"
    return tok

tokenizer = _get_tokenizer()

def ensure_single_token(text):
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) != 1:
        raise ValueError(f"{text!r} must be exactly 1 token. Got {ids}")
    return ids[0]

TARGET_ID = ensure_single_token(TARGET_TEXT)
RIVAL_ID  = ensure_single_token(RIVAL_TEXT)

# ---------------------------
# Batch tokenize prompts
# ---------------------------
enc = tokenizer(PROMPTS, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN)
input_ids = enc["input_ids"].to(DEVICE)
attention_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

B, T = input_ids.shape
lengths = attention_mask.sum(dim=1).long()
t_last = (lengths - 1).clamp(min=0)  # [B]

def late_bounds(t_last_1d, late_w):
    bounds = []
    for b in range(t_last_1d.shape[0]):
        te = int(t_last_1d[b].item())
        t0 = max(0, te - late_w + 1)
        t1 = te
        if SKIP_T0 and t0 == 0:
            t0 = 1
        bounds.append((t0, t1))
    return bounds

bounds = late_bounds(t_last, LATE_W)

# ---------------------------
# Forward helper
# ---------------------------
@torch.no_grad()
def forward_with_info(ids, mask):
    model.eval()
    logits, infos = model(ids, attention_mask=mask, return_info=True, routing_mode="softmax")
    return logits, infos

@torch.no_grad()
def paris_margin_from_logits(logits):
    logp = F.log_softmax(logits, dim=-1)
    idx = torch.arange(B, device=logits.device)
    m = (logp[idx, t_last, TARGET_ID] - logp[idx, t_last, RIVAL_ID]).mean().item()
    return m

# ---------------------------
# PCA fit (stable SVD on CPU)
# ---------------------------
def fit_pca_svd(X_cpu, d=3):
    # X_cpu: [N,K] on CPU, finite
    mu = X_cpu.mean(dim=0, keepdim=True)
    Xc = X_cpu - mu
    U, S, Vh = torch.linalg.svd(Xc, full_matrices=False)
    W = Vh[:d].T.contiguous()  # [K,d]
    s2 = (S**2)
    evr = (s2[:d] / s2.sum().clamp_min(EPS)).cpu().numpy()
    return mu, W, evr

# ---------------------------
# Routing override that ADDS delta to read_logits before softmax
# at specified tokens (late window), per example.
# ---------------------------
def clear_routing_override():
    for li in range(len(model.blocks)):
        model.blocks[li].asa.routing_override = None

def set_additive_readlogits_override(layer_idx, delta_btK, bounds, rtemp=1.0):
    """
    delta_btK: [B,T,K] additive perturbation (mostly zeros except late window)
    bounds: list[(t0,t1)] per example; used only for safety checks
    """
    clear_routing_override()

    def _override(t0, t1, read_logits_chunk, read_logits_key, read_logits_content, ctx):
        # read_logits_chunk: [B,H,L,K], chunk time is [t0,t1)
        Bc, Hc, Lc, Kc = read_logits_chunk.shape
        # softmax weights from (read_logits + delta)
        # Build delta chunk: [B,1,L,K]
        # Note: delta_btK is full length T; slice to this chunk.
        delta_chunk = delta_btK[:, t0:t1, :].to(read_logits_chunk.device)  # [B,L,K]
        delta_chunk = delta_chunk.unsqueeze(1)  # [B,1,L,K]
        rl = read_logits_chunk + delta_chunk  # broadcast heads
        w = torch.softmax(rl / max(1e-6, float(rtemp)), dim=-1)
        w = w / w.sum(dim=-1, keepdim=True).clamp_min(1e-8)
        return w

    model.blocks[layer_idx].asa.routing_override = _override

# ---------------------------
# Main scan
# ---------------------------
# First baseline forward to get all infos + determine layers
logits0, infos0 = forward_with_info(input_ids, attention_mask)
margin0 = paris_margin_from_logits(logits0)

n_layers = len(infos0)
K = infos0[0]["read_logits"].shape[-1]
H = infos0[0]["read_logits"].shape[1]

print(f"[info] detected layers={n_layers} | B={B} T={T} K={K} H={H} | baseline margin={margin0:.4f}")

rows = []
pbar = tqdm(range(n_layers), desc="FD Jacobian alignment per layer", leave=True)

for layer in pbar:
    if "read_logits" not in infos0[layer]:
        rows.append({"layer": layer, "ok": False, "reason": "no_read_logits"})
        continue

    # collect late-window read_logits head-mean from this layer
    rl_bhTK = infos0[layer]["read_logits"]  # [B,H,T,K]
    rl_btK = rl_bhTK.mean(dim=1).float()    # [B,T,K]

    # build PCA dataset from late window
    pts = []
    for b in range(B):
        t0, t1 = bounds[b]
        if t0 > t1:
            continue
        pts.append(rl_btK[b, t0:t1+1, :])
    X = torch.cat(pts, dim=0)  # [N,K]

    # sanitize finites
    finite = torch.isfinite(X).all(dim=-1)
    X = X[finite]
    n_good = int(X.shape[0])
    if n_good < MIN_PCA_POINTS:
        rows.append({"layer": layer, "ok": False, "reason": f"too_few_points({n_good})"})
        continue

    mu, W, evr = fit_pca_svd(X.detach().cpu(), d=PCA_D)  # mu [1,K], W [K,3] on CPU

    # FD gradients in PC coords: g_i = d margin / d alpha_i
    g_pc = np.zeros((PCA_D,), dtype=np.float64)

    for i in range(PCA_D):
        v = W[:, i].detach().cpu().numpy().astype(np.float32)  # [K], unit direction

        # build delta tensor [B,T,K] (mostly zeros; apply to late window tokens)
        delta = torch.zeros((B, T, K), device=DEVICE, dtype=torch.float32)
        for b in range(B):
            t0, t1 = bounds[b]
            if t0 > t1:
                continue
            delta[b, t0:t1+1, :] = torch.from_numpy(v).to(DEVICE)  # broadcast over time

        # +eps
        set_additive_readlogits_override(layer, delta * float(FD_EPS), bounds, rtemp=1.0)
        lp, _ = forward_with_info(input_ids, attention_mask)
        m_plus = paris_margin_from_logits(lp)

        # -eps
        set_additive_readlogits_override(layer, delta * float(-FD_EPS), bounds, rtemp=1.0)
        lm, _ = forward_with_info(input_ids, attention_mask)
        m_minus = paris_margin_from_logits(lm)

        clear_routing_override()

        g_pc[i] = (m_plus - m_minus) / (2.0 * float(FD_EPS))

    # energy fractions and angle
    g2 = float((g_pc**2).sum()) + 1e-18
    frac = (g_pc**2) / g2
    ang12 = float(math.atan2(g_pc[1], g_pc[0])) if PCA_D >= 2 else float("nan")

    rows.append({
        "layer": layer,
        "ok": True,
        "n_points": n_good,
        "evr1": float(evr[0]),
        "evr2": float(evr[1]),
        "evr3": float(evr[2]),
        "g_pc1": float(g_pc[0]),
        "g_pc2": float(g_pc[1]),
        "g_pc3": float(g_pc[2]),
        "frac_pc1": float(frac[0]),
        "frac_pc2": float(frac[1]),
        "frac_pc3": float(frac[2]),
        "angle_pc12": ang12,
    })

# ---------------------------
# Report + plots
# ---------------------------
ok = [r for r in rows if r.get("ok", False)]
bad = [r for r in rows if not r.get("ok", False)]
print(f"[done] ok={len(ok)} skipped={len(bad)} FD_EPS={FD_EPS}")

print("\n=== Per-layer FD Jacobian (compact) ===")
for r in ok:
    print(
        f"L{r['layer']:2d} | frac=[{r['frac_pc1']:.3f},{r['frac_pc2']:.3f},{r['frac_pc3']:.3f}] | "
        f"g=[{r['g_pc1']:+.3g},{r['g_pc2']:+.3g},{r['g_pc3']:+.3g}] | "
        f"EVR1={r['evr1']:.3f} | ang12={r['angle_pc12']:+.2f} | n={r['n_points']}"
    )

if PLOT and ok:
    layers = [r["layer"] for r in ok]
    frac1 = [r["frac_pc1"] for r in ok]
    frac2 = [r["frac_pc2"] for r in ok]
    frac3 = [r["frac_pc3"] for r in ok]
    ang12 = [r["angle_pc12"] for r in ok]

    plt.figure(figsize=(8,4.5))
    plt.plot(layers, frac1, marker="o", label="PC1")
    plt.plot(layers, frac2, marker="o", label="PC2")
    plt.plot(layers, frac3, marker="o", label="PC3")
    plt.title("FD Jacobian energy fraction in PCA coords vs layer (Paris-margin)")
    plt.xlabel("layer")
    plt.ylabel("frac of ||grad||^2")
    plt.grid(alpha=0.25)
    plt.legend()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8,4.5))
    plt.plot(layers, ang12, marker="o")
    plt.title("FD mean Jacobian direction angle in (PC1,PC2) vs layer")
    plt.xlabel("layer")
    plt.ylabel("atan2(g2, g1) [rad]")
    plt.grid(alpha=0.25)
    plt.tight_layout()
    plt.show()

In [ ]:

# ================================================================
#@title Next Cell: Tangent Control Field on the Simplex Sphere (Late-Window Centroid)
# For layers [0,5,13]:
#   - Extract router read_logits trajectories (head-mean) per prompt
#   - Fit PCA->3D on late-window points, project to S^2
#   - Compute FD gradient of Paris-margin w.r.t. a 3D PCA coordinate at each late-window point
#   - Average per-prompt gradients across late-window points
#   - Project gradient to tangent plane at each prompt's late-window centroid on the sphere
#   - Plot: trajectories + centroid points (colored by Paris-margin) + tangent arrows (control direction)
#
# Robustness:
#   - Enforces RIGHT padding + right padding side to avoid NaN / mask issues
#   - Sanitizes non-finite values
#   - Uses FD (finite difference) so no need for autograd plumbing through infos
#   - Includes tqdm progress bars
#
# Assumptions:
#   - model, cfg, DEVICE exist
#   - model(..., return_info=True, routing_mode="softmax") -> (logits, infos)
#   - infos[layer]["read_logits"] exists, shape [B,H,T,K]
# ================================================================
import math
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from tqdm.auto import tqdm

# ---------------------------
# SETTINGS
# ---------------------------
LAYERS = [0, 5, 13]
LATE_W = 5                   # late-window size (last W valid tokens)
FD_EPS = 1e-2                # finite difference step in PCA coords
FD_MODE = "central"          # "central" or "forward"
PLOT_STRIDE = 1              # downsample trajectory points
ARROW_SCALE = 0.25           # arrow length scaling in sphere coords (visual)
ARROW_ALPHA = 0.9
TRAJ_ALPHA = 0.45
TRAJ_LW = 1.8
SPHERE_ALPHA = 0.06
SPHERE_WIREFRAME = True
LABEL_MAX = 10               # label top prompts by Paris-margin
SKIP_T0_IN_PCA = True        # skip token index 0 (often special / unstable)
SANITIZE_MODE = "drop"       # "drop" or "clamp" for non-finite read_logits in PCA fit
CLAMP_VAL = 5.0              # used if SANITIZE_MODE="clamp"

# prompts (use your current set; replace if desired)
PROMPTS = [
    "The capital of France is",
    "France's capital city is",
    "In France, the capital is",
    "The seat of government of France is",
    "The city that is France's capital is",
    "The Eiffel Tower is located in",
    "The Louvre is in",
    "The Seine river flows through",
    "During the French Revolution, the city of",
    "Napoleon crowned himself in",
    "In World War II, the liberation of",
    "The largest city in France is",
    "A major European city in France is",
    "A famous city on the Seine is",
    "The capital of Germany is",
    "The capital of England is",
    "The capital of Italy is",
]

ENDPOINT_COLOR_MODE = "paris_margin"  # only mode supported here (for colorbar)
RIVAL_TEXT = " the"                   # single-token rival for margin

# ---------------------------
# Tokenizer helpers (RIGHT padding enforced)
# ---------------------------
def _get_tokenizer_from_context():
    if "tokenizer" in globals():
        tok = globals()["tokenizer"]
        # enforce right padding (critical)
        tok.padding_side = "right"
        if tok.pad_token is None:
            tok.pad_token = tok.eos_token
        return tok
    from transformers import AutoTokenizer
    tok = AutoTokenizer.from_pretrained(getattr(cfg, "tokenizer_name", "gpt2"))
    tok.padding_side = "right"
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok

tokenizer = _get_tokenizer_from_context()

def ensure_single_token(text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) != 1:
        raise ValueError(f"{text!r} must be exactly 1 token. Got ids={ids}")
    return ids[0]

PARIS_ID = ensure_single_token(" Paris")
RIVAL_ID = ensure_single_token(RIVAL_TEXT)

# ---------------------------
# Forward with info
# ---------------------------
def forward_with_info(ids, mask, routing_mode="softmax"):
    logits, infos = model(ids, attention_mask=mask, return_info=True, routing_mode=routing_mode)
    return logits, infos

# ---------------------------
# Batch tokenize (RIGHT padding)
# ---------------------------
enc = tokenizer(
    PROMPTS,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=cfg.max_seq_len,
)
input_ids = enc["input_ids"].to(DEVICE)
attention_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

B, T = input_ids.shape
lengths = attention_mask.sum(dim=1).long()
t_last = (lengths - 1).clamp(min=0)

def late_window_indices_for_b(b: int):
    """Return list of valid token indices in the late window for prompt b."""
    last = int(t_last[b].item())
    t0 = max(0, last - (LATE_W - 1))
    idxs = list(range(t0, last + 1))
    if SKIP_T0_IN_PCA:
        idxs = [t for t in idxs if t != 0]
    return idxs

late_idxs = [late_window_indices_for_b(b) for b in range(B)]

# ---------------------------
# Baseline logits (for Paris margin)
# ---------------------------
with torch.no_grad():
    logits_base, _infos_base = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
    logp = F.log_softmax(logits_base, dim=-1)  # [B,T,V]
    idx = torch.arange(B, device=DEVICE)
    lp_paris = logp[idx, t_last, PARIS_ID].detach().cpu().numpy()
    lp_rival = logp[idx, t_last, RIVAL_ID].detach().cpu().numpy()
    paris_margin = lp_paris - lp_rival

# ---------------------------
# PCA fit utilities (PyTorch SVD on GPU; fallback to CPU if needed)
# ---------------------------
def _finite_mask(x: torch.Tensor) -> torch.Tensor:
    return torch.isfinite(x).all(dim=-1)

def fit_pca3_from_points(X_points: torch.Tensor):
    """
    X_points: [N,D] float32
    returns: mu [D], W3 [D,3], EVR [3], evals [3]
    """
    # sanitize
    if SANITIZE_MODE == "clamp":
        X_points = torch.nan_to_num(X_points, nan=0.0, posinf=CLAMP_VAL, neginf=-CLAMP_VAL)
        good = _finite_mask(X_points)
    else:
        good = _finite_mask(X_points)
        X_points = X_points[good]

    n_good = int(X_points.shape[0])
    if n_good < 10:
        raise RuntimeError(f"Too few finite points to fit PCA (good={n_good}).")

    mu = X_points.mean(dim=0, keepdim=True)
    Xc = X_points - mu

    # Attempt GPU SVD; fallback to CPU robustly
    try:
        U, S, Vh = torch.linalg.svd(Xc, full_matrices=False)
    except Exception:
        Xc_cpu = Xc.detach().cpu()
        U, S, Vh = torch.linalg.svd(Xc_cpu, full_matrices=False)
        # move back to original device for W/mu usage
        Vh = Vh.to(Xc.device)
        S = S.to(Xc.device)

    W3 = Vh[:3].T.contiguous()  # [D,3]

    # explained variance ratio via singular values
    s2 = (S ** 2)
    evr3 = (s2[:3] / s2.sum().clamp_min(1e-12)).detach().cpu().numpy()
    evals3 = s2[:3].detach().cpu().numpy()
    return mu.squeeze(0), W3, evr3, evals3, n_good

def project_to_sphere(Z: torch.Tensor, mu: torch.Tensor, W3: torch.Tensor, eps=1e-9):
    """
    Z: [N,D]
    returns: XYZ [N,3] (raw PCA), XYZs [N,3] (sphere)
    """
    X3 = (Z - mu[None, :]) @ W3  # [N,3]
    X3_np = X3.detach().cpu().numpy()
    n = np.linalg.norm(X3_np, axis=1, keepdims=True)
    X3s_np = X3_np / np.clip(n, eps, None)
    return X3_np, X3s_np

# ---------------------------
# Routing override machinery (forces read_weights at chosen positions)
# ---------------------------
def clear_routing_override():
    for li in range(len(model.blocks)):
        model.blocks[li].asa.routing_override = None

def set_override_for_layer(layer_idx: int, r_bhk: torch.Tensor, t_targets_cpu: np.ndarray):
    """
    r_bhk: [B,H,K] override distribution for each example at its chosen position (here, t_last)
    t_targets_cpu: [B] indices in [0..T-1]
    """
    clear_routing_override()

    def _override(t0, t1, read_logits_chunk, read_logits_key, read_logits_content, ctx):
        # read_logits_chunk: [B,H,L,K]
        Bc, Hc, Lc, Kc = read_logits_chunk.shape
        rtemp = float(ctx.get("rtemp", 1.0))
        w = torch.softmax(read_logits_chunk / max(1e-6, rtemp), dim=-1)
        for b in range(Bc):
            tt = int(t_targets_cpu[b])
            if t0 <= tt < t1:
                lp = tt - t0
                w[b, :, lp, :] = r_bhk[b]  # [H,K]
        w = w / w.sum(dim=-1, keepdim=True).clamp_min(1e-8)
        return w

    model.blocks[layer_idx].asa.routing_override = _override

@torch.no_grad()
def paris_margin_under_forced_routing(layer_idx: int, r_bhk: torch.Tensor):
    """
    Force routing at t_last for the specified layer only, and return mean Paris-margin.
    """
    t_targets_cpu = t_last.detach().cpu().numpy()
    set_override_for_layer(layer_idx, r_bhk, t_targets_cpu)
    logits, _infos = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
    clear_routing_override()
    logp = F.log_softmax(logits, dim=-1)
    idx = torch.arange(B, device=logits.device)
    lpP = logp[idx, t_last, PARIS_ID]
    lpR = logp[idx, t_last, RIVAL_ID]
    m = (lpP - lpR).mean().item()
    return m

# ---------------------------
# Main per-layer computation
# ---------------------------
def layer_control_field(layer_idx: int):
    """
    Returns:
      dict with trajectories on sphere, centroids, tangent arrows, PCA EVR, etc.
    """
    # Forward to collect read_logits for this layer
    with torch.no_grad():
        _logits, infos = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
    rl = infos[layer_idx].get("read_logits", None)
    if rl is None:
        raise KeyError(f"infos[{layer_idx}] missing 'read_logits'.")

    # rl: [B,H,T,K] -> Z: [B,T,K] (head mean)
    Z = rl.mean(dim=1).float()  # [B,T,K]
    K = Z.shape[-1]
    H = rl.shape[1]

    # Collect late-window points for PCA fit: points are in D=K space
    pts = []
    owner = []  # (b, t) mapping for each point
    for b in range(B):
        for t in late_idxs[b]:
            pts.append(Z[b, t, :])
            owner.append((b, t))
    Zfit = torch.stack(pts, dim=0)  # [N,D]

    # Fit PCA3
    mu, W3, evr3, evals3, n_good = fit_pca3_from_points(Zfit)

    # Project all late points to sphere for trajectories + centroids
    # We'll also project ALL time points for visualization (but PCA fit is late-window)
    # For plotting trajectories we project each prompt's valid points.
    traj_sphere = []
    traj_raw = []
    for b in range(B):
        valid_t = [t for t in range(T) if int(attention_mask[b, t].item()) == 1]
        if SKIP_T0_IN_PCA:
            valid_t = [t for t in valid_t if t != 0]
        if len(valid_t) == 0:
            traj_sphere.append(np.zeros((0, 3)))
            traj_raw.append(np.zeros((0, 3)))
            continue
        Zb = Z[b, valid_t, :]  # [Tv,K]
        X3_np, X3s_np = project_to_sphere(Zb, mu, W3)
        traj_raw.append(X3_np[::PLOT_STRIDE])
        traj_sphere.append(X3s_np[::PLOT_STRIDE])

    # Late-window centroids on sphere (mean of sphere points then renormalize)
    centroids = np.zeros((B, 3), dtype=np.float32)
    late_points_s = [[] for _ in range(B)]
    for (b, t), z in zip(owner, pts):
        X3_np, X3s_np = project_to_sphere(z[None, :], mu, W3)
        late_points_s[b].append(X3s_np[0])
    for b in range(B):
        arr = np.array(late_points_s[b], dtype=np.float32)
        if arr.size == 0:
            centroids[b] = np.array([1.0, 0.0, 0.0], dtype=np.float32)
        else:
            c = arr.mean(axis=0)
            c = c / max(1e-9, float(np.linalg.norm(c)))
            centroids[b] = c

    # ----- FD gradient in PCA coords (3D) per prompt -----
    # For each late point (b,t), define a routing distribution from the PCA coordinate:
    #   z' = mu + (x3 @ W3^T)   (inverse map into K-space)
    #   r = softmax(z')
    # Then we apply that same r at t_last across the batch, and measure mean Paris-margin.
    #
    # Important: This FD is "global scalar margin sensitivity" to changing that K-dim routing mixture
    # at a chosen layer, expressed in the PCA(3) coordinates used for visualization.
    #
    # We'll compute per-(b,t) FD grads, then average over late points for each prompt b.
    def pca3_to_zk(x3: torch.Tensor):
        # x3: [N,3] -> z: [N,K]
        return mu.to(x3.device)[None, :] + (x3 @ W3.T.to(x3.device))

    def x3_to_rk(x3: torch.Tensor):
        zk = pca3_to_zk(x3)  # [N,K]
        return torch.softmax(zk, dim=-1)

    # baseline r at each late point:
    # compute x3 for each late point (owner order)
    # We'll do this on GPU
    x3_list = []
    with torch.no_grad():
        for z in pts:
            x3 = ((z - mu.to(z.device)) @ W3.to(z.device))  # [3]
            x3_list.append(x3)
    X3 = torch.stack(x3_list, dim=0).to(DEVICE)  # [N,3]

    # FD gradients for each late point
    N = X3.shape[0]
    grads = torch.zeros((N, 3), device=DEVICE, dtype=torch.float32)

    def eval_margin_for_x3(x3_point: torch.Tensor):
        # x3_point: [3] -> r_k [K] -> r_bhk [B,H,K] (broadcast same mixture across batch)
        r_k = x3_to_rk(x3_point[None, :])[0]  # [K]
        r_bhk = r_k.view(1, 1, K).expand(B, H, K).contiguous()
        return paris_margin_under_forced_routing(layer_idx, r_bhk)

    # Use a progress bar: per-layer FD is N*(2*3) evals in central mode
    it = tqdm(range(N), desc=f"FD grads layer {layer_idx}", leave=False)
    for i in it:
        x = X3[i]
        if FD_MODE == "central":
            for d in range(3):
                e = torch.zeros_like(x)
                e[d] = FD_EPS
                m_plus = eval_margin_for_x3(x + e)
                m_minus = eval_margin_for_x3(x - e)
                grads[i, d] = (m_plus - m_minus) / (2.0 * FD_EPS)
        else:
            m0 = eval_margin_for_x3(x)
            for d in range(3):
                e = torch.zeros_like(x)
                e[d] = FD_EPS
                m_plus = eval_margin_for_x3(x + e)
                grads[i, d] = (m_plus - m0) / FD_EPS

    # Average per prompt over its late points
    g_prompt = torch.zeros((B, 3), device=DEVICE)
    counts = torch.zeros((B,), device=DEVICE)
    for i, (b, t) in enumerate(owner):
        g_prompt[b] += grads[i]
        counts[b] += 1.0
    g_prompt = g_prompt / counts.clamp_min(1.0).unsqueeze(-1)

    # Project gradient to tangent plane at centroid x on sphere (in R^3)
    g_np = g_prompt.detach().cpu().numpy()
    tangent = np.zeros_like(g_np)
    for b in range(B):
        x = centroids[b]
        g = g_np[b]
        # remove radial component
        tangent[b] = g - np.dot(g, x) * x
        # normalize for plotting (avoid huge variance)
        n = np.linalg.norm(tangent[b])
        if n > 1e-9:
            tangent[b] = tangent[b] / n
        else:
            tangent[b] = np.zeros(3, dtype=np.float32)

    return {
        "layer": layer_idx,
        "traj_sphere": traj_sphere,
        "centroids": centroids,
        "tangent_dir": tangent,
        "paris_margin": paris_margin,
        "evr3": evr3,
        "evals3": evals3,
        "n_fit": n_good,
    }

# ---------------------------
# Plotting: sphere + trajectories + centroid points + tangent arrows
# ---------------------------
def plot_layer_field(res):
    layer = res["layer"]
    traj_s = res["traj_sphere"]
    cent = res["centroids"]
    tang = res["tangent_dir"]
    pm = res["paris_margin"]
    evr3 = res["evr3"]

    fig = plt.figure(figsize=(11.0, 8.8))
    ax = fig.add_subplot(111, projection="3d")

    # faint sphere
    u = np.linspace(0, 2*np.pi, 48)
    v = np.linspace(0, np.pi, 24)
    xs = np.outer(np.cos(u), np.sin(v))
    ys = np.outer(np.sin(u), np.sin(v))
    zs = np.outer(np.ones_like(u), np.cos(v))
    if SPHERE_WIREFRAME:
        ax.plot_wireframe(xs, ys, zs, rstride=3, cstride=3, linewidth=0.5, alpha=SPHERE_ALPHA)
    else:
        ax.plot_surface(xs, ys, zs, alpha=SPHERE_ALPHA, linewidth=0)

    # plot trajectories
    for b in range(B):
        pts = traj_s[b]
        if pts.shape[0] >= 2:
            ax.plot(pts[:,0], pts[:,1], pts[:,2], linewidth=TRAJ_LW, alpha=TRAJ_ALPHA)

    # centroid points colored by Paris-margin
    import matplotlib as mpl
    normc = mpl.colors.Normalize(vmin=float(np.min(pm)), vmax=float(np.max(pm)))
    cmap = plt.cm.viridis

    colors = cmap(normc(pm))
    ax.scatter(cent[:,0], cent[:,1], cent[:,2], c=colors, s=70, alpha=0.95)

    # tangent arrows (quiver)
    # draw small arrows starting at centroid, pointing along tangent_dir
    for b in range(B):
        x = cent[b]
        d = tang[b]
        if np.linalg.norm(d) < 1e-9:
            continue
        ax.quiver(
            x[0], x[1], x[2],
            d[0], d[1], d[2],
            length=ARROW_SCALE,
            normalize=True,
            alpha=ARROW_ALPHA,
            linewidth=1.5,
        )

    # label top prompts by Paris-margin
    order = np.argsort(-pm)
    for i in order[: min(LABEL_MAX, B)]:
        x = cent[i]
        ax.text(x[0], x[1], x[2], f" {i}", fontsize=10)

    # colorbar
    sm = mpl.cm.ScalarMappable(cmap=cmap, norm=normc)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.08)
    cbar.set_label(f"logP(' Paris') - logP({RIVAL_TEXT!r})")

    ax.set_title(
        f"Tangent Control Field on Simplex Sphere (late-window centroid)\n"
        f"layer={layer} | late_W={LATE_W} | FD_EPS={FD_EPS} ({FD_MODE}) | PCA EVR(PC1..3)≈{np.round(evr3,4)}"
    )
    ax.set_xlabel("PC1 (sphere)")
    ax.set_ylabel("PC2 (sphere)")
    ax.set_zlabel("PC3 (sphere)")
    ax.set_box_aspect([1,1,1])
    plt.tight_layout()
    plt.show()

    # quick textual readout
    print(f"[layer {layer}] PCA EVR(PC1..3)≈ {np.round(evr3,4)} | margins: mean={pm.mean():.4f} med={np.median(pm):.4f} min={pm.min():.4f} max={pm.max():.4f}")
    top = order[:5].tolist()
    print("[top Paris-margin prompts]", [(i, float(pm[i]), PROMPTS[i][:40]) for i in top])

# ---------------------------
# Run all requested layers
# ---------------------------
print(f"[info] prompts B={B} | seq_len T={T} | late_W={LATE_W} | layers={LAYERS}")
print("[note] This cell performs FD evaluations and may take a bit; tqdm shows progress per layer.")

results = {}
for L in LAYERS:
    res = layer_control_field(int(L))
    results[int(L)] = res
    plot_layer_field(res)

# ---------------------------
# Optional: compact comparison summary
# ---------------------------
print("\n=== Tangent field summary (per layer) ===")
for L in LAYERS:
    res = results[int(L)]
    # mean resultant length of tangent directions (proxy for "field coherence")
    tang = res["tangent_dir"]
    v = tang.mean(axis=0)
    coh = float(np.linalg.norm(v))
    print(f"L{L:2d} | PCA EVR={np.round(res['evr3'],3)} | tangent_coherence={coh:.3f}")

In [ ]:

# ================================================================
#@title Next Cell (FULL REPLACEMENT): Prompt-Conditional Tangent Control Field on Simplex Sphere (Late-Window Centroid)
# For layers [0,5,13]:
#   - Extract router read_logits trajectories (head-mean) per prompt
#   - Fit PCA->3D on late-window points, project to S^2
#   - Compute **PROMPT-CONDITIONAL** FD gradient of each prompt's Paris-margin
#       wrt a 3D PCA coordinate at each late-window point
#       (override routing ONLY for that prompt b at t_last[b], not the whole batch)
#   - Average per-prompt gradients across late-window points
#   - Project gradient to tangent plane at each prompt's late-window centroid on the sphere
#   - Plot: trajectories + centroid points (colored by Paris-margin) + tangent arrows (control direction)
#   - PLUS: alignment diagnostics (control tangent vs late velocity), and centroid-distance vs margin scatter
#
# Robustness:
#   - Enforces RIGHT padding + right padding side (important to avoid NaNs)
#   - Sanitizes non-finite values in PCA fit
#   - Uses FD (finite difference), no autograd through infos required
#   - Includes tqdm progress bars
#
# Assumptions:
#   - model, cfg, DEVICE exist
#   - model(..., return_info=True, routing_mode="softmax") -> (logits, infos)
#   - infos[layer]["read_logits"] exists, shape [B,H,T,K]
#   - model.blocks[li].asa.routing_override exists and is consulted by ASA
# ================================================================
import math
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from tqdm.auto import tqdm

# ---------------------------
# SETTINGS
# ---------------------------
LAYERS = [0, 5, 13]
LATE_W = 5                    # late-window size (last W valid tokens)
FD_EPS = 1e-2                 # finite difference step in PCA coords
FD_MODE = "central"           # "central" or "forward"
PLOT_STRIDE = 1               # downsample trajectory points
ARROW_SCALE = 0.22            # arrow length scaling on sphere
ARROW_ALPHA = 0.9
TRAJ_ALPHA = 0.45
TRAJ_LW = 1.8
SPHERE_ALPHA = 0.06
SPHERE_WIREFRAME = True
LABEL_MAX = 10                # label top prompts by Paris-margin
SKIP_T0_IN_PCA = True         # skip token index 0 (often special / unstable)
SANITIZE_MODE = "drop"        # "drop" or "clamp" for non-finite read_logits in PCA fit
CLAMP_VAL = 5.0               # used if SANITIZE_MODE="clamp"
MIN_PCA_POINTS = 12           # minimal good points for PCA

# Alignment diagnostics
DO_ALIGNMENT_PANELS = True
VEL_EPS = 1e-9

# prompts
PROMPTS = [
    "The capital of France is",
    "France's capital city is",
    "In France, the capital is",
    "The seat of government of France is",
    "The city that is France's capital is",
    "The Eiffel Tower is located in",
    "The Louvre is in",
    "The Seine river flows through",
    "During the French Revolution, the city of",
    "Napoleon crowned himself in",
    "In World War II, the liberation of",
    "The largest city in France is",
    "A major European city in France is",
    "A famous city on the Seine is",
    "The capital of Germany is",
    "The capital of England is",
    "The capital of Italy is",
]

RIVAL_TEXT = " the"  # single-token rival for Paris margin

# ---------------------------
# Tokenizer helpers (RIGHT padding enforced)
# ---------------------------
def _get_tokenizer_from_context():
    if "tokenizer" in globals():
        tok = globals()["tokenizer"]
        tok.padding_side = "right"
        if tok.pad_token is None:
            tok.pad_token = tok.eos_token
        return tok
    from transformers import AutoTokenizer
    tok = AutoTokenizer.from_pretrained(getattr(cfg, "tokenizer_name", "gpt2"))
    tok.padding_side = "right"
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok

tokenizer = _get_tokenizer_from_context()

def ensure_single_token(text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) != 1:
        raise ValueError(f"{text!r} must be exactly 1 token. Got ids={ids}")
    return ids[0]

PARIS_ID = ensure_single_token(" Paris")
RIVAL_ID = ensure_single_token(RIVAL_TEXT)

# ---------------------------
# Forward with info
# ---------------------------
def forward_with_info(ids, mask, routing_mode="softmax"):
    logits, infos = model(ids, attention_mask=mask, return_info=True, routing_mode=routing_mode)
    return logits, infos

# ---------------------------
# Batch tokenize (RIGHT padding)
# ---------------------------
enc = tokenizer(
    PROMPTS,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=cfg.max_seq_len,
)
input_ids = enc["input_ids"].to(DEVICE)
attention_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

B, T = input_ids.shape
lengths = attention_mask.sum(dim=1).long()
t_last = (lengths - 1).clamp(min=0)

def valid_ts_for_b(b: int):
    ts = [t for t in range(T) if int(attention_mask[b, t].item()) == 1]
    if SKIP_T0_IN_PCA:
        ts = [t for t in ts if t != 0]
    return ts

def late_window_indices_for_b(b: int):
    last = int(t_last[b].item())
    t0 = max(0, last - (LATE_W - 1))
    idxs = list(range(t0, last + 1))
    if SKIP_T0_IN_PCA:
        idxs = [t for t in idxs if t != 0]
    return idxs

late_idxs = [late_window_indices_for_b(b) for b in range(B)]
valid_idxs = [valid_ts_for_b(b) for b in range(B)]

# ---------------------------
# Baseline logits (for Paris margin, per prompt)
# ---------------------------
with torch.no_grad():
    logits_base, _infos_base = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
    logp = F.log_softmax(logits_base, dim=-1)  # [B,T,V]
    idx = torch.arange(B, device=DEVICE)
    lp_paris = logp[idx, t_last, PARIS_ID].detach().cpu().numpy()
    lp_rival = logp[idx, t_last, RIVAL_ID].detach().cpu().numpy()
    paris_margin = lp_paris - lp_rival
baseline_mean_margin = float(np.mean(paris_margin))

# ---------------------------
# PCA fit utilities (SVD with robust fallback)
# ---------------------------
def _finite_mask_lastdim(x: torch.Tensor) -> torch.Tensor:
    return torch.isfinite(x).all(dim=-1)

def fit_pca3_from_points(X_points: torch.Tensor):
    """
    X_points: [N,D] float32
    returns: mu [D], W3 [D,3], EVR [3], evals [3], n_good
    """
    if SANITIZE_MODE == "clamp":
        X_points = torch.nan_to_num(X_points, nan=0.0, posinf=CLAMP_VAL, neginf=-CLAMP_VAL)
        good = _finite_mask_lastdim(X_points)
    else:
        good = _finite_mask_lastdim(X_points)
        X_points = X_points[good]

    n_good = int(X_points.shape[0])
    if n_good < MIN_PCA_POINTS:
        raise RuntimeError(f"Too few finite points to fit PCA (good={n_good}).")

    mu = X_points.mean(dim=0, keepdim=True)
    Xc = X_points - mu

    # try GPU SVD then fallback to CPU
    try:
        U, S, Vh = torch.linalg.svd(Xc, full_matrices=False)
        Vh = Vh.to(Xc.device)
        S = S.to(Xc.device)
    except Exception:
        Xc_cpu = Xc.detach().cpu()
        U, S, Vh = torch.linalg.svd(Xc_cpu, full_matrices=False)
        Vh = Vh.to(Xc.device)
        S = S.to(Xc.device)

    W3 = Vh[:3].T.contiguous()  # [D,3]
    s2 = (S ** 2)
    evr3 = (s2[:3] / s2.sum().clamp_min(1e-12)).detach().cpu().numpy()
    evals3 = s2[:3].detach().cpu().numpy()
    return mu.squeeze(0), W3, evr3, evals3, n_good

def project_to_sphere_from_Z(Z_points: torch.Tensor, mu: torch.Tensor, W3: torch.Tensor, eps=1e-9):
    """
    Z_points: [N,D] on device
    returns: X3_np [N,3] raw, X3s_np [N,3] sphere
    """
    X3 = (Z_points - mu[None, :]) @ W3  # [N,3]
    X3_np = X3.detach().cpu().numpy()
    n = np.linalg.norm(X3_np, axis=1, keepdims=True)
    X3s_np = X3_np / np.clip(n, eps, None)
    return X3_np, X3s_np

# ---------------------------
# Routing override machinery (PROMPT-CONDITIONAL forcing)
# ---------------------------
def clear_routing_override():
    for li in range(len(model.blocks)):
        model.blocks[li].asa.routing_override = None

def set_override_for_layer_prompt_only(layer_idx: int, b_force: int, t_force: int, r_hk: torch.Tensor):
    """
    Force routing ONLY for example b_force at token position t_force, for this layer.
    r_hk: [H,K] distribution per head (typically same across heads)
    """
    clear_routing_override()

    def _override(t0, t1, read_logits_chunk, read_logits_key, read_logits_content, ctx):
        # read_logits_chunk: [B,H,L,K]
        Bc, Hc, Lc, Kc = read_logits_chunk.shape
        rtemp = float(ctx.get("rtemp", 1.0))
        w = torch.softmax(read_logits_chunk / max(1e-6, rtemp), dim=-1)

        # override only if this chunk contains t_force
        if t0 <= t_force < t1:
            lp = t_force - t0
            w[b_force, :, lp, :] = r_hk  # [H,K]

        w = w / w.sum(dim=-1, keepdim=True).clamp_min(1e-8)
        return w

    model.blocks[layer_idx].asa.routing_override = _override

@torch.no_grad()
def paris_margin_for_prompt_under_forced_routing(layer_idx: int, b_force: int, r_k: torch.Tensor):
    """
    Force routing for prompt b_force at t_last[b_force] in given layer, return that prompt's Paris-margin (scalar).
    r_k: [K] distribution; broadcast to [H,K]
    """
    t_force = int(t_last[b_force].item())

    # build [H,K] for heads
    # (head-specific not needed; copy across heads)
    with torch.no_grad():
        # Determine H from model/instrumentation by running once? We'll pass H from enclosing scope where needed.
        pass

# We'll fill the above with H once we know H from read_logits per layer.

# ---------------------------
# Main per-layer computation
# ---------------------------
def layer_control_field_prompt_conditional(layer_idx: int):
    """
    Returns dict with trajectories on sphere, per-prompt centroids, tangent arrows,
    plus alignment diagnostics and centroid-distance stats.
    """
    # Forward to collect read_logits for this layer
    with torch.no_grad():
        _logits, infos = forward_with_info(input_ids, attention_mask, routing_mode="softmax")

    rl = infos[layer_idx].get("read_logits", None)
    if rl is None:
        raise KeyError(f"infos[{layer_idx}] missing 'read_logits'. Keys: {list(infos[layer_idx].keys())}")

    # rl: [B,H,T,K] -> Z: [B,T,K] (head mean)
    H = rl.shape[1]
    Z = rl.mean(dim=1).float()  # [B,T,K]
    K = Z.shape[-1]

    # Collect late-window points for PCA fit: points are in D=K space
    pts = []
    owner = []  # list of (b,t) for each point
    for b in range(B):
        for t in late_idxs[b]:
            pts.append(Z[b, t, :])
            owner.append((b, t))
    Zfit = torch.stack(pts, dim=0)  # [N,K]

    # Fit PCA3
    mu, W3, evr3, evals3, n_good = fit_pca3_from_points(Zfit)

    # Precompute per-prompt trajectories projected to sphere (for display)
    traj_sphere = []
    traj_raw = []
    for b in range(B):
        ts = valid_idxs[b]
        if len(ts) == 0:
            traj_sphere.append(np.zeros((0, 3)))
            traj_raw.append(np.zeros((0, 3)))
            continue
        Zb = Z[b, ts, :]  # [Tv,K]
        X3_np, X3s_np = project_to_sphere_from_Z(Zb, mu, W3)
        traj_raw.append(X3_np[::PLOT_STRIDE])
        traj_sphere.append(X3s_np[::PLOT_STRIDE])

    # Late-window centroids on sphere (mean of late sphere points then renormalize)
    centroids = np.zeros((B, 3), dtype=np.float32)
    late_points_s = [[] for _ in range(B)]
    for (b, t) in owner:
        z = Z[b, t, :][None, :]
        _, x_s = project_to_sphere_from_Z(z, mu, W3)
        late_points_s[b].append(x_s[0])
    for b in range(B):
        arr = np.array(late_points_s[b], dtype=np.float32)
        if arr.size == 0:
            centroids[b] = np.array([1.0, 0.0, 0.0], dtype=np.float32)
        else:
            c = arr.mean(axis=0)
            c = c / max(1e-9, float(np.linalg.norm(c)))
            centroids[b] = c

    # Late-window velocity directions (on sphere): average normalized step directions in late window
    vel_dir = np.zeros((B, 3), dtype=np.float32)
    for b in range(B):
        ts = late_idxs[b]
        if len(ts) < 2:
            continue
        # use the projected sphere points at those times
        pts_s = []
        for t in ts:
            z = Z[b, t, :][None, :]
            _, x_s = project_to_sphere_from_Z(z, mu, W3)
            pts_s.append(x_s[0])
        pts_s = np.array(pts_s, dtype=np.float32)
        steps = pts_s[1:] - pts_s[:-1]
        n = np.linalg.norm(steps, axis=1, keepdims=True)
        steps_u = steps / np.clip(n, VEL_EPS, None)
        v = steps_u.mean(axis=0)
        nv = float(np.linalg.norm(v))
        if nv > 1e-9:
            vel_dir[b] = v / nv

    # Helper: inverse map PCA3 -> K-space logits -> routing dist
    def pca3_to_rk(x3: torch.Tensor):
        # x3: [3] -> z_k: [K] -> softmax
        z_k = mu.to(x3.device) + (x3 @ W3.T.to(x3.device))  # [K]
        return torch.softmax(z_k, dim=-1)  # [K]

    # PROMPT-CONDITIONAL FD grads:
    # For each late point belonging to prompt b, we:
    #   - create routing dist r_k from that point's PCA3 coordinate
    #   - override only prompt b at t_last[b] with r (broadcast across heads)
    #   - evaluate that prompt b's Paris-margin only (not mean over batch)
    #
    # We estimate grad wrt PCA3 coords at that point, then average per prompt b.
    N = len(owner)
    X3_points = []
    with torch.no_grad():
        for (b, t) in owner:
            x3 = (Z[b, t, :] - mu.to(Z.device)) @ W3.to(Z.device)  # [3]
            X3_points.append(x3)
    X3_points = torch.stack(X3_points, dim=0).to(DEVICE)  # [N,3]

    @torch.no_grad()
    def eval_prompt_margin_at_x3(layer_idx: int, b_force: int, x3: torch.Tensor):
        r_k = pca3_to_rk(x3)  # [K]
        r_hk = r_k.view(1, K).expand(H, K).contiguous()  # [H,K]

        t_force = int(t_last[b_force].item())
        set_override_for_layer_prompt_only(layer_idx, b_force=b_force, t_force=t_force, r_hk=r_hk)

        logits, _infos = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
        clear_routing_override()

        logp_loc = F.log_softmax(logits, dim=-1)
        tt = int(t_last[b_force].item())
        lpP = float(logp_loc[b_force, tt, PARIS_ID].item())
        lpR = float(logp_loc[b_force, tt, RIVAL_ID].item())
        return lpP - lpR

    grads = torch.zeros((N, 3), device=DEVICE, dtype=torch.float32)

    # tqdm over points; inner loop over 3 dims with 2 evals for central
    pbar = tqdm(range(N), desc=f"FD prompt-conditional grads | layer {layer_idx}", leave=False)
    for i in pbar:
        b_force, _t_src = owner[i]
        x = X3_points[i]

        if FD_MODE == "central":
            for d in range(3):
                e = torch.zeros_like(x)
                e[d] = FD_EPS
                m_plus = eval_prompt_margin_at_x3(layer_idx, b_force, x + e)
                m_minus = eval_prompt_margin_at_x3(layer_idx, b_force, x - e)
                grads[i, d] = (m_plus - m_minus) / (2.0 * FD_EPS)
        else:
            m0 = eval_prompt_margin_at_x3(layer_idx, b_force, x)
            for d in range(3):
                e = torch.zeros_like(x)
                e[d] = FD_EPS
                m_plus = eval_prompt_margin_at_x3(layer_idx, b_force, x + e)
                grads[i, d] = (m_plus - m0) / FD_EPS

    # Average per prompt across its late points
    g_prompt = torch.zeros((B, 3), device=DEVICE, dtype=torch.float32)
    counts = torch.zeros((B,), device=DEVICE, dtype=torch.float32)
    for i, (b, t) in enumerate(owner):
        g_prompt[b] += grads[i]
        counts[b] += 1.0
    g_prompt = g_prompt / counts.clamp_min(1.0).unsqueeze(-1)
    g_np = g_prompt.detach().cpu().numpy()

    # Project to tangent plane at centroid and normalize for plotting
    tangent_dir = np.zeros((B, 3), dtype=np.float32)
    tangent_mag = np.zeros((B,), dtype=np.float32)
    for b in range(B):
        x = centroids[b]
        g = g_np[b]
        g_tan = g - float(np.dot(g, x)) * x
        mag = float(np.linalg.norm(g_tan))
        tangent_mag[b] = mag
        if mag > 1e-9:
            tangent_dir[b] = (g_tan / mag).astype(np.float32)

    # Tangent coherence: norm of mean unit tangent (skip near-zero)
    use = np.linalg.norm(tangent_dir, axis=1) > 0
    if use.any():
        mean_vec = tangent_dir[use].mean(axis=0)
        tangent_coh = float(np.linalg.norm(mean_vec))
    else:
        tangent_coh = float("nan")

    # Alignment cosines: tangent_dir vs velocity_dir (both unit) (skip invalid)
    align = np.full((B,), np.nan, dtype=np.float32)
    for b in range(B):
        if np.linalg.norm(tangent_dir[b]) > 0 and np.linalg.norm(vel_dir[b]) > 0:
            align[b] = float(np.dot(tangent_dir[b], vel_dir[b]))

    # "Paris-attractor centroid": centroid of top-k prompts by Paris margin in this layer's sphere coords
    order = np.argsort(-paris_margin)
    topk = min(5, B)
    attract = centroids[order[:topk]].mean(axis=0)
    attract = attract / max(1e-9, float(np.linalg.norm(attract)))

    # Distances to attractor (geodesic-ish via dot; also euclidean on sphere is fine)
    dot = np.clip((centroids * attract[None, :]).sum(axis=1), -1.0, 1.0)
    ang = np.arccos(dot)  # [0,pi]
    dist_to_attractor = ang.astype(np.float32)

    return {
        "layer": int(layer_idx),
        "traj_sphere": traj_sphere,
        "centroids": centroids,
        "tangent_dir": tangent_dir,
        "tangent_mag": tangent_mag,
        "vel_dir": vel_dir,
        "align": align,
        "paris_margin": paris_margin.copy(),
        "evr3": evr3,
        "evals3": evals3,
        "n_fit": int(n_good),
        "attractor": attract,
        "dist_to_attractor": dist_to_attractor,
        "tangent_coherence": tangent_coh,
    }

# ---------------------------
# Plotting helpers
# ---------------------------
def plot_layer_field(res):
    layer = res["layer"]
    traj_s = res["traj_sphere"]
    cent = res["centroids"]
    tang = res["tangent_dir"]
    pm = res["paris_margin"]
    evr3 = res["evr3"]
    align = res["align"]
    dist = res["dist_to_attractor"]
    attract = res["attractor"]

    import matplotlib as mpl
    normc = mpl.colors.Normalize(vmin=float(np.min(pm)), vmax=float(np.max(pm)))
    cmap = plt.cm.viridis
    colors = cmap(normc(pm))

    # --- Main globe plot ---
    fig = plt.figure(figsize=(11.0, 8.8))
    ax = fig.add_subplot(111, projection="3d")

    # faint sphere
    u = np.linspace(0, 2*np.pi, 48)
    v = np.linspace(0, np.pi, 24)
    xs = np.outer(np.cos(u), np.sin(v))
    ys = np.outer(np.sin(u), np.sin(v))
    zs = np.outer(np.ones_like(u), np.cos(v))
    if SPHERE_WIREFRAME:
        ax.plot_wireframe(xs, ys, zs, rstride=3, cstride=3, linewidth=0.5, alpha=SPHERE_ALPHA)
    else:
        ax.plot_surface(xs, ys, zs, alpha=SPHERE_ALPHA, linewidth=0)

    # trajectories
    for b in range(B):
        pts = traj_s[b]
        if pts.shape[0] >= 2:
            ax.plot(pts[:, 0], pts[:, 1], pts[:, 2], linewidth=TRAJ_LW, alpha=TRAJ_ALPHA)

    # centroid points
    ax.scatter(cent[:, 0], cent[:, 1], cent[:, 2], c=colors, s=70, alpha=0.95)

    # attractor marker
    ax.scatter([attract[0]], [attract[1]], [attract[2]], s=220, marker="X", alpha=0.95)

    # tangent arrows (unit direction, constant length)
    for b in range(B):
        x = cent[b]
        d = tang[b]
        if float(np.linalg.norm(d)) < 1e-9:
            continue
        ax.quiver(
            x[0], x[1], x[2],
            d[0], d[1], d[2],
            length=ARROW_SCALE,
            normalize=True,
            alpha=ARROW_ALPHA,
            linewidth=1.4,
        )

    # label top prompts by Paris-margin
    order = np.argsort(-pm)
    for i in order[: min(LABEL_MAX, B)]:
        x = cent[i]
        ax.text(x[0], x[1], x[2], f" {i}", fontsize=10)

    # colorbar for margin
    sm = mpl.cm.ScalarMappable(cmap=cmap, norm=normc)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.08)
    cbar.set_label(f"logP(' Paris') - logP({RIVAL_TEXT!r})")

    ax.set_title(
        f"Prompt-Conditional Tangent Control Field (late-window centroid)\n"
        f"layer={layer} | late_W={LATE_W} | FD_EPS={FD_EPS} ({FD_MODE}) | PCA EVR(PC1..3)≈{np.round(evr3,4)}"
    )
    ax.set_xlabel("PC1 (sphere)")
    ax.set_ylabel("PC2 (sphere)")
    ax.set_zlabel("PC3 (sphere)")
    ax.set_box_aspect([1, 1, 1])
    plt.tight_layout()
    plt.show()

    # --- Diagnostics panels ---
    if DO_ALIGNMENT_PANELS:
        fig = plt.figure(figsize=(14.5, 4.6))

        # 1) alignment histogram
        ax1 = fig.add_subplot(1, 3, 1)
        a = align[np.isfinite(align)]
        if a.size > 0:
            ax1.hist(a, bins=12)
            ax1.axvline(float(np.mean(a)), linestyle="--")
            ax1.set_title(f"cos(tangent, late-velocity)\nmean={float(np.mean(a)):.3f}  n={a.size}")
        else:
            ax1.text(0.5, 0.5, "no valid alignments", ha="center", va="center")
            ax1.set_title("cos(tangent, late-velocity)")
        ax1.set_xlabel("cosine")
        ax1.set_ylabel("count")

        # 2) dist-to-attractor vs margin
        ax2 = fig.add_subplot(1, 3, 2)
        ax2.scatter(dist, pm, s=60)
        ax2.set_title("distance to attractor vs Paris-margin")
        ax2.set_xlabel("angle distance (rad)")
        ax2.set_ylabel("Paris-margin")
        # correlation (ignore NaNs)
        if np.isfinite(dist).all():
            corr = float(np.corrcoef(dist, pm)[0, 1])
            ax2.text(0.02, 0.98, f"corr={corr:+.3f}", transform=ax2.transAxes, va="top")

        # 3) 2D centroid map (PC1/PC2 in sphere coords)
        ax3 = fig.add_subplot(1, 3, 3)
        ax3.scatter(cent[:, 0], cent[:, 1], c=pm, s=70)
        ax3.scatter([attract[0]], [attract[1]], s=180, marker="X")
        for i in order[: min(LABEL_MAX, B)]:
            ax3.text(cent[i, 0], cent[i, 1], f" {i}", fontsize=9)
        ax3.set_title("centroids on sphere (PC1,PC2)")
        ax3.set_xlabel("PC1(sphere)")
        ax3.set_ylabel("PC2(sphere)")

        plt.tight_layout()
        plt.show()

    # quick textual readout
    print(f"[layer {layer}] PCA EVR(PC1..3)≈ {np.round(evr3,4)} | PCA_fit_good={res['n_fit']} points")
    print(f"  margins: mean={pm.mean():.4f} med={np.median(pm):.4f} min={pm.min():.4f} max={pm.max():.4f}")
    top = order[:5].tolist()
    print("  [top Paris-margin prompts]", [(i, float(pm[i]), PROMPTS[i][:50]) for i in top])
    print(f"  tangent_coherence={res['tangent_coherence']:.3f}")

# ---------------------------
# Run all requested layers
# ---------------------------
print(f"[info] detected layers={len(model.blocks)} | B={B} T={T} | late_W={LATE_W} | layers={LAYERS}")
print(f"[baseline] mean Paris-margin (no forcing) = {baseline_mean_margin:.4f}")
print("[note] This cell runs prompt-conditional FD and can take time; tqdm shows progress per layer.")

results = {}
for L in LAYERS:
    res = layer_control_field_prompt_conditional(int(L))
    results[int(L)] = res
    plot_layer_field(res)

# ---------------------------
# Compact summary across layers
# ---------------------------
print("\n=== Summary across layers ===")
for L in LAYERS:
    res = results[int(L)]
    pm = res["paris_margin"]
    dist = res["dist_to_attractor"]
    a = res["align"]
    a_mean = float(np.nanmean(a)) if np.isfinite(a).any() else float("nan")
    corr = float(np.corrcoef(dist, pm)[0, 1]) if np.isfinite(dist).all() else float("nan")
    print(
        f"L{L:2d} | EVR={np.round(res['evr3'],3)} | coh={res['tangent_coherence']:.3f} | "
        f"align_mean={a_mean:+.3f} | corr(dist,margin)={corr:+.3f}"
    )

In [ ]:

# ================================================================
#@title Experiment: Residual-vs-Routing Intervention Parity Test (FD + PCA, late-window centroid)
# Tests competing hypothesis:
#   Facts retrieved in residual stream; routing gates expression.
#
# For a chosen layer:
#   (A) ROUTING intervention: move in PCA(read_logits) space -> inverse -> softmax -> force routing at t_last
#   (B) RESIDUAL intervention: move in PCA(residual) space -> inverse -> add delta to hidden state at t_last
#
# Compare efficiency: Δ(Paris-margin) per unit ε.
#
# Assumptions:
#   - model, cfg, DEVICE exist
#   - model(input_ids, attention_mask=..., return_info=True, routing_mode="softmax") -> (logits, infos)
#   - infos[layer]["read_logits"] exists with shape [B,H,T,K]
#   - model.blocks[layer] returns hidden states shaped [B,T,D] (or compatible)
# ================================================================
import math
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

# ---------------------------
# SETTINGS
# ---------------------------
LAYER_TEST = 13           # try 13 first; then 0 and 5
LATE_W = 5                # late-window size
SKIP_T0 = True            # skip token 0 in PCA fits
FD_EPS = 1e-2             # FD step in PCA coords for direction estimation
FD_MODE = "central"       # "central" or "forward"
FORCE_ROUTING_RTEMP = 1.0 # router softmax temp inside override
EPS_SWEEP = [0.0, 0.01, 0.02, 0.04, 0.08]  # intervention magnitudes (in PCA coords)
N_SLOPE_FIT = 3           # use first N eps points to estimate slope near 0

# Prompts (use your current set)
PROMPTS = [
    "The capital of France is",
    "France's capital city is",
    "In France, the capital is",
    "The seat of government of France is",
    "The city that is France's capital is",
    "The Eiffel Tower is located in",
    "The Louvre is in",
    "The Seine river flows through",
    "During the French Revolution, the city of",
    "Napoleon crowned himself in",
    "In World War II, the liberation of",
    "The largest city in France is",
    "A major European city in France is",
    "A famous city on the Seine is",
    "The capital of Germany is",
    "The capital of England is",
    "The capital of Italy is",
]

RIVAL_TEXT = " the"       # single-token rival for margin

# ---------------------------
# Tokenizer (RIGHT padding enforced)
# ---------------------------
def _get_tokenizer_from_context():
    if "tokenizer" in globals():
        tok = globals()["tokenizer"]
        tok.padding_side = "right"
        if tok.pad_token is None:
            tok.pad_token = tok.eos_token
        return tok
    from transformers import AutoTokenizer
    tok = AutoTokenizer.from_pretrained(getattr(cfg, "tokenizer_name", "gpt2"))
    tok.padding_side = "right"
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok

tokenizer = _get_tokenizer_from_context()

def ensure_single_token(text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) != 1:
        raise ValueError(f"{text!r} must be exactly 1 token. Got ids={ids}")
    return ids[0]

PARIS_ID = ensure_single_token(" Paris")
RIVAL_ID = ensure_single_token(RIVAL_TEXT)

# ---------------------------
# Forward with info
# ---------------------------
def forward_with_info(ids, mask, routing_mode="softmax"):
    logits, infos = model(ids, attention_mask=mask, return_info=True, routing_mode=routing_mode)
    return logits, infos

# ---------------------------
# Batch tokenize
# ---------------------------
enc = tokenizer(
    PROMPTS,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=cfg.max_seq_len,
)
input_ids = enc["input_ids"].to(DEVICE)
attention_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

B, T = input_ids.shape
lengths = attention_mask.sum(dim=1).long()
t_last = (lengths - 1).clamp(min=0)

def late_window_indices(b: int):
    last = int(t_last[b].item())
    t0 = max(0, last - (LATE_W - 1))
    idxs = list(range(t0, last + 1))
    if SKIP_T0:
        idxs = [t for t in idxs if t != 0]
    return idxs

late_idxs = [late_window_indices(b) for b in range(B)]

# ---------------------------
# Baseline Paris margin
# ---------------------------
@torch.no_grad()
def paris_margin_from_logits(logits):
    logp = F.log_softmax(logits, dim=-1)
    idx = torch.arange(B, device=logits.device)
    lpP = logp[idx, t_last, PARIS_ID]
    lpR = logp[idx, t_last, RIVAL_ID]
    return (lpP - lpR)  # [B]

with torch.no_grad():
    logits_base, infos_base = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
    margin_base_b = paris_margin_from_logits(logits_base).detach().cpu().numpy()
    margin_base = float(margin_base_b.mean())

print(f"[info] LAYER_TEST={LAYER_TEST} | B={B} T={T} | baseline mean margin={margin_base:.4f}")

# ================================================================
# 1) PCA utilities (robust, CPU fallback)
# ================================================================
def fit_pca3(X: torch.Tensor):
    """
    X: [N,D] float32
    returns: mu [D], W3 [D,3], EVR [3]
    """
    X = X.float()
    good = torch.isfinite(X).all(dim=-1)
    Xg = X[good]
    if Xg.shape[0] < 10:
        raise RuntimeError(f"Too few finite PCA points: {int(Xg.shape[0])}")
    mu = Xg.mean(dim=0, keepdim=True)
    Xc = Xg - mu
    try:
        U, S, Vh = torch.linalg.svd(Xc, full_matrices=False)
    except Exception:
        Xc_cpu = Xc.detach().cpu()
        U, S, Vh = torch.linalg.svd(Xc_cpu, full_matrices=False)
        Vh = Vh.to(Xc.device)
        S = S.to(Xc.device)
    W3 = Vh[:3].T.contiguous()
    s2 = S**2
    evr3 = (s2[:3] / s2.sum().clamp_min(1e-12)).detach().cpu().numpy()
    return mu.squeeze(0), W3, evr3

def pca_encode(X: torch.Tensor, mu: torch.Tensor, W3: torch.Tensor):
    # X: [N,D] -> [N,3]
    return (X - mu[None, :]) @ W3

def pca_decode(x3: torch.Tensor, mu: torch.Tensor, W3: torch.Tensor):
    # x3: [N,3] -> [N,D]
    return mu[None, :] + (x3 @ W3.T)

# ================================================================
# 2) ROUTING intervention machinery (force read_weights at t_last)
# ================================================================
def clear_routing_override():
    for li in range(len(model.blocks)):
        model.blocks[li].asa.routing_override = None

def set_override_for_layer(layer_idx: int, r_bhk: torch.Tensor, t_targets_cpu: np.ndarray):
    clear_routing_override()

    def _override(t0, t1, read_logits_chunk, read_logits_key, read_logits_content, ctx):
        # read_logits_chunk: [B,H,L,K]
        Bc, Hc, Lc, Kc = read_logits_chunk.shape
        rtemp = float(ctx.get("rtemp", FORCE_ROUTING_RTEMP))
        w = torch.softmax(read_logits_chunk / max(1e-6, rtemp), dim=-1)
        for b in range(Bc):
            tt = int(t_targets_cpu[b])
            if t0 <= tt < t1:
                lp = tt - t0
                w[b, :, lp, :] = r_bhk[b]
        return w / w.sum(dim=-1, keepdim=True).clamp_min(1e-8)

    model.blocks[layer_idx].asa.routing_override = _override

@torch.no_grad()
def run_with_forced_routing(layer_idx: int, r_bhk: torch.Tensor):
    t_targets_cpu = t_last.detach().cpu().numpy()
    set_override_for_layer(layer_idx, r_bhk, t_targets_cpu)
    logits, _ = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
    clear_routing_override()
    mb = paris_margin_from_logits(logits).detach().cpu().numpy()
    return float(mb.mean())

# ================================================================
# 3) RESIDUAL intervention machinery (forward hook inject at t_last)
# ================================================================
class ResidualInjector:
    def __init__(self, layer_idx: int):
        self.layer_idx = layer_idx
        self.delta_btD = None  # [B,T,D] sparse-ish (only t_last filled)
        self.hook = None

    def install(self):
        # Hook the block output: assumed [B,T,D]
        def _hook(module, inp, out):
            if self.delta_btD is None:
                return out
            # out could be tuple; handle common patterns
            if isinstance(out, tuple):
                h = out[0]
                rest = out[1:]
                h2 = h + self.delta_btD.to(h.device, dtype=h.dtype)
                return (h2,) + rest
            else:
                return out + self.delta_btD.to(out.device, dtype=out.dtype)

        self.hook = model.blocks[self.layer_idx].register_forward_hook(_hook)

    def uninstall(self):
        if self.hook is not None:
            self.hook.remove()
            self.hook = None
        self.delta_btD = None

@torch.no_grad()
def capture_layer_hidden(layer_idx: int):
    """
    Returns hidden states at output of model.blocks[layer_idx]: [B,T,D] on DEVICE (float32).
    """
    cache = {}
    def _cap_hook(module, inp, out):
        if isinstance(out, tuple):
            cache["h"] = out[0].detach()
        else:
            cache["h"] = out.detach()

    hhook = model.blocks[layer_idx].register_forward_hook(_cap_hook)
    logits, _ = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
    hhook.remove()
    if "h" not in cache:
        raise RuntimeError("Failed to capture hidden state from block hook.")
    return cache["h"].float()

@torch.no_grad()
def run_with_residual_injection(layer_idx: int, delta_bD: torch.Tensor):
    """
    delta_bD: [B,D] added at each example's t_last position at output of block layer_idx.
    """
    h0 = capture_layer_hidden(layer_idx)  # just to get D safely if needed
    D = h0.shape[-1]

    inj = ResidualInjector(layer_idx)
    inj.install()

    delta_btD = torch.zeros((B, T, D), device=DEVICE, dtype=torch.float32)
    for b in range(B):
        tt = int(t_last[b].item())
        delta_btD[b, tt, :] = delta_bD[b].to(DEVICE, dtype=torch.float32)
    inj.delta_btD = delta_btD

    logits, _ = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
    inj.uninstall()

    mb = paris_margin_from_logits(logits).detach().cpu().numpy()
    return float(mb.mean())

# ================================================================
# 4) Build PCA bases for routing and residual, using late-window points
# ================================================================
# ---- Routing read_logits points (K-dim) ----
with torch.no_grad():
    info = infos_base[LAYER_TEST]
    rl = info.get("read_logits", None)
    if rl is None:
        raise KeyError(f"infos[{LAYER_TEST}] missing read_logits.")
    # [B,H,T,K] -> [B,T,K]
    Z = rl.mean(dim=1).float()
    K = Z.shape[-1]
    H = rl.shape[1]

Z_pts = []
for b in range(B):
    for t in late_idxs[b]:
        Z_pts.append(Z[b, t, :])
Z_pts = torch.stack(Z_pts, dim=0).to(DEVICE)  # [N,K]

mu_r, W3_r, evr_r = fit_pca3(Z_pts)
print(f"[routing PCA] K={K} | EVR(PC1..3)≈{np.round(evr_r,4)} | points={Z_pts.shape[0]}")

# ---- Residual hidden points (D-dim) ----
h_layer = capture_layer_hidden(LAYER_TEST)  # [B,T,D]
D = h_layer.shape[-1]

H_pts = []
for b in range(B):
    for t in late_idxs[b]:
        H_pts.append(h_layer[b, t, :])
H_pts = torch.stack(H_pts, dim=0).to(DEVICE)  # [N,D]

mu_h, W3_h, evr_h = fit_pca3(H_pts)
print(f"[residual PCA] D={D} | EVR(PC1..3)≈{np.round(evr_h,4)} | points={H_pts.shape[0]}")

# ================================================================
# 5) Late-window centroid per prompt in each PCA space
# ================================================================
def centroid_pca_per_prompt(points_btD: torch.Tensor, mu: torch.Tensor, W3: torch.Tensor):
    """
    points_btD: [B,T,D] (or [B,T,K]) points in native space
    returns centroids: [B,3] in PCA coords (mean over late window)
    """
    cents = torch.zeros((B, 3), device=DEVICE, dtype=torch.float32)
    for b in range(B):
        idxs = late_idxs[b]
        Xb = torch.stack([points_btD[b, t, :] for t in idxs], dim=0)  # [w,D]
        x3 = pca_encode(Xb, mu, W3)                                   # [w,3]
        cents[b] = x3.mean(dim=0)
    return cents

cent_r = centroid_pca_per_prompt(Z, mu_r, W3_r)        # [B,3]
cent_h = centroid_pca_per_prompt(h_layer, mu_h, W3_h)  # [B,3]

# ================================================================
# 6) Estimate a GLOBAL control direction via FD at centroids
#    - Routing: direction in PCA(read_logits) coords
#    - Residual: direction in PCA(hidden) coords
# ================================================================
def routing_from_x3(x3: torch.Tensor):
    """
    x3: [B,3] in routing PCA coords -> z_k -> softmax -> r_bhk (same per head)
    """
    z_k = pca_decode(x3, mu_r, W3_r)          # [B,K]
    r_bk = torch.softmax(z_k, dim=-1)         # [B,K]
    r_bhk = r_bk[:, None, :].expand(B, H, K).contiguous()
    return r_bhk

def residual_delta_from_x3(x3: torch.Tensor):
    """
    x3: [B,3] in residual PCA coords -> delta_h [B,D]
    """
    dh = pca_decode(x3, mu_h, W3_h) - mu_h[None, :]  # convert coords to displacement around mean
    return dh

def fd_global_dir_routing():
    # Compute grad of mean margin w.r.t. x3 centroid (same direction shared)
    # We'll average per-prompt grads to get a global direction.
    grads = torch.zeros((B, 3), device=DEVICE, dtype=torch.float32)

    base_mean = run_with_forced_routing(LAYER_TEST, routing_from_x3(cent_r))

    for b in tqdm(range(B), desc="FD routing dir (per prompt)", leave=False):
        x = cent_r[b].clone()
        for d in range(3):
            e = torch.zeros_like(x); e[d] = FD_EPS
            if FD_MODE == "central":
                x_plus = cent_r.clone(); x_plus[b] = x + e
                x_minus = cent_r.clone(); x_minus[b] = x - e
                m_plus = run_with_forced_routing(LAYER_TEST, routing_from_x3(x_plus))
                m_minus = run_with_forced_routing(LAYER_TEST, routing_from_x3(x_minus))
                grads[b, d] = (m_plus - m_minus) / (2.0 * FD_EPS)
            else:
                x_plus = cent_r.clone(); x_plus[b] = x + e
                m_plus = run_with_forced_routing(LAYER_TEST, routing_from_x3(x_plus))
                grads[b, d] = (m_plus - base_mean) / FD_EPS

    g = grads.mean(dim=0)
    g = g / g.norm().clamp_min(1e-9)
    return g.detach().cpu().numpy()

def fd_global_dir_residual():
    grads = torch.zeros((B, 3), device=DEVICE, dtype=torch.float32)

    # baseline injection = 0
    base_mean = float(margin_base)

    for b in tqdm(range(B), desc="FD residual dir (per prompt)", leave=False):
        x = cent_h[b].clone()
        for d in range(3):
            e = torch.zeros_like(x); e[d] = FD_EPS
            if FD_MODE == "central":
                x_plus = cent_h.clone(); x_plus[b] = x + e
                x_minus = cent_h.clone(); x_minus[b] = x - e
                dh_plus = residual_delta_from_x3(x_plus - cent_h)   # interpret as displacement from centroid
                dh_minus = residual_delta_from_x3(x_minus - cent_h)
                m_plus = run_with_residual_injection(LAYER_TEST, dh_plus)
                m_minus = run_with_residual_injection(LAYER_TEST, dh_minus)
                grads[b, d] = (m_plus - m_minus) / (2.0 * FD_EPS)
            else:
                x_plus = cent_h.clone(); x_plus[b] = x + e
                dh_plus = residual_delta_from_x3(x_plus - cent_h)
                m_plus = run_with_residual_injection(LAYER_TEST, dh_plus)
                grads[b, d] = (m_plus - base_mean) / FD_EPS

    g = grads.mean(dim=0)
    g = g / g.norm().clamp_min(1e-9)
    return g.detach().cpu().numpy()

print("[step] estimating global routing control direction (FD)...")
g_r = fd_global_dir_routing()
print("  g_r (PCA coords):", np.round(g_r, 4).tolist())

print("[step] estimating global residual control direction (FD)...")
g_h = fd_global_dir_residual()
print("  g_h (PCA coords):", np.round(g_h, 4).tolist())

# ================================================================
# 7) Parity sweep: apply ε * global_dir in each space
#    - Routing: shift all prompt centroids in routing PCA coords by ε*g_r
#    - Residual: inject δh from ε*g_h at each prompt t_last
# ================================================================
def apply_routing_eps(eps: float):
    x3 = cent_r + torch.tensor(eps * g_r, device=DEVICE, dtype=torch.float32)[None, :]
    r_bhk = routing_from_x3(x3)
    return run_with_forced_routing(LAYER_TEST, r_bhk)

def apply_residual_eps(eps: float):
    # per-prompt delta in PCA coords = eps*g_h, decode to D space
    x3 = torch.tensor(eps * g_h, device=DEVICE, dtype=torch.float32)[None, :].expand(B, 3)
    dh = pca_decode(x3, mu_h, W3_h) - mu_h[None, :]  # [B,D]
    return run_with_residual_injection(LAYER_TEST, dh)

routing_means = []
resid_means = []

print("[run] ε sweep (routing vs residual)...")
for eps in tqdm(EPS_SWEEP, desc="parity sweep"):
    mr = apply_routing_eps(float(eps))
    mh = apply_residual_eps(float(eps))
    routing_means.append(mr)
    resid_means.append(mh)

routing_means = np.array(routing_means, dtype=np.float32)
resid_means = np.array(resid_means, dtype=np.float32)

d_r = routing_means - margin_base
d_h = resid_means - margin_base

# slope near 0 (simple least squares on first N points)
def fit_slope(eps_list, delta_list, n=3):
    x = np.array(eps_list[:n], dtype=np.float32)
    y = np.array(delta_list[:n], dtype=np.float32)
    # y = a*x (no intercept) is appropriate if eps=0 => delta=0 numerically
    denom = float((x*x).sum()) + 1e-12
    a = float((x*y).sum() / denom)
    return a

slope_r = fit_slope(EPS_SWEEP, d_r, n=min(N_SLOPE_FIT, len(EPS_SWEEP)))
slope_h = fit_slope(EPS_SWEEP, d_h, n=min(N_SLOPE_FIT, len(EPS_SWEEP)))

# ================================================================
# 8) Plots + readout
# ================================================================
plt.figure(figsize=(8.5, 5.8))
plt.plot(EPS_SWEEP, d_r, marker="o", label="routing override (Δmargin)")
plt.plot(EPS_SWEEP, d_h, marker="o", label="residual injection (Δmargin)")
plt.axhline(0.0, linewidth=1)
plt.xlabel("ε (step in PCA coord direction)")
plt.ylabel("Δ mean Paris-margin")
plt.title(f"Residual vs Routing Parity Test | layer={LAYER_TEST} | baseline={margin_base:.3f}\n"
          f"local slope near 0: routing={slope_r:.3f}  residual={slope_h:.3f}")
plt.grid(True, alpha=0.25)
plt.legend()
plt.tight_layout()
plt.show()

print("\n=== Summary ===")
print(f"layer={LAYER_TEST} | baseline mean margin={margin_base:.4f}")
print(f"routing PCA EVR={np.round(evr_r,4)} | residual PCA EVR={np.round(evr_h,4)}")
print(f"local slope near 0 (Δmargin/ε): routing={slope_r:.4f} | residual={slope_h:.4f}")
print("eps | mean_margin_routing | mean_margin_residual | Δr | Δh")
for eps, mr, mh, dr, dh in zip(EPS_SWEEP, routing_means, resid_means, d_r, d_h):
    print(f"{eps:>4.2f} | {mr:>+8.4f}        | {mh:>+8.4f}         | {dr:>+7.4f} | {dh:>+7.4f}")

In [ ]:

# ================================================================
#@title Experiment 3: Residual controllability with ROUTING FROZEN (decisive A vs B test)
#
# Goal:
#   Test whether residual-state edits can move Paris-margin when routing is held fixed
#   to its baseline behavior.
#
# Method:
#   For each layer L in LAYERS:
#     1) Run baseline forward, cache baseline routing weights w_base[L] (softmax(read_logits))
#     2) Install routing_override that ALWAYS returns w_base[L] (routing frozen)
#     3) Capture baseline residual states h_base[L] at the output of block L (B,T,D)
#     4) Fit PCA(3) on late-window residual points for that layer
#     5) Estimate a "best" residual-control direction g_h in PCA coords via small FD
#     6) Sweep epsilon along g_h, inject into residual at t_last, and measure mean Paris-margin
#
# Interpretation:
#   If residual injection still cannot move margin (flat response) even with routing frozen,
#   that strongly supports "routing-as-address" (Hypothesis A).
#   If residual injection becomes effective (meaningfully moves margin), that supports
#   "routing-as-decoder-key" (Hypothesis B).
#
# Assumptions:
#   - model, cfg, DEVICE exist
#   - model(..., return_info=True, routing_mode="softmax") -> (logits, infos)
#   - infos[L]["read_logits"] exists, shape [B,H,T,K]
#   - model.blocks is indexable (len gives #layers)
#   - Each model.blocks[L](...) returns hidden states as first output or a tensor
# ================================================================

import math
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# ---------------------------
# SETTINGS
# ---------------------------
LAYERS = [0, 5, 13]        # edit as desired
LATE_W = 5                 # last W valid tokens for PCA fit
FD_EPS = 0.02              # FD step in residual PCA coords for direction estimate
EPS_SWEEP = [0.00, 0.01, 0.02, 0.04, 0.08]  # injection magnitudes along g_h
SKIP_T0 = True             # skip token index 0
SANITIZE_MODE = "drop"     # "drop" or "clamp"
CLAMP_VAL = 5.0

RIVAL_TEXT = " the"
TARGET_TEXT = " Paris"

PROMPTS = [
    "The capital of France is",
    "France's capital city is",
    "In France, the capital is",
    "The seat of government of France is",
    "The city that is France's capital is",
    "The Eiffel Tower is located in",
    "The Louvre is in",
    "The Seine river flows through",
    "During the French Revolution, the city of",
    "Napoleon crowned himself in",
    "In World War II, the liberation of",
    "The largest city in France is",
    "A major European city in France is",
    "A famous city on the Seine is",
    "The capital of Germany is",
    "The capital of England is",
    "The capital of Italy is",
]

# ---------------------------
# Tokenizer (RIGHT padding enforced)
# ---------------------------
def _get_tokenizer_from_context():
    if "tokenizer" in globals():
        tok = globals()["tokenizer"]
        tok.padding_side = "right"
        if tok.pad_token is None:
            tok.pad_token = tok.eos_token
        return tok
    from transformers import AutoTokenizer
    tok = AutoTokenizer.from_pretrained(getattr(cfg, "tokenizer_name", "gpt2"))
    tok.padding_side = "right"
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok

tokenizer = _get_tokenizer_from_context()

def ensure_single_token(text: str) -> int:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) != 1:
        raise ValueError(f"{text!r} must be exactly 1 token under tokenizer. Got ids={ids}")
    return ids[0]

PARIS_ID = ensure_single_token(TARGET_TEXT)
RIVAL_ID = ensure_single_token(RIVAL_TEXT)

# ---------------------------
# Batch tokenize
# ---------------------------
enc = tokenizer(
    PROMPTS,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=cfg.max_seq_len,
)
input_ids = enc["input_ids"].to(DEVICE)
attention_mask = enc.get("attention_mask", torch.ones_like(input_ids)).to(DEVICE)

B, T = input_ids.shape
lengths = attention_mask.sum(dim=1).long()
t_last = (lengths - 1).clamp(min=0)

def late_window_indices(b: int):
    last = int(t_last[b].item())
    t0 = max(0, last - (LATE_W - 1))
    idxs = list(range(t0, last + 1))
    if SKIP_T0:
        idxs = [t for t in idxs if t != 0]
    return idxs

late_idxs = [late_window_indices(b) for b in range(B)]

# ---------------------------
# Forward helper
# ---------------------------
def forward_with_info(ids, mask, routing_mode="softmax"):
    return model(ids, attention_mask=mask, return_info=True, routing_mode=routing_mode)

@torch.no_grad()
def mean_paris_margin_from_logits(logits: torch.Tensor) -> float:
    logp = F.log_softmax(logits, dim=-1)
    idx = torch.arange(B, device=logits.device)
    lpP = logp[idx, t_last, PARIS_ID]
    lpR = logp[idx, t_last, RIVAL_ID]
    return float((lpP - lpR).mean().item())

# ---------------------------
# Routing freeze: cache baseline weights and override to return them
# ---------------------------
def clear_all_routing_overrides():
    for li in range(len(model.blocks)):
        blk = model.blocks[li]
        if hasattr(blk, "asa") and hasattr(blk.asa, "routing_override"):
            blk.asa.routing_override = None

def set_frozen_routing(layer_idx: int, w_base_bhtk: torch.Tensor):
    """
    w_base_bhtk: [B,H,T,K] baseline routing weights on DEVICE.
    Override will return baseline weights for the chunk positions.
    """
    clear_all_routing_overrides()

    def _override(t0, t1, read_logits_chunk, read_logits_key, read_logits_content, ctx):
        # read_logits_chunk: [B,H,L,K], where L = t1-t0
        Bc, Hc, Lc, Kc = read_logits_chunk.shape
        w = w_base_bhtk[:, :, t0:t1, :].contiguous()
        # safety: ensure correct shape
        if w.shape != (Bc, Hc, Lc, Kc):
            # fallback: recompute base from current chunk if mismatch (should not happen)
            rtemp = float(ctx.get("rtemp", 1.0))
            w = torch.softmax(read_logits_chunk / max(1e-6, rtemp), dim=-1)
        w = w / w.sum(dim=-1, keepdim=True).clamp_min(1e-8)
        return w

    model.blocks[layer_idx].asa.routing_override = _override

# ---------------------------
# Residual capture + injection hooks
# ---------------------------
def _as_tensor_out(out):
    # handle blocks that return (hidden, ...) vs hidden
    if isinstance(out, (tuple, list)):
        return out[0]
    return out

class ResidualCapture:
    def __init__(self, layer_idx: int):
        self.layer_idx = layer_idx
        self.h = None
        self.hook = None

    def __enter__(self):
        def fn(module, inp, out):
            self.h = _as_tensor_out(out).detach()
            return out
        self.hook = model.blocks[self.layer_idx].register_forward_hook(fn)
        return self

    def __exit__(self, exc_type, exc, tb):
        if self.hook is not None:
            self.hook.remove()
        self.hook = None

class ResidualInjector:
    def __init__(self, layer_idx: int, delta_btd: torch.Tensor):
        """
        delta_btd: [B,T,D] additive delta applied to block output at layer_idx.
        Typically only nonzero at t_last positions.
        """
        self.layer_idx = layer_idx
        self.delta = delta_btd
        self.hook = None

    def __enter__(self):
        def fn(module, inp, out):
            h = _as_tensor_out(out)
            h2 = h + self.delta.to(h.device, dtype=h.dtype)
            if isinstance(out, (tuple, list)):
                out = (h2,) + tuple(out[1:])
                return out
            return h2
        self.hook = model.blocks[self.layer_idx].register_forward_hook(fn)
        return self

    def __exit__(self, exc_type, exc, tb):
        if self.hook is not None:
            self.hook.remove()
        self.hook = None

# ---------------------------
# PCA utilities (torch SVD, robust)
# ---------------------------
def _finite_mask_rows(X: torch.Tensor) -> torch.Tensor:
    return torch.isfinite(X).all(dim=-1)

def fit_pca3(X_points: torch.Tensor):
    """
    X_points: [N,D] float
    returns mu [D], W3 [D,3], evr3 [3], n_good
    """
    X = X_points
    if SANITIZE_MODE == "clamp":
        X = torch.nan_to_num(X, nan=0.0, posinf=CLAMP_VAL, neginf=-CLAMP_VAL)
        good = _finite_mask_rows(X)
    else:
        good = _finite_mask_rows(X)
        X = X[good]

    n_good = int(X.shape[0])
    if n_good < 10:
        raise RuntimeError(f"Too few good PCA points: {n_good}")

    mu = X.mean(dim=0, keepdim=True)
    Xc = X - mu
    try:
        U, S, Vh = torch.linalg.svd(Xc, full_matrices=False)
    except Exception:
        Xc_cpu = Xc.detach().cpu()
        U, S, Vh = torch.linalg.svd(Xc_cpu, full_matrices=False)
        Vh = Vh.to(Xc.device)
        S = S.to(Xc.device)

    W3 = Vh[:3].T.contiguous()  # [D,3]
    s2 = S**2
    evr3 = (s2[:3] / s2.sum().clamp_min(1e-12)).detach().cpu().numpy()
    return mu.squeeze(0), W3, evr3, n_good

def project_pca3(h_btd: torch.Tensor, mu: torch.Tensor, W3: torch.Tensor):
    """
    h_btd: [B,T,D]  -> x_btd3: [B,T,3]
    """
    return (h_btd - mu.view(1, 1, -1)) @ W3  # [B,T,3]

# ---------------------------
# Direction estimation (FD in residual PCA coords, routing frozen)
# ---------------------------
@torch.no_grad()
def estimate_global_residual_dir(layer_idx: int, mu: torch.Tensor, W3: torch.Tensor, h_base_btd: torch.Tensor, w_base_bhtk: torch.Tensor):
    """
    Estimate FD gradient of mean Paris-margin w.r.t a 3D residual PCA coordinate,
    evaluated at the GLOBAL late-point centroid (mean across all late points).
    Returns unit vector g_dir (3,) in PCA coords.
    """
    # Build centroid in PCA coords from late-window points
    x3 = project_pca3(h_base_btd, mu, W3)  # [B,T,3]
    pts = []
    for b in range(B):
        for t in late_idxs[b]:
            pts.append(x3[b, t])
    Xc = torch.stack(pts, dim=0)  # [N,3]
    x0 = Xc.mean(dim=0)           # [3]

    # helper: run model with injection corresponding to x = x0 + delta in PCA coords at each b's t_last
    def eval_at_x(x_vec3: torch.Tensor) -> float:
        # Convert PCA delta -> residual delta in D
        # We inject ONLY at each prompt's t_last: h += (x_vec3 @ W3^T)  (same delta for all prompts)
        d_h = (x_vec3 @ W3.T).view(1, 1, -1)  # [1,1,D]
        delta = torch.zeros_like(h_base_btd)
        for b in range(B):
            tt = int(t_last[b].item())
            delta[b, tt, :] = d_h[0, 0, :]
        set_frozen_routing(layer_idx, w_base_bhtk)
        with ResidualInjector(layer_idx, delta):
            logits, _infos = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
        clear_all_routing_overrides()
        return mean_paris_margin_from_logits(logits)

    g = torch.zeros(3, device=DEVICE)
    for d in range(3):
        e = torch.zeros(3, device=DEVICE)
        e[d] = FD_EPS
        m_plus = eval_at_x(x0 + e)
        m_minus = eval_at_x(x0 - e)
        g[d] = (m_plus - m_minus) / (2.0 * FD_EPS)

    g_norm = g.norm().clamp_min(1e-12)
    g_dir = (g / g_norm).detach().cpu().numpy()
    return g_dir, g.detach().cpu().numpy()

# ---------------------------
# Main run
# ---------------------------
@torch.no_grad()
def run_layer(layer_idx: int):
    # baseline forward: cache routing weights + capture residual
    clear_all_routing_overrides()

    # capture residual at layer output
    with ResidualCapture(layer_idx) as cap:
        logits_base, infos_base = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
    base_margin = mean_paris_margin_from_logits(logits_base)

    rl = infos_base[layer_idx]["read_logits"]            # [B,H,T,K]
    w_base = torch.softmax(rl, dim=-1).detach()          # [B,H,T,K]
    h_base = cap.h                                       # [B,T,D]
    assert h_base is not None, "Failed to capture residual. Check block output structure."

    # fit PCA on late-window residual points
    pts = []
    for b in range(B):
        for t in late_idxs[b]:
            pts.append(h_base[b, t])
    Hfit = torch.stack(pts, dim=0).float()  # [N,D]
    mu, W3, evr3, n_good = fit_pca3(Hfit)

    # estimate global direction in residual PCA coords, routing frozen
    g_dir, g_raw = estimate_global_residual_dir(layer_idx, mu.to(DEVICE), W3.to(DEVICE), h_base.float(), w_base)

    # sweep eps along that direction
    margins = []
    deltas = []
    for eps in EPS_SWEEP:
        x_vec3 = torch.tensor(g_dir, device=DEVICE, dtype=torch.float32) * float(eps)
        d_h = (x_vec3 @ W3.to(DEVICE).T).view(1, 1, -1)  # [1,1,D]
        delta = torch.zeros_like(h_base).float()
        for b in range(B):
            tt = int(t_last[b].item())
            delta[b, tt, :] = d_h[0, 0, :]
        set_frozen_routing(layer_idx, w_base)
        with ResidualInjector(layer_idx, delta):
            logits, _infos = forward_with_info(input_ids, attention_mask, routing_mode="softmax")
        clear_all_routing_overrides()
        m = mean_paris_margin_from_logits(logits)
        margins.append(m)
        deltas.append(m - base_margin)

    # local slope estimate around 0 using first nonzero point
    slope = None
    if len(EPS_SWEEP) >= 2 and EPS_SWEEP[1] > 0:
        slope = (deltas[1] - deltas[0]) / EPS_SWEEP[1]

    return {
        "layer": layer_idx,
        "base_margin": base_margin,
        "evr3": evr3,
        "n_good": n_good,
        "g_raw": g_raw,
        "g_dir": g_dir,
        "eps": np.array(EPS_SWEEP, dtype=np.float32),
        "margins": np.array(margins, dtype=np.float32),
        "deltas": np.array(deltas, dtype=np.float32),
        "slope0": slope,
    }

results = {}
print(f"[info] B={B} T={T} | detected layers={len(model.blocks)} | testing layers={LAYERS}")
print("[note] Routing is frozen per-layer to baseline weights; only residual is perturbed.\n")

for L in LAYERS:
    out = run_layer(int(L))
    results[int(L)] = out
    print(f"=== Layer {L} ===")
    print(f"baseline mean margin = {out['base_margin']:+.4f}")
    print(f"residual PCA EVR(PC1..3)≈ {np.round(out['evr3'],4)} | PCA_fit_good={out['n_good']}")
    print(f"FD grad in PCA coords (unnormalized) g_raw = {np.round(out['g_raw'],4)}")
    print(f"unit dir g_dir = {np.round(out['g_dir'],4)}")
    if out["slope0"] is not None:
        print(f"local slope near 0 (Δmargin/ε) ≈ {out['slope0']:+.4f}")
    print("eps | mean_margin | Δmargin")
    for e, m, d in zip(out["eps"], out["margins"], out["deltas"]):
        print(f"{e:>4.2f} | {m:+.4f}    | {d:+.4f}")
    print()

# ---------------------------
# Plot summary
# ---------------------------
plt.figure(figsize=(8.5, 5.4))
for L in LAYERS:
    out = results[int(L)]
    plt.plot(out["eps"], out["deltas"], marker="o", label=f"L{L} (slope≈{(out['slope0'] if out['slope0'] is not None else float('nan')):+.2f})")
plt.axhline(0.0, linewidth=1.0)
plt.title("Residual Injection with Routing FROZEN: Δ(Paris-margin) vs ε")
plt.xlabel("ε (step along estimated residual-control direction)")
plt.ylabel("Δ mean Paris-margin")
plt.grid(True, alpha=0.25)
plt.legend()
plt.tight_layout()
plt.show()

print("\n[done] If curves are ~flat (all layers), residual is not locally controllable under frozen routing (supports Hypothesis A).")
print("[done] If one or more layers show strong monotone Δmargin vs ε, residual becomes controllable when routing is fixed (supports Hypothesis B).")

In [ ]:

#@title Run WikiText Benchmark
import torch
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import AutoTokenizer

# ============================
# Configuration
# ============================
BATCH_SIZE = 8  # Adjust based on your GPU memory
MAX_LENGTH = cfg.max_seq_len  # Use model's max sequence length
STRIDE = MAX_LENGTH // 2  # Sliding window stride for long sequences

# ============================
# Load WikiText-2 Test Set
# ============================
#print("Loading WikiText-103-raw test set...")
#dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="test")

# Load dataset
print("Loading WikiText-103 test set...")
dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="test")
print(f"Number of examples: {len(dataset)}")

# Get tokenizer
#tokenizer = ckpt["tokenizer"]
#print(f"✓ Loaded tokenizer (vocab size: {len(tokenizer)})")
if ckpt["cfg"].get("tokenizer_name", None) is not None:
    tokenizer = AutoTokenizer.from_pretrained(ckpt["cfg"].get("tokenizer_name", "gpt2"), use_fast=True)

# Tokenize and concatenate properly
print("Tokenizing dataset...")
all_tokens = []
for example in tqdm(dataset, desc="Processing"):
    text = example["text"].strip()
    if text:  # Skip empty lines
        tokens = tokenizer.encode(text)
        all_tokens.extend(tokens)

all_tokens = torch.tensor(all_tokens, dtype=torch.long)
print(f"Total tokens: {len(all_tokens):,}")
print(f"Expected: ~3.4M tokens")


# ============================
# Evaluate perplexity
# ============================
@torch.no_grad()
def evaluate_perplexity(model, tokens, batch_size=BATCH_SIZE, max_length=MAX_LENGTH, stride=STRIDE):
    model.eval()
    total_loss = 0.0
    total_tokens = 0

    # Create sliding windows
    sequences = []
    for i in range(0, len(tokens) - 1, stride):
        end_idx = min(i + max_length, len(tokens))
        if end_idx - i > 1:  # Need at least 2 tokens (input + target)
            sequences.append(tokens[i:end_idx])

    print(f"Created {len(sequences)} sequences with stride {stride}")

    # Process in batches
    for batch_start in tqdm(range(0, len(sequences), batch_size), desc="Evaluating"):
        batch_seqs = sequences[batch_start:batch_start + batch_size]

        # Pad sequences to same length in batch
        max_len_in_batch = max(len(s) for s in batch_seqs)
        batch_inputs = []
        batch_targets = []
        batch_masks = []

        for seq in batch_seqs:
            input_seq = seq[:-1]
            target_seq = seq[1:]

            # Pad if needed
            pad_len = max_len_in_batch - 1 - len(input_seq)
            if pad_len > 0:
                input_seq = torch.cat([input_seq, torch.zeros(pad_len, dtype=torch.long)])
                target_seq = torch.cat([target_seq, torch.zeros(pad_len, dtype=torch.long)])
                mask = torch.cat([torch.ones(len(seq) - 1), torch.zeros(pad_len)])
            else:
                mask = torch.ones(len(input_seq))

            batch_inputs.append(input_seq)
            batch_targets.append(target_seq)
            batch_masks.append(mask)

        # Stack into tensors
        inputs = torch.stack(batch_inputs).to(DEVICE)
        targets = torch.stack(batch_targets).to(DEVICE)
        masks = torch.stack(batch_masks).to(DEVICE)

        # Forward pass
        logits = model(inputs)  # Shape: [B, T, vocab_size]

        # Calculate loss
        loss = F.cross_entropy(
            logits.reshape(-1, logits.size(-1)),
            targets.reshape(-1),
            reduction='none'
        )
        loss = loss.view(targets.shape)

        # Apply mask and accumulate
        masked_loss = (loss * masks).sum()
        masked_tokens = masks.sum()

        total_loss += masked_loss.item()
        total_tokens += masked_tokens.item()

    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)

    return perplexity, avg_loss

# ============================
# Run evaluation
# ============================
print("\n" + "="*50)
print("Running WikiText-2 Benchmark")
print("="*50)

perplexity, avg_loss = evaluate_perplexity(model, all_tokens)

print("\n" + "="*50)
print("RESULTS")
print("="*50)
print(f"Perplexity: {perplexity:.2f}")
print(f"Avg Loss: {avg_loss:.4f}")
print(f"Model: {CKPT_PATH}")
print_cfg_summary(cfg, model)
print("="*50)

In [ ]:

#@title Experiment 1: Slotspace Ablation Study
import torch
import torch.nn.functional as F
from datasets import load_dataset
from tqdm.auto import tqdm
import pandas as pd
import matplotlib.pyplot as plt

# ============================
# Configuration
# ============================
BASE_CONFIG = {
    "vocab_size": 50257,
    "embed_dim": 384,
    "num_layers": 12,  # Smaller for faster experiments
    "num_heads": 8,
    "num_slots": 16,
    "max_seq_len": 1024,
    "mlp_ratio": 4.0,
    "dropout": 0.1,
    "read_temperature": 1.0,
    "write_temperature": 1.0,
    "state_fp32": True,
    "slot_dropout": 0.05,
    "normalize_k": False,
    "tie_weights": True,
    "use_abs_pos": False,
    "use_rope_keys": True,
    "rope_base": 10000.0,
    "use_alibi_write": True,
    "alibi_strength_init": 0.1,
    "learn_alibi_strength": True,
    "min_strength": 0.0,
    "use_content_read": True,
    "content_read_init": -4.0,
    "content_read_max_gamma": 3.0,
    "slotspace_dim": 32,
    "slotspace_gate_init": -4.0,
    "slotspace_dropout": 0.05,
    "slotspace_signed_weights": True,
    "use_rope_slotspace": True,
    "rope_base_slotspace": 100000.0,
    "write_chunk_size": 128,
    "slotspace_chunk_size": 128,
}

# Ablation configurations
ABLATIONS = {
    "no_slotspace": {
        "use_slotspace_refine": False,
        "description": "No slotspace refinement (baseline)"
    },
    "full": {
        "use_slotspace_refine": True,
        "description": "Full model (your current best)"
    },
    "no_signed_weights": {
        "use_slotspace_refine": True,
        "slotspace_signed_weights": False,
        "description": "Slotspace with softmax instead of tanh"
    },
    "no_rope_slotspace": {
        "use_slotspace_refine": True,
        "use_rope_slotspace": False,
        "description": "Slotspace without RoPE"
    },
    "weak_gate_init": {
        "use_slotspace_refine": True,
        "slotspace_gate_init": -8.0,  # More closed initially
        "description": "Slotspace with weaker initial gate"
    },
}

# Training hyperparameters
TRAIN_CONFIG = {
    "batch_size": 32,
    "learning_rate": 3e-4,
    "weight_decay": 0.01,
    "warmup_steps": 500,
    "total_steps": 10_000,  # Short training for quick comparison
    "eval_interval": 500,
    "log_interval": 100,
    "grad_clip": 1.0,
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================
# Training Loop
# ============================
def train_ablation(config_name, model_config, train_config, tokenizer):
    """Train a single ablation"""
    print(f"\n{'='*60}")
    print(f"Training: {config_name}")
    print(f"Description: {ABLATIONS[config_name]['description']}")
    print(f"{'='*60}")

    # Build model
    cfg = ASMTrainConfig(**{**BASE_CONFIG, **model_config})
    model = build_model_from_cfg(cfg).to(DEVICE)

    n_params = sum(p.numel() for p in model.parameters())
    print(f"Parameters: {n_params/1e6:.2f}M")

    # Optimizer
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=train_config["learning_rate"],
        weight_decay=train_config["weight_decay"],
        betas=(0.9, 0.95),
    )

    # Load dataset (use smaller subset for speed)
    dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")

    # Simple training loop
    model.train()
    losses = []
    steps = []

    step = 0
    running_loss = 0.0

    pbar = tqdm(total=train_config["total_steps"], desc=f"Training {config_name}")

    while step < train_config["total_steps"]:
        # Get batch (simplified - you'd want proper batching)
        batch_texts = []
        for _ in range(train_config["batch_size"]):
            idx = torch.randint(0, len(dataset), (1,)).item()
            text = dataset[idx]["text"]
            if len(text.strip()) > 0:
                batch_texts.append(text)

        if len(batch_texts) == 0:
            continue

        # Tokenize
        tokens = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        input_ids = tokens["input_ids"].to(DEVICE)

        if input_ids.size(1) < 2:
            continue

        # Forward
        logits = model(input_ids[:, :-1])
        targets = input_ids[:, 1:]

        loss = F.cross_entropy(
            logits.reshape(-1, logits.size(-1)),
            targets.reshape(-1),
            ignore_index=tokenizer.pad_token_id,
        )

        # Backward
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), train_config["grad_clip"])
        optimizer.step()

        running_loss += loss.item()
        step += 1

        if step % train_config["log_interval"] == 0:
            avg_loss = running_loss / train_config["log_interval"]
            losses.append(avg_loss)
            steps.append(step)
            pbar.set_postfix({"loss": f"{avg_loss:.4f}"})
            running_loss = 0.0

        pbar.update(1)

    pbar.close()

    # Quick eval
    model.eval()
    eval_loss = 0.0
    eval_steps = 0

    val_dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="validation")

    with torch.no_grad():
        for _ in range(100):  # Sample 100 validation examples
            idx = torch.randint(0, len(val_dataset), (1,)).item()
            text = val_dataset[idx]["text"]
            if len(text.strip()) == 0:
                continue

            tokens = tokenizer([text], return_tensors="pt", truncation=True, max_length=512)
            input_ids = tokens["input_ids"].to(DEVICE)

            if input_ids.size(1) < 2:
                continue

            logits = model(input_ids[:, :-1])
            targets = input_ids[:, 1:]

            loss = F.cross_entropy(
                logits.reshape(-1, logits.size(-1)),
                targets.reshape(-1),
                reduction='sum',
            )

            eval_loss += loss.item()
            eval_steps += targets.numel()

    final_loss = eval_loss / eval_steps if eval_steps > 0 else float('inf')
    final_ppl = torch.exp(torch.tensor(final_loss)).item()

    print(f"\nFinal validation loss: {final_loss:.4f}")
    print(f"Final validation PPL: {final_ppl:.2f}")

    return {
        "config_name": config_name,
        "description": ABLATIONS[config_name]['description'],
        "train_losses": losses,
        "train_steps": steps,
        "final_val_loss": final_loss,
        "final_val_ppl": final_ppl,
        "n_params": n_params,
    }

# ============================
# Run All Ablations
# ============================
# Load tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

results = []

for config_name, ablation_config in ABLATIONS.items():
    result = train_ablation(config_name, ablation_config, TRAIN_CONFIG, tokenizer)
    results.append(result)

    # Save intermediate results
    torch.save(results, "ablation_results.pt")

# ============================
# Visualize Results
# ============================
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Training curves
ax = axes[0]
for result in results:
    ax.plot(result["train_steps"], result["train_losses"], label=result["config_name"], linewidth=2)
ax.set_xlabel("Training Steps")
ax.set_ylabel("Training Loss")
ax.set_title("Training Loss Curves")
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Final validation PPL
ax = axes[1]
names = [r["config_name"] for r in results]
ppls = [r["final_val_ppl"] for r in results]
colors = ['red' if 'no_slotspace' in n else 'green' if n == 'full' else 'blue' for n in names]
bars = ax.bar(range(len(names)), ppls, color=colors, alpha=0.7)
ax.set_xticks(range(len(names)))
ax.set_xticklabels(names, rotation=45, ha='right')
ax.set_ylabel("Validation Perplexity")
ax.set_title("Final Validation PPL (lower is better)")
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, ppl in zip(bars, ppls):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{ppl:.1f}',
            ha='center', va='bottom', fontsize=9)

# Plot 3: Summary table
ax = axes[2]
ax.axis('tight')
ax.axis('off')

table_data = []
for r in results:
    table_data.append([
        r["config_name"],
        f"{r['final_val_ppl']:.2f}",
        f"{r['n_params']/1e6:.1f}M"
    ])

table = ax.table(cellText=table_data,
                colLabels=['Config', 'Val PPL', 'Params'],
                cellLoc='center',
                loc='center',
                colWidths=[0.4, 0.3, 0.3])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

# Color code rows
for i, (cell_key) in enumerate(table.get_celld().keys()):
    cell = table[cell_key]
    if cell_key[0] == 0:  # Header
        cell.set_facecolor('#40466e')
        cell.set_text_props(weight='bold', color='white')
    elif 'no_slotspace' in table_data[cell_key[0]-1][0]:
        cell.set_facecolor('#ffcccc')
    elif table_data[cell_key[0]-1][0] == 'full':
        cell.set_facecolor('#ccffcc')

plt.tight_layout()
plt.savefig('ablation_study.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "="*60)
print("ABLATION STUDY COMPLETE")
print("="*60)

# Print summary
df = pd.DataFrame([
    {
        "Config": r["config_name"],
        "Description": r["description"],
        "Val PPL": f"{r['final_val_ppl']:.2f}",
        "Params": f"{r['n_params']/1e6:.1f}M"
    }
    for r in results
])
print(df.to_string(index=False))

# Calculate improvements
baseline = [r for r in results if r["config_name"] == "no_slotspace"][0]
full_model = [r for r in results if r["config_name"] == "full"][0]

improvement = ((baseline["final_val_ppl"] - full_model["final_val_ppl"]) / baseline["final_val_ppl"]) * 100
print(f"\nSlotspace provides {improvement:.1f}% PPL improvement over baseline")

In [ ]:


#@title FAST Paper-Grade Probing with Proper Progress Tracking

import re, random
from dataclasses import dataclass
from collections import Counter, defaultdict

import numpy as np
import torch

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score, f1_score

import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from transformers import AutoTokenizer

# ===== SAME HELPER FUNCTIONS =====
def set_all_seeds(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def make_nonce_word(rng: np.random.Generator, min_len=4, max_len=8):
    letters = np.array(list("abcdefghijklmnopqrstuvwxyz"))
    L = int(rng.integers(min_len, max_len + 1))
    return "".join(rng.choice(letters, size=L))

def apply_control(words, labels, control: str, seed: int):
    if control == "none":
        return words, labels

    n = min(len(words), len(labels))
    words = words[:n]
    labels = labels[:n]
    rng = np.random.default_rng(seed)

    if control == "nonce":
        return [make_nonce_word(rng) for _ in range(n)], list(labels)
    if control == "shuffle":
        pairs = list(zip(words, labels))
        rng.shuffle(pairs)
        w2, l2 = zip(*pairs) if pairs else ([], [])
        return list(w2), list(l2)
    raise ValueError(f"Unknown control={control!r}")

def depths_for_sent(sent):
    root = sent.root
    depth = {root.i: 0}
    stack = [root]
    while stack:
        tok = stack.pop()
        for child in tok.children:
            if child.i < sent.start or child.i >= sent.end:
                continue
            if child.i not in depth:
                depth[child.i] = depth[tok.i] + 1
                stack.append(child)
    return [int(depth.get(t.i, 0)) for t in sent]

def build_spacy_labeled_sentences(
    max_sents=2000,
    text_source="wikitext",
    spacy_model="en_core_web_sm",
    seed=42,
    min_len=3,
    max_len=40,
):
    set_all_seeds(seed)
    from datasets import load_dataset
    import spacy

    nlp = spacy.load(spacy_model, disable=["ner"])

    if text_source == "wikitext":
        ds = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")
        texts = ds["text"]
    else:
        ds = load_dataset("wikipedia", "20220301.en", split="train[:1%]")
        texts = ds["text"]

    idx = np.arange(len(texts))
    rng = np.random.default_rng(seed)
    rng.shuffle(idx)

    examples = []
    sent_id = 0

    for i in idx:
        line = texts[int(i)]
        if not isinstance(line, str):
            continue
        line = line.strip()
        if len(line) < 20 or (line.startswith("=") and line.endswith("=")):
            continue

        doc = nlp(line)
        for sent in doc.sents:
            words = [t.text for t in sent]
            if not (min_len <= len(words) <= max_len):
                continue

            upos = [t.pos_ for t in sent]
            depth = depths_for_sent(sent)

            text = " ".join(words)
            examples.append({
                "text": text,
                "words": words,
                "upos": upos,
                "depth": depth,
                "sent_id": sent_id
            })
            sent_id += 1
            if sent_id >= max_sents:
                return examples

    return examples

WORD_RE = re.compile(r"\S+")

def word_spans(text: str):
    return [(m.start(), m.end(), m.group()) for m in WORD_RE.finditer(text)]

def token_overlaps(t_start, t_end, w_start, w_end):
    return (t_start < w_end) and (t_end > w_start)

class ASALayerFeatureExtractor:
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tok = tokenizer
        self.device = device

    def _forward_to_layer(self, input_ids, layer_idx):
        x = self.model.tok(input_ids)
        if getattr(self.model, "use_abs_pos", False):
            pos = torch.arange(input_ids.size(1), device=input_ids.device).unsqueeze(0)
            x = x + self.model.pos(pos)
        x = self.model.drop(x)

        info_at = None
        for i, blk in enumerate(self.model.blocks):
            x, info = blk(x, return_info=True)
            if i == layer_idx:
                info_at = info
                break
        if info_at is None:
            raise ValueError(f"Layer {layer_idx} not found. blocks={len(self.model.blocks)}")
        return info_at

    def extract_word_features(self, text, layer_idx, feature_type):
        toks = self.tok(
            text,
            return_tensors="pt",
            return_offsets_mapping=True,
            add_special_tokens=False,
        )
        input_ids = toks["input_ids"].to(self.device)
        offsets = toks["offset_mapping"][0].tolist()

        spans = word_spans(text)

        self.model.eval()
        with torch.no_grad():
            info = self._forward_to_layer(input_ids, layer_idx)

            if feature_type == "content":
                feats = info.get("read_logits_content", None)
                if feats is None:
                    raise ValueError("read_logits_content is None")
            elif feature_type == "key":
                feats = info.get("read_logits_key", None)
                if feats is None:
                    raise ValueError("read_logits_key missing")
            elif feature_type == "routing":
                feats = info.get("read_weights", None)
                if feats is None:
                    raise ValueError("read_weights missing")
            else:
                raise ValueError(f"Unknown feature_type={feature_type!r}")

            feats = feats.mean(dim=1).squeeze(0).detach().cpu()
            T = feats.shape[0]

        word_feats = []
        for (w_start, w_end, _) in spans:
            idx = []
            for t, (t_start, t_end) in enumerate(offsets):
                if t_start == t_end == 0:
                    continue
                if 0 <= t < T and token_overlaps(t_start, t_end, w_start, w_end):
                    idx.append(t)
            if len(idx) == 0:
                word_feats.append(None)
            else:
                word_feats.append(feats[idx].mean(dim=0).numpy())
        return word_feats

# ===== CACHE FEATURES (MAJOR SPEEDUP!) =====

def cache_all_features(examples, extractor, feature_types, tasks, controls, seed_base=42):
    """
    Pre-extract ALL features for all layers to avoid redundant forward passes.
    This is the KEY optimization!
    """
    n_layers = len(extractor.model.blocks)

    # Cache structure: cache[sent_id][control][feature_type][layer_idx] = word_features
    cache = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

    total_extractions = len(examples) * len(controls) * len(feature_types) * n_layers

    print(f"Pre-extracting features for {len(examples)} sentences...")
    print(f"Total forward passes: {total_extractions}")

    with tqdm(total=total_extractions, desc="Caching features") as pbar:
        for ex in examples:
            sent_id = ex["sent_id"]
            words = ex["words"]

            for control in controls:
                # Apply control once per sentence
                words_c, _ = apply_control(words, ex["upos"], control, seed_base + sent_id)
                text_c = " ".join(words_c)

                for ft in feature_types:
                    for layer_idx in range(n_layers):
                        try:
                            feats = extractor.extract_word_features(text_c, layer_idx, ft)
                            cache[sent_id][control][ft][layer_idx] = feats
                        except Exception as e:
                            print(f"Error caching sent={sent_id}, ctrl={control}, ft={ft}, layer={layer_idx}: {e}")
                            cache[sent_id][control][ft][layer_idx] = None

                        pbar.update(1)

    return cache

def build_arrays_from_cache(examples, feature_cache, layer_idx, feature_type, task, control):
    """Build probe arrays from pre-cached features"""
    X, y, groups = [], [], []

    for ex in examples:
        sent_id = ex["sent_id"]
        labels = ex[task]

        feats_word = feature_cache.get(sent_id, {}).get(control, {}).get(feature_type, {}).get(layer_idx, None)

        if feats_word is None:
            continue

        n = min(len(feats_word), len(labels))
        for i in range(n):
            if feats_word[i] is None:
                continue
            X.append(feats_word[i])
            y.append(labels[i])
            groups.append(sent_id)

    return X, y, groups

# ===== FAST PROBE RUNNER =====

def fit_eval_probe(X, y, groups, seed):
    X = np.asarray(X)
    y = np.asarray(y)
    groups = np.asarray(groups)

    counts = Counter(y.tolist())
    majority = max(counts.values()) / len(y)
    uniform = 1.0 / len(counts)

    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
    tr, te = next(gss.split(X, y, groups))
    Xtr, Xte = X[tr], X[te]
    ytr, yte = y[tr], y[te]

    clf = LogisticRegression(
        max_iter=3000,
        random_state=seed,
        class_weight="balanced",
    )
    clf.fit(Xtr, ytr)
    pred = clf.predict(Xte)

    acc = accuracy_score(yte, pred)
    f1 = f1_score(yte, pred, average="macro", zero_division=0)
    return float(acc), float(f1), float(uniform), float(majority)

def run_all_layers_fast(examples, feature_cache, feature_types, tasks, controls, seeds):
    """Fast version using cached features"""
    n_layers = len(extractor.model.blocks)
    results = {task: {control: {ft: {"acc": [], "f1": [], "uniform": [], "majority": []}
                                for ft in feature_types}
                      for control in controls}
               for task in tasks}

    total_probes = n_layers * len(tasks) * len(controls) * len(feature_types)

    with tqdm(total=total_probes, desc="Running probes") as pbar:
        for task in tasks:
            for control in controls:
                for ft in feature_types:
                    for layer_idx in range(n_layers):
                        accs, f1s, unis, majs = [], [], [], []

                        for s in seeds:
                            X, y, g = build_arrays_from_cache(
                                examples, feature_cache, layer_idx, ft, task, control
                            )

                            if len(X) < 200 or len(set(y)) < 2:
                                acc, f1, uni, maj = (np.nan, np.nan, np.nan, np.nan)
                            else:
                                acc, f1, uni, maj = fit_eval_probe(X, y, g, seed=s)

                            accs.append(acc)
                            f1s.append(f1)
                            unis.append(uni)
                            majs.append(maj)

                        results[task][control][ft]["acc"].append((np.nanmean(accs), np.nanstd(accs)))
                        results[task][control][ft]["f1"].append((np.nanmean(f1s), np.nanstd(f1s)))
                        results[task][control][ft]["uniform"].append(np.nanmean(unis))
                        results[task][control][ft]["majority"].append(np.nanmean(majs))

                        pbar.set_postfix({
                            'task': task,
                            'ctrl': control,
                            'ft': ft,
                            'layer': layer_idx
                        })
                        pbar.update(1)

    return results

def plot_task(results, task, control, feature_types):
    n_layers = len(next(iter(results[task][control].values()))["acc"])
    layers = np.arange(n_layers)

    fig, axes = plt.subplots(1, 2, figsize=(16, 5))

    # Accuracy
    ax = axes[0]
    for ft in feature_types:
        m = np.array([x[0] for x in results[task][control][ft]["acc"]], dtype=float)
        s = np.array([x[1] for x in results[task][control][ft]["acc"]], dtype=float)
        ax.plot(layers, m, marker="o", linewidth=2, label=f"{ft}", markersize=4)
        ax.fill_between(layers, m-s, m+s, alpha=0.15)

    uni = np.array(results[task][control]["key"]["uniform"], dtype=float)
    maj = np.array(results[task][control]["key"]["majority"], dtype=float)
    ax.plot(layers, uni, linestyle="--", linewidth=2, label="Uniform", alpha=0.7)
    ax.plot(layers, maj, linestyle=":",  linewidth=2, label="Majority", alpha=0.7)

    ax.set_title(f"{task.upper()} Probing Accuracy (control={control})", fontsize=13, fontweight='bold')
    ax.set_xlabel("Layer", fontsize=11)
    ax.set_ylabel("Accuracy", fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=10)

    # Macro-F1
    ax = axes[1]
    for ft in feature_types:
        m = np.array([x[0] for x in results[task][control][ft]["f1"]], dtype=float)
        s = np.array([x[1] for x in results[task][control][ft]["f1"]], dtype=float)
        ax.plot(layers, m, marker="o", linewidth=2, label=f"{ft}", markersize=4)
        ax.fill_between(layers, m-s, m+s, alpha=0.15)

    ax.set_title(f"{task.upper()} Macro-F1 (control={control})", fontsize=13, fontweight='bold')
    ax.set_xlabel("Layer", fontsize=11)
    ax.set_ylabel("Macro-F1", fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=10)

    plt.tight_layout()
    plt.savefig(f'probing_{task}_{control}.png', dpi=150, bbox_inches='tight')
    plt.show()

# ===== MAIN =====
set_all_seeds(42)

print("Loading model...")
model, cfg, _, ckpt = load_model_and_cfg(CKPT_PATH)

tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
extractor = ASALayerFeatureExtractor(model, tokenizer, DEVICE)

print("\nBuilding labeled sentences from WikiText with spaCy...")
print("(This may take a few minutes on first run...)")
examples = build_spacy_labeled_sentences(max_sents=1000, text_source="wikitext", seed=42)
print(f"✓ Prepared {len(examples)} labeled sentences.")

feature_types = ("content", "key", "routing")
tasks = ("upos", "depth")
controls = ("none", "nonce")  # Skip shuffle for speed
seeds = (1, 2, 3)  # Reduced for speed

print("\n" + "="*70)
print("PHASE 1: Caching Features (this is the slow part)")
print("="*70)
feature_cache = cache_all_features(examples, extractor, feature_types, tasks, controls)
print("✓ Feature caching complete!")

print("\n" + "="*70)
print("PHASE 2: Running Probes (this is fast!)")
print("="*70)
results = run_all_layers_fast(examples, feature_cache, feature_types, tasks, controls, seeds)
print("✓ All probes complete!")

print("\n" + "="*70)
print("Generating plots...")
print("="*70)
for task in tasks:
    for control in controls:
        plot_task(results, task, control, feature_types)

print("\n✓ All done! Check the generated plots.")

In [ ]:

#@title Experiment 3: Slot Intervention Experiments

import torch
import torch.nn.functional as F
from collections import defaultdict

# ============================
# Intervention Functions
# ============================

def mask_slots(model, slot_indices, layer_idx=6):
    """
    Mask specific slots at a specific layer
    Returns a hook that will be registered
    """
    def hook(module, input, output):
        # output is (out, info)
        out, info = output

        # Create slot mask
        B, H, T, K = info["read_weights"].shape
        mask = torch.ones((K,), device=out.device)
        mask[slot_indices] = 0.0

        # Apply mask to read weights
        masked_weights = info["read_weights"] * mask.view(1, 1, 1, K)
        masked_weights = masked_weights / masked_weights.sum(dim=-1, keepdim=True).clamp_min(1e-8)

        # Re-run the attention with masked weights
        # (This is simplified - you'd need to access slot states)

        return out, info

    # Register hook on specific layer
    handle = model.blocks[layer_idx].asa.register_forward_hook(hook)
    return handle

def intervene_and_evaluate(model, tokenizer, test_sentences, intervention_fn, device):
    """
    Apply intervention and measure effect on predictions
    """
    results = []

    model.eval()
    with torch.no_grad():
        for sent in tqdm(test_sentences, desc="Evaluating intervention"):
            tokens = tokenizer(sent, return_tensors="pt").to(device)
            input_ids = tokens["input_ids"]

            if input_ids.size(1) < 2:
                continue

            # Baseline prediction
            logits_base = model(input_ids)
            probs_base = F.softmax(logits_base[0, -1], dim=-1)
            top_base = torch.topk(probs_base, k=5)

            # Intervened prediction
            handle = intervention_fn(model)
            logits_int = model(input_ids)
            handle.remove()

            probs_int = F.softmax(logits_int[0, -1], dim=-1)
            top_int = torch.topk(probs_int, k=5)

            # Measure change
            kl_div = F.kl_div(
                torch.log(probs_int + 1e-8),
                probs_base,
                reduction='sum'
            ).item()

            results.append({
                "sentence": sent,
                "kl_divergence": kl_div,
                "top_tokens_base": [tokenizer.decode([t]) for t in top_base.indices],
                "top_tokens_int": [tokenizer.decode([t]) for t in top_int.indices],
                "prob_shift": (probs_base[top_base.indices[0]] - probs_int[top_base.indices[0]]).item()
            })

    return results

# ============================
# Specific Intervention Tests
# ============================

# Load model
print("Loading model...")
model, cfg, _, ckpt = load_model_and_cfg(CKPT_PATH)
tokenizer = ckpt["tokenizer"]

test_sentences = [
    "The capital of France is",
    "The quick brown fox jumps over the",
    "In the beginning there was",
    "She walked to the",
    "The president of the United States is",
    "Two plus two equals",
    "The color of the sky is",
    "The largest planet in our solar system is",
    "Water boils at",
    "The opposite of hot is",
]

# Test 1: Mask high-usage slots
print("\n" + "="*60)
print("INTERVENTION 1: Masking Top-Used Slots")
print("="*60)

# First, identify which slots are used most
slot_usage = defaultdict(float)

model.eval()
with torch.no_grad():
    for sent in test_sentences[:5]:
        tokens = tokenizer(sent, return_tensors="pt").to(DEVICE)
        logits, infos = model(tokens["input_ids"], return_info=True)

        # Aggregate read weights from middle layer
        read_w = infos[6]["read_weights"]  # [B, H, T, K]
        usage = read_w.mean(dim=(0, 1, 2))  # [K]

        for k in range(usage.size(0)):
            slot_usage[k] += usage[k].item()

# Sort by usage
sorted_slots = sorted(slot_usage.items(), key=lambda x: x[1], reverse=True)
print("Top 5 most-used slots:")
for slot_idx, usage in sorted_slots[:5]:
    print(f"  Slot {slot_idx}: {usage:.4f}")

top_slots = [s[0] for s in sorted_slots[:3]]

# Intervention: mask top slots
intervention_top = lambda m: mask_slots(m, top_slots, layer_idx=6)
results_top = intervene_and_evaluate(model, tokenizer, test_sentences, intervention_top, DEVICE)

print("\nEffect of masking top-used slots:")
print(f"Average KL divergence: {np.mean([r['kl_divergence'] for r in results_top]):.4f}")

# Test 2: Mask bottom-used slots (should have less effect)
print("\n" + "="*60)
print("INTERVENTION 2: Masking Least-Used Slots")
print("="*60)

bottom_slots = [s[0] for s in sorted_slots[-3:]]
intervention_bottom = lambda m: mask_slots(m, bottom_slots, layer_idx=6)
results_bottom = intervene_and_evaluate(model, tokenizer, test_sentences, intervention_bottom, DEVICE)

print("\nEffect of masking least-used slots:")
print(f"Average KL divergence: {np.mean([r['kl_divergence'] for r in results_bottom]):.4f}")

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: KL divergence comparison
ax = axes[0, 0]
conditions = ['Baseline\n(no masking)', 'Mask Top-3\nSlots', 'Mask Bottom-3\nSlots']
kls = [
    0,
    np.mean([r['kl_divergence'] for r in results_top]),
    np.mean([r['kl_divergence'] for r in results_bottom])
]
bars = ax.bar(conditions, kls, color=['green', 'red', 'blue'], alpha=0.7)
ax.set_ylabel('KL Divergence')
ax.set_title('Effect of Slot Masking on Predictions')
ax.grid(True, alpha=0.3, axis='y')

for bar, kl in zip(bars, kls):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{kl:.3f}',
            ha='center', va='bottom')

# Plot 2: Per-sentence effects
ax = axes[0, 1]
sent_names = [f"S{i+1}" for i in range(len(test_sentences))]
kls_top = [r['kl_divergence'] for r in results_top]
kls_bottom = [r['kl_divergence'] for r in results_bottom]

x = np.arange(len(sent_names))
width = 0.35

ax.bar(x - width/2, kls_top, width, label='Mask Top Slots', color='red', alpha=0.7)
ax.bar(x + width/2, kls_bottom, width, label='Mask Bottom Slots', color='blue', alpha=0.7)

ax.set_xlabel('Sentence')
ax.set_ylabel('KL Divergence')
ax.set_title('Intervention Effect Per Sentence')
ax.set_xticks(x)
ax.set_xticklabels(sent_names, rotation=45)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 3: Example predictions
ax = axes[1, 0]
ax.axis('off')
example_idx = 0
example = results_top[example_idx]

text = f"Sentence: '{example['sentence']}'\n\n"
text += "Top predictions (baseline):\n"
for i, tok in enumerate(example['top_tokens_base'][:3]):
    text += f"  {i+1}. {tok}\n"
text += "\nTop predictions (masked top slots):\n"
for i, tok in enumerate(example['top_tokens_int'][:3]):
    text += f"  {i+1}. {tok}\n"
text += f"\nKL divergence: {example['kl_divergence']:.4f}"

ax.text(0.1, 0.5, text, fontsize=11, family='monospace',
        verticalalignment='center')

# Plot 4: Slot usage distribution
ax = axes[1, 1]
slot_ids = list(range(len(sorted_slots)))
usages = [sorted_slots[i][1] for i in range(len(sorted_slots))]

bars = ax.bar(slot_ids, usages, color='steelblue', alpha=0.7)

# Highlight intervened slots
for slot_idx in top_slots:
    bars[slot_idx].set_color('red')
    bars[slot_idx].set_alpha(1.0)

for slot_idx in bottom_slots:
    bars[slot_idx].set_color('blue')
    bars[slot_idx].set_alpha(1.0)

ax.set_xlabel('Slot Index')
ax.set_ylabel('Average Usage')
ax.set_title('Slot Usage (Red=Top, Blue=Bottom)')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('intervention_results.png', dpi=150)
plt.show()

print("\n" + "="*60)
print("INTERVENTION SUMMARY")
print("="*60)
print(f"Masking top-used slots causes {np.mean([r['kl_divergence'] for r in results_top]):.3f} KL divergence")
print(f"Masking bottom-used slots causes {np.mean([r['kl_divergence'] for r in results_bottom]):.3f} KL divergence")
print(f"\nRatio: {np.mean([r['kl_divergence'] for r in results_top]) / np.mean([r['kl_divergence'] for r in results_bottom]):.2f}x more effect")
print("\nSlots are causally important! ✓")

In [ ]:

#@title Run WikiText-103 Benchmark on Multiple Models
import torch
from datasets import load_dataset
from tqdm.auto import tqdm
import math

# ============================
# Configuration
# ============================
BATCH_SIZE = 8
STRIDE = 512

class TestResults:
    def __init__(self, cfg, avg_loss, ppl, num_params):
        self.cfg = cfg
        self.avg_loss = avg_loss
        self.ppl = ppl
        self.num_params = num_params

    def __repr__(self):
        return f"TestResults(ppl={self.ppl:.2f}, loss={self.avg_loss:.4f}, params={self.num_params/1e6:.1f}M)"

CKPT_PATHS = [
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_32sd_16s_35l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_256d_16a_11l_75ksteps/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_16a_15l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_32a_5l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_3072t_384d_32sd_16s_7l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_640d_64sd_24s_12l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_16sd_8s_21l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_768d_32sd_32cs_17l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_640d_64sd_24s_17l/best.pt",
]

# ============================
# Load and tokenize dataset ONCE
# ============================
print("="*60)
print("Loading WikiText-103 test set...")
print("="*60)
dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="test")
print(f"Number of examples: {len(dataset)}")

# Get tokenizer from first checkpoint
print("\nLoading tokenizer from first checkpoint...")
first_ckpt = torch.load(CKPT_PATHS[0], map_location="cpu")
if first_ckpt["cfg"].get("tokenizer_name", None) is not None:
    tokenizer = AutoTokenizer.from_pretrained(ckpt["cfg"].get("tokenizer_name", "gpt2"), use_fast=True)
print(f"✓ Loaded tokenizer (vocab size: {len(tokenizer)})")

# Concatenate ALL text first, then tokenize
print("\nConcatenating all text...")
full_text = "\n".join(dataset["text"])
print(f"Total characters: {len(full_text):,}")

print("Tokenizing entire dataset...")
all_tokens = torch.tensor(tokenizer.encode(full_text), dtype=torch.long)
print(f"Total tokens: {len(all_tokens):,}")
print(f"Expected: ~3.4M tokens")

if len(all_tokens) < 3_000_000:
    print("⚠ Warning: Token count seems low. Expected ~3.4M tokens.")
else:
    print("✓ Token count looks good!")

# ============================
# Evaluation function
# ============================
@torch.no_grad()
def evaluate_perplexity(model, tokens, batch_size=BATCH_SIZE, stride=STRIDE):
    model.eval()
    max_length = model.cfg.max_seq_len if hasattr(model, 'cfg') else 1024

    total_loss = 0.0
    total_tokens = 0

    # Create sliding windows
    sequences = []
    for i in range(0, len(tokens) - 1, stride):
        end_idx = min(i + max_length, len(tokens))
        if end_idx - i > 1:
            sequences.append(tokens[i:end_idx])

    print(f"  Created {len(sequences)} sequences (stride={stride}, max_len={max_length})")

    # Process in batches
    for batch_start in tqdm(range(0, len(sequences), batch_size), desc="  Evaluating", leave=False):
        batch_seqs = sequences[batch_start:batch_start + batch_size]
        max_len_in_batch = max(len(s) for s in batch_seqs)

        batch_inputs, batch_targets, batch_masks = [], [], []

        for seq in batch_seqs:
            input_seq = seq[:-1]
            target_seq = seq[1:]
            pad_len = max_len_in_batch - 1 - len(input_seq)

            if pad_len > 0:
                input_seq = torch.cat([input_seq, torch.zeros(pad_len, dtype=torch.long)])
                target_seq = torch.cat([target_seq, torch.zeros(pad_len, dtype=torch.long)])
                mask = torch.cat([torch.ones(len(seq) - 1), torch.zeros(pad_len)])
            else:
                mask = torch.ones(len(input_seq))

            batch_inputs.append(input_seq)
            batch_targets.append(target_seq)
            batch_masks.append(mask)

        inputs = torch.stack(batch_inputs).to(DEVICE)
        targets = torch.stack(batch_targets).to(DEVICE)
        masks = torch.stack(batch_masks).to(DEVICE)

        logits = model(inputs)
        loss = F.cross_entropy(
            logits.reshape(-1, logits.size(-1)),
            targets.reshape(-1),
            reduction='none'
        )
        loss = loss.view(targets.shape)

        total_loss += (loss * masks).sum().item()
        total_tokens += masks.sum().item()

    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    return perplexity, avg_loss

# ============================
# Evaluate all models
# ============================
print("\n" + "="*60)
print("EVALUATING MODELS")
print("="*60)

results = []

for i, CKPT_PATH in enumerate(CKPT_PATHS, 1):
    print(f"\n[{i}/{len(CKPT_PATHS)}] Loading: {CKPT_PATH.split('/')[-2]}")

    try:
        model, cfg, cfg_dict, ckpt = load_model_and_cfg(CKPT_PATH)
        print_cfg_summary(cfg, model)

        print("  Running evaluation...")
        perplexity, avg_loss = evaluate_perplexity(model, all_tokens)

        num_params, _ = count_params(model)
        results.append(TestResults(cfg, avg_loss, perplexity, num_params))

        print(f"  ✓ Perplexity: {perplexity:.2f} | Loss: {avg_loss:.4f}")

        # Free memory
        del model, cfg, cfg_dict, ckpt
        torch.cuda.empty_cache()

    except Exception as e:
        print(f"  ✗ Error: {e}")
        continue

# ============================
# Summary results
# ============================
print("\n" + "="*60)
print("SUMMARY RESULTS")
print("="*60)

# Sort by perplexity
results_sorted = sorted(results, key=lambda x: x.ppl)

print(f"\n{'Rank':<6}{'PPL':<10}{'Loss':<10}{'Params':<12}{'Layers':<8}{'d_model':<10}{'Slots':<8}")
print("-"*70)

for rank, result in enumerate(results_sorted, 1):
    cfg_sum = cfg_summary(result.cfg)
    print(f"{rank:<6}{result.ppl:<10.2f}{result.avg_loss:<10.4f}{result.num_params/1e6:<12.1f}"
          f"{cfg_sum['layers']:<8}{cfg_sum['d_model']:<10}{cfg_sum['slots']:<8}")

print("\n" + "="*60)
print(f"Best model: PPL = {results_sorted[0].ppl:.2f}")
print("="*60)

In [ ]:



class TestResults:
    def __init__(cfg, avg_loss, ppl):
        self.cfg = cfg
        self.avg_loss = avg_loss
        self.ppl = ppl


CKPT_PATHS = [
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_32sd_16s_35l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_256d_16a_11l_75ksteps/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_16a_15l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_32a_5l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_3072t_384d_32sd_16s_7l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_640d_64sd_24s_12l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_384d_16sd_8s_21l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_768d_32sd_32cs_17l/best.pt",
    "./drive/MyDrive/asm_outputs/asm_wikitext_1024t_640d_64sd_24s_17l/best.pt",
]

results = []

for CKPT_PATH in CKPT_PATHS:
    print("loading next model...")
    model, cfg, cfg_dict, ckpt = load_model_and_cfg(CKPT_PATH)
    print_cfg_summary(cfg, model)

    print("testing model...")
    perplexity, avg_loss = evaluate_perplexity(model, all_tokens)
    results.append(TestResults(cfg, avg_loss, perplexity))
    print(f"Perplexity: {perplexity:.2f}")

print(results)

In [ ]:

#@title Batch Data Generation

import os, random, pickle
from typing import List, Tuple, Optional, Iterator, Dict

import torch
from torch.utils.data import Dataset, IterableDataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer


# -------------------------
# Minimal dataset classes (same as training)
# -------------------------
class StableValidationDataset(Dataset):
    def __init__(self, samples: List[Tuple[torch.Tensor, torch.Tensor]]):
        self.samples = samples
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx]

def _ensure_dir(path: str):
    d = os.path.dirname(path)
    if d:
        os.makedirs(d, exist_ok=True)

def build_or_load_token_stream(
    *,
    cache_path: str,
    dataset_name: str,
    dataset_config: str,
    split: str,
    tokenizer_name: str,
    min_chars: int = 1,
    add_eos_between_rows: bool = True,
    max_rows: Optional[int] = None,
) -> List[int]:
    if os.path.exists(cache_path):
        with open(cache_path, "rb") as f:
            stream = pickle.load(f)
        return stream

    _ensure_dir(cache_path)
    tok = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
    eos = tok.eos_token_id
    assert eos is not None, "Tokenizer must have eos_token_id"

    ds = load_dataset(dataset_name, dataset_config, split=split)

    stream: List[int] = []
    used = 0
    for row in ds:
        if max_rows is not None and used >= max_rows:
            break
        text = (row.get("text") or "").strip()
        if len(text) < min_chars:
            continue
        ids = tok.encode(text, add_special_tokens=False)
        if not ids:
            continue
        stream.extend(ids)
        if add_eos_between_rows:
            stream.append(eos)
        used += 1

    with open(cache_path, "wb") as f:
        pickle.dump(stream, f)
    return stream

def build_or_load_validation_windows(
    *,
    cache_path: str,
    token_stream: List[int],
    max_seq_len: int,
    stride_frac: float,
    val_samples_target: int,
) -> StableValidationDataset:
    if os.path.exists(cache_path):
        with open(cache_path, "rb") as f:
            samples = pickle.load(f)
        return StableValidationDataset(samples)

    _ensure_dir(cache_path)

    T = int(max_seq_len)
    stride = max(1, int(T * float(stride_frac)))
    need = int(val_samples_target)

    max_start = len(token_stream) - (T + 1)
    if max_start <= 0:
        raise ValueError("Validation token stream too small for max_seq_len+1")

    samples: List[Tuple[torch.Tensor, torch.Tensor]] = []
    for start in range(0, max_start + 1, stride):
        chunk = token_stream[start:start + T + 1]
        if len(chunk) < T + 1:
            break
        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        samples.append((x, y))
        if len(samples) >= need:
            break

    with open(cache_path, "wb") as f:
        pickle.dump(samples, f)
    return StableValidationDataset(samples)

class WikiTextRandomWindowStream(IterableDataset):
    def __init__(self, token_stream: List[int], max_seq_len: int, train_samples_target: int, seed: int):
        super().__init__()
        self.stream = token_stream
        self.T = int(max_seq_len)
        self.target = int(train_samples_target)
        self.seed = int(seed)
        self.max_start = len(self.stream) - (self.T + 1)
        if self.max_start <= 0:
            raise ValueError("Train token stream too small for max_seq_len+1")

    def __iter__(self):
        info = torch.utils.data.get_worker_info()
        wid = info.id if info is not None else 0
        rng = random.Random(self.seed + 17 * wid)

        yielded = 0
        while yielded < self.target:
            start = rng.randint(0, self.max_start)
            chunk = self.stream[start:start + self.T + 1]
            x = torch.tensor(chunk[:-1], dtype=torch.long)
            y = torch.tensor(chunk[1:], dtype=torch.long)
            yield x, y
            yielded += 1


# -------------------------
# Analysis batch generator
# -------------------------
def make_batch_generator(
    cfg,
    *,
    split: str = "val",                 # "val" or "train"
    device: Optional[torch.device] = None,
    seq_len: int = 0,
    num_workers: int = 0,               # 0 is safest for Colab analysis
    pin_memory: bool = True,
    batches_per_epoch: Optional[int] = None,  # if set, generator stops after N batches
    infinite: bool = True,              # if True, cycles (val) or keeps sampling (train)
    seed: Optional[int] = None,         # override cfg.seed for sampling
) -> Iterator[Tuple[torch.Tensor, torch.Tensor]]:
    """
    Yields xb,yb batches shaped [B,T] int64.

    - split="val": deterministic stable windows (cached) then cycles if infinite=True
    - split="train": random windows from cached token stream; infinite is naturally True
    """
    assert split in ("val", "train")
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Resolve cfg fields with fallback (works for dict-like or dataclass)
    def get(name, default=None):
        return getattr(cfg, name, cfg.get(name, default) if isinstance(cfg, dict) else default)

    cache_dir = get("cache_dir")
    dataset_name = get("dataset_name")
    dataset_config = get("dataset_config")
    tokenizer_name = get("tokenizer_name")
    max_seq_len = seq_len if seq_len > 0 else int(get("max_seq_len"))
    batch_size = int(get("batch_size"))
    stride_frac_val = float(get("stride_frac_val", 0.5))
    val_samples_target = int(get("val_samples_target", 25_000))
    val_windows_cache = get("val_windows_cache")
    train_samples_target = int(get("train_samples_target", 100_000_000))
    used_seed = int(seed if seed is not None else get("seed", 1337))

    os.makedirs(cache_dir, exist_ok=True)
    train_stream_cache = os.path.join(cache_dir, f"{dataset_config}_train_stream.pkl")
    val_stream_cache   = os.path.join(cache_dir, f"{dataset_config}_val_stream.pkl")

    if split == "train":
        train_stream = build_or_load_token_stream(
            cache_path=train_stream_cache,
            dataset_name=dataset_name,
            dataset_config=dataset_config,
            split="train",
            tokenizer_name=tokenizer_name,
            min_chars=1,
            add_eos_between_rows=True,
        )
        ds = WikiTextRandomWindowStream(
            token_stream=train_stream,
            max_seq_len=max_seq_len,
            train_samples_target=train_samples_target,
            seed=used_seed,
        )
        loader = DataLoader(
            ds,
            batch_size=batch_size,
            num_workers=num_workers,
            pin_memory=pin_memory and torch.cuda.is_available(),
        )

        yielded = 0
        for xb, yb in loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            yield xb, yb
            yielded += 1
            if batches_per_epoch is not None and yielded >= batches_per_epoch:
                if infinite:
                    yielded = 0
                    continue
                break

    else:
        # val
        val_stream = build_or_load_token_stream(
            cache_path=val_stream_cache,
            dataset_name=dataset_name,
            dataset_config=dataset_config,
            split="validation",
            tokenizer_name=tokenizer_name,
            min_chars=1,
            add_eos_between_rows=True,
        )
        val_ds = build_or_load_validation_windows(
            cache_path=val_windows_cache,
            token_stream=val_stream,
            max_seq_len=max_seq_len,
            stride_frac=stride_frac_val,
            val_samples_target=val_samples_target,
        )
        loader = DataLoader(
            val_ds,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=pin_memory and torch.cuda.is_available(),
        )

        while True:
            yielded = 0
            for xb, yb in loader:
                xb = xb.to(device, non_blocking=True)
                yb = yb.to(device, non_blocking=True)
                yield xb, yb
                yielded += 1
                if batches_per_epoch is not None and yielded >= batches_per_epoch:
                    break
            if not infinite:
                break


# -------------------------
# Quick smoke test
# -------------------------

gen = make_batch_generator(cfg, batches_per_epoch=50, split="val", device=torch.device("cuda"))
xb, yb = next(gen)
print(xb.shape, yb.shape, xb.device, xb.dtype)


#

In [ ]:

#@title Forward Pass, Collect Out Infos

@torch.no_grad()
def run_with_infos(model, xb: torch.Tensor, attention_mask=None):
    out = model(xb, return_info=True, attention_mask=attention_mask)
    if isinstance(out, tuple) and len(out) == 2:
        logits, infos = out
    else:
        raise ValueError("Model did not return (logits, infos) under return_info=True")
    return logits, infos

logits, infos = run_with_infos(model, xb[:4])
print("logits:", logits.shape, logits.dtype)
print("num layers infos:", len(infos))
print("info keys example:", list(infos[0].keys()))
print("read_weights shape:", infos[0]["read_weights"].shape)


#

In [ ]:
#@title crafted gen
import math
from dataclasses import dataclass
from typing import Any, Dict, Optional, Tuple, Union, List

import torch
import torch.nn.functional as F


# -----------------------------------------------------------------------------
# Helpers
# -----------------------------------------------------------------------------
def _top_k_top_p_filtering(
    logits: torch.Tensor,
    top_k: int = 0,
    top_p: float = 1.0,
    min_tokens_to_keep: int = 1,
) -> torch.Tensor:
    """
    Filter a distribution of logits using top-k and/or nucleus (top-p).
    logits: [V]
    """
    if top_k > 0:
        top_k = min(max(top_k, min_tokens_to_keep), logits.size(-1))
        kth = torch.topk(logits, top_k).values[-1]
        logits = logits.masked_fill(logits < kth, float("-inf"))

    if top_p < 1.0:
        sorted_logits, sorted_idx = torch.sort(logits, descending=True)
        probs = F.softmax(sorted_logits, dim=-1)
        cumprobs = probs.cumsum(dim=-1)

        # Remove tokens with cumulative prob above threshold
        cutoff = cumprobs > top_p
        # Keep at least min_tokens_to_keep
        cutoff[:min_tokens_to_keep] = False

        sorted_logits = sorted_logits.masked_fill(cutoff, float("-inf"))
        logits = logits.scatter(0, sorted_idx, sorted_logits)

    return logits


def _apply_repetition_penalty(
    logits: torch.Tensor,
    generated_ids: torch.Tensor,
    penalty: float,
) -> torch.Tensor:
    """
    Classic repetition penalty (GPT-2 style): penalize logits of previously generated tokens.
    logits: [V], generated_ids: [t]
    """
    if penalty is None or penalty == 1.0 or generated_ids.numel() == 0:
        return logits
    uniq = torch.unique(generated_ids)
    # If logit > 0: divide by penalty; else multiply by penalty
    l = logits[uniq]
    logits[uniq] = torch.where(l > 0, l / penalty, l * penalty)
    return logits


def _no_repeat_ngram_ban(
    logits: torch.Tensor,
    generated_ids: torch.Tensor,
    no_repeat_ngram_size: int,
) -> torch.Tensor:
    """
    Ban tokens that would create a repeated n-gram of size N in the generated sequence.
    logits: [V], generated_ids: [t]
    """
    n = int(no_repeat_ngram_size or 0)
    if n <= 1 or generated_ids.numel() < n - 1:
        return logits

    seq = generated_ids.tolist()
    prefix = seq[-(n - 1):]  # length n-1
    # Build set of next tokens seen after this prefix in the past
    banned = set()
    for i in range(len(seq) - n + 1):
        if seq[i:i + n - 1] == prefix:
            banned.add(seq[i + n - 1])

    if banned:
        banned = torch.tensor(list(banned), device=logits.device, dtype=torch.long)
        logits[banned] = float("-inf")
    return logits


# -----------------------------------------------------------------------------
# ASA/ASM-specific generation
# -----------------------------------------------------------------------------
@torch.no_grad()
def asa_generate(
    prompt: Union[str, List[int], torch.Tensor],
    model: torch.nn.Module,
    gen: Dict[str, Any],
) -> Dict[str, Any]:
    """
    Generation crafted for ASA/ASM models:
      - Uses soft sampling by default (hard routing variants are unstable per your ablations).
      - Optionally uses ASA internal telemetry (return_info=True, return_light_stats=True)
        to perform *router-aware fallback* when EOS-risk is high early, or when routing is
        pathologically branchy.
      - Supports standard sampling controls + mild anti-repetition.
      - Keeps inference dropout off.

    Args
    ----
    prompt:
        - str: requires gen["tokenizer"] providing encode/decode
        - List[int] / 1D torch.Tensor: token ids
    model:
        ASMLanguageModel (or compatible) returning logits or (logits, infos) if return_info=True
    gen params (dict):
        Required (if prompt is str):
          tokenizer: a HF tokenizer with encode/decode
        Common:
          max_new_tokens: int (default 128)
          temperature: float (default 0.8)
          top_p: float (default 0.9)
          top_k: int (default 50)
          min_new_tokens: int (default 0)
          eos_token_id: int (default tokenizer.eos_token_id if available)
          pad_token_id: int (optional)
          do_sample: bool (default True)
          repetition_penalty: float (default 1.05)
          no_repeat_ngram_size: int (default 3)
          device: torch.device or str (default model device)
        ASA-aware controls:
          asa_info: bool (default True) -> request return_info+light_stats and use them
          eos_risk_threshold: float (default 0.25)
          early_steps: int (default 24) -> window in which to apply EOS-risk mitigations
          branchy_entropy_threshold: float (default None) -> if set, triggers extra sharpening
          rescue_mode: str in {"none","scaffold","resample"} (default "resample")
              - "resample": if EOS risk triggers, resample with lower temp / higher top_k keep
              - "scaffold": if tokenizer provided and prompt looks like a known template,
                            inject a short scaffold (see below) once at the start
          rescue_temp: float (default 0.65)
          rescue_top_p: float (default 0.85)
          rescue_top_k: int (default 80)
          max_resample_tries: int (default 4)
        Return:
          return_text: bool (default True if tokenizer present else False)

    Returns
    -------
    dict with:
      "input_ids": [1, T+new]
      "generated_ids": [new]
      "text": optional
      "info_trace": optional list of per-step ASA stats (if asa_info=True)
    """
    model.eval()

    tokenizer = gen.get("tokenizer", None)
    device = gen.get("device", None)
    if device is None:
        device = next(model.parameters()).device

    # --- tokenize prompt ---
    if isinstance(prompt, str):
        if tokenizer is None:
            raise ValueError("prompt is str but gen['tokenizer'] was not provided.")
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    elif isinstance(prompt, list):
        input_ids = torch.tensor(prompt, device=device, dtype=torch.long).unsqueeze(0)
    elif isinstance(prompt, torch.Tensor):
        if prompt.dim() == 1:
            input_ids = prompt.to(device=device, dtype=torch.long).unsqueeze(0)
        elif prompt.dim() == 2:
            input_ids = prompt.to(device=device, dtype=torch.long)
        else:
            raise ValueError("prompt tensor must be 1D or 2D token ids.")
    else:
        raise TypeError("prompt must be str, List[int], or torch.Tensor of token ids.")

    max_new = int(gen.get("max_new_tokens", 128))
    min_new = int(gen.get("min_new_tokens", 0))
    do_sample = bool(gen.get("do_sample", True))

    temperature = float(gen.get("temperature", 0.8))
    top_p = float(gen.get("top_p", 0.9))
    top_k = int(gen.get("top_k", 50))

    repetition_penalty = float(gen.get("repetition_penalty", 1.05))
    no_repeat_ngram_size = int(gen.get("no_repeat_ngram_size", 3))

    eos_token_id = gen.get("eos_token_id", None)
    if eos_token_id is None and tokenizer is not None:
        eos_token_id = tokenizer.eos_token_id
    if eos_token_id is None:
        eos_token_id = -1  # disable EOS logic if unknown

    asa_info = bool(gen.get("asa_info", True))
    eos_risk_threshold = float(gen.get("eos_risk_threshold", 0.25))
    early_steps = int(gen.get("early_steps", 24))
    branchy_entropy_threshold = gen.get("branchy_entropy_threshold", None)
    rescue_mode = str(gen.get("rescue_mode", "resample")).lower()
    rescue_temp = float(gen.get("rescue_temp", 0.65))
    rescue_top_p = float(gen.get("rescue_top_p", 0.85))
    rescue_top_k = int(gen.get("rescue_top_k", 80))
    max_resample_tries = int(gen.get("max_resample_tries", 4))

    # Optional scaffold injection (architecture-aware: helps route trajectory)
    if rescue_mode == "scaffold" and tokenizer is not None and isinstance(prompt, str):
        # Very small, conservative scaffold set—extend as you like
        scaffolds = [
            ("The capital of", " the city of"),
            ("Albert Einstein was born", " in"),
            ("The scientific method involves", " the process of"),
            ("The algorithm proceeds as follows", " 1."),
        ]
        for k, s in scaffolds:
            if prompt.strip().startswith(k) and not prompt.strip().endswith(s.strip()):
                input_ids = tokenizer.encode(prompt + s, return_tensors="pt").to(device)
                break

    info_trace: List[Dict[str, float]] = []

    # Generation loop
    cur_ids = input_ids
    for step in range(max_new):
        # Model forward
        if asa_info:
            out = model(cur_ids, return_info=True, return_light_stats=True)
            logits, infos = out
            # infos is list per layer; take last block's light stats if present
            last = infos[-1] if isinstance(infos, list) and len(infos) > 0 else None
            stat = {}
            if isinstance(last, dict):
                # these are CPU tensors in your module; cast to float if present
                for k in ["entropy_mean", "top1freq_mean", "content_read_gamma_mean", "slotspace_gate_mean", "slotspace_delta_norm"]:
                    if k in last and last[k] is not None:
                        try:
                            stat[k] = float(last[k].item())
                        except Exception:
                            pass
            # Store later for debugging
        else:
            logits = model(cur_ids)
            stat = None

        next_logits = logits[0, -1, :]  # [V]

        # Basic constraints
        if step < min_new and eos_token_id >= 0:
            next_logits = next_logits.clone()
            next_logits[eos_token_id] = float("-inf")

        # Anti-repetition (mild, usually good for ASA because content-read is self-referential)
        gen_so_far = cur_ids[0, input_ids.shape[1]:]  # only newly generated, if any
        next_logits = _apply_repetition_penalty(next_logits, gen_so_far, repetition_penalty)
        next_logits = _no_repeat_ngram_ban(next_logits, cur_ids[0], no_repeat_ngram_size)

        # Router-aware rescue (early EOS / excessive branchiness)
        # Use next-token EOS risk; optionally sharpen if branchy.
        tries = 0
        used_temp, used_top_p, used_top_k = temperature, top_p, top_k
        while True:
            l = next_logits
            if used_temp and used_temp > 0:
                l = l / used_temp

            l = _top_k_top_p_filtering(l, top_k=used_top_k, top_p=used_top_p)

            probs = F.softmax(l, dim=-1)
            p_eos = float(probs[eos_token_id].item()) if eos_token_id >= 0 else 0.0
            ent = float(-(probs.clamp_min(1e-12) * probs.clamp_min(1e-12).log()).sum().item())

            # Condition: early EOS risk is too high
            eos_risky = (eos_token_id >= 0) and (step < early_steps) and (p_eos > eos_risk_threshold)

            # Condition: branchy token distribution (optional) -> reduce temperature a bit
            branchy = False
            if branchy_entropy_threshold is not None and step < early_steps:
                branchy = ent > float(branchy_entropy_threshold)

            if (eos_risky or branchy) and rescue_mode == "resample" and tries < max_resample_tries:
                used_temp = min(used_temp, rescue_temp)
                used_top_p = min(used_top_p, rescue_top_p)
                used_top_k = max(used_top_k, rescue_top_k)
                tries += 1
                continue

            # Choose token
            if do_sample:
                next_id = torch.multinomial(probs, num_samples=1)
            else:
                next_id = torch.argmax(probs, dim=-1, keepdim=True)

            break

        # Log trace
        if asa_info:
            rec = {"step": float(step), "token_entropy": float(ent), "p_eos": float(p_eos)}
            if stat:
                for k, v in stat.items():
                    rec[k] = float(v)
            # record rescue adjustments
            rec["temp_used"] = float(used_temp)
            rec["top_p_used"] = float(used_top_p)
            rec["top_k_used"] = float(used_top_k)
            info_trace.append(rec)

        # Append token
        cur_ids = torch.cat([cur_ids, next_id.view(1, 1)], dim=1)

        # Stop on EOS
        if eos_token_id >= 0 and int(next_id.item()) == int(eos_token_id) and step >= min_new:
            break

    generated_ids = cur_ids[:, input_ids.shape[1]:]

    out: Dict[str, Any] = {
        "input_ids": cur_ids,
        "generated_ids": generated_ids,
    }
    if asa_info:
        out["info_trace"] = info_trace

    return_text = bool(gen.get("return_text", tokenizer is not None))
    if return_text and tokenizer is not None:
        out["text"] = tokenizer.decode(cur_ids[0].tolist(), skip_special_tokens=False)

    return out


# =========================
# PATCH 1: wrappers for your crafted asa_generate
# =========================

@torch.no_grad()
def asa_greedy_suffix(
    prompt: str,
    model: torch.nn.Module,
    gen: dict,
    max_new_tokens: int = 8,
    strip: bool = True,
) -> str:
    """
    Runs your asa_generate in greedy mode and returns ONLY the suffix after `prompt`.
    This is what you want for exact-match checks / scoring.
    """
    # Copy gen so we can override safely
    g = dict(gen)
    g["do_sample"] = False
    g["max_new_tokens"] = int(max_new_tokens)

    out = asa_generate(prompt, model, g)
    text = out.get("text", None)
    if text is None:
        # Fallback: decode manually
        tok = g.get("tokenizer", None)
        if tok is None:
            raise ValueError("No decoded text available; provide gen['tokenizer'].")
        text = tok.decode(out["input_ids"][0].tolist(), skip_special_tokens=False)

    # Suffix (best-effort): if prompt string matches prefix of decoded text
    if text.startswith(prompt):
        suf = text[len(prompt):]
    else:
        # Robust fallback: try to locate the prompt inside the decoded text
        idx = text.find(prompt)
        suf = text[idx + len(prompt):] if idx >= 0 else text

    if strip:
        suf = suf.replace("\n", " ").strip()
    return suf


@torch.no_grad()
def asa_generate_many(
    prompts: list,
    model: torch.nn.Module,
    gen: dict,
    do_sample: bool = False,
    max_new_tokens: int = 8,
) -> list:
    """
    Convenience wrapper: runs asa_generate per prompt (loop) and returns decoded texts.
    """
    g = dict(gen)
    g["do_sample"] = bool(do_sample)
    g["max_new_tokens"] = int(max_new_tokens)

    outs = []
    for p in prompts:
        out = asa_generate(p, model, g)
        text = out.get("text", None)
        if text is None:
            tok = g.get("tokenizer", None)
            if tok is None:
                raise ValueError("No decoded text available; provide gen['tokenizer'].")
            text = tok.decode(out["input_ids"][0].tolist(), skip_special_tokens=False)
        outs.append(text)
    return outs

@torch.no_grad()
def score_next_token_rank(
    prompt: str,
    target_token: str,
    model: torch.nn.Module,
    gen: dict,
) -> dict:
    """
    Computes P(target) and rank for the *next token* only, matching your printed diagnostics.
    """
    tok = gen["tokenizer"]
    device = next(model.parameters()).device

    # encode prompt
    input_ids = tok.encode(prompt, return_tensors="pt").to(device)

    # encode target token as a single token (best-effort)
    target_ids = tok.encode(target_token, add_special_tokens=False)
    if len(target_ids) != 1:
        return {"ok": False, "reason": f"target_token maps to {len(target_ids)} tokens", "target_ids": target_ids}

    target_id = target_ids[0]

    model.eval()
    logits = model(input_ids)  # if your model needs return_info=False default
    if isinstance(logits, (tuple, list)):
        logits = logits[0]
    next_logits = logits[0, -1, :]

    probs = torch.softmax(next_logits, dim=-1)
    p_t = float(probs[target_id].item())

    # rank: 1 = best
    sorted_idx = torch.argsort(next_logits, descending=True)
    rank = int((sorted_idx == target_id).nonzero(as_tuple=False).item()) + 1

    return {"ok": True, "p_target": p_t, "rank": rank, "target_id": target_id}

#

In [ ]:

from contextlib import contextmanager

@contextmanager
def asa_temperatures(model, read_temperature=None, write_temperature=None):
    """
    Temporarily override ASA read/write temperatures across all ASA modules.
    Restores original values afterwards.
    """
    # Collect ASA modules (by attribute presence, avoids needing class import)
    asa_mods = []
    for m in model.modules():
        if hasattr(m, "read_temperature") and hasattr(m, "write_temperature") and hasattr(m, "num_slots"):
            # heuristic: ASA-like module
            asa_mods.append(m)

    old = [(m.read_temperature, m.write_temperature) for m in asa_mods]
    try:
        for m in asa_mods:
            if read_temperature is not None:
                m.read_temperature = float(read_temperature)
            if write_temperature is not None:
                m.write_temperature = float(write_temperature)
        yield
    finally:
        for m, (rt, wt) in zip(asa_mods, old):
            m.read_temperature = rt
            m.write_temperature = wt


def sweep_asa_temps(
    model, tokenizer, gener, examples,
    read_grid=(0.7, 0.85, 1.0, 1.15),
    write_grid=(0.8, 1.0, 1.2),
    max_new_tokens=8,
    n_gen_samples=6,
    seed=1337,
):
    rng = random.Random(seed)
    sample = rng.sample(examples, min(n_gen_samples, len(examples)))

    rows = []
    for rt in read_grid:
        for wt in write_grid:
            with asa_temperatures(model, read_temperature=rt, write_temperature=wt):
                acc = eval_exact_match(examples, model, gener, max_new_tokens=max_new_tokens)
                # next-token diagnostic on a tiny subset (optional)
                rows.append((rt, wt, acc))

            print(f"\n=== read_temperature={rt:.2f} | write_temperature={wt:.2f} | exact_match={acc:.3f} ===")
            with asa_temperatures(model, read_temperature=rt, write_temperature=wt):
                for ex in sample:
                    pred = greedy_suffix(ex["prompt"], model, gener, max_new_tokens=max_new_tokens)
                    print(f"- {ex['tag']:<18} prompt={ex['prompt']!r} -> {pred[:60]!r}")

    # sort best first
    rows.sort(key=lambda x: -x[2])
    print("\nTop settings:")
    for rt, wt, acc in rows[:10]:
        print(f"  rt={rt:.2f} wt={wt:.2f} acc={acc:.3f}")

In [ ]:

#@title multigen
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)

gener = dict(
    tokenizer=tokenizer,
    max_new_tokens=32,
    #min_new_tokens=4,
    temperature=0.1,
    top_p=0.95,
    top_k=80,
    repetition_penalty=1.03,
    no_repeat_ngram_size=3, # 3
    asa_info=False,
    rescue_mode=None, # "resample", None
    #eos_risk_threshold=0.25,
    #early_steps=24,
    #branchy_entropy_threshold=7.5,   # optional; depends on vocab size and filtering
)

print("#"*5, "Countries", "#"*5)
finishers = ["is", "sounds like", "consists of", "is a form of", "all changed when"]
qualities = ["capital", "language", "geography", "government", "history"]
countries = ["France", "Spain", "the United States", "Russia", "Italy", "Japan", "Egypt", "Germany", "Brazil", "Iraq"]
for country in countries:
    for quality, finisher in zip(qualities, finishers):
        out = asa_generate(f"The {quality} of {country} {finisher}", model, gener)
        print(out["text"])

print("#"*5, "People", "#"*5)
people = ["Albert Einstein", "George Patton", "Charles Darwin", "George Washington", "Winston Churchill", "Jesus of Nazareth"]
factoids = ["was born", "contributed", "accomplished", "had a strong opinion about", "died"]
for person in people:
    for factoid in factoids:
        out = asa_generate(f"{person} {factoid}", model, gener)
        print(out["text"])


# Optionally inspect router-aware trace:
# out["info_trace"][:5]

In [ ]:

#@title synthetic fine tune data


def is_single_token(s: str, tokenizer) -> bool:
    ids = tokenizer.encode(s, add_special_tokens=False)
    return len(ids) == 1

def build_pairs_expanded(tokenizer):
    """
    Massively expanded dataset generation with WikiText-103 style templates.
    Includes geographical, historical, scientific, cultural, and biographical facts.
    """
    pairs = []

    # ========================================
    # GEOGRAPHY SECTION (Massively Expanded)
    # ========================================

    # ---- Capitals (comprehensive list)
    capitals = {
        # Europe
        "France": " Paris",
        "Germany": " Berlin",
        "Italy": " Rome",
        "Spain": " Madrid",
        "Portugal": " Lisbon",
        "Greece": " Athens",
        "Austria": " Vienna",
        "Poland": " Warsaw",
        "Norway": " Oslo",
        "Sweden": " Stockholm",
        "Finland": " Helsinki",
        "Denmark": " Copenhagen",
        "Ireland": " Dublin",
        "Belgium": " Brussels",
        "Netherlands": " Amsterdam",
        "Switzerland": " Bern",
        "Czech Republic": " Prague",
        "Hungary": " Budapest",
        "Romania": " Bucharest",
        "Bulgaria": " Sofia",
        "Croatia": " Zagreb",
        "Serbia": " Belgrade",
        "Slovakia": " Bratislava",
        "Slovenia": " Ljubljana",
        "Lithuania": " Vilnius",
        "Latvia": " Riga",
        "Estonia": " Tallinn",
        "Iceland": " Reykjavik",
        "Luxembourg": " Luxembourg",
        "Malta": " Valletta",
        "Cyprus": " Nicosia",

        # Asia
        "Japan": " Tokyo",
        "China": " Beijing",
        "India": " Delhi",
        "South Korea": " Seoul",
        "North Korea": " Pyongyang",
        "Thailand": " Bangkok",
        "Vietnam": " Hanoi",
        "Indonesia": " Jakarta",
        "Philippines": " Manila",
        "Malaysia": " Kuala",
        "Singapore": " Singapore",
        "Myanmar": " Naypyidaw",
        "Cambodia": " Phnom",
        "Laos": " Vientiane",
        "Bangladesh": " Dhaka",
        "Pakistan": " Islamabad",
        "Afghanistan": " Kabul",
        "Iran": " Tehran",
        "Iraq": " Baghdad",
        "Saudi Arabia": " Riyadh",
        "Turkey": " Ankara",
        "Israel": " Jerusalem",
        "Jordan": " Amman",
        "Lebanon": " Beirut",
        "Syria": " Damascus",
        "Yemen": " Sanaa",
        "Oman": " Muscat",
        "Kuwait": " Kuwait",
        "Qatar": " Doha",
        "Bahrain": " Manama",
        "United Arab Emirates": " Abu",
        "Nepal": " Kathmandu",
        "Sri Lanka": " Colombo",
        "Mongolia": " Ulaanbaatar",
        "Kazakhstan": " Astana",
        "Uzbekistan": " Tashkent",

        # Africa
        "Egypt": " Cairo",
        "South Africa": " Pretoria",
        "Nigeria": " Abuja",
        "Kenya": " Nairobi",
        "Ethiopia": " Addis",
        "Morocco": " Rabat",
        "Algeria": " Algiers",
        "Tunisia": " Tunis",
        "Libya": " Tripoli",
        "Sudan": " Khartoum",
        "Ghana": " Accra",
        "Tanzania": " Dodoma",
        "Uganda": " Kampala",
        "Angola": " Luanda",
        "Mozambique": " Maputo",
        "Zimbabwe": " Harare",
        "Zambia": " Lusaka",
        "Senegal": " Dakar",
        "Ivory Coast": " Yamoussoukro",
        "Cameroon": " Yaounde",

        # Americas
        "United States": " Washington",
        "Canada": " Ottawa",
        "Mexico": " Mexico",
        "Brazil": " Brasilia",
        "Argentina": " Buenos",
        "Chile": " Santiago",
        "Colombia": " Bogota",
        "Peru": " Lima",
        "Venezuela": " Caracas",
        "Ecuador": " Quito",
        "Bolivia": " La",
        "Paraguay": " Asuncion",
        "Uruguay": " Montevideo",
        "Cuba": " Havana",
        "Jamaica": " Kingston",
        "Costa Rica": " San",
        "Panama": " Panama",
        "Guatemala": " Guatemala",
        "Honduras": " Tegucigalpa",
        "Nicaragua": " Managua",

        # Oceania
        "Australia": " Canberra",
        "New Zealand": " Wellington",
        "Papua New Guinea": " Port",
        "Fiji": " Suva",

        # Former USSR
        "Russia": " Moscow",
        "Ukraine": " Kyiv",
        "Belarus": " Minsk",
        "Georgia": " Tbilisi",
        "Armenia": " Yerevan",
        "Azerbaijan": " Baku",
    }

    for c, cap in capitals.items():
        pairs.append({"prompt": f"The capital of {c} is", "completion": cap, "tag": f"capital:{c}"})
        pairs.append({"prompt": f"{c}'s capital city is", "completion": cap, "tag": f"capital:{c}"})
        pairs.append({"prompt": f"{c} has its capital in", "completion": cap, "tag": f"capital:{c}"})

    # ---- Languages (comprehensive)
    languages = {
        "France": " French",
        "Germany": " German",
        "Italy": " Italian",
        "Japan": " Japanese",
        "Spain": " Spanish",
        "Russia": " Russian",
        "Brazil": " Portuguese",
        "Portugal": " Portuguese",
        "Egypt": " Arabic",
        "China": " Chinese",
        "India": " Hindi",
        "Mexico": " Spanish",
        "Argentina": " Spanish",
        "Netherlands": " Dutch",
        "Greece": " Greek",
        "Poland": " Polish",
        "Turkey": " Turkish",
        "Iran": " Persian",
        "Israel": " Hebrew",
        "Sweden": " Swedish",
        "Norway": " Norwegian",
        "Denmark": " Danish",
        "Finland": " Finnish",
        "Czech Republic": " Czech",
        "Hungary": " Hungarian",
        "Romania": " Romanian",
        "Thailand": " Thai",
        "Vietnam": " Vietnamese",
        "South Korea": " Korean",
    }
    for c, lang in languages.items():
        pairs.append({"prompt": f"The language of {c} is", "completion": lang, "tag": f"language:{c}"})
        pairs.append({"prompt": f"The official language of {c} is", "completion": lang, "tag": f"language:{c}"})
        pairs.append({"prompt": f"People in {c} speak", "completion": lang, "tag": f"language:{c}"})

    # ---- Currencies (comprehensive)
    currencies = {
        "Japan": " yen",
        "Russia": " ruble",
        "India": " rupee",
        "Mexico": " peso",
        "China": " yuan",
        "United Kingdom": " pound",
        "United States": " dollar",
        "Canada": " dollar",
        "Australia": " dollar",
        "Germany": " euro",
        "France": " euro",
        "Italy": " euro",
        "Spain": " euro",
        "Portugal": " euro",
        "Greece": " euro",
        "Austria": " euro",
        "Netherlands": " euro",
        "Belgium": " euro",
        "Poland": " zloty",
        "Czech Republic": " koruna",
        "Sweden": " krona",
        "Norway": " krone",
        "Denmark": " krone",
        "Switzerland": " franc",
        "Brazil": " real",
        "South Africa": " rand",
        "Turkey": " lira",
        "Thailand": " baht",
        "Indonesia": " rupiah",
    }
    for c, cur in currencies.items():
        pairs.append({"prompt": f"The currency of {c} is the", "completion": cur, "tag": f"currency:{c}"})
        pairs.append({"prompt": f"{c} uses the", "completion": cur, "tag": f"currency:{c}"})

    # ---- Continents (expanded with variations)
    continents = {
        "France": " Europe",
        "Germany": " Europe",
        "Italy": " Europe",
        "Spain": " Europe",
        "Poland": " Europe",
        "Greece": " Europe",
        "Sweden": " Europe",
        "Norway": " Europe",
        "Russia": " Europe",
        "Egypt": " Africa",
        "Nigeria": " Africa",
        "Kenya": " Africa",
        "South Africa": " Africa",
        "Morocco": " Africa",
        "Japan": " Asia",
        "China": " Asia",
        "India": " Asia",
        "Thailand": " Asia",
        "Vietnam": " Asia",
        "Indonesia": " Asia",
        "Brazil": " South",
        "Argentina": " South",
        "Chile": " South",
        "Peru": " South",
        "Colombia": " South",
        "Canada": " North",
        "United States": " North",
        "Mexico": " North",
        "Australia": " Oceania",
        "New Zealand": " Oceania",
    }
    for c, cont in continents.items():
        pairs.append({"prompt": f"{c} is in", "completion": cont, "tag": f"continent:{c}"})
        pairs.append({"prompt": f"{c} is located in", "completion": cont, "tag": f"continent:{c}"})

    # ---- Major Rivers
    rivers = {
        "The Nile flows through": " Egypt",
        "The Amazon flows through": " Brazil",
        "The Thames flows through": " London",
        "The Seine flows through": " Paris",
        "The Danube flows through": " Europe",
        "The Rhine flows through": " Germany",
        "The Ganges flows through": " India",
        "The Yangtze flows through": " China",
        "The Mississippi flows through": " America",
        "The Nile is located in": " Africa",
        "The Amazon is in": " South",
        "The Rhine is in": " Europe",
    }
    for p, comp in rivers.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "rivers"})

    # ---- Mountain ranges and peaks
    mountains = {
        "Mount Everest is in": " Nepal",
        "The Alps are in": " Europe",
        "The Himalayas are in": " Asia",
        "The Andes are in": " South",
        "The Rocky Mountains are in": " North",
        "Mount Fuji is in": " Japan",
        "The Pyrenees are between France and": " Spain",
    }
    for p, comp in mountains.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "mountains"})

    # ---- Oceans and seas
    oceans = {
        "The Pacific Ocean is the": " largest",
        "The Atlantic Ocean is the": " second",
        "The Mediterranean Sea is in": " Europe",
        "The Caribbean Sea is in": " Central",
        "The Baltic Sea is in": " Northern",
    }
    for p, comp in oceans.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "oceans"})

    # ========================================
    # HISTORICAL FACTS (Massively Expanded)
    # ========================================

    # ---- Birth locations (expanded)
    born_in = {
        "Albert Einstein was born in": " Germany",
        "Charles Darwin was born in": " England",
        "George Washington was born in": " Virginia",
        "Winston Churchill was born in": " England",
        "Napoleon Bonaparte was born in": " Corsica",
        "Leonardo da Vinci was born in": " Italy",
        "William Shakespeare was born in": " England",
        "Isaac Newton was born in": " England",
        "Marie Curie was born in": " Poland",
        "Galileo Galilei was born in": " Italy",
        "Aristotle was born in": " Greece",
        "Plato was born in": " Athens",
        "Confucius was born in": " China",
        "Buddha was born in": " Nepal",
        "Muhammad Ali was born in": " Kentucky",
        "Martin Luther King was born in": " Georgia",
        "Abraham Lincoln was born in": " Kentucky",
        "Thomas Edison was born in": " Ohio",
        "Nikola Tesla was born in": " Croatia",
        "Sigmund Freud was born in": " Czech",
    }
    for p, comp in born_in.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "born_in"})

    # ---- Death years (single token years)
    death_years = {
        "Albert Einstein died in": " 1955",
        "Isaac Newton died in": " 1727",
        "Charles Darwin died in": " 1882",
        "Leonardo da Vinci died in": " 1519",
        "William Shakespeare died in": " 1616",
        "George Washington died in": " 1799",
        "Napoleon Bonaparte died in": " 1821",
        "Abraham Lincoln died in": " 1865",
        "Marie Curie died in": " 1934",
        "Nikola Tesla died in": " 1943",
    }
    for p, comp in death_years.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "death_year"})

    # ---- Historical events and dates
    historical_events = {
        "World War I began in": " 1914",
        "World War II began in": " 1939",
        "World War II ended in": " 1945",
        "The American Revolution began in": " 1775",
        "The French Revolution began in": " 1789",
        "The Russian Revolution was in": " 1917",
        "The fall of the Berlin Wall was in": " 1989",
        "The September 11 attacks occurred in": " 2001",
        "The moon landing was in": " 1969",
        "Christopher Columbus sailed in": " 1492",
        "The Declaration of Independence was signed in": " 1776",
        "The Civil War began in": " 1861",
        "The Great Depression began in": " 1929",
        "The Cold War began after": " 1945",
    }
    for p, comp in historical_events.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "historical_event"})

    # ---- Century associations
    centuries = {
        "The Renaissance occurred in the": " 15th",
        "The Industrial Revolution began in the": " 18th",
        "The Enlightenment was in the": " 18th",
        "The Victorian Era was in the": " 19th",
        "World War I was in the": " 20th",
    }
    for p, comp in centuries.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "century"})

    # ---- Leaders and rulers
    leaders = {
        "Julius Caesar was a": " Roman",
        "Alexander the Great was a": " Macedonian",
        "Cleopatra was the queen of": " Egypt",
        "Queen Victoria ruled": " Britain",
        "Napoleon was the emperor of": " France",
        "Peter the Great ruled": " Russia",
        "Elizabeth I was queen of": " England",
        "Henry VIII was king of": " England",
        "Louis XIV was king of": " France",
    }
    for p, comp in leaders.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "leader"})

    # ========================================
    # SCIENTIFIC FACTS (Massively Expanded)
    # ========================================

    # ---- Physics facts
    physics = {
        "The speed of light is approximately": " 300",
        "Gravity was discovered by": " Newton",
        "Einstein developed the theory of": " relativity",
        "The atomic bomb was developed during": " World",
        "Newton's laws describe": " motion",
        "Electrons have a": " negative",
        "Protons have a": " positive",
        "The Earth orbits the": " Sun",
        "The Moon orbits the": " Earth",
    }
    for p, comp in physics.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "physics"})

    # ---- Chemistry facts
    chemistry = {
        "Water is composed of hydrogen and": " oxygen",
        "The symbol for gold is": " Au",
        "The symbol for silver is": " Ag",
        "The symbol for iron is": " Fe",
        "The periodic table was created by": " Mendeleev",
        "Oxygen has atomic number": " 8",
        "Carbon has atomic number": " 6",
        "Hydrogen has atomic number": " 1",
        "Salt is composed of sodium and": " chloride",
    }
    for p, comp in chemistry.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "chemistry"})

    # ---- Biology facts
    biology = {
        "DNA stands for deoxyribonucleic": " acid",
        "Photosynthesis occurs in": " plants",
        "The heart pumps": " blood",
        "Humans have": " 46",
        "Evolution was proposed by": " Darwin",
        "Cells are the basic unit of": " life",
        "Mitochondria produce": " energy",
        "The largest organ is the": " skin",
    }
    for p, comp in biology.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "biology"})

    # ---- Mathematics facts
    mathematics = {
        "Pi is approximately": " 3",
        "A triangle has": " three",
        "A square has": " four",
        "A circle has": " 360",
        "The Pythagorean theorem relates": " triangles",
        "Calculus was invented by": " Newton",
        "Algebra originated in": " ancient",
    }
    for p, comp in mathematics.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "mathematics"})

    # ---- Astronomy facts
    astronomy = {
        "The Sun is a": " star",
        "Jupiter is a": " gas",
        "Mars is the": " red",
        "Saturn has": " rings",
        "The Solar System has": " eight",
        "The Milky Way is a": " galaxy",
        "A light year measures": " distance",
        "The nearest star to Earth is the": " Sun",
        "Pluto was reclassified as a": " dwarf",
    }
    for p, comp in astronomy.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "astronomy"})

    # ========================================
    # CULTURAL FACTS (Massively Expanded)
    # ========================================

    # ---- Literature and authors
    literature = {
        "Shakespeare wrote": " Hamlet",
        "Homer wrote the": " Odyssey",
        "Tolkien wrote The Lord of the": " Rings",
        "George Orwell wrote": " 1984",
        "Jane Austen wrote Pride and": " Prejudice",
        "Mark Twain wrote The Adventures of": " Tom",
        "Charles Dickens wrote A Tale of": " Two",
        "Ernest Hemingway wrote The Old Man and the": " Sea",
        "F. Scott Fitzgerald wrote The Great": " Gatsby",
        "Leo Tolstoy wrote War and": " Peace",
        "Fyodor Dostoevsky wrote Crime and": " Punishment",
        "Victor Hugo wrote Les": " Miserables",
        "Miguel de Cervantes wrote Don": " Quixote",
        "Dante wrote The Divine": " Comedy",
        "Virgil wrote the": " Aeneid",
    }
    for p, comp in literature.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "literature"})

    # ---- Art and artists
    art = {
        "Leonardo da Vinci painted the Mona": " Lisa",
        "Vincent van Gogh painted Starry": " Night",
        "Pablo Picasso was a": " Spanish",
        "Michelangelo painted the Sistine": " Chapel",
        "Claude Monet was an": " Impressionist",
        "Salvador Dali was a": " Surrealist",
        "Rembrandt was a": " Dutch",
        "Andy Warhol was a": " Pop",
    }
    for p, comp in art.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "art"})

    # ---- Music and composers
    music = {
        "Mozart was a": " composer",
        "Beethoven wrote": " symphonies",
        "Bach was a": " Baroque",
        "Chopin was a": " Polish",
        "Tchaikovsky was a": " Russian",
        "Wagner was a": " German",
        "Vivaldi wrote The Four": " Seasons",
        "Handel wrote": " Messiah",
    }
    for p, comp in music.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "music"})

    # ---- Sports facts
    sports = {
        "The Olympics originated in": " Greece",
        "Soccer is called football in": " Europe",
        "Basketball was invented in": " America",
        "Baseball is popular in": " America",
        "Cricket is popular in": " India",
        "The World Cup is held every": " four",
        "Tennis is played on a": " court",
        "Golf is played on a": " course",
    }
    for p, comp in sports.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "sports"})

    # ========================================
    # TECHNOLOGY AND INVENTIONS
    # ========================================

    technology = {
        "The telephone was invented by": " Bell",
        "The light bulb was invented by": " Edison",
        "The airplane was invented by the Wright": " Brothers",
        "The printing press was invented by": " Gutenberg",
        "The steam engine was invented by": " Watt",
        "The radio was invented by": " Marconi",
        "The computer was invented in the": " 20th",
        "The internet was developed in": " America",
        "Apple was founded by Steve": " Jobs",
        "Microsoft was founded by Bill": " Gates",
        "Facebook was founded by Mark": " Zuckerberg",
    }
    for p, comp in technology.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "technology"})

    # ========================================
    # ARCHITECTURE AND LANDMARKS
    # ========================================

    landmarks = {
        "The Eiffel Tower is in": " Paris",
        "The Colosseum is in": " Rome",
        "The Taj Mahal is in": " India",
        "The Great Wall is in": " China",
        "The Statue of Liberty is in": " New",
        "Big Ben is in": " London",
        "The Pyramids are in": " Egypt",
        "The Parthenon is in": " Athens",
        "The Kremlin is in": " Moscow",
        "Machu Picchu is in": " Peru",
        "Petra is in": " Jordan",
        "Angkor Wat is in": " Cambodia",
        "The Sydney Opera House is in": " Australia",
    }
    for p, comp in landmarks.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "landmarks"})

    # ========================================
    # ANIMALS AND NATURE
    # ========================================

    animals = {
        "The largest animal is the blue": " whale",
        "The fastest land animal is the": " cheetah",
        "The tallest animal is the": " giraffe",
        "Lions are found in": " Africa",
        "Pandas are native to": " China",
        "Kangaroos are native to": " Australia",
        "Penguins live in": " Antarctica",
        "Tigers are native to": " Asia",
        "Elephants are found in": " Africa",
        "Polar bears live in the": " Arctic",
    }
    for p, comp in animals.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "animals"})

    # ========================================
    # RELIGIONS AND MYTHOLOGY
    # ========================================

    religions = {
        "Christianity originated in": " Israel",
        "Islam originated in": " Saudi",
        "Buddhism originated in": " India",
        "Hinduism originated in": " India",
        "Judaism originated in": " Israel",
        "The Bible is the holy book of": " Christianity",
        "The Quran is the holy book of": " Islam",
        "Zeus was the king of the": " Greek",
        "Thor was a": " Norse",
        "Ra was an": " Egyptian",
    }
    for p, comp in religions.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "religion"})

    # ========================================
    # FOOD AND CUISINE
    # ========================================

    cuisine = {
        "Pizza originated in": " Italy",
        "Sushi originated in": " Japan",
        "Tacos originated in": " Mexico",
        "Hamburgers are popular in": " America",
        "Pasta is from": " Italy",
        "Croissants are from": " France",
        "Curry is from": " India",
        "Paella is from": " Spain",
        "Kimchi is from": " Korea",
    }
    for p, comp in cuisine.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "cuisine"})

    # ========================================
    # ECONOMIC AND POLITICAL FACTS
    # ========================================

    economics = {
        "The largest economy is": " America",
        "The European Union uses the": " euro",
        "OPEC stands for Organization of": " Petroleum",
        "The World Bank is headquartered in": " Washington",
        "The United Nations is headquartered in": " New",
        "NATO stands for North Atlantic": " Treaty",
        "GDP stands for Gross Domestic": " Product",
    }
    for p, comp in economics.items():
        pairs.append({"prompt": p, "completion": comp, "tag": "economics"})

    # ---- Filter to single-token completions
    kept = []
    dropped = []
    for ex in pairs:
        if is_single_token(ex["completion"], tokenizer):
            kept.append(ex)
        else:
            dropped.append(ex)

    # ---- Basic reporting
    from collections import Counter
    counts = Counter(ex["tag"].split(":")[0] for ex in kept)
    print("Kept counts by task:", dict(counts))
    print(f"\nTotal generated pairs: {len(pairs)}")
    print(f"Single-token completions: {len(kept)}")
    print(f"Multi-token completions (dropped): {len(dropped)}")

    if dropped:
        print(f"\nShowing first 20 dropped (multi-token) examples:")
        for ex in dropped[:20]:
            ids = tokenizer.encode(ex["completion"], add_special_tokens=False)
            print(f"  {ex['tag']:<20} {ex['prompt']:<50} -> {repr(ex['completion']):<20} token_ids={ids}")

    return kept

tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
pairs = build_pairs_expanded(tokenizer)

In [ ]:

# ==========================================
#@title Expanded Mini-alignment dataset + WikiText mix + optional slot-attn-only finetune + rerun generations
# (Aligned to ASMLanguageModel + your asa_generate)
# ==========================================

import random
import math
import re
import itertools
import torch
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

# -----------------------
# 0) Repro & device
# -----------------------
SEED = 1337
random.seed(SEED)
torch.manual_seed(SEED)
device = next(model.parameters()).device

# -----------------------
# 0.5) Config knobs (NEW)
# -----------------------
CFG = {
    # mix in WikiText
    "use_wiki": True,
    "wiki_dataset_name": "wikitext",
    "wiki_config_candidates": ["wikitext-103-raw-v1", "wikitext-2-raw-v1"],  # fallback
    "wiki_num_samples": 1536,         # number of wiki chunks (not lines)
    "wiki_chunk_chars_min": 400,      # filter small chunks
    "wiki_chunk_chars_max": 1200,     # chunk size (chars) before tokenization

    # training
    "max_len": 128,                   # increased since wiki chunks are longer
    "batch_size": 16,
    "steps": 777,
    "lr": 7e-6,
    "weight_decay": 0.007,
    "grad_clip": 1.0,

    # finetune mode
    # "all" trains everything; "slot_attn_only" freezes everything except slot-space attention op
    "finetune_mode": "all",  # or "all" or  "slot_attn_only"

    # which params count as "slot attention" (adjust to your module names)
    #"slot_train_name_regex": r"(slot|slots).*(attn|attention)|((attn|attention).*(slot|slots))",

    "slot_train_name_regex":r"(^|\.)(slot_in|slot_q|slot_k|slot_v|slot_out)\.weight$|(^|\.)(_slotspace_gate_raw)$",


}

# -----------------------
# 1) Utilities (aligned to your model call style)
# -----------------------
@torch.no_grad()
def next_token_stats(prompt: str, target_token_str: str, model, tokenizer):
    model.eval()
    inp = tokenizer.encode(prompt, return_tensors="pt").to(device)

    tgt_ids = tokenizer.encode(target_token_str, add_special_tokens=False)
    if len(tgt_ids) != 1:
        return {"ok": False, "reason": f"target string maps to {len(tgt_ids)} tokens", "target_ids": tgt_ids}

    tgt = tgt_ids[0]

    out = model(inp)
    logits = out[0] if isinstance(out, (tuple, list)) else out
    last = logits[0, -1, :]
    probs = torch.softmax(last, dim=-1)

    p = float(probs[tgt].item())
    rank = int((torch.argsort(last, descending=True) == tgt).nonzero(as_tuple=False).item()) + 1
    top1_id = int(torch.argmax(last).item())
    top1 = tokenizer.decode([top1_id])

    return {"ok": True, "p_target": p, "rank": rank, "top1": top1, "target_id": tgt}

@torch.no_grad()
def greedy_suffix(prompt: str, model, gen, max_new_tokens=8):
    g = dict(gen)
    g["do_sample"] = False
    g["max_new_tokens"] = int(max_new_tokens)
    out = asa_generate(prompt, model, g)
    text = out["text"]
    if text.startswith(prompt):
        return text[len(prompt):].replace("\n", " ").strip()
    idx = text.find(prompt)
    if idx >= 0:
        return text[idx+len(prompt):].replace("\n", " ").strip()
    return text.replace("\n", " ").strip()

@torch.no_grad()
def eval_exact_match(examples, model, gen, max_new_tokens=8):
    model.eval()
    ok = 0
    for ex in examples:
        pred = greedy_suffix(ex["prompt"], model, gen, max_new_tokens=max_new_tokens)
        gold = ex["completion"].replace("\n", " ").strip()
        ok += int(pred.startswith(gold))
    return ok / max(1, len(examples))

# -----------------------
# 2) Dataset builders
# -----------------------


def load_wikitext_chunks(tokenizer, num_samples=2048, chunk_chars_min=400, chunk_chars_max=1200):
    """
    Produces wiki training examples as plain LM text chunks:
      ex = {"prompt": "", "completion": "<wiki chunk>", "tag": "wiki"}
    We chunk by chars first, then token-truncate later in dataset.
    """
    try:
        from datasets import load_dataset
    except Exception as e:
        print("[WikiText] datasets not available; skipping WikiText mix.")
        return []

    ds = None
    used_cfg = None
    for cfg in CFG["wiki_config_candidates"]:
        try:
            ds = load_dataset(CFG["wiki_dataset_name"], cfg, split="train")
            used_cfg = cfg
            break
        except Exception:
            ds = None

    if ds is None:
        print("[WikiText] Could not load WikiText (tried configs:", CFG["wiki_config_candidates"], "). Skipping.")
        return []

    print(f"[WikiText] Loaded {CFG['wiki_dataset_name']} / {used_cfg} train split with {len(ds)} rows.")

    # Pull raw text field (wikitext uses 'text')
    texts = [t for t in ds["text"] if isinstance(t, str) and len(t.strip()) > 0]

    # Make chunks: concatenate consecutive lines until size bound, filter small chunks
    chunks = []
    buf = []
    buf_len = 0

    # shuffle deterministically
    rng = random.Random(SEED)
    rng.shuffle(texts)

    for line in texts:
        line = line.strip()
        # skip headings markup lines; keep normal prose
        if line.startswith("=") and line.endswith("="):
            continue
        if not line:
            continue

        # add line to buffer
        if buf_len + len(line) + 1 <= chunk_chars_max:
            buf.append(line)
            buf_len += len(line) + 1
        else:
            chunk = " ".join(buf).strip()
            if len(chunk) >= chunk_chars_min:
                chunks.append(chunk)
            buf = [line]
            buf_len = len(line) + 1

        if len(chunks) >= num_samples:
            break

    # flush
    if len(chunks) < num_samples:
        chunk = " ".join(buf).strip()
        if len(chunk) >= chunk_chars_min:
            chunks.append(chunk)

    # create examples
    wiki_examples = [{"prompt": "", "completion": c, "tag": "wiki"} for c in chunks[:num_samples]]
    print(f"[WikiText] Prepared {len(wiki_examples)} wiki chunks.")

    return wiki_examples

# -----------------------
# 3) Split synthetic (entity-holdout) + build mixed train set
# -----------------------
#pairs = build_pairs_expanded(tokenizer)

from collections import Counter

holdout_capitals = {
    "Spain", "Canada", "Poland", "Portugal", "Greece", "Austria",
    "Norway", "Ireland", "Romania", "Croatia", "Argentina", "Chile"
}
holdout_languages = {"Brazil", "Mexico", "Netherlands", "Sweden", "Finland"}
holdout_currencies = {"Japan", "Switzerland", "South Africa", "Thailand"}
holdout_continents = {"Kenya", "Vietnam", "Peru", "New Zealand"}

train_examples, holdout_examples = [], []
for ex in pairs:
    task = ex["tag"].split(":")[0]

    if task == "capital":
        country = ex["tag"].split(":", 1)[1]
        (holdout_examples if country in holdout_capitals else train_examples).append(ex)
    elif task == "language":
        country = ex["tag"].split(":", 1)[1]
        (holdout_examples if country in holdout_languages else train_examples).append(ex)
    elif task == "currency":
        country = ex["tag"].split(":", 1)[1]
        (holdout_examples if country in holdout_currencies else train_examples).append(ex)
    elif task == "continent":
        country = ex["tag"].split(":", 1)[1]
        (holdout_examples if country in holdout_continents else train_examples).append(ex)
    else:
        if random.random() < 0.2:
            holdout_examples.append(ex)
        else:
            train_examples.append(ex)

print(f"\n[Synthetic] Total kept pairs: {len(pairs)} | Train: {len(train_examples)} | Holdout: {len(holdout_examples)}")
print(f"[Synthetic] Split: {len(train_examples)/len(pairs)*100:.1f}% / {len(holdout_examples)/len(pairs)*100:.1f}%")
holdout_by_cat = Counter(ex["tag"].split(":")[0] for ex in holdout_examples)
print("[Synthetic] Holdout by category:", dict(holdout_by_cat))

# NEW: load wiki and mix into TRAIN ONLY
wiki_examples = []
if CFG["use_wiki"]:
    wiki_examples = load_wikitext_chunks(
        tokenizer,
        num_samples=CFG["wiki_num_samples"],
        chunk_chars_min=CFG["wiki_chunk_chars_min"],
        chunk_chars_max=CFG["wiki_chunk_chars_max"],
    )

mixed_train_examples = train_examples + wiki_examples
print(f"\n[Mix] Train synthetic={len(train_examples)} + wiki={len(wiki_examples)} => mixed_train={len(mixed_train_examples)}")
print(f"[Mix] Holdout (synthetic only) = {len(holdout_examples)}")

# -----------------------
# 4) Tiny finetune dataset (teacher forcing)
#    Works for BOTH: prompt+completion pairs and raw wiki chunks (prompt="")
# -----------------------
class PromptCompletionDataset(Dataset):
    def __init__(self, examples, tokenizer, max_len=128):
        self.examples = examples
        self.tok = tokenizer
        self.max_len = int(max_len)

    def __len__(self): return len(self.examples)

    def __getitem__(self, idx):
        ex = self.examples[idx]
        text = ex["prompt"] + ex["completion"]
        ids = self.tok.encode(text)

        # keep the tail; for wiki, this acts like "random suffix LM"
        ids = ids[-self.max_len:]

        x = torch.tensor(ids[:-1], dtype=torch.long)
        y = torch.tensor(ids[1:], dtype=torch.long)
        return x, y, ex

def collate_pad(batch):
    xs, ys, exs = zip(*batch)
    maxT = max(x.size(0) for x in xs)
    pad_id = tokenizer.eos_token_id  # GPT-2 no pad token

    X = torch.full((len(xs), maxT), pad_id, dtype=torch.long)
    Y = torch.full((len(xs), maxT), -100, dtype=torch.long)

    for i, (x, y) in enumerate(zip(xs, ys)):
        T = x.size(0)
        X[i, :T] = x
        Y[i, :T] = y
    return X.to(device), Y.to(device), exs

train_ds = PromptCompletionDataset(mixed_train_examples, tokenizer, max_len=CFG["max_len"])
train_dl = DataLoader(
    train_ds,
    batch_size=min(CFG["batch_size"], len(train_ds)),
    shuffle=True,
    collate_fn=collate_pad
)

# -----------------------
# 5) Optional: freeze everything except slot-space attention (NEW)
# -----------------------
def configure_finetune_mode(model, mode: str, name_regex: str):
    """
    mode:
      - "all": train everything
      - "slot_attn_only": only train parameters whose full name matches `name_regex`
    """
    if mode == "all":
        for p in model.parameters():
            p.requires_grad = True
        trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total = sum(p.numel() for p in model.parameters())
        print(f"[Finetune] mode=all trainable={trainable}/{total} ({trainable/total*100:.2f}%)")
        return

    if mode != "slot_attn_only":
        raise ValueError(f"Unknown finetune_mode={mode}")

    rx = re.compile(name_regex, flags=re.IGNORECASE)

    # freeze everything
    for _, p in model.named_parameters():
        p.requires_grad = False

    # unfreeze matching params
    matched = []
    for n, p in model.named_parameters():
        if rx.search(n) is not None:
            p.requires_grad = True
            matched.append((n, p.numel()))

    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"[Finetune] mode=slot_attn_only regex={name_regex!r}")
    print(f"[Finetune] trainable={trainable}/{total} ({trainable/total*100:.4f}%) matched_tensors={len(matched)}")

    # show top matches by size
    matched.sort(key=lambda x: -x[1])
    for n, k in matched[:25]:
        print(f"  [trainable] {k:>10}  {n}")

configure_finetune_mode(model, CFG["finetune_mode"], CFG["slot_train_name_regex"])

# -----------------------
# 6) Pre-eval (synthetic only, as before)
# -----------------------
print("\n" + "="*80)
print("PRE-TRAINING EVALUATION (synthetic only)")
print("="*80)

pre_acc_train = eval_exact_match(train_examples, model, gener, max_new_tokens=8)
pre_acc_hold  = eval_exact_match(holdout_examples, model, gener, max_new_tokens=8)

print(f"\n[PRE] Exact-match accuracy:")
print(f"  Train:   {pre_acc_train:.3f} ({int(pre_acc_train*len(train_examples))}/{len(train_examples)})")
print(f"  Holdout: {pre_acc_hold:.3f} ({int(pre_acc_hold*len(holdout_examples))}/{len(holdout_examples)})")

print("\n[PRE] Next-token stats for sample of single-token targets (synthetic only):")
sample_for_stats = random.sample(pairs, min(30, len(pairs)))
for ex in sample_for_stats:
    stats = next_token_stats(ex["prompt"], ex["completion"], model, tokenizer)
    if stats["ok"]:
        print(f"  {ex['tag']:<25} P={stats['p_target']:.4f} rank={stats['rank']:>5} top1={stats['top1']!r}")
    else:
        print(f"  {ex['tag']:<25} (skip) {stats['reason']}")

# -----------------------
# 7) Light training (mixed: synthetic + wiki)
# -----------------------
print("\n" + "="*80)
print("TRAINING (mixed synthetic + wiki)")
print("="*80)

model.train()

# IMPORTANT: optimizer must only see trainable params (esp for slot_attn_only)
trainable_params = [p for p in model.parameters() if p.requires_grad]
if len(trainable_params) == 0:
    raise RuntimeError("No trainable parameters. Check CFG['finetune_mode'] and regex.")

opt = torch.optim.AdamW(
    trainable_params,
    lr=CFG["lr"],
    betas=(0.9, 0.95),
    weight_decay=CFG["weight_decay"]
)

steps = int(CFG["steps"])
grad_clip = float(CFG["grad_clip"])

print(f"Training for {steps} steps with batch_size={train_dl.batch_size}")
print(f"Total mixed_train_examples={len(mixed_train_examples)} | synthetic={len(train_examples)} | wiki={len(wiki_examples)}\n")

# stable batch stream (avoid re-instantiating iter(train_dl) each step)
batch_iter = itertools.cycle(train_dl)

for step in range(steps):
    X, Y, _ = next(batch_iter)
    opt.zero_grad(set_to_none=True)

    logits = model(X)
    logits = logits[0] if isinstance(logits, (tuple, list)) else logits

    loss = F.cross_entropy(
        logits.view(-1, logits.size(-1)),
        Y.view(-1),
        ignore_index=-100
    )
    loss.backward()
    torch.nn.utils.clip_grad_norm_(trainable_params, grad_clip)
    opt.step()

    if (step + 1) % 50 == 0:
        print(f"  [train] step {step+1:>4}/{steps} loss={float(loss.item()):.4f}")

model.eval()

# -----------------------
# 8) Post-eval (synthetic only, as before)
# -----------------------
print("\n" + "="*80)
print("POST-TRAINING EVALUATION (synthetic only)")
print("="*80)

post_acc_train = eval_exact_match(train_examples, model, gener, max_new_tokens=8)
post_acc_hold  = eval_exact_match(holdout_examples, model, gener, max_new_tokens=8)

print(f"\n[POST] Exact-match accuracy:")
print(f"  Train:   {post_acc_train:.3f} ({int(post_acc_train*len(train_examples))}/{len(train_examples)})")
print(f"  Holdout: {post_acc_hold:.3f} ({int(post_acc_hold*len(holdout_examples))}/{len(holdout_examples)})")

print(f"\n[DELTA] Accuracy change:")
print(f"  Train:   {pre_acc_train:.3f} -> {post_acc_train:.3f} (Δ={post_acc_train-pre_acc_train:+.3f})")
print(f"  Holdout: {pre_acc_hold:.3f} -> {post_acc_hold:.3f} (Δ={post_acc_hold-pre_acc_hold:+.3f})")

print("\n[POST] Next-token stats for same sample (synthetic only):")
for ex in sample_for_stats:
    stats = next_token_stats(ex["prompt"], ex["completion"], model, tokenizer)
    if stats["ok"]:
        print(f"  {ex['tag']:<25} P={stats['p_target']:.4f} rank={stats['rank']:>5} top1={stats['top1']!r}")

# -----------------------
# 9) Generations (synthetic categories only)
# -----------------------
print("\n" + "="*80)
print("GENERATION SAMPLES (greedy decoding) (synthetic only)")
print("="*80)

generation_samples = []
by_category = {}
for ex in pairs:
    cat = ex["tag"].split(":")[0]
    by_category.setdefault(cat, []).append(ex)

for cat, exs in sorted(by_category.items()):
    generation_samples.extend(exs[:2])

generation_samples = generation_samples[:25]

for ex in generation_samples:
    raw = greedy_suffix(ex["prompt"], model, gener, max_new_tokens=12)

    tag_base = ex["tag"].split(":")[0]
    if tag_base == "capital":
        scaffold_prompt = ex["prompt"] + " the city of"
    elif tag_base == "language":
        scaffold_prompt = ex["prompt"] + " primarily"
    elif tag_base == "currency":
        scaffold_prompt = ex["prompt"]
    else:
        scaffold_prompt = ex["prompt"]

    sca = greedy_suffix(scaffold_prompt, model, gener, max_new_tokens=12)

    print(f"\n{'─'*80}")
    print(f"CATEGORY: {ex['tag']:<25} TARGET: {ex['completion']!r}")
    print(f"PROMPT:   {ex['prompt']!r}")
    print(f"RAW:      {raw[:100]}")
    if scaffold_prompt != ex["prompt"]:
        print(f"SCAFFOLD: {sca[:100]}")

print("\n" + "="*80)
print("EVALUATION COMPLETE")
print("="*80)

In [ ]:

sweep_asa_temps(
    model, tokenizer, gener,
    holdout_examples,
    read_grid=(0.4, 0.1, 1.0, 0.05),
    write_grid=(1.0,),
    max_new_tokens=8,
    n_gen_samples=8,
)

# Final

In [ ]:



#@title Vanilla Transformer (GPT-style) baseline with matching training loop + comparable metrics

import math
import os
import time
import random
from dataclasses import dataclass, asdict
from typing import Optional, Tuple, Dict, List

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

# =========================================================
# RoPE (same helper style as your ASA)
# =========================================================
def _rotate_half(x: torch.Tensor) -> torch.Tensor:
    x1 = x[..., ::2]
    x2 = x[..., 1::2]
    return torch.stack((-x2, x1), dim=-1).flatten(-2)

class RotaryEmbedding(nn.Module):
    def __init__(self, dim: int, base: float = 10000.0):
        super().__init__()
        assert dim % 2 == 0, "RoPE requires even dim"
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)
        self._cos_cached = None
        self._sin_cached = None
        self._t_cached = None
        self._device_cached = None

    def get_cos_sin(self, T: int, device, dtype):
        if (
            self._t_cached == T
            and self._cos_cached is not None
            and self._device_cached == device
        ):
            return self._cos_cached.to(dtype=dtype), self._sin_cached.to(dtype=dtype)

        t = torch.arange(T, device=device, dtype=self.inv_freq.dtype)
        freqs = torch.einsum("t,f->tf", t, self.inv_freq)  # [T, d/2]
        emb = torch.cat([freqs, freqs], dim=-1)            # [T, d]
        cos = emb.cos()[None, None, :, :]                  # [1,1,T,d]
        sin = emb.sin()[None, None, :, :]                  # [1,1,T,d]

        self._t_cached = T
        self._device_cached = device
        self._cos_cached = cos
        self._sin_cached = sin
        return cos.to(dtype=dtype), sin.to(dtype=dtype)

def apply_rope(x: torch.Tensor, cos: torch.Tensor, sin: torch.Tensor) -> torch.Tensor:
    return (x * cos) + (_rotate_half(x) * sin)


# =========================================================
# Config
# =========================================================
@dataclass
class TFTrainConfig:
    # Data
    dataset_name: str = "wikitext"
    dataset_config: str = "wikitext-103-raw-v1"
    tokenizer_name: str = "gpt2"

    max_seq_len: int = 1024
    stride_frac_val: float = 0.50
    seed: int = 1337

    # Sample budgets
    train_samples_target: int = 100_000_000
    val_samples_target: int = 25_000

    # Training
    batch_size: int = 32
    learning_rate: float = 3e-4
    weight_decay: float = 0.01
    betas: Tuple[float, float] = (0.9, 0.95)
    grad_clip: float = 1.0
    warmup_steps: int = 1_000
    total_steps: int = 75_000
    eval_interval: int = 1_000
    log_interval: int = 100

    # Model
    vocab_size: int = 50257
    embed_dim: int = 384
    num_layers: int = 13
    num_heads: int = 8
    mlp_ratio: float = 4.0
    dropout: float = 0.1
    tie_weights: bool = True

    # Positions
    use_abs_pos: bool = False          # keep False to mirror your ASA runs
    use_rope: bool = True
    rope_base: float = 10000.0

    # Analytics
    eval_max_batches: int = 150
    analytics_last_k: int = 32

    # IO / caches
    output_dir: str = "./drive/MyDrive/tf_outputs"
    tag: str = "tf_wikitext"
    cache_dir: str = "./drive/MyDrive/asm_caches"  # reuse your cached streams
    val_windows_cache: str = "./drive/MyDrive/asm_nlp/val_cache_wikitext_windows_512.pkl"  # reuse


# =========================================================
# Model: GPT-style Transformer
# =========================================================
class CausalSelfAttention(nn.Module):
    def __init__(
        self,
        embed_dim: int,
        num_heads: int,
        dropout: float,
        use_rope: bool = True,
        rope_base: float = 10000.0,
    ):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.use_rope = bool(use_rope)

        self.qkv = nn.Linear(embed_dim, 3 * embed_dim, bias=False)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=False)
        self.attn_drop = nn.Dropout(dropout)
        self.resid_drop = nn.Dropout(dropout)

        self.rope = RotaryEmbedding(self.head_dim, base=float(rope_base)) if self.use_rope else None

    def forward(
        self,
        x: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,  # [B,T] bool/int (1=valid)
        return_info: bool = False,
    ):
        B, T, C = x.shape
        H, d = self.num_heads, self.head_dim

        qkv = self.qkv(x)  # [B,T,3C]
        q, k, v = qkv.split(C, dim=-1)

        # [B,H,T,d]
        q = q.view(B, T, H, d).transpose(1, 2)
        k = k.view(B, T, H, d).transpose(1, 2)
        v = v.view(B, T, H, d).transpose(1, 2)

        if self.use_rope:
            cos, sin = self.rope.get_cos_sin(T, device=x.device, dtype=q.dtype)
            q = apply_rope(q, cos, sin)
            k = apply_rope(k, cos, sin)

        # scaled dot-product attention logits: [B,H,T,T]
        att = torch.einsum("bhid,bhjd->bhij", q, k) / math.sqrt(d)

        # causal mask
        causal = torch.tril(torch.ones((T, T), device=x.device, dtype=torch.bool))
        att = att.masked_fill(~causal.view(1, 1, T, T), float("-inf"))

        # optional key padding mask (mask keys that are padding)
        if attention_mask is not None:
            valid = attention_mask.to(dtype=torch.bool)  # [B,T]
            att = att.masked_fill(~valid.view(B, 1, 1, T), float("-inf"))

        w = torch.softmax(att, dim=-1)  # [B,H,T,T]
        w = self.attn_drop(w)

        y = torch.einsum("bhij,bhjd->bhid", w, v)  # [B,H,T,d]
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.resid_drop(self.out_proj(y))

        info = None
        if return_info:
            # Store attention weights (can be big); use eval_max_batches to control cost.
            info = {
                "attn_weights": w.detach(),               # [B,H,T,T]
                "attn_logits": att.detach().to(torch.float32),  # [B,H,T,T] for COM/diagnostics if needed
            }
        return y, info


class TFBlock(nn.Module):
    def __init__(
        self,
        embed_dim: int,
        num_heads: int,
        mlp_ratio: float,
        dropout: float,
        use_rope: bool,
        rope_base: float,
    ):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = CausalSelfAttention(
            embed_dim=embed_dim,
            num_heads=num_heads,
            dropout=dropout,
            use_rope=use_rope,
            rope_base=rope_base,
        )
        self.norm2 = nn.LayerNorm(embed_dim)

        hidden = int(embed_dim * float(mlp_ratio))
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, hidden, bias=False),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, embed_dim, bias=False),
            nn.Dropout(dropout),
        )

    def forward(self, x, attention_mask=None, return_info=False):
        a, info = self.attn(self.norm1(x), attention_mask=attention_mask, return_info=return_info)
        x = x + a
        x = x + self.mlp(self.norm2(x))
        return x, info


class TransformerLM(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        embed_dim: int,
        num_layers: int,
        num_heads: int,
        max_seq_len: int,
        mlp_ratio: float,
        dropout: float,
        tie_weights: bool,
        use_abs_pos: bool,
        use_rope: bool,
        rope_base: float,
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.max_seq_len = max_seq_len
        self.use_abs_pos = bool(use_abs_pos)

        self.tok = nn.Embedding(vocab_size, embed_dim)
        self.pos = nn.Embedding(max_seq_len, embed_dim) if self.use_abs_pos else None
        self.drop = nn.Dropout(dropout)

        self.blocks = nn.ModuleList([
            TFBlock(
                embed_dim=embed_dim,
                num_heads=num_heads,
                mlp_ratio=mlp_ratio,
                dropout=dropout,
                use_rope=use_rope,
                rope_base=rope_base,
            )
            for _ in range(num_layers)
        ])

        self.norm = nn.LayerNorm(embed_dim)
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
        if tie_weights:
            self.lm_head.weight = self.tok.weight

        self.apply(self._init)

    def _init(self, m):
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, std=0.02)
        elif isinstance(m, nn.Embedding):
            nn.init.normal_(m.weight, std=0.02)
        elif isinstance(m, nn.LayerNorm):
            nn.init.ones_(m.weight)
            nn.init.zeros_(m.bias)

    def forward(self, input_ids, attention_mask=None, return_info=False):
        B, T = input_ids.shape
        #assert T <= self.max_seq_len, f"T={T} exceeds max_seq_len={self.max_seq_len}"

        x = self.tok(input_ids)
        if self.use_abs_pos:
            pos = torch.arange(T, device=input_ids.device).unsqueeze(0).expand(B, -1)
            x = x + self.pos(pos)
        x = self.drop(x)

        infos = []
        for blk in self.blocks:
            x, info = blk(x, attention_mask=attention_mask, return_info=return_info)
            if return_info:
                infos.append(info)

        x = self.norm(x)
        logits = self.lm_head(x)
        return (logits, infos) if return_info else logits


# =========================================================
# Comparable metrics for Transformer attention
# =========================================================
@torch.no_grad()
def _attn_entropy_mean(attn_w: torch.Tensor) -> float:
    # attn_w: [B,H,T,T] entropy over keys for each query
    eps = 1e-8
    p = attn_w.clamp_min(eps)
    ent = -(p * p.log()).sum(dim=-1)  # [B,H,T]
    return float(ent.mean().cpu().item())

@torch.no_grad()
def _attn_top1freq_mean(attn_w: torch.Tensor) -> float:
    # "most common top1 key index" frequency, averaged over batch+heads
    top1 = attn_w.argmax(dim=-1)  # [B,H,T]
    flat = top1.reshape(-1).cpu()
    T = attn_w.shape[-1]
    hist = torch.bincount(flat, minlength=T).float()
    denom = hist.sum().clamp_min(1.0)
    return float((hist.max() / denom).item())

@torch.no_grad()
def _attn_com_and_lastk(attn_w: torch.Tensor, last_k: int) -> Tuple[float, float]:
    # center-of-mass over keys (0..1) and last-k mass, averaged over B,H,T
    B, H, Tq, Tk = attn_w.shape
    assert Tq == Tk
    pos = torch.arange(Tk, device=attn_w.device, dtype=attn_w.dtype).view(1, 1, 1, Tk)
    com = (attn_w * pos).sum(dim=-1) / max(1.0, float(Tk - 1))  # [B,H,T]
    last_k = min(max(1, int(last_k)), Tk)
    lastk_mass = attn_w[..., -last_k:].sum(dim=-1)              # [B,H,T]
    return float(com.mean().cpu().item()), float(lastk_mass.mean().cpu().item())


# =========================================================
# Eval for Transformer (same CE loss, similar printed stats)
# =========================================================
@torch.no_grad()
def evaluate_tf(model, val_loader, max_batches=50, last_k=32):
    model.eval()
    losses = []

    ent_acc = 0.0
    top1_acc = 0.0
    com_acc = 0.0
    lastk_acc = 0.0
    n_batches = 0

    for i, (xb, yb) in enumerate(val_loader):
        if i >= max_batches:
            break
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)

        with torch.autocast(device_type=device.type, dtype=amp_dtype, enabled=use_amp):
            logits, infos = model(xb, return_info=True)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), yb.view(-1))
        losses.append(float(loss.item()))

        # layer-avg attention diagnostics
        eL, tL, cL, lL = [], [], [], []
        for info in infos:
            w = info.get("attn_weights", None)
            if w is None:
                continue
            eL.append(_attn_entropy_mean(w))
            tL.append(_attn_top1freq_mean(w))
            com, lastm = _attn_com_and_lastk(w, last_k=last_k)
            cL.append(com); lL.append(lastm)

        if eL:
            ent_acc += sum(eL) / len(eL)
            top1_acc += sum(tL) / len(tL)
            com_acc += sum(cL) / len(cL)
            lastk_acc += sum(lL) / len(lL)
            n_batches += 1

    mean = sum(losses) / max(1, len(losses))
    ppl = float(math.exp(min(20.0, mean)))

    stats = {}
    if n_batches > 0:
        stats["attn_entropy_mean"] = ent_acc / n_batches
        stats["attn_top1freq_mean"] = top1_acc / n_batches
        stats["attn_com_mean"] = com_acc / n_batches
        stats["attn_lastk_mass_mean"] = lastk_acc / n_batches

    model.train()
    return mean, ppl, stats

def _fmt_stats_tf(stats: Dict[str, float], last_k: int) -> str:
    keys = ["attn_entropy_mean", "attn_top1freq_mean", "attn_com_mean", "attn_lastk_mass_mean"]
    parts = []
    for k in keys:
        if k in stats:
            if k == "attn_lastk_mass_mean":
                parts.append(f"{k}(last_k={last_k})={stats[k]:.4f}")
            else:
                parts.append(f"{k}={stats[k]:.4f}")
    return " | ".join(parts)


# =========================================================
# Train loop (reuses your cached stream/window builders + WarmupCosine + save_ckpt)
# Requires: build_or_load_token_stream, build_or_load_validation_windows, WikiTextRandomWindowStream,
#           WarmupCosine, save_ckpt from your notebook (already defined above).
# =========================================================
def train_tf(cfg: TFTrainConfig, bail=False):
    random.seed(cfg.seed)
    torch.manual_seed(cfg.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(cfg.seed)

    os.makedirs(cfg.cache_dir, exist_ok=True)
    train_stream_cache = os.path.join(cfg.cache_dir, f"{cfg.dataset_config}_train_stream.pkl")
    val_stream_cache   = os.path.join(cfg.cache_dir, f"{cfg.dataset_config}_val_stream.pkl")

    train_stream = build_or_load_token_stream(
        cache_path=train_stream_cache,
        dataset_name=cfg.dataset_name,
        dataset_config=cfg.dataset_config,
        split="train",
        tokenizer_name=cfg.tokenizer_name,
        min_chars=1,
        add_eos_between_rows=True,
    )
    val_stream = build_or_load_token_stream(
        cache_path=val_stream_cache,
        dataset_name=cfg.dataset_name,
        dataset_config=cfg.dataset_config,
        split="validation",
        tokenizer_name=cfg.tokenizer_name,
        min_chars=1,
        add_eos_between_rows=True,
    )

    val_dataset = build_or_load_validation_windows(
        cache_path=cfg.val_windows_cache,
        token_stream=val_stream,
        max_seq_len=cfg.max_seq_len,
        stride_frac=cfg.stride_frac_val,
        val_samples_target=cfg.val_samples_target,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=torch.cuda.is_available(),
    )

    train_ds = WikiTextRandomWindowStream(
        token_stream=train_stream,
        max_seq_len=cfg.max_seq_len,
        train_samples_target=cfg.train_samples_target,
        seed=cfg.seed,
    )
    train_loader = DataLoader(
        train_ds,
        batch_size=cfg.batch_size,
        num_workers=3,
        pin_memory=torch.cuda.is_available(),
    )

    model = TransformerLM(
        vocab_size=cfg.vocab_size,
        embed_dim=cfg.embed_dim,
        num_layers=cfg.num_layers,
        num_heads=cfg.num_heads,
        max_seq_len=cfg.max_seq_len,
        mlp_ratio=cfg.mlp_ratio,
        dropout=cfg.dropout,
        tie_weights=cfg.tie_weights,
        use_abs_pos=cfg.use_abs_pos,
        use_rope=cfg.use_rope,
        rope_base=cfg.rope_base,
    ).to(device)

    if bail:
        return model

    out_dir = os.path.join(cfg.output_dir, cfg.tag)
    os.makedirs(out_dir, exist_ok=True)

    n_params = sum(p.numel() for p in model.parameters())
    print("=" * 108)
    print(f"Training [TF {cfg.tag}] on {cfg.dataset_name}/{cfg.dataset_config}")
    print(f"Params: {n_params:,}")
    print(f"Train tokens: {len(train_stream):,} | Val tokens: {len(val_stream):,} | Val windows: {len(val_dataset):,}")
    print(f"T={cfg.max_seq_len} | val_stride_frac={cfg.stride_frac_val} | last_k={cfg.analytics_last_k}")
    print(f"amp={use_amp}({amp_dtype}) | abs_pos={cfg.use_abs_pos} | rope={cfg.use_rope}(base={cfg.rope_base:g})")
    print("=" * 108)

    opt = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate, weight_decay=cfg.weight_decay, betas=cfg.betas)
    sched = WarmupCosine(opt, cfg.warmup_steps, cfg.total_steps, cfg.learning_rate)

    # initial eval + ckpt
    best_val = float("inf")
    vloss, vppl, vstats = evaluate_tf(model, val_loader, max_batches=cfg.eval_max_batches, last_k=cfg.analytics_last_k)
    best_val = vloss
    save_ckpt(os.path.join(out_dir, "best.pt"), cfg, model, opt, 0, best_val)

    print(f"[VAL step 0] loss={vloss:.3f} ppl={vppl:.2f}")
    if vstats:
        print("  " + _fmt_stats_tf(vstats, last_k=cfg.analytics_last_k))

    running = 0.0
    step = 0
    t_last = time.time()

    pbar = tqdm(total=cfg.total_steps, desc=f"[TF {cfg.tag}]")
    for xb, yb in train_loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        with torch.autocast(device_type=device.type, dtype=amp_dtype, enabled=use_amp):
            logits = model(xb)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), yb.view(-1))

        opt.zero_grad(set_to_none=True)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)
        opt.step()
        lr = sched.step()

        step += 1
        running += float(loss.item())
        pbar.update(1)

        if step % cfg.log_interval == 0:
            avg = running / cfg.log_interval
            running = 0.0
            it_s = cfg.log_interval / max(1e-9, (time.time() - t_last))
            t_last = time.time()
            pbar.set_postfix({
                "loss": f"{avg:.3f}",
                "ppl": f"{math.exp(min(20.0, avg)):.2f}",
                "lr": f"{lr:.2e}",
                "it/s": f"{it_s:.2f}",
            })
            print(f"[step {step}] train_loss={avg:.3f} ppl={math.exp(min(20.0, avg)):.2f} lr={lr:.2e} it/s={it_s:.2f}")

        if step % cfg.eval_interval == 0:
            vloss, vppl, vstats = evaluate_tf(model, val_loader, max_batches=cfg.eval_max_batches, last_k=cfg.analytics_last_k)
            print(f"\n[VAL step {step}] loss={vloss:.3f} ppl={vppl:.2f}")
            if vstats:
                print("  " + _fmt_stats_tf(vstats, last_k=cfg.analytics_last_k))
            if vloss < best_val:
                best_val = vloss
                save_ckpt(os.path.join(out_dir, "best.pt"), cfg, model, opt, step, best_val)

        if step >= cfg.total_steps:
            break

    save_ckpt(os.path.join(out_dir, "final.pt"), cfg, model, opt, step, best_val)
    print(f"[TF {cfg.tag}] Done. Best val loss: {best_val:.4f}")
    return model


def count_params(model):
    n = sum(p.numel() for p in model.parameters())
    n_train = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return n, n_train

# =========================================================
# Quick start snippet (edit tag + total_steps to 25000 if you want parity)
# =========================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_amp = torch.cuda.is_available()
amp_dtype = torch.bfloat16  # A100-friendly

print("Using device:", device)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")


tf_cfg = TFTrainConfig(
    max_seq_len=1024,
    batch_size=32,
    embed_dim=384,
    num_layers=13,
    num_heads=8,
    total_steps=25000,  # match your interrupted ASA point
    tag="tf_wikitext_1024t_384d_8h_13l_25ksteps",
)

print(tf_cfg)
tf_model = train_tf(tf_cfg, True)
n, n_trainable = count_params(tf_model)
print(f"Params: {n/1e6:.2f}M")
#

In [ ]:

#@title Batch Data Generation (UPDATED): supports variable seq_len for extrapolation

import os, random, pickle
from typing import List, Tuple, Optional, Iterator

import torch
from torch.utils.data import Dataset, IterableDataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer


# -------------------------
# Minimal dataset classes (same as training)
# -------------------------
class StableValidationDataset(Dataset):
    def __init__(self, samples: List[Tuple[torch.Tensor, torch.Tensor]]):
        self.samples = samples
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx]

def _ensure_dir(path: str):
    d = os.path.dirname(path)
    if d:
        os.makedirs(d, exist_ok=True)

def build_or_load_token_stream(
    *,
    cache_path: str,
    dataset_name: str,
    dataset_config: str,
    split: str,
    tokenizer_name: str,
    min_chars: int = 1,
    add_eos_between_rows: bool = True,
    max_rows: Optional[int] = None,
) -> List[int]:
    if os.path.exists(cache_path):
        with open(cache_path, "rb") as f:
            return pickle.load(f)

    _ensure_dir(cache_path)
    tok = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
    eos = tok.eos_token_id
    assert eos is not None, "Tokenizer must have eos_token_id"

    ds = load_dataset(dataset_name, dataset_config, split=split)

    stream: List[int] = []
    used = 0
    for row in ds:
        if max_rows is not None and used >= max_rows:
            break
        text = (row.get("text") or "").strip()
        if len(text) < min_chars:
            continue
        ids = tok.encode(text, add_special_tokens=False)
        if not ids:
            continue
        stream.extend(ids)
        if add_eos_between_rows:
            stream.append(eos)
        used += 1

    with open(cache_path, "wb") as f:
        pickle.dump(stream, f)
    return stream

def build_or_load_validation_windows(
    *,
    cache_path: str,
    token_stream: List[int],
    max_seq_len: int,
    stride_frac: float,
    val_samples_target: int,
) -> StableValidationDataset:
    """
    Same as before: stable deterministic val windows at a specific max_seq_len.
    """
    if os.path.exists(cache_path):
        with open(cache_path, "rb") as f:
            samples = pickle.load(f)
        return StableValidationDataset(samples)

    _ensure_dir(cache_path)

    T = int(max_seq_len)
    stride = max(1, int(T * float(stride_frac)))
    need = int(val_samples_target)

    max_start = len(token_stream) - (T + 1)
    if max_start <= 0:
        raise ValueError("Validation token stream too small for max_seq_len+1")

    samples: List[Tuple[torch.Tensor, torch.Tensor]] = []
    for start in range(0, max_start + 1, stride):
        chunk = token_stream[start:start + T + 1]
        if len(chunk) < T + 1:
            break
        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        samples.append((x, y))
        if len(samples) >= need:
            break

    with open(cache_path, "wb") as f:
        pickle.dump(samples, f)
    return StableValidationDataset(samples)

class WikiTextRandomWindowStream(IterableDataset):
    def __init__(self, token_stream: List[int], seq_len: int, train_samples_target: int, seed: int):
        super().__init__()
        self.stream = token_stream
        self.T = int(seq_len)
        self.target = int(train_samples_target)
        self.seed = int(seed)
        self.max_start = len(self.stream) - (self.T + 1)
        if self.max_start <= 0:
            raise ValueError(f"Train token stream too small for seq_len+1 (seq_len={self.T})")

    def __iter__(self):
        info = torch.utils.data.get_worker_info()
        wid = info.id if info is not None else 0
        rng = random.Random(self.seed + 17 * wid)

        yielded = 0
        while yielded < self.target:
            start = rng.randint(0, self.max_start)
            chunk = self.stream[start:start + self.T + 1]
            x = torch.tensor(chunk[:-1], dtype=torch.long)
            y = torch.tensor(chunk[1:], dtype=torch.long)
            yield x, y
            yielded += 1


# -------------------------
# New: stable val windows for arbitrary seq_len (no cache explosion)
# -------------------------
class StableValWindowsFromStream(Dataset):
    """
    Deterministic windows for *any* seq_len using a token stream.
    This avoids writing a giant cache file for 8k+ windows.
    """
    def __init__(self, token_stream: List[int], seq_len: int, stride_frac: float, n_samples: int):
        self.stream = token_stream
        self.T = int(seq_len)
        self.stride = max(1, int(self.T * float(stride_frac)))
        self.n = int(n_samples)

        max_start = len(self.stream) - (self.T + 1)
        if max_start <= 0:
            raise ValueError(f"Val token stream too small for seq_len+1 (seq_len={self.T})")

        # deterministically pick starts: 0, stride, 2*stride, ...
        starts = list(range(0, max_start + 1, self.stride))
        if len(starts) == 0:
            raise ValueError("No valid starts computed.")
        # truncate deterministically
        self.starts = starts[: self.n]

    def __len__(self): return len(self.starts)

    def __getitem__(self, idx):
        s = self.starts[idx]
        chunk = self.stream[s:s + self.T + 1]
        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        return x, y


# -------------------------
# Analysis batch generator (UPDATED)
# -------------------------
def make_batch_generator(
    cfg,
    *,
    split: str = "val",                 # "val" or "train"
    device: Optional[torch.device] = None,
    num_workers: int = 0,               # 0 is safest for Colab analysis
    pin_memory: bool = True,
    batches_per_epoch: Optional[int] = None,
    set_batch_size: Optional[int] = -1,
    infinite: bool = True,
    seed: Optional[int] = None,
    seq_len: Optional[int] = None,      # <-- NEW: override cfg.max_seq_len
) -> Iterator[Tuple[torch.Tensor, torch.Tensor]]:
    """
    Yields xb,yb batches shaped [B,T] int64.

    NEW:
      - seq_len lets you request longer contexts (e.g., 8192) for extrapolation.
      - For val:
          * if seq_len == cfg.max_seq_len: uses cached stable windows (as before)
          * else: uses deterministic StableValWindowsFromStream (no cache explosion)
      - For train: uses random windows at seq_len.
    """
    assert split in ("val", "train")
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Resolve cfg fields with fallback (works for dict-like or dataclass)
    def get(name, default=None):
        return getattr(cfg, name, cfg.get(name, default) if isinstance(cfg, dict) else default)

    cache_dir = get("cache_dir")
    dataset_name = get("dataset_name")
    dataset_config = get("dataset_config")
    tokenizer_name = get("tokenizer_name")
    train_T = int(get("max_seq_len"))
    T = int(seq_len) if seq_len is not None else train_T

    batch_size = set_batch_size if set_batch_size != -1 else int(get("batch_size"))
    stride_frac_val = float(get("stride_frac_val", 0.5))
    val_samples_target = int(get("val_samples_target", 25_000))
    val_windows_cache = get("val_windows_cache")
    train_samples_target = int(get("train_samples_target", 100_000_000))
    used_seed = int(seed if seed is not None else get("seed", 1337))

    os.makedirs(cache_dir, exist_ok=True)
    train_stream_cache = os.path.join(cache_dir, f"{dataset_config}_train_stream.pkl")
    val_stream_cache   = os.path.join(cache_dir, f"{dataset_config}_val_stream.pkl")

    if split == "train":
        train_stream = build_or_load_token_stream(
            cache_path=train_stream_cache,
            dataset_name=dataset_name,
            dataset_config=dataset_config,
            split="train",
            tokenizer_name=tokenizer_name,
            min_chars=1,
            add_eos_between_rows=True,
        )
        ds = WikiTextRandomWindowStream(
            token_stream=train_stream,
            seq_len=T,
            train_samples_target=train_samples_target,
            seed=used_seed,
        )
        loader = DataLoader(
            ds,
            batch_size=batch_size,
            num_workers=num_workers,
            pin_memory=pin_memory and torch.cuda.is_available(),
        )

        yielded = 0
        for xb, yb in loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            yield xb, yb
            yielded += 1
            if batches_per_epoch is not None and yielded >= batches_per_epoch:
                if infinite:
                    yielded = 0
                    continue
                break

    else:
        # val stream
        val_stream = build_or_load_token_stream(
            cache_path=val_stream_cache,
            dataset_name=dataset_name,
            dataset_config=dataset_config,
            split="validation",
            tokenizer_name=tokenizer_name,
            min_chars=1,
            add_eos_between_rows=True,
        )

        # If we're at training length, preserve exact old behavior (cached stable windows)
        if T == train_T:
            val_ds = build_or_load_validation_windows(
                cache_path=val_windows_cache,
                token_stream=val_stream,
                max_seq_len=T,
                stride_frac=stride_frac_val,
                val_samples_target=val_samples_target,
            )
            loader = DataLoader(
                val_ds,
                batch_size=batch_size,
                shuffle=False,
                num_workers=num_workers,
                pin_memory=pin_memory and torch.cuda.is_available(),
            )
        else:
            # NEW: deterministic stable windows directly from stream for arbitrary T
            # We only need a small number of batches for analysis, so keep it modest.
            # If batches_per_epoch is None, default to enough samples for ~50 batches.
            if batches_per_epoch is None:
                target_batches = 50
            else:
                target_batches = int(batches_per_epoch)

            n_samples = target_batches * batch_size
            val_ds = StableValWindowsFromStream(
                token_stream=val_stream,
                seq_len=T,
                stride_frac=stride_frac_val,
                n_samples=n_samples,
            )
            loader = DataLoader(
                val_ds,
                batch_size=batch_size,
                shuffle=False,  # deterministic
                num_workers=num_workers,
                pin_memory=pin_memory and torch.cuda.is_available(),
            )

        while True:
            yielded = 0
            for xb, yb in loader:
                xb = xb.to(device, non_blocking=True)
                yb = yb.to(device, non_blocking=True)
                yield xb, yb
                yielded += 1
                if batches_per_epoch is not None and yielded >= batches_per_epoch:
                    break
            if not infinite:
                break


# -------------------------
# Quick smoke tests
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# training-length batch
gen = make_batch_generator(cfg, split="val", device=device, batches_per_epoch=1, set_batch_size=4, infinite=False)
xb, yb = next(gen)
print("train-len val:", xb.shape, yb.shape, xb.device, xb.dtype)

# extrapolation-length batch (example: 8192) — will work as long as val_stream is long enough
try:
    gen2 = make_batch_generator(cfg, split="val", device=device, seq_len=8192, batches_per_epoch=1, set_batch_size=4, infinite=False)
    xb2, yb2 = next(gen2)
    print("8192-len val:", xb2.shape, yb2.shape, xb2.device, xb2.dtype)
except Exception as e:
    print("⚠ 8192 smoke test failed (usually stream too short or RAM constraints):", repr(e))

In [ ]:

#@title Throughput & Scaling Benchmark: Verify O(T×K) Complexity

import time
import gc
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

# ============================
# Configuration
# ============================
BENCHMARK_CONFIG = {
    # Sequence lengths to test
    'seq_lengths': [512, 1024, 2048, 4096, 8192],

    # Batch sizes (can vary by seq_len to fit in memory)
    'batch_size_map': {
        256: 16,
        512: 16,
        1024: 8,
        2048: 4,
        4096: 2,
        8192: 1,
    },

    # Number of warmup iterations (for stable timing)
    'warmup_iters': 10,

    # Number of measurement iterations
    'measure_iters': 50,

    # Whether to test backward pass (memory intensive)
    'test_backward': True,

    # Whether to measure peak memory
    'measure_memory': True,
}

# ============================
# Helper Functions
# ============================

def clear_gpu_memory():
    """Clear GPU memory cache."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

def get_gpu_memory_mb() -> float:
    """Get current GPU memory usage in MB."""
    if not torch.cuda.is_available():
        return 0.0
    return torch.cuda.memory_allocated() / 1024**2

def get_peak_gpu_memory_mb() -> float:
    """Get peak GPU memory usage in MB."""
    if not torch.cuda.is_available():
        return 0.0
    return torch.cuda.max_memory_allocated() / 1024**2

def reset_peak_memory():
    """Reset peak memory stats."""
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()

def benchmark_forward_pass(
    model: torch.nn.Module,
    seq_len: int,
    batch_size: int,
    vocab_size: int,
    device: torch.device,
    warmup_iters: int = 3,
    measure_iters: int = 10,
) -> Dict[str, float]:
    """
    Benchmark forward pass only.
    Returns: {
        'mean_time_ms': float,
        'std_time_ms': float,
        'tokens_per_sec': float,
        'memory_mb': float,
    }
    """
    model.eval()
    clear_gpu_memory()
    reset_peak_memory()

    # Create dummy input
    input_ids = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)

    # Warmup
    with torch.no_grad():
        for _ in range(warmup_iters):
            _ = model(input_ids)
            if torch.cuda.is_available():
                torch.cuda.synchronize()

    # Measure
    times = []
    with torch.no_grad():
        for _ in range(measure_iters):
            if torch.cuda.is_available():
                torch.cuda.synchronize()

            start = time.perf_counter()
            _ = model(input_ids)

            if torch.cuda.is_available():
                torch.cuda.synchronize()

            end = time.perf_counter()
            times.append((end - start) * 1000)  # Convert to ms

    mean_time_ms = np.mean(times)
    std_time_ms = np.std(times)

    # Calculate throughput
    total_tokens = batch_size * seq_len
    tokens_per_sec = total_tokens / (mean_time_ms / 1000)

    # Memory
    memory_mb = get_peak_gpu_memory_mb()

    return {
        'mean_time_ms': mean_time_ms,
        'std_time_ms': std_time_ms,
        'tokens_per_sec': tokens_per_sec,
        'memory_mb': memory_mb,
    }

def benchmark_forward_backward(
    model: torch.nn.Module,
    seq_len: int,
    batch_size: int,
    vocab_size: int,
    device: torch.device,
    warmup_iters: int = 3,
    measure_iters: int = 10,
) -> Dict[str, float]:
    """
    Benchmark forward + backward pass.
    Returns: {
        'mean_time_ms': float,
        'std_time_ms': float,
        'tokens_per_sec': float,
        'memory_mb': float,
    }
    """
    model.train()
    clear_gpu_memory()
    reset_peak_memory()

    # Create dummy input
    input_ids = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
    targets = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)

    # Warmup
    for _ in range(warmup_iters):
        logits = model(input_ids)
        loss = F.cross_entropy(
            logits.view(-1, vocab_size),
            targets.view(-1)
        )
        loss.backward()
        model.zero_grad()
        if torch.cuda.is_available():
            torch.cuda.synchronize()

    # Measure
    times = []
    for _ in range(measure_iters):
        if torch.cuda.is_available():
            torch.cuda.synchronize()

        start = time.perf_counter()

        logits = model(input_ids)
        loss = F.cross_entropy(
            logits.view(-1, vocab_size),
            targets.view(-1)
        )
        loss.backward()
        model.zero_grad()

        if torch.cuda.is_available():
            torch.cuda.synchronize()

        end = time.perf_counter()
        times.append((end - start) * 1000)  # Convert to ms

    mean_time_ms = np.mean(times)
    std_time_ms = np.std(times)

    # Calculate throughput
    total_tokens = batch_size * seq_len
    tokens_per_sec = total_tokens / (mean_time_ms / 1000)

    # Memory
    memory_mb = get_peak_gpu_memory_mb()

    model.eval()  # Reset to eval

    return {
        'mean_time_ms': mean_time_ms,
        'std_time_ms': std_time_ms,
        'tokens_per_sec': tokens_per_sec,
        'memory_mb': memory_mb,
    }

# ============================
# Main Benchmark
# ============================

def run_scaling_benchmark(
    model: torch.nn.Module,
    config: Dict,
    device: torch.device,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Run comprehensive scaling benchmark.

    Returns:
        forward_df: DataFrame with forward pass results
        backward_df: DataFrame with forward+backward results (if enabled)
    """
    vocab_size = model.vocab_size
    K = cfg.num_slots
    H = cfg.num_heads

    forward_results = []
    backward_results = []

    print("="*80)
    print(f"SCALING BENCHMARK: {cfg.num_layers}L, {cfg.embed_dim}D, {H}H, {K} slots")
    print(f"Expected complexity: O(T × K) = O(T × {K})")
    print("="*80)

    for seq_len in config['seq_lengths']:
        batch_size = config['batch_size_map'].get(seq_len, 1)

        print(f"\n{'─'*80}")
        print(f"Testing T={seq_len:5d}, B={batch_size:2d}")
        print(f"{'─'*80}")

        try:
            # Forward pass benchmark
            print("  Forward pass...", end=" ", flush=True)
            fwd_stats = benchmark_forward_pass(
                model=model,
                seq_len=seq_len,
                batch_size=batch_size,
                vocab_size=vocab_size,
                device=device,
                warmup_iters=config['warmup_iters'],
                measure_iters=config['measure_iters'],
            )

            forward_results.append({
                'seq_len': seq_len,
                'batch_size': batch_size,
                'time_ms': fwd_stats['mean_time_ms'],
                'time_std_ms': fwd_stats['std_time_ms'],
                'tokens_per_sec': fwd_stats['tokens_per_sec'],
                'memory_mb': fwd_stats['memory_mb'],
                'time_per_token_us': (fwd_stats['mean_time_ms'] * 1000) / (batch_size * seq_len),
            })

            print(f"✓ {fwd_stats['mean_time_ms']:.1f}ms ± {fwd_stats['std_time_ms']:.1f}ms "
                  f"({fwd_stats['tokens_per_sec']:.0f} tok/s, {fwd_stats['memory_mb']:.0f} MB)")

            # Forward + Backward benchmark
            if config['test_backward']:
                print("  Fwd+Bwd pass...", end=" ", flush=True)
                bwd_stats = benchmark_forward_backward(
                    model=model,
                    seq_len=seq_len,
                    batch_size=batch_size,
                    vocab_size=vocab_size,
                    device=device,
                    warmup_iters=config['warmup_iters'],
                    measure_iters=config['measure_iters'],
                )

                backward_results.append({
                    'seq_len': seq_len,
                    'batch_size': batch_size,
                    'time_ms': bwd_stats['mean_time_ms'],
                    'time_std_ms': bwd_stats['std_time_ms'],
                    'tokens_per_sec': bwd_stats['tokens_per_sec'],
                    'memory_mb': bwd_stats['memory_mb'],
                    'time_per_token_us': (bwd_stats['mean_time_ms'] * 1000) / (batch_size * seq_len),
                })

                print(f"✓ {bwd_stats['mean_time_ms']:.1f}ms ± {bwd_stats['std_time_ms']:.1f}ms "
                      f"({bwd_stats['tokens_per_sec']:.0f} tok/s, {bwd_stats['memory_mb']:.0f} MB)")

            clear_gpu_memory()

        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                print(f"✗ OOM at T={seq_len}, B={batch_size}")
                clear_gpu_memory()
            else:
                raise

    forward_df = pd.DataFrame(forward_results)
    backward_df = pd.DataFrame(backward_results) if backward_results else None

    return forward_df, backward_df

# ============================
# Visualization
# ============================

def plot_scaling_analysis(
    forward_df: pd.DataFrame,
    backward_df: Optional[pd.DataFrame],
    K: int,
    save_path: Optional[str] = None,
):
    """
    Create comprehensive scaling analysis plots.
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle(f'ASA Scaling Analysis (K={K} slots per head)', fontsize=16, fontweight='bold')

    T = forward_df['seq_len'].values

    # --- Plot 1: Time vs Sequence Length (log-log) ---
    ax = axes[0, 0]
    ax.errorbar(T, forward_df['time_ms'], yerr=forward_df['time_std_ms'],
                marker='o', label='Forward', capsize=4, linewidth=2)
    if backward_df is not None:
        ax.errorbar(T, backward_df['time_ms'], yerr=backward_df['time_std_ms'],
                    marker='s', label='Forward+Backward', capsize=4, linewidth=2)

    # Add theoretical curves
    T_theory = np.array([T.min(), T.max()])

    # O(T) reference
    t_linear_ref = T_theory / T_theory[0] * forward_df['time_ms'].iloc[0]
    ax.plot(T_theory, t_linear_ref, 'k--', alpha=0.5, linewidth=1.5, label='O(T) reference')

    # O(T²) reference
    t_quadratic_ref = (T_theory / T_theory[0])**2 * forward_df['time_ms'].iloc[0]
    ax.plot(T_theory, t_quadratic_ref, 'r:', alpha=0.5, linewidth=1.5, label='O(T²) reference')

    ax.set_xscale('log', base=2)
    ax.set_yscale('log')
    ax.set_xlabel('Sequence Length (T)', fontsize=11)
    ax.set_ylabel('Time (ms)', fontsize=11)
    ax.set_title('Latency vs Sequence Length', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # --- Plot 2: Throughput vs Sequence Length ---
    ax = axes[0, 1]
    ax.plot(T, forward_df['tokens_per_sec'], marker='o', linewidth=2, label='Forward')
    if backward_df is not None:
        ax.plot(T, backward_df['tokens_per_sec'], marker='s', linewidth=2, label='Forward+Backward')

    ax.set_xscale('log', base=2)
    ax.set_xlabel('Sequence Length (T)', fontsize=11)
    ax.set_ylabel('Throughput (tokens/sec)', fontsize=11)
    ax.set_title('Throughput vs Sequence Length', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # --- Plot 3: Memory vs Sequence Length ---
    ax = axes[0, 2]
    ax.plot(T, forward_df['memory_mb'], marker='o', linewidth=2, label='Forward')
    if backward_df is not None:
        ax.plot(T, backward_df['memory_mb'], marker='s', linewidth=2, label='Forward+Backward')

    # O(T) memory reference
    mem_linear_ref = T_theory / T_theory[0] * forward_df['memory_mb'].iloc[0]
    ax.plot(T_theory, mem_linear_ref, 'k--', alpha=0.5, linewidth=1.5, label='O(T) reference')

    ax.set_xscale('log', base=2)
    ax.set_yscale('log')
    ax.set_xlabel('Sequence Length (T)', fontsize=11)
    ax.set_ylabel('Peak Memory (MB)', fontsize=11)
    ax.set_title('Memory Usage vs Sequence Length', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # --- Plot 4: Time per Token (should be constant for O(T×K)) ---
    ax = axes[1, 0]
    ax.plot(T, forward_df['time_per_token_us'], marker='o', linewidth=2, label='Forward')
    if backward_df is not None:
        ax.plot(T, backward_df['time_per_token_us'], marker='s', linewidth=2, label='Forward+Backward')

    # Add mean reference line
    fwd_mean = forward_df['time_per_token_us'].mean()
    ax.axhline(fwd_mean, color='blue', linestyle='--', alpha=0.5, label=f'Fwd mean: {fwd_mean:.2f}μs')

    ax.set_xscale('log', base=2)
    ax.set_xlabel('Sequence Length (T)', fontsize=11)
    ax.set_ylabel('Time per Token (μs)', fontsize=11)
    ax.set_title('Time per Token (should be ~constant for O(T×K))', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # --- Plot 5: Scaling Factor Analysis ---
    ax = axes[1, 1]

    # Calculate actual scaling exponent
    T_ratios = T[1:] / T[:-1]
    time_ratios_fwd = forward_df['time_ms'].values[1:] / forward_df['time_ms'].values[:-1]

    # Theoretical: for O(T^n), time_ratio = T_ratio^n, so n = log(time_ratio)/log(T_ratio)
    scaling_exponents_fwd = np.log(time_ratios_fwd) / np.log(T_ratios)

    ax.plot(T[1:], scaling_exponents_fwd, marker='o', linewidth=2, label='Forward (measured)')

    if backward_df is not None:
        time_ratios_bwd = backward_df['time_ms'].values[1:] / backward_df['time_ms'].values[:-1]
        scaling_exponents_bwd = np.log(time_ratios_bwd) / np.log(T_ratios)
        ax.plot(T[1:], scaling_exponents_bwd, marker='s', linewidth=2, label='Fwd+Bwd (measured)')

    ax.axhline(1.0, color='green', linestyle='--', alpha=0.7, linewidth=2, label='O(T) target')
    ax.axhline(2.0, color='red', linestyle=':', alpha=0.7, linewidth=2, label='O(T²) reference')

    ax.set_xscale('log', base=2)
    ax.set_xlabel('Sequence Length (T)', fontsize=11)
    ax.set_ylabel('Scaling Exponent', fontsize=11)
    ax.set_title('Empirical Scaling Exponent (target: 1.0)', fontweight='bold')
    ax.set_ylim(0.5, 2.5)
    ax.legend()
    ax.grid(True, alpha=0.3)

    # --- Plot 6: Summary Statistics Table ---
    ax = axes[1, 2]
    ax.axis('off')

    # Calculate summary stats
    mean_exponent_fwd = scaling_exponents_fwd.mean()
    std_exponent_fwd = scaling_exponents_fwd.std()

    summary_text = f"""
    SUMMARY STATISTICS
    {'─'*35}

    Model Configuration:
      • Layers: {cfg.num_layers}
      • Embed dim: {cfg.embed_dim}
      • Heads: {cfg.num_heads}
      • Slots/head: {K}
      • Total params: {sum(p.numel() for p in model.parameters())/1e6:.1f}M

    Scaling Analysis (Forward):
      • Mean exponent: {mean_exponent_fwd:.3f} ± {std_exponent_fwd:.3f}
      • Target (O(T×K)): 1.000
      • Deviation: {abs(mean_exponent_fwd - 1.0):.3f}

    Throughput (T=1024):
      • Forward: {forward_df[forward_df['seq_len']==1024]['tokens_per_sec'].values[0]:.0f} tok/s
    """

    if backward_df is not None:
        mean_exponent_bwd = scaling_exponents_bwd.mean()
        std_exponent_bwd = scaling_exponents_bwd.std()
        summary_text += f"""
    Scaling Analysis (Fwd+Bwd):
      • Mean exponent: {mean_exponent_bwd:.3f} ± {std_exponent_bwd:.3f}
      • Deviation: {abs(mean_exponent_bwd - 1.0):.3f}

    Throughput (T=1024):
      • Fwd+Bwd: {backward_df[backward_df['seq_len']==1024]['tokens_per_sec'].values[0]:.0f} tok/s
        """

    ax.text(0.05, 0.95, summary_text, transform=ax.transAxes,
            fontsize=10, verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"\n✓ Saved plot to {save_path}")

    plt.show()

    return fig

# ============================
# Run Benchmark
# ============================

print(f"\n🚀 Starting benchmark on {DEVICE}")
print(f"Model: {cfg.num_layers}L × {cfg.embed_dim}D × {cfg.num_heads}H × {cfg.num_slots}K")

forward_df, backward_df = run_scaling_benchmark(
    model=model,
    config=BENCHMARK_CONFIG,
    device=DEVICE,
)

# ============================
# Display Results
# ============================

print("\n" + "="*80)
print("FORWARD PASS RESULTS")
print("="*80)
print(forward_df.to_string(index=False))

if backward_df is not None:
    print("\n" + "="*80)
    print("FORWARD + BACKWARD RESULTS")
    print("="*80)
    print(backward_df.to_string(index=False))

# ============================
# Complexity Verification
# ============================

print("\n" + "="*80)
print("COMPLEXITY VERIFICATION")
print("="*80)

T_vals = forward_df['seq_len'].values
time_vals = forward_df['time_ms'].values

# Fit to power law: time = a * T^b
log_T = np.log(T_vals)
log_time = np.log(time_vals)
b, log_a = np.polyfit(log_T, log_time, 1)
a = np.exp(log_a)

print(f"\nFitted power law: time = {a:.4f} × T^{b:.4f}")
print(f"Expected for O(T×K): exponent ≈ 1.0")
print(f"Expected for O(T²): exponent ≈ 2.0")
print(f"\nActual exponent: {b:.4f}")

if abs(b - 1.0) < 0.15:
    print("✓ CONFIRMED: Empirical scaling matches O(T) / O(T×K) complexity!")
elif abs(b - 2.0) < 0.15:
    print("✗ WARNING: Empirical scaling suggests O(T²) complexity")
else:
    print(f"⚠ NOTICE: Empirical scaling exponent is {b:.3f}")

# ============================
# Visualization
# ============================

fig = plot_scaling_analysis(
    forward_df=forward_df,
    backward_df=backward_df,
    K=cfg.num_slots,
    save_path=None,  # Set to path if you want to save
)

print("\n✓ Benchmark complete!")

In [ ]:

#@title Hungarian Alignment: table + summary + plots

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------------
# Info capture policy for this experiment
# ----------------------------
# We only need read_weights.
ASA_ALIGN_INFO_CFG = dict(
    store_read_weights=True,
    store_read_logits=False,
    store_write_logits=False,
    store_slot_state_norm=False,
    detach_to_cpu=False,
    time_stride=1,
    batch_stride=1,
)

# ----------------------------
# Core: cosine + Hungarian
# ----------------------------

def _cosine_sim_matrix(A_kd: torch.Tensor, B_kd: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    """Return S [K,K] where S[i,j] = cos(A[i], B[j])."""
    A = A_kd.float()
    B = B_kd.float()
    A = A / A.norm(dim=-1, keepdim=True).clamp_min(eps)
    B = B / B.norm(dim=-1, keepdim=True).clamp_min(eps)
    return A @ B.t()

def hungarian_match_cos(A_kd: torch.Tensor, B_kd: torch.Tensor, eps: float = 1e-8):
    """
    A_kd, B_kd: [K,D] slot feature matrices.
    Returns diag_mean, best_mean, perm (LongTensor[K] where perm[i] = matched j in B for i in A).
    """
    if A_kd.shape != B_kd.shape:
        raise ValueError(f"Shape mismatch: A={tuple(A_kd.shape)} B={tuple(B_kd.shape)}")

    S = _cosine_sim_matrix(A_kd, B_kd, eps=eps).detach().cpu().numpy()  # [K,K]
    diag = float(np.diag(S).mean())

    try:
        from scipy.optimize import linear_sum_assignment
        row, col = linear_sum_assignment(-S)  # maximize
        best = float(S[row, col].mean())
        perm = np.empty((S.shape[0],), dtype=np.int64)
        perm[row] = col
        perm = torch.from_numpy(perm)
    except Exception:
        # deterministic greedy fallback
        K = S.shape[0]
        used = np.zeros((K,), dtype=bool)
        perm_g = np.full((K,), -1, dtype=np.int64)
        vals = []
        for i in range(K):
            j = int(np.argmax(np.where(used, -1e9, S[i])))
            used[j] = True
            perm_g[i] = j
            vals.append(S[i, j])
        best = float(np.mean(vals))
        perm = torch.from_numpy(perm_g)

    return diag, best, perm


# ----------------------------
# Feature builders (all -> [K,D])
# ----------------------------

def feat_basic_role_mass(read_weights_bhtk: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    """
    read_weights: [B,H,T,K] -> feature per slot = mean role mass across (B,T) as a vector over heads.
    Returns [K,H].
    """
    hk = read_weights_bhtk.float().mean(dim=(0, 2))   # [H,K]
    kd = hk.t().contiguous()                          # [K,H]
    return kd / kd.norm(dim=-1, keepdim=True).clamp_min(eps)

def feat_timebins(read_weights_bhtk: torch.Tensor, num_bins: int = 8, eps: float = 1e-8) -> torch.Tensor:
    """
    [B,H,T,K] -> [K, H*num_bins]
    """
    B, H, T, K = read_weights_bhtk.shape
    p = read_weights_bhtk.float()
    edges = torch.linspace(0, T, num_bins + 1, device=p.device).round().long().clamp(0, T)

    chunks = []
    for b in range(num_bins):
        t0, t1 = int(edges[b]), int(edges[b + 1])
        if t1 <= t0:
            m = torch.zeros((H, K), device=p.device)
        else:
            m = p[:, :, t0:t1, :].mean(dim=(0, 2))  # [H,K]
        chunks.append(m)

    feats = torch.stack(chunks, dim=0).permute(2, 1, 0).reshape(K, H * num_bins).contiguous()
    return feats / feats.norm(dim=-1, keepdim=True).clamp_min(eps)

def feat_inertia(read_weights_bhtk: torch.Tensor, max_lag: int = 32, per_head: bool = True, eps: float = 1e-8) -> torch.Tensor:
    """
    Autocorr signature of routing weights vs lag.
    If per_head: feature is [K, H*L] else [K, L].
    """
    B, H, T, K = read_weights_bhtk.shape
    L = min(int(max_lag), T - 1)
    p = read_weights_bhtk.float()

    if per_head:
        s = p.mean(dim=0)  # [H,T,K]
        feats = torch.zeros((K, H * L), device=p.device)
        for k in range(K):
            rows = []
            for h in range(H):
                v = s[h, :, k]  # [T]
                sims = []
                for lag in range(1, L + 1):
                    a = v[:-lag]
                    b = v[lag:]
                    sim = (a @ b) / (a.norm() * b.norm().clamp_min(eps))
                    sims.append(sim)
                rows.append(torch.stack(sims))
            feats[k] = torch.cat(rows)
    else:
        s = p.mean(dim=(0, 1))  # [T,K]
        feats = torch.zeros((K, L), device=p.device)
        for k in range(K):
            v = s[:, k]
            sims = []
            for lag in range(1, L + 1):
                a = v[:-lag]
                b = v[lag:]
                sim = (a @ b) / (a.norm() * b.norm().clamp_min(eps))
                sims.append(sim)
            feats[k] = torch.stack(sims)

    return feats / feats.norm(dim=-1, keepdim=True).clamp_min(eps)


# ----------------------------
# Experiment runner
# ----------------------------

@torch.no_grad()
def compute_adjacent_layer_alignment(
    infos,
    *,
    num_bins: int = 8,
    max_lag: int = 32,
    per_head_inertia: bool = True,
):
    rows = []
    for l in range(len(infos) - 1):
        rwA = infos[l].get("read_weights", None) if infos[l] is not None else None
        rwB = infos[l + 1].get("read_weights", None) if infos[l + 1] is not None else None
        if rwA is None or rwB is None:
            continue

        A_basic = feat_basic_role_mass(rwA)
        B_basic = feat_basic_role_mass(rwB)
        d, b, _ = hungarian_match_cos(A_basic, B_basic)
        rows.append({"method": "basic", "layer": l, "diag": d, "best": b, "improvement": b - d})

        A_tb = feat_timebins(rwA, num_bins=num_bins)
        B_tb = feat_timebins(rwB, num_bins=num_bins)
        d, b, _ = hungarian_match_cos(A_tb, B_tb)
        rows.append({"method": "timebins", "layer": l, "diag": d, "best": b, "improvement": b - d})

        A_in = feat_inertia(rwA, max_lag=max_lag, per_head=per_head_inertia)
        B_in = feat_inertia(rwB, max_lag=max_lag, per_head=per_head_inertia)
        d, b, _ = hungarian_match_cos(A_in, B_in)
        rows.append({"method": "inertia", "layer": l, "diag": d, "best": b, "improvement": b - d})

        A_c = torch.cat([A_tb, A_in], dim=-1)
        B_c = torch.cat([B_tb, B_in], dim=-1)
        A_c = A_c / A_c.norm(dim=-1, keepdim=True).clamp_min(1e-8)
        B_c = B_c / B_c.norm(dim=-1, keepdim=True).clamp_min(1e-8)
        d, b, _ = hungarian_match_cos(A_c, B_c)
        rows.append({"method": "combined", "layer": l, "diag": d, "best": b, "improvement": b - d})

    df = pd.DataFrame(rows)
    if not df.empty:
        df["transition"] = df["layer"].astype(str) + "→" + (df["layer"] + 1).astype(str)
        df = df[["method", "layer", "transition", "diag", "best", "improvement"]]
    return df

def summarize_alignment_df(df: pd.DataFrame):
    if df.empty:
        print("No alignment rows computed.")
        return
    g = df.groupby("method")[["best", "improvement"]].mean().sort_values("best", ascending=False)
    print("Average (over adjacent layer transitions)")
    print(g.to_string(float_format=lambda x: f"{x:.4f}"))

def plot_alignment_suite(df: pd.DataFrame, *, title="Adjacent-layer slot alignment", figsize=(14, 9)):
    if df.empty:
        print("No data to plot.")
        return

    methods = ["basic", "timebins", "inertia", "combined"]

    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(2, 2, hspace=0.35, wspace=0.25)

    ax1 = fig.add_subplot(gs[0, 0])  # best vs layer
    ax2 = fig.add_subplot(gs[0, 1])  # improvement vs layer
    ax3 = fig.add_subplot(gs[1, 0])  # distribution of best by method
    ax4 = fig.add_subplot(gs[1, 1])  # scatter: diag vs best

    for ax in (ax1, ax2, ax3, ax4):
        ax.grid(True, alpha=0.2)

    # best cosine across depth
    for method in methods:
        sub = df[df["method"] == method].sort_values("layer")
        if sub.empty:
            continue
        ax1.plot(sub["layer"].values, sub["best"].values, marker="o", linewidth=2, label=method)
    ax1.set_title("Alignment quality across depth")
    ax1.set_xlabel("Layer transition index (l → l+1)")
    ax1.set_ylabel("Mean cosine (optimal matching)")
    ax1.set_ylim(0, 1.05)
    ax1.legend(title="Feature space", frameon=True)

    # improvement across depth
    for method in methods:
        sub = df[df["method"] == method].sort_values("layer")
        if sub.empty:
            continue
        ax2.plot(sub["layer"].values, sub["improvement"].values, marker="o", linewidth=2, label=method)
    ax2.set_title("Gain from remapping across depth")
    ax2.set_xlabel("Layer transition index (l → l+1)")
    ax2.set_ylabel("best − diagonal")
    ax2.axhline(0, linewidth=1)
    ax2.legend(title="Feature space", frameon=True)

    # distribution of best across transitions
    best_lists, labels = [], []
    for method in methods:
        vals = df.loc[df["method"] == method, "best"].values
        if len(vals) == 0:
            continue
        best_lists.append(vals)
        labels.append(method)
    ax3.boxplot(best_lists, labels=labels, showmeans=True)
    ax3.set_title("Distribution of alignment quality")
    ax3.set_ylabel("Mean cosine (optimal matching)")
    ax3.set_ylim(0, 1.05)

    # diag vs best scatter
    for method in methods:
        sub = df[df["method"] == method]
        if sub.empty:
            continue
        ax4.scatter(sub["diag"].values, sub["best"].values, s=40, label=method, alpha=0.85)

    lo = float(df["diag"].min())
    hi = float(df["best"].max())
    ax4.plot([lo, hi], [lo, hi], linewidth=1)  # y=x

    ax4.set_title("Diagonal vs optimal matching")
    ax4.set_xlabel("Mean cosine (diagonal / identity)")
    ax4.set_ylabel("Mean cosine (optimal matching)")
    ax4.set_xlim(max(0, lo - 0.02), min(1.05, hi + 0.02))
    ax4.set_ylim(max(0, lo - 0.02), 1.05)
    ax4.legend(title="Feature space", frameon=True)

    fig.suptitle(title, y=1.02)
    fig.tight_layout()
    plt.show()


# ----------------------------
# Run (directly uses model(...) new args)
# ----------------------------

NUM_BINS = 8
MAX_LAG = 32
PER_HEAD_INERTIA = True

# Recompute infos with the new ASA capture controls
with torch.no_grad():
    logits, infos = model(
        xb[:4],
        return_info=True,
        attention_mask=None,
        info_level="basic",          # "basic" is sufficient here
        info_cfg=ASA_ALIGN_INFO_CFG, # only read_weights
    )

df_align = compute_adjacent_layer_alignment(
    infos,
    num_bins=NUM_BINS,
    max_lag=MAX_LAG,
    per_head_inertia=PER_HEAD_INERTIA,
)

display(df_align)

print()
summarize_alignment_df(df_align)

print()
plot_alignment_suite(
    df_align,
    title=f"Adjacent-layer slot alignment (bins={NUM_BINS}, lag≤{MAX_LAG})"
)

In [ ]:

#@title Policy Inertia: curves + uncertainty + half-life summary

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------------
# Capture policy (new ASA plumbing)
# ----------------------------
ASA_INERTIA_INFO_CFG = dict(
    store_read_weights=True,
    store_read_logits=False,
    store_write_logits=False,
    store_slot_state_norm=False,
    detach_to_cpu=False,   # keep on GPU for speed; df/plots go to cpu anyway
    time_stride=1,
    batch_stride=1,
)

# ----------------------------
# Core: inertia curve
# ----------------------------
@torch.no_grad()
def routing_inertia_curve(
    t2a: torch.Tensor,
    max_lag: int = 64,
    *,
    reduce: str = "mean",          # "mean" | "per_head" | "none"
    eps: float = 1e-8,
) -> torch.Tensor:
    """
    t2a: [B,H,T,K] probabilities (or weights)
    returns:
      - reduce=="mean":     [L]     mean over (B,H,T')
      - reduce=="per_head": [H,L]   mean over (B,T')
      - reduce=="none":     [B,H,L] mean over (T')
    where L = min(max_lag, T-1) and T' = T-L
    """
    if t2a is None:
        raise ValueError("routing_inertia_curve: got t2a=None")
    if t2a.dim() != 4:
        raise ValueError(f"routing_inertia_curve expects [B,H,T,K], got shape={tuple(t2a.shape)}")

    B, H, T, K = t2a.shape
    if T < 2:
        if reduce == "mean":
            return torch.empty((0,), device=t2a.device, dtype=torch.float32)
        if reduce == "per_head":
            return torch.empty((H, 0), device=t2a.device, dtype=torch.float32)
        if reduce == "none":
            return torch.empty((B, H, 0), device=t2a.device, dtype=torch.float32)
        raise ValueError(f"Unknown reduce={reduce}")

    L = int(min(max_lag, T - 1))
    p = t2a.float()

    # normalize per (B,H,T) vector over K
    p = p / (p.norm(dim=-1, keepdim=True).clamp_min(eps))

    # windows: [B,H, T-L, K, L+1] -> [B,H, T-L, L+1, K]
    windows = p.unfold(dimension=2, size=L + 1, step=1).permute(0, 1, 2, 4, 3).contiguous()

    # anchor t vs t+Δ
    a = windows[:, :, :, :1, :]      # [B,H,T-L,1,K]
    b = windows[:, :, :, 1:, :]      # [B,H,T-L,L,K]
    sim = (a * b).sum(dim=-1)        # [B,H,T-L,L]

    if reduce == "mean":
        return sim.mean(dim=(0, 1, 2))      # [L]
    elif reduce == "per_head":
        return sim.mean(dim=(0, 2))         # [H,L]
    elif reduce == "none":
        return sim.mean(dim=2)              # [B,H,L]
    else:
        raise ValueError(f"Unknown reduce={reduce}")


def inertia_half_life(curve_L: torch.Tensor, *, threshold: float = 0.90) -> float:
    """
    curve_L: [L] mean cosine at lags 1..L
    returns the smallest Δ where curve(Δ) <= threshold (linear interp), else L+.
    """
    y = curve_L.detach().float().cpu().numpy()
    if y.size == 0:
        return float("nan")
    # Find first index where below threshold
    idx = np.where(y <= threshold)[0]
    if len(idx) == 0:
        return float(len(y) + 1)  # "beyond range"
    i = int(idx[0])
    if i == 0:
        return 1.0
    # linear interpolation between (i, y[i]) and (i-1, y[i-1])
    y0, y1 = y[i - 1], y[i]
    if abs(y1 - y0) < 1e-12:
        return float(i + 1)
    frac = (threshold - y0) / (y1 - y0)
    # lag values are 1..L, and i corresponds to lag=i+1
    return float((i) + 1 + frac)


# ----------------------------
# Run model with new plumbing
# ----------------------------
MAX_LAG = 64
BATCH_FOR_PAPER = 8  # bump to 16/32 if you want smoother SEM; still cheap on A100

with torch.no_grad():
    logits, infos = model(
        xb[:BATCH_FOR_PAPER],
        return_info=True,
        info_level="basic",
        info_cfg=ASA_INERTIA_INFO_CFG,
    )

# ----------------------------
# Compute curves + uncertainty + half-life
# ----------------------------
curves_mean = {}
curves_sem  = {}
rows = []

for li, info in enumerate(infos):
    rw = None if (info is None) else info.get("read_weights", None)
    if rw is None:
        continue

    # mean curve [L]
    mean_L = routing_inertia_curve(rw, max_lag=MAX_LAG, reduce="mean")  # [L]

    # per (B,H) curve [B,H,L] -> mean±SEM over BH for uncertainty bands
    bh_L = routing_inertia_curve(rw, max_lag=MAX_LAG, reduce="none")    # [B,H,L]
    bh = bh_L.reshape(-1, bh_L.shape[-1]).detach().float().cpu().numpy() # [BH,L]

    mu = bh.mean(axis=0)
    sem = bh.std(axis=0, ddof=1) / max(1.0, np.sqrt(bh.shape[0]))

    curves_mean[li] = mean_L.detach().float().cpu()
    curves_sem[li]  = torch.from_numpy(sem).float()

    # Half-lives at a couple thresholds (paper-friendly)
    hl_90 = inertia_half_life(curves_mean[li], threshold=0.90)
    hl_80 = inertia_half_life(curves_mean[li], threshold=0.80)

    # Fixed-lag summaries (helps interpret “how inertial?”)
    def at_lag(lag: int) -> float:
        if lag <= 0: return float("nan")
        if lag > curves_mean[li].numel(): return float("nan")
        return float(curves_mean[li][lag - 1])

    rows.append(dict(
        layer=li,
        sim_d1=at_lag(1),
        sim_d8=at_lag(8),
        sim_d32=at_lag(32),
        half_life_0p90=hl_90,
        half_life_0p80=hl_80,
        L=int(curves_mean[li].numel()),
        BH=int(bh.shape[0]),
    ))

df_inertia = pd.DataFrame(rows).sort_values("layer")
display(df_inertia)

# ----------------------------
# Plots (paper-facing)
# ----------------------------
# Choose a small set of layers to avoid a spaghetti plot.
def pick_layers(n_layers: int, picks=(0, 1, 2, -1)):
    out = []
    for p in picks:
        idx = (n_layers + p) if p < 0 else p
        if 0 <= idx < n_layers:
            out.append(idx)
    # de-dup while preserving order
    seen = set()
    out2 = []
    for x in out:
        if x not in seen:
            out2.append(x)
            seen.add(x)
    return out2

layer_list = sorted(list(curves_mean.keys()))
show_layers = pick_layers(len(layer_list), picks=(0, 1, 2, -1))
show_layers = [layer_list[i] for i in show_layers] if len(layer_list) > 0 else []

# 1) Inertia curves with uncertainty bands
plt.figure(figsize=(8, 4.5))
for li in show_layers:
    y = curves_mean[li].numpy()
    s = curves_sem[li].numpy()
    x = np.arange(1, len(y) + 1)
    plt.plot(x, y, linewidth=2, label=f"layer {li}")
    plt.fill_between(x, y - s, y + s, alpha=0.15)

plt.axhline(0.90, linewidth=1, linestyle="--")
plt.axhline(0.80, linewidth=1, linestyle="--")
plt.xlabel("Lag Δ")
plt.ylabel("Mean cosine(p_t, p_{t+Δ})")
plt.title(f"Policy inertia curves (mean ± SEM), max_lag={MAX_LAG}")
plt.legend(title="Layer", frameon=True)
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

# 2) Half-life vs depth (two thresholds)
if not df_inertia.empty:
    plt.figure(figsize=(7.5, 4))
    plt.plot(df_inertia["layer"].values, df_inertia["half_life_0p90"].values, marker="o", linewidth=2, label="threshold 0.90")
    plt.plot(df_inertia["layer"].values, df_inertia["half_life_0p80"].values, marker="o", linewidth=2, label="threshold 0.80")
    plt.xlabel("Layer")
    plt.ylabel("Lag Δ at threshold crossing")
    plt.title("Routing inertia half-life vs depth")
    plt.legend(frameon=True)
    plt.grid(True, alpha=0.25)
    plt.tight_layout()
    plt.show()

# 3) Optional: similarity at fixed lags across depth (quick “how inertial?” view)
if not df_inertia.empty:
    plt.figure(figsize=(7.5, 4))
    plt.plot(df_inertia["layer"].values, df_inertia["sim_d1"].values, marker="o", linewidth=2, label="Δ=1")
    plt.plot(df_inertia["layer"].values, df_inertia["sim_d8"].values, marker="o", linewidth=2, label="Δ=8")
    plt.plot(df_inertia["layer"].values, df_inertia["sim_d32"].values, marker="o", linewidth=2, label="Δ=32")
    plt.xlabel("Layer")
    plt.ylabel("Mean cosine similarity")
    plt.title("Routing similarity at fixed lags vs depth")
    plt.ylim(0, 1.05)
    plt.legend(frameon=True)
    plt.grid(True, alpha=0.25)
    plt.tight_layout()
    plt.show()

In [ ]:

#@title Slot Write Half-Life: capture + timescales + ESS + plots (FIXED for new ASA info_level)

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------------
# Capture policy (new ASA plumbing)
# ----------------------------
# NOTE: With your ASA class, write logits are only stored when info_level="full".
ASA_WRITE_INFO_CFG = dict(
    store_read_weights=False,
    store_read_logits=False,
    store_write_logits=True,     # <-- required
    store_slot_state_norm=False,
    detach_to_cpu=False,         # keep on GPU for speed; derived values go to CPU
    time_stride=1,               # you can set >1 for cheaper capture
    batch_stride=1,
)

# ----------------------------
# Prefix stats from write logits
# ----------------------------
@torch.no_grad()
def _prefix_tail_halflife_and_ess_at_t(
    logits_bhkt: torch.Tensor,     # [B,H,K,T]
    t_points: torch.Tensor,        # [P] int
    *,
    threshold: float = 0.5,
    eps: float = 1e-12,
):
    """
    For each requested t in t_points, define p_i ∝ exp(logits[..., i]) over i<=t (prefix softmax).

    Returns:
      hl  : [B,H,K,P] int32   (smallest h s.t. sum_{i=t-h+1..t} p_i >= threshold)
      ess : [B,H,K,P] float32 (ESS = 1 / sum_i p_i^2 over i<=t)
      t_points_used: [P] cpu long
    """
    if logits_bhkt is None or logits_bhkt.dim() != 4:
        raise ValueError(f"Expected logits [B,H,K,T], got {None if logits_bhkt is None else tuple(logits_bhkt.shape)}")

    B, H, K, T = logits_bhkt.shape
    device = logits_bhkt.device

    t_points = t_points.to(device=device, dtype=torch.long)
    t_points = torch.unique(t_points.clamp(0, T - 1))
    P = int(t_points.numel())
    thr = float(threshold)

    hl_out  = torch.empty((B, H, K, P), device=device, dtype=torch.int32)
    ess_out = torch.empty((B, H, K, P), device=device, dtype=torch.float32)

    for pi, t in enumerate(t_points.tolist()):
        L = int(t) + 1
        x = logits_bhkt[..., :L].float()                 # [B,H,K,L]
        m = x.max(dim=-1, keepdim=True).values           # [B,H,K,1]
        z = torch.exp(x - m)                             # [B,H,K,L]
        Z = z.sum(dim=-1).clamp_min(eps)                 # [B,H,K]
        p = z / Z.unsqueeze(-1)                          # [B,H,K,L]

        ess_out[..., pi] = 1.0 / p.square().sum(dim=-1).clamp_min(eps)

        tail = torch.flip(p, dims=[-1]).cumsum(dim=-1)   # [B,H,K,L]
        crossed = (tail >= thr)
        first = crossed.float().argmax(dim=-1)           # [B,H,K]
        never = ~crossed.any(dim=-1)
        first = torch.where(never, torch.full_like(first, L - 1), first)
        hl_out[..., pi] = (first + 1).to(torch.int32)

    return hl_out, ess_out, t_points.detach().cpu()


@torch.no_grad()
def slot_write_timescale_analysis(
    model,
    xb: torch.Tensor,
    attention_mask=None,
    *,
    threshold: float = 0.5,
    n_timepoints: int = 32,
    min_t: int = 32,
    max_t: int | None = None,
    per_head: bool = True,
    max_sample_per_layer: int = 20000,  # keeps pooled plots light
):
    """
    Computes per-layer write tail half-life + ESS using ASA write logits.

    IMPORTANT for your ASA:
      - must call model(..., return_info=True, info_level="full", info_cfg with store_write_logits=True)
    """
    model.eval()
    B, T = xb.shape
    if T < 2:
        raise ValueError("Need T>=2")

    if max_t is None:
        max_t = T - 1
    max_t = min(int(max_t), T - 1)
    min_t = min(int(min_t), max_t)

    tp = torch.linspace(min_t, max_t, steps=int(n_timepoints), device=xb.device).round().long()
    tp = torch.unique(tp)

    # --- CRITICAL FIX: info_level must be "full" for write_logits to be stored in your ASA.
    logits, infos = model(
        xb,
        attention_mask=attention_mask,
        return_info=True,
        info_level="full",
        info_cfg=ASA_WRITE_INFO_CFG,
    )

    out = dict(
        threshold=float(threshold),
        t_points=tp.detach().cpu().tolist(),
        per_layer=[],
    )

    # quick contract sanity
    if not infos:
        print("⚠ No infos returned by model.")
        return out
    if infos[0] is not None:
        k0 = sorted(list(infos[0].keys()))
        if ("write_logits" not in k0) and ("write_logits_raw" not in k0):
            print("⚠ info dict has no write logits keys. Keys:", k0)

    for layer, info in enumerate(infos):
        if info is None:
            continue

        wl = info.get("write_logits", None)
        if wl is None:
            # Some runs may only store raw; accept that too.
            wl = info.get("write_logits_raw", None)
        if wl is None:
            continue

        if wl.dim() != 4:
            print(f"⚠ layer {layer}: write_logits shape {tuple(wl.shape)} (expected [B,H,K,T])")
            continue
        if wl.shape[-1] != T:
            print(f"⚠ layer {layer}: write_logits T={wl.shape[-1]} != xb T={T} (skipping)")
            continue

        hl, ess, _ = _prefix_tail_halflife_and_ess_at_t(wl, tp, threshold=threshold)
        hl_f = hl.float()

        rec = dict(
            layer=int(layer),
            B=int(wl.shape[0]),
            H=int(wl.shape[1]),
            K=int(wl.shape[2]),
            T=int(wl.shape[3]),
            P=int(hl.shape[-1]),
            mean_tail_half_life=float(hl_f.mean().detach().cpu()),
            median_tail_half_life=float(hl_f.median().detach().cpu()),
            mean_ess=float(ess.mean().detach().cpu()),
            median_ess=float(ess.median().detach().cpu()),
        )

        # pooled samples for distribution plots (true pooled values, not head×slot means)
        flat = hl_f.reshape(-1).detach().cpu().numpy()
        if flat.size > max_sample_per_layer:
            idx = np.random.choice(flat.size, size=max_sample_per_layer, replace=False)
            flat = flat[idx]
        rec["half_life_sample"] = flat

        if per_head:
            rec["per_head_mean_tail_half_life"] = hl_f.mean(dim=(0, 2, 3)).detach().cpu().numpy()  # [H]
            rec["per_head_mean_ess"] = ess.mean(dim=(0, 2, 3)).detach().cpu().numpy()              # [H]
            rec["head_slot_half_life"] = hl_f.mean(dim=(0, 3)).detach().cpu().numpy()               # [H,K]
            rec["head_slot_ess"] = ess.mean(dim=(0, 3)).detach().cpu().numpy()                      # [H,K]

        out["per_layer"].append(rec)

    return out


# ----------------------------
# DataFrame + plots
# ----------------------------
def _as_df(res):
    rows = []
    for r in res.get("per_layer", []):
        rows.append(dict(
            layer=r["layer"],
            mean_tail_half_life=r["mean_tail_half_life"],
            median_tail_half_life=r["median_tail_half_life"],
            mean_ess=r["mean_ess"],
            median_ess=r["median_ess"],
        ))
    df = pd.DataFrame(rows)
    if df.empty:
        return pd.DataFrame(columns=["layer","mean_tail_half_life","median_tail_half_life","mean_ess","median_ess"])
    return df.sort_values("layer")


def plot_write_timescale_paper(res, figsize=(13, 4.5)):
    df = _as_df(res)
    if df.empty:
        print("No layers found with write logits (df empty).")
        return

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize, sharex=True)

    ax1.plot(df["layer"], df["mean_tail_half_life"], marker="o", linewidth=2, label="mean")
    ax1.plot(df["layer"], df["median_tail_half_life"], marker="s", linewidth=2, linestyle="--", alpha=0.85, label="median")
    ax1.set_title("Write tail half-life vs depth")
    ax1.set_xlabel("Layer")
    ax1.set_ylabel(f"Tokens to cover ≥{int(res['threshold']*100)}% mass (tail)")
    ax1.grid(True, alpha=0.25)
    ax1.legend(frameon=True)

    ax2.plot(df["layer"], df["mean_ess"], marker="o", linewidth=2, label="mean")
    ax2.plot(df["layer"], df["median_ess"], marker="s", linewidth=2, linestyle="--", alpha=0.85, label="median")
    ax2.set_title("Write distribution ESS vs depth")
    ax2.set_xlabel("Layer")
    ax2.set_ylabel("ESS ≈ 1 / Σ p² (prefix)")
    ax2.grid(True, alpha=0.25)
    ax2.legend(frameon=True)

    fig.suptitle(f"Slot write timescales (threshold={res['threshold']}, P={len(res['t_points'])} timepoints)", y=1.02)
    fig.tight_layout()
    plt.show()


def plot_tail_half_life_distribution(res, *, log_x=True, bins=45, figsize=(7.5, 4.5)):
    samples = []
    for r in res.get("per_layer", []):
        if "half_life_sample" in r:
            samples.append(np.asarray(r["half_life_sample"]))
    if not samples:
        print("No pooled samples available for half-life distribution.")
        return

    vals = np.concatenate(samples)
    vals = vals[np.isfinite(vals) & (vals > 0)]

    plt.figure(figsize=figsize)
    plt.hist(vals, bins=bins, density=True, alpha=0.75)
    if log_x:
        plt.xscale("log")
    plt.xlabel("Tail half-life (tokens)")
    plt.ylabel("Density")
    plt.title("Pooled distribution of write tail half-life")
    plt.grid(True, alpha=0.25)
    plt.tight_layout()
    plt.show()

    print(
        f"Pooled tail half-life stats: "
        f"mean={vals.mean():.1f}, median={np.median(vals):.1f}, "
        f"p90={np.percentile(vals, 90):.1f}, p99={np.percentile(vals, 99):.1f}, "
        f"n={vals.size}"
    )


def plot_head_slot_heatmaps(res, *, which="half_life", figsize=(13, 2.8)):
    layers = res.get("per_layer", [])
    if not layers:
        print("No data.")
        return

    if which == "half_life":
        k = "head_slot_half_life"
        title = "Mean write tail half-life (head × slot)"
    elif which == "ess":
        k = "head_slot_ess"
        title = "Mean write ESS (head × slot)"
    else:
        raise ValueError("which must be 'half_life' or 'ess'")

    if k not in layers[0]:
        print(f"Missing heatmap key: {k}")
        return

    for r in layers:
        mat = np.asarray(r[k])
        plt.figure(figsize=figsize)
        plt.imshow(mat, aspect="auto", interpolation="nearest")
        plt.colorbar()
        plt.xlabel("Slot")
        plt.ylabel("Head")
        plt.title(f"Layer {r['layer']} — {title}")
        plt.tight_layout()
        plt.show()


# ----------------------------
# Run
# ----------------------------
print("Computing slot write half-life + ESS...\n")

BATCH_FOR_PAPER = 8
res_write = slot_write_timescale_analysis(
    model,
    xb[:BATCH_FOR_PAPER],
    attention_mask=None,
    threshold=0.50,
    n_timepoints=32,
    min_t=32,
    max_t=None,
    per_head=True,
)

df_write = _as_df(res_write)
display(df_write)

if df_write.empty:
    print("\n⚠ Still no write logits captured.")
    print("Sanity checks:")
    print("  1) Are ASMBlock / ASMLanguageModel forwarding info_level and info_cfg into ASA?")
    print("  2) Does each layer's info dict contain write_logits/write_logits_raw keys at all?")
    if res_write.get("per_layer") is None:
        print("  (no per_layer records were produced)")
else:
    print("\nPaper plots...\n")
    plot_write_timescale_paper(res_write)

    print("\nPooled distribution...\n")
    plot_tail_half_life_distribution(res_write)

    print("\nAppendix: head×slot half-life maps...\n")
    plot_head_slot_heatmaps(res_write, which="half_life")
    plot_head_slot_heatmaps(res_write, which="ess")

In [ ]:

#@title Big-picture head stratification: quantile fans (+ optional stratification index)

import numpy as np
import matplotlib.pyplot as plt

def _stack_per_head(res, key):
    layers = res.get("per_layer", [])
    if not layers:
        raise ValueError("res has no per_layer records.")
    mats = []
    layer_ids = []
    for r in layers:
        if key not in r:
            raise KeyError(f"Missing key '{key}' in per_layer records.")
        v = np.asarray(r[key], dtype=np.float64)  # [H]
        mats.append(v)
        layer_ids.append(int(r["layer"]))
    M = np.stack(mats, axis=0)  # [L,H]
    return np.asarray(layer_ids, dtype=int), M

def _quantiles(M_LH, qs=(0.10, 0.25, 0.50, 0.75, 0.90)):
    # returns dict q -> [L]
    out = {}
    for q in qs:
        out[q] = np.quantile(M_LH, q, axis=1)
    return out

def _strat_index(M_LH, eps=1e-12):
    """
    Simple, interpretable stratification score per layer:
      log10(p90/p10). Larger = more stratified across heads.
    Works for positive quantities (ESS, half-life).
    """
    p10 = np.quantile(M_LH, 0.10, axis=1)
    p90 = np.quantile(M_LH, 0.90, axis=1)
    return np.log10((p90 + eps) / (p10 + eps))

def plot_quantile_fan(layer_ids, M_LH, *, title, ylabel, logy=True, show_index=True, figsize=(8.8, 4.8)):
    Q = _quantiles(M_LH, qs=(0.10, 0.25, 0.50, 0.75, 0.90))
    med = Q[0.50]

    fig, ax = plt.subplots(1, 1, figsize=figsize)
    ax.grid(True, alpha=0.25)

    # Bands: 10–90 (light) and 25–75 (darker)
    ax.fill_between(layer_ids, Q[0.10], Q[0.90], alpha=0.15, label="10–90% across heads")
    ax.fill_between(layer_ids, Q[0.25], Q[0.75], alpha=0.25, label="25–75% across heads")
    ax.plot(layer_ids, med, marker="o", linewidth=2.2, label="median head")

    ax.set_title(title)
    ax.set_xlabel("Layer")
    ax.set_ylabel(ylabel)
    if logy:
        ax.set_yscale("log")

    ax.legend(frameon=True, loc="best")
    plt.tight_layout()
    plt.show()

    # Optional: stratification index (single number per layer)
    if show_index:
        s = _strat_index(M_LH)
        plt.figure(figsize=(7.6, 3.8))
        plt.plot(layer_ids, s, marker="o", linewidth=2.2)
        plt.grid(True, alpha=0.25)
        plt.xlabel("Layer")
        plt.ylabel("log10(p90/p10)")
        plt.title(f"Stratification index vs depth (derived from: {title})")
        plt.tight_layout()
        plt.show()

# ---- build matrices from res_write ----
layer_ids, HL = _stack_per_head(res_write, "per_head_mean_tail_half_life")  # [L,H]
_, ESS = _stack_per_head(res_write, "per_head_mean_ess")                   # [L,H]

# ---- Panel E: big-picture quantile fans ----
plot_quantile_fan(
    layer_ids, HL,
    title="(E) Head population structure: write tail half-life distribution vs depth",
    ylabel="Tail half-life (tokens)",
    logy=True,
    show_index=True,
)

plot_quantile_fan(
    layer_ids, ESS,
    title="(E2) Head population structure: write ESS distribution vs depth",
    ylabel="ESS (prefix)",
    logy=True,
    show_index=True,
)

In [ ]:

#@title Big-picture: head stratification across layers (rank stability panels)

import numpy as np
import matplotlib.pyplot as plt

def _stack_per_head(res, key):
    layers = res.get("per_layer", [])
    if not layers:
        raise ValueError("res has no per_layer records.")
    mats = []
    layer_ids = []
    for r in layers:
        if key not in r:
            raise KeyError(f"Missing key '{key}' in per_layer records.")
        mats.append(np.asarray(r[key], dtype=np.float64))  # [H]
        layer_ids.append(int(r["layer"]))
    M = np.stack(mats, axis=0)  # [L,H]
    return np.asarray(layer_ids, dtype=int), M

def _rankdata_ordinal(x):
    """
    Ordinal ranks (1..H) with average for ties (rare here).
    Higher values => rank 1 (so rank order corresponds to "largest is best").
    """
    # sort descending
    order = np.argsort(-x, kind="mergesort")
    ranks = np.empty_like(order, dtype=np.float64)
    ranks[order] = np.arange(1, len(x) + 1, dtype=np.float64)

    # tie handling (average ranks)
    # if exact ties exist, average their ranks
    xs = x[order]
    i = 0
    while i < len(xs):
        j = i + 1
        while j < len(xs) and xs[j] == xs[i]:
            j += 1
        if j - i > 1:
            avg = ranks[order[i:j]].mean()
            ranks[order[i:j]] = avg
        i = j
    return ranks

def _spearman_corr(a, b, eps=1e-12):
    # a,b are ranks (floats)
    a = a - a.mean()
    b = b - b.mean()
    denom = (np.sqrt((a*a).sum()) * np.sqrt((b*b).sum())) + eps
    return float((a*b).sum() / denom)

def rank_correlation_matrix(M_LH):
    """
    M_LH: [L,H] metric values per layer per head.
    Returns:
      R_LL: [L,L] Spearman correlation of head ranks between layers.
      ranks_LH: [L,H] ranks per layer (1..H, 1 = highest metric).
    """
    L, H = M_LH.shape
    ranks = np.stack([_rankdata_ordinal(M_LH[i]) for i in range(L)], axis=0)  # [L,H]
    R = np.zeros((L, L), dtype=np.float64)
    for i in range(L):
        for j in range(L):
            R[i, j] = _spearman_corr(ranks[i], ranks[j])
    return R, ranks

def consensus_order_and_error(ranks_LH):
    """
    ranks_LH: [L,H], 1 = highest.
    Returns:
      consensus_order: [H] head indices sorted by mean rank (ascending).
      mean_abs_err_L: [L] mean absolute rank error per layer vs consensus ranks.
    """
    mean_rank = ranks_LH.mean(axis=0)  # [H]
    consensus_order = np.argsort(mean_rank, kind="mergesort")  # best (low rank) first

    # consensus ranks: map head -> rank position 1..H
    consensus_rank = np.empty_like(consensus_order, dtype=np.float64)
    consensus_rank[consensus_order] = np.arange(1, len(consensus_order)+1, dtype=np.float64)

    mean_abs_err = np.mean(np.abs(ranks_LH - consensus_rank[None, :]), axis=1)  # [L]
    return consensus_order, mean_abs_err, mean_rank

def plot_rank_stability_panels(layer_ids, M_LH, *, metric_name, figsize=(13.5, 4.8)):
    R, ranks = rank_correlation_matrix(M_LH)
    consensus_order, mean_abs_err, mean_rank = consensus_order_and_error(ranks)

    L, H = M_LH.shape

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)

    # ---- Panel F: Spearman rank-correlation matrix ----
    im = ax1.imshow(R, vmin=0.0, vmax=1.0, interpolation="nearest", aspect="auto")
    ax1.set_title(f"(F) Rank consistency across layers ({metric_name})\nSpearman( rank(head, layer i), rank(head, layer j) )")
    ax1.set_xticks(range(L)); ax1.set_yticks(range(L))
    ax1.set_xticklabels([f"L{l}" for l in layer_ids], rotation=45, ha="right")
    ax1.set_yticklabels([f"L{l}" for l in layer_ids])
    cbar = plt.colorbar(im, ax=ax1, fraction=0.046, pad=0.04)
    cbar.set_label("Spearman ρ (1 = identical ordering)")

    # ---- Panel G: agreement to consensus ordering ----
    ax2.plot(layer_ids, mean_abs_err, marker="o", linewidth=2.2)
    ax2.grid(True, alpha=0.25)
    ax2.set_title(f"(G) Agreement with consensus head ordering ({metric_name})")
    ax2.set_xlabel("Layer")
    ax2.set_ylabel("Mean |rank_layer − rank_consensus|  (lower = more stable)")

    fig.tight_layout()
    plt.show()

    # Print a compact “consensus head order” for the paper notebook text output
    # (best heads first according to average rank)
    print(f"\nConsensus head order by {metric_name} (best → worst):")
    print("  " + " ".join([f"H{h}" for h in consensus_order.tolist()]))

    # Also show mean rank per head (optional, useful for debugging)
    # print("Mean rank per head:", {f"H{i}": float(mean_rank[i]) for i in range(H)})

# ---- Build matrices from res_write ----
layer_ids, HL  = _stack_per_head(res_write, "per_head_mean_tail_half_life")  # [L,H]
_,        ESS = _stack_per_head(res_write, "per_head_mean_ess")              # [L,H]

# ---- Run panels for both metrics ----
plot_rank_stability_panels(layer_ids, HL,  metric_name="write tail half-life")
plot_rank_stability_panels(layer_ids, ESS, metric_name="write ESS")

In [ ]:

#@title Build event_toks + ctrl_toks (paper-facing): model-driven events + matched controls

import numpy as np
import torch

# ==========================================================
# Produces:
#   event_toks: np.ndarray[int64]  token indices in [0, T)
#   ctrl_toks : np.ndarray[int64]  matched control indices in [0, T)
#
# Requires:
#   model, xb   (xb: [B,T] input ids)
#
# Options:
#   event_mode="model"  -> derive "event" tokens from ASA routing dynamics
#   event_mode="given"  -> use provided EVENT_TOKS_GIVEN list/array
#
# Control matching:
#   - always matches position bins (prevents trivial "late tokens differ")
#   - optionally matches token id distribution (stronger control)
# ==========================================================

# ----------------------------
# Capture policy for event discovery (new ASA plumbing)
# ----------------------------
ASA_EVENT_INFO_CFG = dict(
    store_read_weights=True,      # needed to compute routing dynamics
    store_read_logits=False,
    store_write_logits=False,
    store_slot_state_norm=False,
    detach_to_cpu=False,
    time_stride=1,
    batch_stride=1,
)

def _sanitize_unique(idxs, T):
    idxs = np.asarray(idxs, dtype=np.int64)
    idxs = idxs[(idxs >= 0) & (idxs < T)]
    idxs = np.unique(idxs)
    return idxs

def _position_bins(T, n_bins):
    # returns bin index for each t in [0..T-1]
    edges = np.linspace(0, T, n_bins + 1).astype(np.int64)
    # ensure monotonic
    edges[0] = 0
    edges[-1] = T
    bins = np.zeros((T,), dtype=np.int64)
    for b in range(n_bins):
        t0, t1 = edges[b], edges[b + 1]
        if t1 > t0:
            bins[t0:t1] = b
    return bins, n_bins

@torch.no_grad()
def _event_scores_from_read_weights(rw_bhtk: torch.Tensor, eps=1e-8):
    """
    rw: [B,H,T,K]  (probabilities over slots)
    Returns three per-token scores (length T), computed on mean-over-B,H routing:
      - delta_l1: mean L1 change in routing between t and t-1
      - delta_cos: 1 - cosine(r_t, r_{t-1})
      - entropy: entropy(r_t)  (peaks when routing is diffuse/uncertain)
    """
    # mean policy across batch+heads: [T,K]
    p = rw_bhtk.float().mean(dim=(0, 1))
    p = p / p.sum(dim=-1, keepdim=True).clamp_min(eps)

    # deltas vs previous step (pad t=0 with 0)
    p0 = p[:-1]
    p1 = p[1:]

    delta_l1 = (p1 - p0).abs().sum(dim=-1)  # [T-1]
    delta_l1 = torch.cat([torch.zeros((1,), device=p.device), delta_l1], dim=0)

    # cosine distance (routing normalized)
    pn = p / p.norm(dim=-1, keepdim=True).clamp_min(eps)
    cos = (pn[1:] * pn[:-1]).sum(dim=-1).clamp(-1, 1)
    delta_cos = 1.0 - cos
    delta_cos = torch.cat([torch.zeros((1,), device=p.device), delta_cos], dim=0)

    # entropy
    ent = -(p * torch.log(p.clamp_min(eps))).sum(dim=-1)

    return delta_l1, delta_cos, ent

def build_event_and_control_tokens(
    model,
    xb: torch.Tensor,
    *,
    event_mode: str = "model",          # "model" | "given"
    EVENT_TOKS_GIVEN=None,              # used if event_mode="given"
    n_events: int = 256,                # target number of event tokens
    min_t: int = 8,                     # ignore earliest positions (often special tokens)
    max_t: int | None = None,
    score: str = "delta_cos",           # "delta_cos" | "delta_l1" | "entropy"
    position_bins: int = 16,            # for matched controls
    match_token_id: bool = True,        # stronger control: match token ids too
    seed: int = 1337,
):
    """
    Returns (event_toks, ctrl_toks).
    """
    assert xb.dim() == 2, "xb must be [B,T]"
    B, T = xb.shape
    if max_t is None:
        max_t = T - 1
    max_t = min(int(max_t), T - 1)
    min_t = min(int(min_t), max_t)

    rng = np.random.default_rng(seed)

    # ------------------------
    # Build event_toks
    # ------------------------
    if event_mode == "given":
        if EVENT_TOKS_GIVEN is None:
            raise ValueError("event_mode='given' requires EVENT_TOKS_GIVEN")
        event_toks = _sanitize_unique(EVENT_TOKS_GIVEN, T)

    elif event_mode == "model":
        with torch.no_grad():
            _, infos = model(
                xb,
                return_info=True,
                info_level="basic",
                info_cfg=ASA_EVENT_INFO_CFG,
            )

        # stack per-layer scores, then take a robust aggregate (median across layers)
        scores_by_layer = []
        for info in infos:
            rw = None if (info is None) else info.get("read_weights", None)
            if rw is None:
                continue
            d_l1, d_cos, ent = _event_scores_from_read_weights(rw)
            if score == "delta_l1":
                s = d_l1
            elif score == "delta_cos":
                s = d_cos
            elif score == "entropy":
                s = ent
            else:
                raise ValueError(f"Unknown score={score}")
            scores_by_layer.append(s.detach())

        if not scores_by_layer:
            raise RuntimeError("No read_weights captured for event discovery. Check ASA_EVENT_INFO_CFG / info_level.")

        S = torch.stack(scores_by_layer, dim=0)   # [L,T]
        s_med = S.median(dim=0).values            # [T]

        # restrict to range
        mask = torch.zeros((T,), device=s_med.device, dtype=torch.bool)
        mask[min_t:max_t+1] = True
        s_med = s_med.masked_fill(~mask, float("-inf"))

        # pick top-n_events
        n_events = int(min(n_events, (max_t - min_t + 1)))
        top = torch.topk(s_med, k=n_events, dim=0).indices.detach().cpu().numpy()
        event_toks = np.sort(np.unique(top.astype(np.int64)))

    else:
        raise ValueError("event_mode must be 'model' or 'given'")

    # apply min/max window
    event_toks = event_toks[(event_toks >= min_t) & (event_toks <= max_t)]
    if event_toks.size == 0:
        raise RuntimeError("No event tokens after applying [min_t, max_t].")

    # ------------------------
    # Build matched controls
    # ------------------------
    # Build position bins and match per-bin counts
    bins, nb = _position_bins(T, position_bins)
    event_bins = bins[event_toks]

    # Candidate pool excludes event tokens
    all_t = np.arange(T, dtype=np.int64)
    is_event = np.zeros((T,), dtype=bool)
    is_event[event_toks] = True

    ctrl = []
    # Optional token-id matching uses xb[0] token IDs as the "position token id".
    # (You can switch to a specific example in batch; for paper this is fine.)
    toks_ids = xb[0].detach().cpu().numpy()  # [T]

    for b in range(nb):
        e_in_bin = event_toks[event_bins == b]
        if e_in_bin.size == 0:
            continue

        cand = all_t[(bins == b) & (~is_event)]
        if cand.size == 0:
            continue

        if match_token_id:
            # For each event position, try to pick a control position in same bin with same token id.
            # Fall back to random in-bin if none exists.
            cand_by_id = {}
            for t in cand:
                cand_by_id.setdefault(int(toks_ids[t]), []).append(int(t))

            # Shuffle each list deterministically
            for k in cand_by_id:
                rng.shuffle(cand_by_id[k])

            used = set()
            for t_ev in e_in_bin:
                tid = int(toks_ids[t_ev])
                picked = None
                lst = cand_by_id.get(tid, [])
                while lst:
                    t = lst.pop()
                    if t not in used:
                        picked = t
                        break
                if picked is None:
                    # fallback: pick any unused cand
                    avail = [int(t) for t in cand if int(t) not in used]
                    if not avail:
                        break
                    picked = int(rng.choice(avail))
                used.add(picked)
                ctrl.append(picked)
        else:
            # simpler: random sample same count in this bin
            k = min(len(e_in_bin), len(cand))
            picked = rng.choice(cand, size=k, replace=False)
            ctrl.extend([int(x) for x in picked])

    ctrl_toks = np.asarray(ctrl, dtype=np.int64)

    # Make lengths match (if token-id constraints limited controls)
    n = min(len(event_toks), len(ctrl_toks))
    event_toks = event_toks[:n]
    ctrl_toks  = ctrl_toks[:n]

    return event_toks, ctrl_toks

# ----------------------------
# Run (recommended defaults)
# ----------------------------
event_toks, ctrl_toks = build_event_and_control_tokens(
    model,
    xb,
    event_mode="model", # or given
    n_events=256,
    min_t=8,
    max_t=None,
    score="delta_cos",       # tends to be very interpretable as “policy shift”
    position_bins=16,
    match_token_id=True,
    seed=1337,
)

print(f"Built event/control tokens: event={len(event_toks)} ctrl={len(ctrl_toks)}")
print("First 20 event_toks:", event_toks[:20])
print("First 20 ctrl_toks :", ctrl_toks[:20])

In [ ]:

#@title Canonicalized Head Semantic Role Alignment (paper-facing): event tokens vs matched controls

import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment

# ==========================================================
# Expected inputs:
#   model, xb
#   event_toks: 1D list/np array of token indices in [0, T-1]
#   ctrl_toks : 1D list/np array of matched control indices in [0, T-1]
#
# Notes:
#   - This cell runs model(...) with ASA info plumbing (no precomputed infos).
#   - It captures read_weights only (OOM-friendly).
# ==========================================================

# ----------------------------
# Capture policy (new ASA plumbing)
# ----------------------------
ASA_CANON_INFO_CFG = dict(
    store_read_weights=True,
    store_read_logits=False,
    store_write_logits=False,
    store_slot_state_norm=False,
    detach_to_cpu=False,    # keep on GPU for speed; we move reduced tensors to CPU
    time_stride=1,
    batch_stride=1,
)

# ----------------
# Small helpers
# ----------------
def _safe_log(x, eps=1e-9):
    return torch.log(x.clamp_min(eps))

def kl_divergence(p, q, eps=1e-9):
    # KL(p||q) over last dim
    return (p * (_safe_log(p, eps) - _safe_log(q, eps))).sum(dim=-1)

def cosine_sim(a, b, eps=1e-9):
    an = a.norm(dim=-1).clamp_min(eps)
    bn = b.norm(dim=-1).clamp_min(eps)
    return (a * b).sum(dim=-1) / (an * bn)

def hungarian_align(A_hd: np.ndarray, B_hd: np.ndarray):
    """
    Align rows of B to rows of A by maximizing cosine similarity.
    A,B: [H,D] numpy arrays
    Returns:
      perm: array length H where perm[i] is B-row matched to A-row i
      q   : mean matched cosine similarity
    """
    A = A_hd.astype(np.float64)
    B = B_hd.astype(np.float64)
    A = A / (np.linalg.norm(A, axis=1, keepdims=True) + 1e-9)
    B = B / (np.linalg.norm(B, axis=1, keepdims=True) + 1e-9)
    S = A @ B.T  # [H,H]
    row, col = linear_sum_assignment(-S)
    perm = np.empty((A.shape[0],), dtype=np.int64)
    perm[row] = col
    return perm, float(S[row, col].mean())

def apply_perm_head0(x_h, perm, device=None):
    """
    x_h: torch tensor with head axis at dim=0: [H,...]
    perm maps canonical head i -> x head perm[i]
    output[i] = x[perm[i]]
    """
    if device is None:
        device = x_h.device
    idx = torch.tensor(perm, device=device, dtype=torch.long)
    return x_h.index_select(0, idx)

# -------------------------------------------
# 0) Run model to get per-layer read_weights
# -------------------------------------------
with torch.no_grad():
    logits, infos = model(
        xb,
        return_info=True,
        info_level="basic",          # read_weights allowed in "basic"
        info_cfg=ASA_CANON_INFO_CFG,
    )

# Collect valid layers
rw_by_layer = {}
valid_layers = []
for l, info in enumerate(infos):
    rw = None if (info is None) else info.get("read_weights", None)
    if rw is None or rw.dim() != 4:
        continue
    # rw: [B,H,T,K]
    rw_by_layer[l] = rw.float()
    valid_layers.append(l)

assert len(valid_layers) >= 2, "Need at least 2 layers with read_weights captured."

L0 = valid_layers[0]
B, H, T, K = rw_by_layer[L0].shape

# ----------------------------
# 1) sanitize token sets
# ----------------------------
event_toks = np.asarray(event_toks, dtype=np.int64)
ctrl_toks  = np.asarray(ctrl_toks, dtype=np.int64)

event_toks = event_toks[(event_toks >= 0) & (event_toks < T)]
ctrl_toks  = ctrl_toks[(ctrl_toks  >= 0) & (ctrl_toks  < T)]

n = min(len(event_toks), len(ctrl_toks))
if n == 0:
    raise ValueError("No valid event/control tokens after clipping to [0, T).")
if len(event_toks) != len(ctrl_toks):
    print(f"⚠ event/control count mismatch after clipping: event={len(event_toks)} ctrl={len(ctrl_toks)}. Using n={n}.")
    event_toks = event_toks[:n]
    ctrl_toks  = ctrl_toks[:n]

print(f"Valid layers: {valid_layers}")
print(f"Shapes: B={B}, H={H}, T={T}, K={K}")
print(f"Event tokens: {len(event_toks)} | Controls: {len(ctrl_toks)}")

# ----------------------------------------------------
# 2) Head feature vectors for cross-layer canonicalization
# ----------------------------------------------------
def build_head_features_from_rw(rw_bhtk: torch.Tensor, max_lag=16):
    """
    rw: [B,H,T,K] probabilities over slots
    Return [H,D] numpy features capturing:
      - inertia_lag1 (mean cosine p_t vs p_{t+1})
      - inertia_mean (mean cosine across lags)
      - inertia_slope (slope of cosine vs lag)
      - entropy_mean (mean H(p_t))
      - eff_slots (exp(entropy_mean))
      - top4_mass (mean sum of top4 probs)
    """
    with torch.no_grad():
        p = rw_bhtk.mean(dim=0)  # [H,T,K]
        p = p / p.sum(dim=-1, keepdim=True).clamp_min(1e-9)

        ent = -(p * _safe_log(p)).sum(dim=-1)     # [H,T]
        ent_mean = ent.mean(dim=-1)               # [H]
        eff_slots = torch.exp(ent_mean)           # [H]

        top4 = torch.topk(p, k=min(4, K), dim=-1).values.sum(dim=-1)  # [H,T]
        top4_mean = top4.mean(dim=-1)                                 # [H]

        L = min(int(max_lag), T - 1)
        cos_lags = []
        for lag in range(1, L + 1):
            a = p[:, :-lag, :]
            b = p[:, lag:, :]
            c = cosine_sim(a, b).mean(dim=-1)  # [H]
            cos_lags.append(c)
        cos_lags = torch.stack(cos_lags, dim=-1)  # [H,L]

        inertia_lag1 = cos_lags[:, 0]
        inertia_mean = cos_lags.mean(dim=-1)

        xs = torch.arange(1, L + 1, device=rw_bhtk.device, dtype=torch.float32)
        xs = (xs - xs.mean()) / xs.std().clamp_min(1e-9)
        ys = (cos_lags - cos_lags.mean(dim=-1, keepdim=True)) / cos_lags.std(dim=-1, keepdim=True).clamp_min(1e-9)
        inertia_slope = (ys * xs).mean(dim=-1)

        feats = torch.stack(
            [inertia_lag1, inertia_mean, inertia_slope, ent_mean, eff_slots, top4_mean], dim=-1
        )
        feats = feats / feats.norm(dim=-1, keepdim=True).clamp_min(1e-9)
        return feats.detach().cpu().numpy()

head_features_by_layer = {l: build_head_features_from_rw(rw_by_layer[l], max_lag=16) for l in valid_layers}
D = head_features_by_layer[L0].shape[1]
print(f"Head feature dim for canonicalization: D={D}")

# -------------------------------------------------------------------
# 3) Canonicalize heads across layers (compose Hungarian permutations)
# -------------------------------------------------------------------
canon_perm_to_layer = {L0: np.arange(H, dtype=np.int64)}
align_quality = {}

for i in range(len(valid_layers) - 1):
    la = valid_layers[i]
    lb = valid_layers[i + 1]
    A = head_features_by_layer[la]  # [H,D]
    Bf = head_features_by_layer[lb] # [H,D]

    perm_ab, q = hungarian_align(A, Bf)          # la-head -> lb-head
    align_quality[(la, lb)] = q

    canon_to_la = canon_perm_to_layer[la]
    canon_to_lb = perm_ab[canon_to_la]           # compose
    canon_perm_to_layer[lb] = canon_to_lb

print("\nAdjacent-layer head alignment quality (mean cosine under matching):")
for (la, lb), q in align_quality.items():
    print(f"  {la:>2} -> {lb:<2}: {q:.4f}")

# ---------------------------------------------------
# 4) Token-conditioned head metrics (event vs control)
# ---------------------------------------------------
def per_layer_head_metrics(rw_bhtk: torch.Tensor, toks):
    """
    rw: [B,H,T,K]
    toks: list/array token indices
    Returns dict of [H] tensors:
      - kl_to_mean: mean KL(p_h(t) || mean_h(p(t))) over tokens
      - entropy: mean entropy(p_h(t)) over tokens
      - top1: mean max prob over tokens
      - arg_conflict: fraction tokens where head argmax != mean-head argmax
    """
    with torch.no_grad():
        p = rw_bhtk.mean(dim=0)  # [H,T,K]
        p = p / p.sum(dim=-1, keepdim=True).clamp_min(1e-9)

        toks = torch.as_tensor(toks, device=p.device, dtype=torch.long)
        pt = p.index_select(1, toks)  # [H,|t|,K]

        pmean = pt.mean(dim=0, keepdim=True)  # [1,|t|,K]
        pmean = pmean / pmean.sum(dim=-1, keepdim=True).clamp_min(1e-9)

        kl = kl_divergence(pt, pmean).mean(dim=1)                              # [H]
        ent = (-(pt * _safe_log(pt)).sum(dim=-1)).mean(dim=1)                  # [H]
        top1 = pt.max(dim=-1).values.mean(dim=1)                                # [H]
        head_arg = pt.argmax(dim=-1)                                            # [H,|t|]
        mean_arg = pmean.argmax(dim=-1).squeeze(0)                              # [|t|]
        arg_conflict = (head_arg != mean_arg.unsqueeze(0)).float().mean(dim=1)  # [H]

        return {"kl_to_mean": kl, "entropy": ent, "top1": top1, "arg_conflict": arg_conflict}

layers = valid_layers
metrics = ["kl_to_mean", "entropy", "top1", "arg_conflict"]

event_mat = {m: torch.zeros((len(layers), H)) for m in metrics}
ctrl_mat  = {m: torch.zeros((len(layers), H)) for m in metrics}

for li, l in enumerate(layers):
    rw = rw_by_layer[l]
    ev = per_layer_head_metrics(rw, event_toks)
    ct = per_layer_head_metrics(rw, ctrl_toks)

    perm = canon_perm_to_layer[l]  # canonical head i corresponds to layer head perm[i]
    for m in metrics:
        event_mat[m][li] = apply_perm_head0(ev[m], perm, device=ev[m].device).detach().cpu()
        ctrl_mat[m][li]  = apply_perm_head0(ct[m], perm, device=ct[m].device).detach().cpu()

diff_mat = {m: (event_mat[m] - ctrl_mat[m]) for m in metrics}

# ---------------------------------------------------
# 5) Falsifiable tests (paper-friendly readouts)
# ---------------------------------------------------
decision_window = [l for l in layers if (l >= 2 and l <= 5)]
dw_idx = [layers.index(l) for l in decision_window] if decision_window else list(range(len(layers)))

branch_score = diff_mat["kl_to_mean"][dw_idx].mean(dim=0).numpy()  # [H]
rank = np.argsort(-branch_score)

print("\nCanonical head 'brancher' ranking (higher = more event-specific divergence):")
for i in rank[:min(H, 12)]:
    print(f"  Hcanon {i:2d}: ΔKL_to_mean (decision-window avg) = {branch_score[i]:+.4f}")

ref = diff_mat["kl_to_mean"][dw_idx].mean(dim=0)
ref = (ref - ref.mean()) / ref.std().clamp_min(1e-9)

layer_corrs = []
for li in range(len(layers)):
    v = diff_mat["kl_to_mean"][li]
    v = (v - v.mean()) / v.std().clamp_min(1e-9)
    layer_corrs.append(float((v * ref).mean()))

print("\nLayerwise correlation of ΔKL_to_mean head profile with reference profile:")
for l, c in zip(layers, layer_corrs):
    print(f"  Layer {l:2d}: corr = {c:+.3f}")

# ---------------------------------------------------
# 6) Plots (heatmaps + role curves + signature scatter)
# ---------------------------------------------------
def plot_heatmap(mat, title, *, vmin=None, vmax=None):
    plt.figure(figsize=(12, 5.5))
    plt.imshow(mat, aspect="auto", interpolation="nearest", vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.yticks(range(len(layers)), [f"L{l}" for l in layers])
    plt.xticks(range(H), [f"H{i}" for i in range(H)], rotation=0)
    plt.xlabel("Canonical head")
    plt.ylabel("Layer")
    plt.title(title)
    plt.tight_layout()
    plt.show()

plot_heatmap(diff_mat["kl_to_mean"].numpy(),
             "Δ KL(head || mean): event − control (canonicalized heads)")

plot_heatmap(diff_mat["entropy"].numpy(),
             "Δ head entropy: event − control (canonicalized heads)")

plot_heatmap(diff_mat["arg_conflict"].numpy(),
             "Δ argmax conflict vs mean: event − control (canonicalized heads)")

# Role curves: top-k branchers
topk = min(4, H)
plt.figure(figsize=(12, 4.8))
for j in range(topk):
    h = rank[j]
    plt.plot(layers, diff_mat["kl_to_mean"][:, h].numpy(), marker="o", label=f"Hcanon {h}")
if decision_window:
    plt.axvspan(min(decision_window), max(decision_window), alpha=0.15)
plt.axhline(0, lw=1)
plt.xlabel("Layer")
plt.ylabel("Δ KL(head || mean)  (event − control)")
plt.title("Event-specific head divergence across depth (canonical heads)")
plt.grid(True, alpha=0.25)
plt.legend(title="Top branchers", frameon=True)
plt.tight_layout()
plt.show()

# "confident split" signature: ΔEntropy vs ΔKL in decision window
sig_dkl  = diff_mat["kl_to_mean"][dw_idx].mean(dim=0).numpy()
sig_dent = diff_mat["entropy"][dw_idx].mean(dim=0).numpy()
sig_top1 = diff_mat["top1"][dw_idx].mean(dim=0).numpy()

plt.figure(figsize=(7.2, 6.2))
plt.scatter(sig_dent, sig_dkl, s=45, alpha=0.9)
for h in range(H):
    plt.text(sig_dent[h], sig_dkl[h], str(h), fontsize=9)
plt.axhline(0, lw=1); plt.axvline(0, lw=1)
plt.xlabel("Δ entropy (event−control)")
plt.ylabel("Δ KL(head||mean) (event−control)")
plt.title("Canonical heads: confident divergence signature (decision window)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

print("\nDone.")

In [ ]:

#@title Position remapping WITHOUT changing forward signatures (monkeypatch RoPE + optional ALiBi disable)
# This cell:
#  1) Reuses the same deterministic long window (T_MAX)
#  2) Computes chunked NLL vs absolute position
#  3) Compares variants that DO NOT require model(..., position_ids=...)
#     - baseline
#     - ALiBi disabled (strength -> 0)   [optional but strongly recommended]
#     - RoPE remap: reset every TRAIN_T  (t -> t % TRAIN_T)
#     - RoPE remap: random block offset (blockwise t -> (t+off_b)%TRAIN_T)
#
# Works by monkeypatching RotaryEmbedding.get_cos_sin() for all modules that look like RoPE,
# plus (optionally) forcing ALiBi strength to zero via safe param override.

import math
import inspect
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Config
# ----------------------------
BATCH = 8
T_MAX = 12288
CHUNK = 2048
SEED  = 1337
SPLIT = "val"

TRAIN_T = int(getattr(cfg, "max_seq_len", getattr(model, "training_max_seq_len", getattr(model, "max_seq_len", 256))))
print(f"Training context length (reference): {TRAIN_T}")

# ----------------------------
# Deterministic long window
# ----------------------------
@torch.no_grad()
def get_same_window(cfg, *, T: int, B: int, split=SPLIT, device=DEVICE):
    gen = make_batch_generator(
        cfg,
        split=split,
        device=device,
        seq_len=int(T),
        batches_per_epoch=1,
        infinite=False,
        num_workers=0,
    )
    xb, yb = next(gen)
    xb = xb[:B].contiguous()
    yb = yb[:B].contiguous()
    if xb.shape[1] != T:
        raise ValueError(f"Expected xb length {T}, got {xb.shape[1]}")
    return xb, yb

xb_long, yb_long = get_same_window(cfg, T=T_MAX, B=BATCH)
print(f"Using same window: B={xb_long.shape[0]} T={xb_long.shape[1]}")

# ----------------------------
# Forward wrapper (no return_info)
# ----------------------------
@torch.no_grad()
def run_logits(model, xb):
    model.eval()
    out = model(xb, attention_mask=None, return_info=False)
    logits = out[0] if isinstance(out, (tuple, list)) else out
    return logits

# ----------------------------
# Chunked NLL curve
# ----------------------------
@torch.no_grad()
def chunk_nll_curve(logits_btv, y_bt, *, chunk=CHUNK):
    B, T, V = logits_btv.shape
    losses = []
    centers = []
    for start in range(0, T, chunk):
        end = min(T, start + chunk)
        l = F.cross_entropy(
            logits_btv[:, start:end, :].reshape(-1, V),
            y_bt[:, start:end].reshape(-1),
            reduction="mean"
        ).item()
        losses.append(l)
        centers.append((start + end - 1) / 2.0)
    return np.asarray(centers), np.asarray(losses)

# ============================================================
# 1) Monkeypatch RoPE: patch *all* modules that look like RotaryEmbedding
# ============================================================
def _is_rope_module(m) -> bool:
    # robust “duck typing”: has inv_freq buffer + get_cos_sin method
    return hasattr(m, "inv_freq") and hasattr(m, "get_cos_sin") and callable(getattr(m, "get_cos_sin"))

def _make_pos_map(T: int, period: int, mode: str, seed: int, device):
    """
    Returns int64 positions [T] to use in RoPE instead of arange(T).
      - mode="baseline": t
      - mode="reset": t % period
      - mode="rand_block": for each block of size period, t -> (t+off_b)%period
    """
    t = torch.arange(T, device=device, dtype=torch.long)
    if mode == "baseline":
        return t
    if period <= 0:
        raise ValueError("period must be > 0")
    if mode == "reset":
        return t % (period//2)
    if mode == "rand_block":
        rng = np.random.default_rng(seed)
        out = torch.empty((T,), device=device, dtype=torch.long)
        n_blocks = (T + period - 1) // period
        for b in range(n_blocks):
            off = int(rng.integers(low=0, high=period))
            s = b * period
            e = min(T, (b + 1) * period)
            out[s:e] = (torch.arange(e - s, device=device, dtype=torch.long) + off) % period
        return out
    raise ValueError(f"Unknown RoPE remap mode: {mode}")

class RopeRemapPatch:
    """
    Context manager: temporarily replaces get_cos_sin(T, device, dtype) on all RoPE modules.
    """
    def __init__(self, model, *, mode: str, period: int, seed: int):
        self.model = model
        self.mode = mode
        self.period = int(period)
        self.seed = int(seed)
        self._saved = []

    def __enter__(self):
        # Find and patch
        rope_modules = [m for m in self.model.modules() if _is_rope_module(m)]
        if len(rope_modules) == 0:
            print("⚠ No RoPE-like modules found to patch.")
            return self

        for rm in rope_modules:
            old = rm.get_cos_sin

            def make_wrapped(rm_ref, old_fn):
                # rm_ref: module with inv_freq
                def wrapped(T: int, device, dtype):
                    # Build mapped positions
                    t_map = _make_pos_map(T, self.period, self.mode, self.seed, device=device)
                    # Use rm_ref.inv_freq dtype for freqs, but final cos/sin in requested dtype
                    inv = rm_ref.inv_freq.to(device=device)
                    # freqs: [T, d/2]
                    freqs = torch.einsum("t,f->tf", t_map.to(inv.dtype), inv)
                    emb = torch.cat([freqs, freqs], dim=-1)            # [T, d]
                    cos = emb.cos()[None, None, :, :].to(dtype=dtype)  # [1,1,T,d]
                    sin = emb.sin()[None, None, :, :].to(dtype=dtype)  # [1,1,T,d]
                    return cos, sin
                return wrapped

            rm.get_cos_sin = make_wrapped(rm, old)
            self._saved.append((rm, old))

        return self

    def __exit__(self, exc_type, exc, tb):
        for rm, old in self._saved:
            rm.get_cos_sin = old
        self._saved.clear()
        return False

# ============================================================
# 2) Optional ALiBi disable (no signature changes)
#    - We try common patterns:
#        a) modules with _alibi_strength_param (softplus param)
#        b) modules with alibi_strength float attribute
# ============================================================
class TempSetAlibiZero:
    def __init__(self, model):
        self.model = model
        self.saved = []

    def __enter__(self):
        for m in self.model.modules():
            if hasattr(m, "_alibi_strength_param") and isinstance(getattr(m, "_alibi_strength_param"), torch.nn.Parameter):
                p = m._alibi_strength_param
                self.saved.append(("param", p, p.data.clone()))
                # make softplus(param)+min_strength ≈ 0 by setting very negative
                p.data.fill_(-30.0)
            elif hasattr(m, "alibi_strength") and isinstance(getattr(m, "alibi_strength"), (float, int)):
                self.saved.append(("attr", m, float(m.alibi_strength)))
                m.alibi_strength = 0.0
        return self

    def __exit__(self, exc_type, exc, tb):
        for kind, obj, old in self.saved:
            if kind == "param":
                obj.data.copy_(old)
            else:
                obj.alibi_strength = old
        self.saved.clear()
        return False

# ============================================================
# 3) Evaluate variants
# ============================================================
def eval_variant(name, *, rope_mode=None, alibi_zero=False):
    if rope_mode is None:
        rope_ctx = nullcontext()
    else:
        rope_ctx = RopeRemapPatch(model, mode=rope_mode, period=TRAIN_T, seed=SEED)

    alibi_ctx = TempSetAlibiZero(model) if alibi_zero else nullcontext()

    with rope_ctx, alibi_ctx:
        logits = run_logits(model, xb_long)
        x, nll = chunk_nll_curve(logits, yb_long, chunk=CHUNK)

    mean_nll = float(nll.mean())
    ppl = float(np.exp(min(20.0, mean_nll)))
    print(f"{name:>22}: mean NLL={mean_nll:.3f} | ppl={ppl:.2f}")
    return x, nll, mean_nll, ppl

# small helper
from contextlib import nullcontext

variants = []
series = {}

# Baseline
x, nll, mn, ppl = eval_variant("baseline", rope_mode=None, alibi_zero=False)
variants.append(dict(variant="baseline", mean_nll=mn, ppl=ppl))
series["baseline"] = (x, nll)

# ALiBi off only
x, nll, mn, ppl = eval_variant("alibi_zero", rope_mode=None, alibi_zero=True)
variants.append(dict(variant="alibi_zero", mean_nll=mn, ppl=ppl))
series["alibi_zero"] = (x, nll)

# RoPE reset only
x, nll, mn, ppl = eval_variant("rope_reset_mod", rope_mode="reset", alibi_zero=False)
variants.append(dict(variant="rope_reset_mod", mean_nll=mn, ppl=ppl))
series["rope_reset_mod"] = (x, nll)

# RoPE reset + ALiBi off
x, nll, mn, ppl = eval_variant("rope_reset+alibi0", rope_mode="reset", alibi_zero=True)
variants.append(dict(variant="rope_reset+alibi0", mean_nll=mn, ppl=ppl))
series["rope_reset+alibi0"] = (x, nll)

# RoPE random block offset only
x, nll, mn, ppl = eval_variant("rope_rand_block", rope_mode="rand_block", alibi_zero=False)
variants.append(dict(variant="rope_rand_block", mean_nll=mn, ppl=ppl))
series["rope_rand_block"] = (x, nll)

# RoPE random block offset + ALiBi off
x, nll, mn, ppl = eval_variant("rope_rand+alibi0", rope_mode="rand_block", alibi_zero=True)
variants.append(dict(variant="rope_rand+alibi0", mean_nll=mn, ppl=ppl))
series["rope_rand+alibi0"] = (x, nll)

df = pd.DataFrame(variants).sort_values("mean_nll")
display(df)

# ============================================================
# 4) Plot: NLL vs absolute position (chunk centers)
# ============================================================
plt.figure(figsize=(10.5, 4.8))
for name, (x, y) in series.items():
    plt.plot(x, y, marker="o", linewidth=2, label=name)
plt.axvline(TRAIN_T, linestyle="--", linewidth=1)
plt.xlabel("Absolute position (chunk center)")
plt.ylabel(f"Mean NLL over chunk (size={CHUNK})")
plt.title(f"RoPE/ALiBi position-remap diagnostics @ T={T_MAX}  (TRAIN_T={TRAIN_T})")
plt.grid(True, alpha=0.25)
plt.legend(frameon=True, ncol=3)
plt.tight_layout()
plt.show()

print("Done.")

In [ ]:

#@title Context Extrapolation — RoPE position remap sweeps + ALiBi strength sweeps (DROP-IN CELL)

import math
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Config
# ----------------------------
TRAIN_T = int(getattr(cfg, "max_seq_len", getattr(model, "training_max_seq_len", getattr(model, "max_seq_len", 256))))
print(f"Training context length (reference): {TRAIN_T}")

BATCH_FOR_PAPER = 1
T_LONG = 12288

# Chunked loss-vs-position diagnostic (optional, but very informative)
CHUNK = 4096

# --- RoPE remap sweeps ---
# effective position: t' = floor(t / s)   (smooth dilation/compression)
ROPE_SCALES = [2, 3, 4, 6, 8, 12]

# --- ALiBi strength sweeps ---
# multiplier on ALiBi term
ALIBI_MULTS = [0.25, 0.5, 1.0, 2.0, 4.0, 16.0]

# ASA capture knobs (keep light; we only need logits here)
INFO_LEVEL = "basic"
ASA_EXTRAP_INFO_CFG = dict(
    store_read_weights=False,  # keep light for sweeps
    store_read_logits=False,
    store_write_logits=False,
    store_slot_state_norm=False,
    detach_to_cpu=False,
    time_stride=4,
    batch_stride=1,
)

# ----------------------------
# Helpers
# ----------------------------
@torch.no_grad()
def lm_loss(logits_btv: torch.Tensor, y_bt: torch.Tensor) -> float:
    return F.cross_entropy(logits_btv.reshape(-1, logits_btv.size(-1)), y_bt.reshape(-1)).item()

@torch.no_grad()
def run_model(model, xb, *, info_level=INFO_LEVEL, info_cfg=ASA_EXTRAP_INFO_CFG):
    model.eval()
    try:
        logits, infos = model(
            xb,
            attention_mask=None,
            return_info=True,
            info_level=info_level,
            info_cfg=info_cfg,
        )
        return logits, infos
    except TypeError:
        logits, infos = model(xb, attention_mask=None, return_info=True)
        return logits, infos

@torch.no_grad()
def get_same_window(cfg, *, T: int, B: int, split="val", device=DEVICE):
    gen = make_batch_generator(
        cfg,
        split=split,
        device=device,
        seq_len=int(T),
        batches_per_epoch=1,
        set_batch_size=BATCH_FOR_PAPER,
        infinite=False,
        num_workers=0,
    )
    xb, yb = next(gen)
    xb = xb[:B].contiguous()
    yb = yb[:B].contiguous()
    if xb.shape[1] != T:
        raise ValueError(f"Expected xb length {T}, got {xb.shape[1]}")
    return xb, yb

def _softplus_inv(y: float) -> float:
    # inverse softplus for y>0: x = log(exp(y)-1)
    y = float(max(1e-12, y))
    return float(np.log(np.expm1(y)))

def _try_get_alibi_param(model):
    """
    Returns a list of (asa_module, attr_name, tensor_param) for ALiBi strength raw params if present.
    Supports common patterns:
      - asa._alibi_strength_param (raw, softplus'd in forward)
    """
    found = []
    if not hasattr(model, "blocks"):
        return found
    for blk in model.blocks:
        asa = getattr(blk, "asa", None)
        if asa is None:
            continue
        if hasattr(asa, "_alibi_strength_param") and isinstance(getattr(asa, "_alibi_strength_param"), torch.nn.Parameter):
            found.append((asa, "_alibi_strength_param", getattr(asa, "_alibi_strength_param")))
    return found

def _try_get_alibi_min_strength(model):
    # optional min_strength used in forward
    mins = []
    if not hasattr(model, "blocks"):
        return mins
    for blk in model.blocks:
        asa = getattr(blk, "asa", None)
        if asa is None:
            continue
        mins.append(float(getattr(asa, "min_strength", 0.0)))
    return mins

def _try_get_alibi_current_strength(model):
    """
    Best-effort: compute current effective alibi strength if the module exposes it.
    If asa._alibi_strength(dtype, device) exists, use that (in eval).
    Else return None.
    """
    vals = []
    if not hasattr(model, "blocks"):
        return None
    for blk in model.blocks:
        asa = getattr(blk, "asa", None)
        if asa is None:
            continue
        fn = getattr(asa, "_alibi_strength", None)
        if callable(fn):
            with torch.no_grad():
                v = float(fn(dtype=torch.float32, device=DEVICE).detach().cpu().item())
            vals.append(v)
        else:
            return None
    if not vals:
        return None
    return float(np.mean(vals))

class _TempOverride:
    def __init__(self):
        self.saved = []

    def set_tensor_(self, tensor, new_data: torch.Tensor):
        self.saved.append((tensor, tensor.data.clone()))
        tensor.data.copy_(new_data)

    def fill_(self, tensor, value: float):
        self.saved.append((tensor, tensor.data.clone()))
        tensor.data.fill_(value)

    def restore_(self):
        for tensor, old in self.saved:
            tensor.data.copy_(old)

# ----------------------------
# RoPE remap monkeypatch
# ----------------------------
def _patch_rope_get_cos_sin(rope_module, pos_map: torch.Tensor):
    """
    Patch rope_module.get_cos_sin(T, device, dtype) to return cos/sin for mapped positions.
    pos_map: [T] long tensor mapping absolute t -> t'
    """
    if rope_module is None or not hasattr(rope_module, "inv_freq"):
        return None

    orig = rope_module.get_cos_sin

    def get_cos_sin_mapped(T: int, device, dtype):
        # pos_map is defined for the specific T
        assert pos_map.numel() == T, "pos_map length mismatch"
        t = pos_map.to(device=device, dtype=rope_module.inv_freq.dtype)  # [T]
        freqs = torch.einsum("t,f->tf", t, rope_module.inv_freq)         # [T, d/2]
        emb = torch.cat([freqs, freqs], dim=-1)                          # [T, d]
        cos = emb.cos()[None, None, :, :].to(dtype=dtype)                # [1,1,T,d]
        sin = emb.sin()[None, None, :, :].to(dtype=dtype)
        return cos, sin

    rope_module.get_cos_sin = get_cos_sin_mapped
    return orig

def _patch_all_asa_ropes(model, pos_map: torch.Tensor, also_slotspace: bool = True):
    """
    Patch both write-side rope (asa.rope) and optionally slotspace rope (asa.rope_slotspace).
    Returns list of (module, original_fn) to restore.
    """
    patches = []
    if not hasattr(model, "blocks"):
        return patches
    for blk in model.blocks:
        asa = getattr(blk, "asa", None)
        if asa is None:
            continue
        # write-side rope
        if getattr(asa, "rope", None) is not None:
            orig = _patch_rope_get_cos_sin(asa.rope, pos_map)
            if orig is not None:
                patches.append((asa.rope, orig))
        # slotspace rope
        if also_slotspace and getattr(asa, "rope_slotspace", None) is not None:
            orig = _patch_rope_get_cos_sin(asa.rope_slotspace, pos_map)
            if orig is not None:
                patches.append((asa.rope_slotspace, orig))
    return patches

def _restore_rope_patches(patches):
    for rope_module, orig in patches:
        rope_module.get_cos_sin = orig

# ----------------------------
# ALiBi strength scaling (best-effort)
# ----------------------------
def _set_alibi_multiplier(model, mult: float):
    """
    Best effort:
      - if asa._alibi_strength_param exists (raw), compute current effective strength s,
        then set raw so new strength ~= mult*s (respecting min_strength).
    Returns override object to restore.
    """
    ov = _TempOverride()
    alibi_params = _try_get_alibi_param(model)
    if not alibi_params:
        return ov, False

    # estimate a per-layer effective strength; if accessible, use it, else approximate from param directly
    # We'll compute per-asa:
    for asa, name, p in alibi_params:
        with torch.no_grad():
            # Try calling _alibi_strength to get current effective s
            fn = getattr(asa, "_alibi_strength", None)
            if callable(fn):
                s0 = float(fn(dtype=torch.float32, device=DEVICE).detach().cpu().item())
            else:
                # fallback: assume softplus(raw)+min_strength
                min_strength = float(getattr(asa, "min_strength", 0.0))
                s0 = float((torch.nn.functional.softplus(p.detach().cpu()) + min_strength).item())

            s1 = float(max(1e-12, mult * s0))
            min_strength = float(getattr(asa, "min_strength", 0.0))
            y = max(1e-12, s1 - min_strength)
            raw_new = torch.tensor(_softplus_inv(y), device=p.device, dtype=p.dtype)
            ov.set_tensor_(p, raw_new)

    return ov, True

# ----------------------------
# Metrics & diagnostics
# ----------------------------
@torch.no_grad()
def eval_mean_nll(model, xb, yb):
    logits, _ = run_model(model, xb)
    return float(lm_loss(logits, yb))

@torch.no_grad()
def eval_chunk_nll_curve(model, xb, yb, chunk=256):
    """
    Returns centers, mean_nll_per_chunk for a given xb/yb (same length).
    """
    logits, _ = run_model(model, xb)
    B, T, V = logits.shape
    losses = []
    centers = []
    for t0 in range(0, T, chunk):
        t1 = min(T, t0 + chunk)
        if t1 - t0 < 8:
            continue
        l = F.cross_entropy(
            logits[:, t0:t1, :].reshape(-1, V),
            yb[:, t0:t1].reshape(-1),
            reduction="mean",
        ).item()
        losses.append(l)
        centers.append((t0 + t1) / 2.0)
    return np.asarray(centers), np.asarray(losses)

def _ppl(nll): return float(np.exp(min(20.0, float(nll))))

# ----------------------------
# Run
# ----------------------------
xb_long, yb_long = get_same_window(cfg, T=T_LONG, B=BATCH_FOR_PAPER, split="val", device=DEVICE)
print(f"Using same window: B={xb_long.shape[0]} T={xb_long.shape[1]}")

# Baseline
base_nll = eval_mean_nll(model, xb_long, yb_long)
print(f"baseline: mean NLL={base_nll:.3f} | ppl={_ppl(base_nll):.2f}")

rows = []
rows.append(dict(kind="baseline", variant="baseline", mean_nll=base_nll, ppl=_ppl(base_nll)))

# ----------------------------
# 1) RoPE scale sweep (smooth dilation)
# ----------------------------
print("\nRoPE scale sweep: t' = floor(t / s)")
for s in ROPE_SCALES:
    pos_map = (torch.arange(T_LONG, dtype=torch.long) // int(s)).clamp_min(0)
    patches = _patch_all_asa_ropes(model, pos_map, also_slotspace=True)
    try:
        nll = eval_mean_nll(model, xb_long, yb_long)
        rows.append(dict(kind="rope_scale", variant=f"rope_scale_s={s}", mean_nll=nll, ppl=_ppl(nll)))
        print(f"  s={s:>2}: mean NLL={nll:.3f} | ppl={_ppl(nll):.2f}")
    finally:
        _restore_rope_patches(patches)

# ----------------------------
# 2) RoPE reset baseline (t % TRAIN_T)
# ----------------------------
print("\nRoPE reset: t' = t % TRAIN_T")
pos_map = (torch.arange(T_LONG, dtype=torch.long) % int(TRAIN_T//2)).clamp_min(0)
patches = _patch_all_asa_ropes(model, pos_map, also_slotspace=True)
try:
    nll = eval_mean_nll(model, xb_long, yb_long)
    rows.append(dict(kind="rope_reset", variant=f"rope_reset_mod_{TRAIN_T}//2", mean_nll=nll, ppl=_ppl(nll)))
    print(f"  reset: mean NLL={nll:.3f} | ppl={_ppl(nll):.2f}")
finally:
    _restore_rope_patches(patches)

# ----------------------------
# 3) ALiBi strength sweep (multiplier)
# ----------------------------
print("\nALiBi strength sweep (multiplier on effective strength):")
alibi_supported = bool(_try_get_alibi_param(model))
if not alibi_supported:
    print("  ⚠ No _alibi_strength_param found in model.blocks[*].asa; skipping ALiBi sweep.")
else:
    s_now = _try_get_alibi_current_strength(model)
    if s_now is not None:
        print(f"  Current mean effective ALiBi strength ≈ {s_now:.6f}")
    for a in ALIBI_MULTS:
        ov, ok = _set_alibi_multiplier(model, a)
        if not ok:
            print("  ⚠ Failed to apply ALiBi scaling (unexpected).")
            break
        try:
            nll = eval_mean_nll(model, xb_long, yb_long)
            rows.append(dict(kind="alibi_mult", variant=f"alibi_mult={a:g}", mean_nll=nll, ppl=_ppl(nll)))
            print(f"  α={a:g}: mean NLL={nll:.3f} | ppl={_ppl(nll):.2f}")
        finally:
            ov.restore_()

# ----------------------------
# 4) Best-of diagnostics: chunk loss vs absolute position
# ----------------------------
df = pd.DataFrame(rows).sort_values(["mean_nll"])
display(df)

# pick best rope_scale and best alibi_mult (if present)
best_rope = df[df["kind"].isin(["rope_scale","rope_reset"])].iloc[0] if np.any(df["kind"].isin(["rope_scale","rope_reset"])) else None
best_alibi = df[df["kind"].isin(["alibi_mult"])].iloc[0] if np.any(df["kind"].isin(["alibi_mult"])) else None

def _run_variant_for_curve(variant_row):
    name = variant_row["variant"]
    kind = variant_row["kind"]

    if kind == "baseline":
        centers, losses = eval_chunk_nll_curve(model, xb_long, yb_long, chunk=CHUNK)
        return name, centers, losses

    if kind in ["rope_scale","rope_reset"]:
        # parse s or reset
        if "rope_scale_s=" in name:
            s = int(name.split("rope_scale_s=")[-1])
            pos_map = (torch.arange(T_LONG, dtype=torch.long) // s).clamp_min(0)
        else:
            pos_map = (torch.arange(T_LONG, dtype=torch.long) % int(TRAIN_T)).clamp_min(0)

        patches = _patch_all_asa_ropes(model, pos_map, also_slotspace=True)
        try:
            centers, losses = eval_chunk_nll_curve(model, xb_long, yb_long, chunk=CHUNK)
            return name, centers, losses
        finally:
            _restore_rope_patches(patches)

    if kind == "alibi_mult":
        a = float(name.split("=")[-1])
        ov, ok = _set_alibi_multiplier(model, a)
        if not ok:
            return None
        try:
            centers, losses = eval_chunk_nll_curve(model, xb_long, yb_long, chunk=CHUNK)
            return name, centers, losses
        finally:
            ov.restore_()

    return None

# Make curves: baseline vs best_rope vs best_alibi (if any)
curves = []
curves.append(_run_variant_for_curve(pd.Series(dict(kind="baseline", variant="baseline"))))
if best_rope is not None:
    curves.append(_run_variant_for_curve(best_rope))
if best_alibi is not None:
    curves.append(_run_variant_for_curve(best_alibi))

plt.figure(figsize=(10.5, 4.6))
for item in curves:
    if item is None:
        continue
    name, centers, losses = item
    plt.plot(centers, losses, marker="o", linewidth=2, label=name)
plt.axvline(TRAIN_T, linestyle="--", linewidth=1)
plt.xlabel(f"Absolute position (chunk center, chunk={CHUNK})")
plt.ylabel("Mean NLL over chunk")
plt.title(f"Position diagnostics @ T={T_LONG}: baseline vs best remap / ALiBi sweep")
plt.grid(True, alpha=0.25)
plt.legend(frameon=True)
plt.tight_layout()
plt.show()

print("Done.")

In [ ]:

#@title Analysis 1 — Cyclic RoPE induces *stationary* routing (periodicity + drift diagnostics)
# This drills into: "cyclic is a feature" by showing that routing stats stop drifting with absolute position.

import math
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Config
# ----------------------------
TRAIN_T = int(getattr(cfg, "max_seq_len", getattr(model, "training_max_seq_len", getattr(model, "max_seq_len", 256))))
T_LONG  = 12288
BATCH   = 2

# Capture knobs (keep light; we only need read_weights summaries)
INFO_LEVEL = "basic"
ASA_INFO_CFG = dict(
    store_read_weights=True,
    store_read_logits=False,
    store_write_logits=False,
    store_slot_state_norm=False,
    detach_to_cpu=False,
    time_stride=4,   # downsample time for long sequences
    batch_stride=1,
)

# ----------------------------
# Helpers: loss + runner
# ----------------------------
@torch.no_grad()
def lm_loss(logits_btv: torch.Tensor, y_bt: torch.Tensor) -> float:
    return F.cross_entropy(logits_btv.reshape(-1, logits_btv.size(-1)), y_bt.reshape(-1)).item()

@torch.no_grad()
def run_model(model, xb, *, info_level=INFO_LEVEL, info_cfg=ASA_INFO_CFG):
    model.eval()
    try:
        return model(xb, attention_mask=None, return_info=True, info_level=info_level, info_cfg=info_cfg)
    except TypeError:
        return model(xb, attention_mask=None, return_info=True)

@torch.no_grad()
def get_same_window(cfg, *, T: int, B: int, split="val", device=DEVICE):
    gen = make_batch_generator(
        cfg, split=split, device=device,
        seq_len=int(T), batches_per_epoch=1, set_batch_size=2, infinite=False,
        num_workers=0
    )
    xb, yb = next(gen)
    xb = xb[:B].contiguous()
    yb = yb[:B].contiguous()
    if xb.shape[1] != T:
        raise ValueError(f"Expected T={T}, got {xb.shape[1]}")
    return xb, yb

# ----------------------------
# RoPE patcher (cyclic or scaled positions) for ASA blocks
# Works without position_ids support.
# ----------------------------
class RopePosRemap:
    def __init__(self, model, *, mode="reset", train_T=1024, scale=1):
        self.model = model
        self.mode = str(mode)
        self.train_T = int(train_T)
        self.scale = int(scale)
        self.saved = []  # (rope_module, original_get_cos_sin)

    def _remap_positions(self, T: int, device, dtype, inv_freq: torch.Tensor):
        # returns cos,sin with shape [1,1,T,d]
        t = torch.arange(T, device=device, dtype=inv_freq.dtype)
        if self.mode == "reset":
            t = t % self.train_T
        elif self.mode == "scale":
            s = max(1, self.scale)
            t = torch.floor_divide(t, s)
        else:
            raise ValueError(f"Unknown mode={self.mode}")

        freqs = torch.einsum("t,f->tf", t, inv_freq)     # [T, d/2]
        emb   = torch.cat([freqs, freqs], dim=-1)        # [T, d]
        cos   = emb.cos()[None, None, :, :].to(dtype=dtype)
        sin   = emb.sin()[None, None, :, :].to(dtype=dtype)
        return cos, sin

    def __enter__(self):
        # Patch every ASA rope used for write keys + (optionally) slotspace rope
        for blk in getattr(self.model, "blocks", []):
            asa = getattr(blk, "asa", None)
            if asa is None:
                continue

            for rope_name in ("rope", "rope_slotspace"):
                rope = getattr(asa, rope_name, None)
                if rope is None:
                    continue
                if not hasattr(rope, "inv_freq") or not hasattr(rope, "get_cos_sin"):
                    continue

                orig = rope.get_cos_sin

                def patched_get_cos_sin(T, device, dtype, *, _rope=rope, _orig=orig):
                    inv = _rope.inv_freq
                    return self._remap_positions(T=int(T), device=device, dtype=dtype, inv_freq=inv)

                self.saved.append((rope, orig))
                rope.get_cos_sin = patched_get_cos_sin

        return self

    def __exit__(self, exc_type, exc, tb):
        for rope, orig in self.saved:
            rope.get_cos_sin = orig
        self.saved = []

# ----------------------------
# Routing summaries: per-chunk stationarity + drift vs chunk0
# ----------------------------
@torch.no_grad()
def summarize_chunks(infos, *, chunk_len: int, time_stride: int):
    """
    Returns:
      df_chunk: per (layer, chunk) scalar routing metrics
      df_drift: per chunk drift stats aggregated across layers
    """
    eps = 1e-9
    rows = []

    for li, info in enumerate(infos):
        if info is None:
            continue
        rw = info.get("read_weights", None)
        if rw is None:
            continue

        # rw is [B,H,T_eff,K] after time_stride downsample
        rw = rw.float()
        B, H, T_eff, K = rw.shape

        # Effective chunk length in downsampled time
        chunk_eff = max(1, int(chunk_len // max(1, time_stride)))
        n_chunks = int(math.ceil(T_eff / chunk_eff))

        # Per-token entropy + maxp
        ent = -(rw * rw.clamp_min(eps).log()).sum(dim=-1)  # [B,H,T_eff]
        maxp = rw.max(dim=-1).values                        # [B,H,T_eff]

        for c in range(n_chunks):
            t0 = c * chunk_eff
            t1 = min(T_eff, (c + 1) * chunk_eff)
            if t1 <= t0:
                continue

            # Chunk routing scalars
            ent_norm = (ent[:, :, t0:t1].mean() / math.log(K)).item()
            maxp_mu  = maxp[:, :, t0:t1].mean().item()

            # Slot usage distribution in this chunk
            usage = rw[:, :, t0:t1, :].mean(dim=(0, 1, 2))              # [K]
            usage = usage / usage.sum().clamp_min(eps)
            usage_ent = float(-(usage * usage.clamp_min(eps).log()).sum().cpu())
            usage_ent_norm = usage_ent / float(math.log(K))

            rows.append(dict(
                layer=int(li),
                chunk=int(c),
                ent_norm=float(ent_norm),
                maxp=float(maxp_mu),
                usage_ent_norm=float(usage_ent_norm),
                T_eff=int(T_eff),
                K=int(K),
            ))

    df = pd.DataFrame(rows)
    if df.empty:
        return df, df

    # Drift vs chunk0: compare usage distributions (approx via entropy proxy + Jensen gap)
    # We also report stationarity of ent_norm/maxp: std across chunks.
    drift_rows = []
    for c in sorted(df["chunk"].unique()):
        sub = df[df["chunk"] == c]
        drift_rows.append(dict(
            chunk=int(c),
            ent_norm_mean=float(sub["ent_norm"].mean()),
            ent_norm_std=float(sub["ent_norm"].std(ddof=0)),
            maxp_mean=float(sub["maxp"].mean()),
            maxp_std=float(sub["maxp"].std(ddof=0)),
            usage_ent_mean=float(sub["usage_ent_norm"].mean()),
            usage_ent_std=float(sub["usage_ent_norm"].std(ddof=0)),
        ))
    df_drift = pd.DataFrame(drift_rows).sort_values("chunk")
    return df.sort_values(["layer", "chunk"]), df_drift

def plot_stationarity(df_drift, *, title, train_T=1024):
    if df_drift.empty:
        print("No chunk summaries to plot.")
        return
    plt.figure(figsize=(10.8, 4.6))
    plt.plot(df_drift["chunk"], df_drift["ent_norm_mean"], marker="o", linewidth=2, label="mean routing entropy (norm)")
    plt.plot(df_drift["chunk"], df_drift["maxp_mean"], marker="o", linewidth=2, label="mean routing maxp")
    plt.plot(df_drift["chunk"], df_drift["usage_ent_mean"], marker="o", linewidth=2, label="mean slot usage entropy (norm)")
    plt.xlabel(f"Chunk index (each ≈ {train_T} tokens)")
    plt.ylabel("Layer-avg metric")
    plt.title(title)
    plt.grid(True, alpha=0.25)
    plt.legend(frameon=True)
    plt.tight_layout()
    plt.show()

# ----------------------------
# Run: baseline vs cyclic RoPE reset
# ----------------------------
xb_long, yb_long = get_same_window(cfg, T=T_LONG, B=BATCH, split="val", device=DEVICE)
print(f"Same window: B={xb_long.shape[0]} T={xb_long.shape[1]} | TRAIN_T={TRAIN_T}")

# Baseline
logits0, infos0 = run_model(model, xb_long, info_level=INFO_LEVEL, info_cfg=ASA_INFO_CFG)
loss0 = lm_loss(logits0, yb_long)
ppl0  = float(np.exp(min(20.0, loss0)))
df0, d0 = summarize_chunks(infos0, chunk_len=TRAIN_T, time_stride=ASA_INFO_CFG["time_stride"])
print(f"baseline: loss={loss0:.4f} ppl={ppl0:.2f} | chunks={len(d0)}")

# Cyclic RoPE reset
with RopePosRemap(model, mode="reset", train_T=TRAIN_T):
    logits1, infos1 = run_model(model, xb_long, info_level=INFO_LEVEL, info_cfg=ASA_INFO_CFG)
loss1 = lm_loss(logits1, yb_long)
ppl1  = float(np.exp(min(20.0, loss1)))
df1, d1 = summarize_chunks(infos1, chunk_len=TRAIN_T, time_stride=ASA_INFO_CFG["time_stride"])
print(f"rope_reset (t%{TRAIN_T}): loss={loss1:.4f} ppl={ppl1:.2f} | chunks={len(d1)}")

# Plot: stationarity across chunks
plot_stationarity(d0, title=f"Baseline routing drift across chunks @ T={T_LONG}", train_T=TRAIN_T)
plot_stationarity(d1, title=f"Cyclic RoPE routing stationarity across chunks @ T={T_LONG}", train_T=TRAIN_T)

# Quick numeric: how stationary?
def stationarity_score(df_drift):
    # lower is more stationary: sum of stds
    if df_drift.empty:
        return np.nan
    return float(df_drift["ent_norm_std"].mean() + df_drift["maxp_std"].mean() + df_drift["usage_ent_std"].mean())

s0 = stationarity_score(d0)
s1 = stationarity_score(d1)
pd.DataFrame([
    dict(variant="baseline", ppl=ppl0, stationarity_score=s0),
    dict(variant=f"rope_reset_mod_{TRAIN_T}", ppl=ppl1, stationarity_score=s1),
]).sort_values("ppl")

In [ ]:

#@title Analysis 2 (DROP-IN, OOM-safe + RELIABLE canonical head+slot matching) — "cyclic is a feature" & "internals behave as intended"
# Complete replacement. Fixes:
#   - OOM: streamed loss uses log-softmax + gather (no giant CE temp); optional CPU eval.
#   - Head match bug: schema-locked head feature vectors (fixed D) so no (11 vs 12) mismatch.
#   - Robust canonicalization: head matching across depth + cross-condition head match at ref layer.
#   - Slot matching: per-head slot descriptor uses time-marginal usage AND inertia signatures, then exact match for K<=8.
#
# Outputs:
#   df_scores: NLL/PPL per condition
#   df_align : head+slot alignment quality train↔baseline and train↔cyclic at ref layer
#   df_depth : adjacent-layer head alignment quality per condition
#
# Notes:
#   - Assumes ASA info plumbing exposes read_weights in info_level="basic" and obeys info_cfg time_stride/detach_to_cpu.
#   - Uses RoPE remap patcher (no position_ids required) by monkeypatching rope.get_cos_sin.

import math
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Config knobs
# ----------------------------
TRAIN_T = int(getattr(cfg, "max_seq_len", getattr(model, "training_max_seq_len", getattr(model, "max_seq_len", 256))))
T_LONG  = 12288
BATCH   = 4

# Streamed loss knobs
LOSS_CHUNK_T = 256           # token chunk size for loss
LOSS_CHUNK_B = 1             # microbatch for loss (reduces peak)
LOSS_ON_CPU  = False         # set True if you keep hitting GPU fragmentation

INFO_LEVEL = "basic"
INFO_CFG = dict(
    store_read_weights=True,
    store_read_logits=False,
    store_write_logits=False,
    store_slot_state_norm=False,
    detach_to_cpu=True,      # IMPORTANT: frees GPU
    time_stride=16,          # IMPORTANT: huge memory win
    batch_stride=1,
)

# Descriptor knobs (applied after time_stride)
MAX_LAG = 64
USE_LAGS = (1, 2, 4, 8, 16, 32, 64)

# ----------------------------
# Helpers: data
# ----------------------------
@torch.no_grad()
def get_same_window(cfg, *, T: int, B: int, split="val", device=DEVICE):
    gen = make_batch_generator(
        cfg, split=split, device=device, seq_len=int(T),
        batches_per_epoch=1, infinite=False, num_workers=0
    )
    xb, yb = next(gen)
    xb = xb[:B].contiguous()
    yb = yb[:B].contiguous()
    if xb.shape[1] != T:
        raise ValueError(f"Expected T={T}, got {xb.shape[1]}")
    return xb, yb

# ----------------------------
# Helpers: OOM-safe streamed NLL (log-softmax + gather)
# ----------------------------
@torch.no_grad()
def streamed_nll_from_logits(logits_btv: torch.Tensor, y_bt: torch.Tensor,
                             *, chunk_t: int = LOSS_CHUNK_T, chunk_b: int = LOSS_CHUNK_B,
                             compute_on_cpu: bool = LOSS_ON_CPU) -> float:
    """
    Computes mean NLL without allocating a huge temporary (like CE sometimes does).
    Uses log_softmax + gather in chunks over time and microbatches over batch.
    """
    if compute_on_cpu:
        logits_btv = logits_btv.detach().to("cpu")
        y_bt = y_bt.detach().to("cpu")

    B, T, V = logits_btv.shape
    total = 0.0
    count = 0

    for b0 in range(0, B, chunk_b):
        b1 = min(B, b0 + chunk_b)
        lg_b = logits_btv[b0:b1]  # [b,T,V]
        y_b  = y_bt[b0:b1]        # [b,T]

        for t0 in range(0, T, chunk_t):
            t1 = min(T, t0 + chunk_t)
            lg = lg_b[:, t0:t1, :]                 # [b,dt,V]
            yy = y_b[:, t0:t1]                     # [b,dt]
            lsm = F.log_softmax(lg, dim=-1)         # [b,dt,V]
            nll = -lsm.gather(-1, yy.unsqueeze(-1)).squeeze(-1)  # [b,dt]
            total += float(nll.sum().item())
            count += int(nll.numel())

    return float(total / max(1, count))

# ----------------------------
# Helpers: model runner
# ----------------------------
@torch.no_grad()
def run_model(model, xb, *, info_level=INFO_LEVEL, info_cfg=INFO_CFG):
    model.eval()
    try:
        return model(xb, attention_mask=None, return_info=True, info_level=info_level, info_cfg=info_cfg)
    except TypeError:
        return model(xb, attention_mask=None, return_info=True)

# ----------------------------
# RoPE remap patcher (works without position_ids)
# ----------------------------
class RopePosRemap:
    def __init__(self, model, *, mode="reset", train_T=1024, scale=1):
        self.model = model
        self.mode = str(mode)
        self.train_T = int(train_T)
        self.scale = int(scale)
        self.saved = []

    def _remap_positions(self, T: int, device, dtype, inv_freq: torch.Tensor):
        t = torch.arange(T, device=device, dtype=inv_freq.dtype)
        if self.mode == "reset":
            t = t % self.train_T
        elif self.mode == "scale":
            s = max(1, self.scale)
            t = torch.floor_divide(t, s)
        else:
            raise ValueError(f"Unknown mode={self.mode}")
        freqs = torch.einsum("t,f->tf", t, inv_freq)   # [T, d/2]
        emb = torch.cat([freqs, freqs], dim=-1)        # [T, d]
        cos = emb.cos()[None, None, :, :].to(dtype=dtype)
        sin = emb.sin()[None, None, :, :].to(dtype=dtype)
        return cos, sin

    def __enter__(self):
        for blk in getattr(self.model, "blocks", []):
            asa = getattr(blk, "asa", None)
            if asa is None:
                continue
            for rope_name in ("rope", "rope_slotspace"):
                rope = getattr(asa, rope_name, None)
                if rope is None or (not hasattr(rope, "inv_freq")) or (not hasattr(rope, "get_cos_sin")):
                    continue
                orig = rope.get_cos_sin

                def patched_get_cos_sin(T, device, dtype, *, _rope=rope):
                    return self._remap_positions(int(T), device, dtype, _rope.inv_freq)

                self.saved.append((rope, orig))
                rope.get_cos_sin = patched_get_cos_sin
        return self

    def __exit__(self, exc_type, exc, tb):
        for rope, orig in self.saved:
            rope.get_cos_sin = orig
        self.saved = []

# ----------------------------
# Canonical assignment (exact for N<=8)
# ----------------------------
def _cosine_sim_matrix(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    A = A.astype(np.float64); B = B.astype(np.float64)
    A = A / (np.linalg.norm(A, axis=1, keepdims=True) + 1e-9)
    B = B / (np.linalg.norm(B, axis=1, keepdims=True) + 1e-9)
    return A @ B.T

def _exact_assignment_maxsim(S: np.ndarray) -> np.ndarray:
    import itertools
    N = S.shape[0]
    best = None
    best_val = -1e18
    for perm in itertools.permutations(range(N)):
        val = 0.0
        for i in range(N):
            val += S[i, perm[i]]
        if val > best_val:
            best_val = val
            best = perm
    return np.asarray(best, dtype=np.int64)

def _greedy_assignment_maxsim(S: np.ndarray) -> np.ndarray:
    N = S.shape[0]
    used_r = set(); used_c = set()
    perm = -np.ones((N,), dtype=np.int64)
    pairs = [(S[i, j], i, j) for i in range(N) for j in range(N)]
    pairs.sort(reverse=True, key=lambda x: x[0])
    for s, i, j in pairs:
        if i in used_r or j in used_c:
            continue
        perm[i] = j
        used_r.add(i); used_c.add(j)
        if len(used_r) == N:
            break
    for i in range(N):
        if perm[i] < 0:
            for j in range(N):
                if j not in used_c:
                    perm[i] = j
                    used_c.add(j)
                    break
    return perm

def match_rows_by_cosine(A: np.ndarray, B: np.ndarray) -> tuple[np.ndarray, float, float]:
    S = _cosine_sim_matrix(A, B)
    N = S.shape[0]
    perm = _exact_assignment_maxsim(S) if N <= 8 else _greedy_assignment_maxsim(S)
    sims = np.asarray([S[i, perm[i]] for i in range(N)], dtype=np.float64)
    return perm, float(sims.mean()), float(sims.min())

# ----------------------------
# Routing inertia curve (per-head, CPU-friendly)
# ----------------------------
def routing_inertia_curve_cpu(p_h_t_k: np.ndarray, max_lag: int = 64, eps: float = 1e-8) -> np.ndarray:
    """
    p_h_t_k: [H,T,K] (already probs)
    Returns [H,L] where L=min(max_lag,T-1): mean cosine over t for each head.
    """
    H, T, K = p_h_t_k.shape
    if T < 2:
        return np.zeros((H, 0), dtype=np.float32)
    L = int(min(max_lag, T - 1))

    pn = p_h_t_k / (np.linalg.norm(p_h_t_k, axis=-1, keepdims=True) + eps)  # [H,T,K]
    out = np.zeros((H, L), dtype=np.float32)
    for lag in range(1, L + 1):
        a = pn[:, :-lag, :]
        b = pn[:,  lag:, :]
        out[:, lag - 1] = (a * b).sum(axis=-1).mean(axis=-1)
    return out

# ----------------------------
# Head role features (SCHEMA-LOCKED, fixed D)
# ----------------------------
FEATURE_KEYS = [
    "inertia_d1", "inertia_d2", "inertia_d4", "inertia_d8", "inertia_d16", "inertia_d32", "inertia_d64",
    "inertia_mean", "inertia_slope",
    "ent_mean", "eff_slots", "maxp_mean", "top2_mass", "top4_mass",
    "usage_ent", "argmax_fliprate",
    "gini",
]

def _gini(x, eps=1e-9):
    # x: [...,K] nonneg
    xs = np.sort(x, axis=-1)
    K = xs.shape[-1]
    idx = np.arange(1, K+1, dtype=np.float64)
    num = np.sum((2*idx - K - 1.0) * xs, axis=-1)
    den = (K * (np.sum(xs, axis=-1) + eps))
    return (num / den).astype(np.float32)

def build_head_role_features_locked(rw_bhtk: torch.Tensor) -> np.ndarray:
    """
    rw: [B,H,T,K] (often CPU already)
    Returns: [H,D] numpy, L2-normalized, D fixed by FEATURE_KEYS
    """
    eps = 1e-9
    rw = rw_bhtk.detach().float().cpu().numpy()  # [B,H,T,K]
    B, H, T, K = rw.shape

    p = rw / (rw.sum(axis=-1, keepdims=True) + eps)        # [B,H,T,K]
    p_htk = p.mean(axis=0)                                 # [H,T,K]

    # entropy / concentration
    ent = -(p_htk * np.log(np.clip(p_htk, eps, None))).sum(axis=-1)  # [H,T]
    ent_mean = ent.mean(axis=-1)                                     # [H]
    eff_slots = np.exp(ent_mean).astype(np.float32)                  # [H]
    maxp_mean = p_htk.max(axis=-1).mean(axis=-1).astype(np.float32)  # [H]

    sorted_p = np.sort(p_htk, axis=-1)
    top2_mass = sorted_p[..., -min(2, K):].sum(axis=-1).mean(axis=-1).astype(np.float32)
    top4_mass = sorted_p[..., -min(4, K):].sum(axis=-1).mean(axis=-1).astype(np.float32)

    usage = p_htk.mean(axis=1)                           # [H,K]
    usage = usage / (usage.sum(axis=-1, keepdims=True) + eps)
    usage_ent = (-(usage * np.log(np.clip(usage, eps, None))).sum(axis=-1)).astype(np.float32)  # [H]

    # argmax fliprate
    arg = p_htk.argmax(axis=-1)          # [H,T]
    flip = (arg[:, 1:] != arg[:, :-1]).mean(axis=-1).astype(np.float32) if T > 1 else np.zeros((H,), dtype=np.float32)

    # inertia
    curve = routing_inertia_curve_cpu(p_htk.astype(np.float32), max_lag=MAX_LAG, eps=eps)  # [H,L]
    L = curve.shape[1]

    def lag_or_zero(d):
        if d <= 0 or d > L: return np.zeros((H,), dtype=np.float32)
        return curve[:, d-1].astype(np.float32)

    inertia_d1  = lag_or_zero(1)
    inertia_d2  = lag_or_zero(2)
    inertia_d4  = lag_or_zero(4)
    inertia_d8  = lag_or_zero(8)
    inertia_d16 = lag_or_zero(16)
    inertia_d32 = lag_or_zero(32)
    inertia_d64 = lag_or_zero(64)
    inertia_mean = curve.mean(axis=-1).astype(np.float32) if L > 0 else np.zeros((H,), dtype=np.float32)

    if L > 1:
        xs = np.arange(1, L+1, dtype=np.float32)
        xs = (xs - xs.mean()) / (xs.std() + eps)
        ys = curve.astype(np.float32)
        ys = (ys - ys.mean(axis=-1, keepdims=True)) / (ys.std(axis=-1, keepdims=True) + eps)
        inertia_slope = (ys * xs[None, :]).mean(axis=-1).astype(np.float32)
    else:
        inertia_slope = np.zeros((H,), dtype=np.float32)

    # gini (mean over time)
    g = _gini(p_htk, eps=eps).mean(axis=-1).astype(np.float32)  # [H]

    key_to_vec = dict(
        inertia_d1=inertia_d1, inertia_d2=inertia_d2, inertia_d4=inertia_d4, inertia_d8=inertia_d8,
        inertia_d16=inertia_d16, inertia_d32=inertia_d32, inertia_d64=inertia_d64,
        inertia_mean=inertia_mean, inertia_slope=inertia_slope,
        ent_mean=ent_mean.astype(np.float32),
        eff_slots=eff_slots,
        maxp_mean=maxp_mean,
        top2_mass=top2_mass,
        top4_mass=top4_mass,
        usage_ent=usage_ent,
        argmax_fliprate=flip,
        gini=g,
    )

    feats = np.stack([key_to_vec[k] for k in FEATURE_KEYS], axis=-1)  # [H,D]
    feats = feats / (np.linalg.norm(feats, axis=-1, keepdims=True) + 1e-9)
    return feats.astype(np.float32)

# ----------------------------
# Canonicalization: heads across depth (adjacent compose)
# ----------------------------
def canonicalize_heads_across_layers(head_feat_by_layer: dict[int, np.ndarray]):
    layers = sorted(head_feat_by_layer.keys())
    if len(layers) == 0:
        return {}, {}, []
    H = head_feat_by_layer[layers[0]].shape[0]
    canon_perm_to_layer = {layers[0]: np.arange(H, dtype=np.int64)}
    align_q = {}
    for i in range(len(layers) - 1):
        la, lb = layers[i], layers[i + 1]
        perm_ab, q, qmin = match_rows_by_cosine(head_feat_by_layer[la], head_feat_by_layer[lb])
        align_q[(la, lb)] = (q, qmin)
        canon_perm_to_layer[lb] = perm_ab[canon_perm_to_layer[la]]
    return canon_perm_to_layer, align_q, layers

# ----------------------------
# Slot descriptors + canonicalization within a head (more informative)
# ----------------------------
def build_slot_descriptors(rw_bhtk: torch.Tensor, *, max_lag=16) -> np.ndarray:
    """
    rw: [B,H,T,K] probs
    Returns desc: [H,K,D] where each slot gets a descriptor:
      [usage_k, mean_p_k, inertia_k_lag1, inertia_k_lag4, inertia_k_lag8, ... , usage_vector(K)]
    Designed to break slot symmetry under cyclic remaps.
    """
    eps = 1e-9
    rw = rw_bhtk.detach().float().cpu().numpy()   # [B,H,T,K]
    B, H, T, K = rw.shape
    p = rw / (rw.sum(axis=-1, keepdims=True) + eps)          # [B,H,T,K]
    p_htk = p.mean(axis=0)                                   # [H,T,K]

    usage = p_htk.mean(axis=1)                               # [H,K]
    usage = usage / (usage.sum(axis=-1, keepdims=True) + eps)

    # slot “self-inertia”: treat scalar series p_k(t) and correlate with lagged itself
    L = int(min(max_lag, T-1)) if T > 1 else 0
    use_lags = [d for d in (1, 4, 8, 16) if d <= L]
    if len(use_lags) == 0:
        use_lags = []

    inertia = np.zeros((H, K, len(use_lags)), dtype=np.float32)
    if len(use_lags) > 0:
        for j, lag in enumerate(use_lags):
            a = p_htk[:-0, :-lag, :] if False else p_htk[:, :-lag, :]  # [H,T-lag,K]
            b = p_htk[:,  lag:, :]                                     # [H,T-lag,K]
            # cosine on scalar series reduces to normalized dot:
            # normalize per (H,K) across time
            a2 = a - a.mean(axis=1, keepdims=True)
            b2 = b - b.mean(axis=1, keepdims=True)
            num = (a2 * b2).mean(axis=1)
            den = (a2.std(axis=1) * b2.std(axis=1) + eps)
            inertia[:, :, j] = (num / den).astype(np.float32)          # [H,K]

    # Assemble descriptor: [usage_k, mean_over_time(p_k), inertia_lags..., usage_vector(K)]
    mean_pk = p_htk.mean(axis=1).astype(np.float32)                    # [H,K]
    parts = [usage[..., None].astype(np.float32), mean_pk[..., None]]
    if len(use_lags) > 0:
        parts.append(inertia)
    # broadcast usage vector into each slot descriptor (context)
    uv = np.repeat(usage[:, None, :], repeats=K, axis=1).astype(np.float32)  # [H,K,K]
    parts.append(uv)

    desc = np.concatenate(parts, axis=-1)  # [H,K,D]
    desc = desc / (np.linalg.norm(desc, axis=-1, keepdims=True) + 1e-9)
    return desc

def canonicalize_slots_between_conditions(descA_hkd: np.ndarray, descB_hkd: np.ndarray):
    H, K, D = descA_hkd.shape
    perms = np.zeros((H, K), dtype=np.int64)
    qs = []
    qmins = []
    for h in range(H):
        perm, q, qmin = match_rows_by_cosine(descA_hkd[h], descB_hkd[h])
        perms[h] = perm
        qs.append(q); qmins.append(qmin)
    return perms, float(np.mean(qs)), float(np.min(qmins))

# ----------------------------
# Extract read_weights per layer
# ----------------------------
def extract_rw_by_layer(infos):
    out = {}
    for li, info in enumerate(infos):
        if info is None:
            continue
        r = info.get("read_weights", None)
        if r is None or (not hasattr(r, "dim")) or r.dim() != 4:
            continue
        out[li] = r  # usually CPU due to detach_to_cpu=True
    return out

# ----------------------------
# Run a condition end-to-end
# ----------------------------
@torch.no_grad()
def summarize_condition(model, xb, yb, *, label, rope_reset=False):
    if rope_reset:
        with RopePosRemap(model, mode="reset", train_T=TRAIN_T):
            logits, infos = run_model(model, xb, info_level=INFO_LEVEL, info_cfg=INFO_CFG)
    else:
        logits, infos = run_model(model, xb, info_level=INFO_LEVEL, info_cfg=INFO_CFG)

    nll = streamed_nll_from_logits(logits, yb, chunk_t=LOSS_CHUNK_T, chunk_b=LOSS_CHUNK_B, compute_on_cpu=LOSS_ON_CPU)
    ppl = float(np.exp(min(20.0, nll)))

    # Proactively free logits now (helps fragmentation)
    del logits
    torch.cuda.empty_cache()
    gc.collect()

    rw_by_layer = extract_rw_by_layer(infos)
    head_feat_by_layer = {}
    for l, rw in rw_by_layer.items():
        head_feat_by_layer[l] = build_head_role_features_locked(rw)

    canon_heads, align_adj, layers = canonicalize_heads_across_layers(head_feat_by_layer)

    return dict(
        label=label,
        mean_nll=float(nll),
        ppl=float(ppl),
        rw_by_layer=rw_by_layer,
        head_feat_by_layer=head_feat_by_layer,
        canon_heads=canon_heads,
        align_adj=align_adj,
        layers=layers,
    )

# ----------------------------
# Main: run 3 conditions on SAME window
# ----------------------------
xb_long, yb_long = get_same_window(cfg, T=T_LONG, B=BATCH, split="val", device=DEVICE)
xb_train = xb_long[:, :TRAIN_T].contiguous()
yb_train = yb_long[:, :TRAIN_T].contiguous()

print(f"TRAIN_T={TRAIN_T} | T_LONG={T_LONG} | B={BATCH} | time_stride={INFO_CFG['time_stride']} | LOSS_CHUNK_T={LOSS_CHUNK_T}")

c_train = summarize_condition(model, xb_train, yb_train, label=f"T={TRAIN_T} (train-length)", rope_reset=False)
c_base  = summarize_condition(model, xb_long,  yb_long,  label=f"T={T_LONG} baseline",       rope_reset=False)
c_cyc   = summarize_condition(model, xb_long,  yb_long,  label=f"T={T_LONG} cyclic RoPE (t%{TRAIN_T})", rope_reset=True)

# ----------------------------
# Pick reference layer (middle of available layers)
# ----------------------------
def pick_ref_layer(cond):
    Ls = cond["layers"]
    return Ls[len(Ls)//2] if Ls else None

ref_layer = pick_ref_layer(c_train)
if ref_layer is None:
    raise RuntimeError("No layers with read_weights captured; check ASA plumbing / INFO_CFG.")

# ----------------------------
# Cross-condition head match at ref layer (schema-locked => no dim mismatch)
# ----------------------------
def head_alignment_at_layer(condA, condB, layer):
    A = condA["head_feat_by_layer"][layer]
    B = condB["head_feat_by_layer"][layer]
    perm, q, qmin = match_rows_by_cosine(A, B)
    return perm, q, qmin

perm_tb, q_tb, qmin_tb = head_alignment_at_layer(c_train, c_base, ref_layer)
perm_tc, q_tc, qmin_tc = head_alignment_at_layer(c_train, c_cyc,  ref_layer)

# ----------------------------
# Slot canonicalization at ref layer (head-aligned first)
# ----------------------------
def slot_alignment_at_layer(condA, condB, layer, head_perm_A_to_B):
    rwA = condA["rw_by_layer"][layer]  # [B,H,T,K]
    rwB = condB["rw_by_layer"][layer]  # [B,H,T,K]
    _, H, _, K = rwA.shape

    descA = build_slot_descriptors(rwA, max_lag=16)  # [H,K,D]
    descB = build_slot_descriptors(rwB, max_lag=16)  # [H,K,D]

    # reorder B heads to match A heads
    descB_aligned = descB[head_perm_A_to_B]

    perm_slots_hk, mean_q, min_q = canonicalize_slots_between_conditions(descA, descB_aligned)
    return perm_slots_hk, mean_q, min_q, dict(H=H, K=K, D=descA.shape[-1])

perm_slots_tb, qslot_tb, qslotmin_tb, hkinfo = slot_alignment_at_layer(c_train, c_base, ref_layer, perm_tb)
perm_slots_tc, qslot_tc, qslotmin_tc, _      = slot_alignment_at_layer(c_train, c_cyc,  ref_layer, perm_tc)

# ----------------------------
# Depth-wise head alignment quality (adjacent transitions)
# ----------------------------
def depth_alignment_df(cond):
    rows = []
    for (la, lb), (q, qmin) in cond["align_adj"].items():
        rows.append(dict(condition=cond["label"], a=int(la), b=int(lb), transition=f"{la}->{lb}", mean=q, min=qmin))
    return pd.DataFrame(rows)

df_depth = pd.concat([depth_alignment_df(c_train), depth_alignment_df(c_base), depth_alignment_df(c_cyc)], ignore_index=True)

# ----------------------------
# Summary tables
# ----------------------------
df_scores = pd.DataFrame([
    dict(condition=c_train["label"], mean_nll=c_train["mean_nll"], ppl=c_train["ppl"]),
    dict(condition=c_base["label"],  mean_nll=c_base["mean_nll"],  ppl=c_base["ppl"]),
    dict(condition=c_cyc["label"],   mean_nll=c_cyc["mean_nll"],   ppl=c_cyc["ppl"]),
]).sort_values("ppl")

df_align = pd.DataFrame([
    dict(compare=f"train ↔ baseline @ L{ref_layer}", head_match_mean=q_tb, head_match_min=qmin_tb, slot_match_mean=qslot_tb, slot_match_min=qslotmin_tb),
    dict(compare=f"train ↔ cyclic   @ L{ref_layer}", head_match_mean=q_tc, head_match_min=qmin_tc, slot_match_mean=qslot_tc, slot_match_min=qslotmin_tc),
])

display(df_scores)
display(df_align)

if not df_depth.empty:
    display(df_depth.groupby("condition")[["mean","min"]].mean().reset_index())

# ----------------------------
# Optional plots
# ----------------------------
if not df_depth.empty:
    plt.figure(figsize=(8.8, 4.2))
    for name, g in df_depth.groupby("condition"):
        g = g.sort_values("a")
        plt.plot(g["a"].values, g["mean"].values, marker="o", linewidth=2, label=name)
    plt.xlabel("Layer (transition start)")
    plt.ylabel("Adjacent-layer head match quality (mean cosine)")
    plt.title("Depth-wise canonical head stability (adjacent matching)")
    plt.grid(True, alpha=0.25)
    plt.legend(frameon=True)
    plt.tight_layout()
    plt.show()

plt.figure(figsize=(7.8, 3.6))
plt.bar([0, 1], [qslot_tb, qslot_tc])
plt.xticks([0, 1], [f"train↔baseline @L{ref_layer}", f"train↔cyclic @L{ref_layer}"], rotation=10)
plt.ylim(0, 1.02)
plt.ylabel("Slot match quality (mean cosine)")
plt.title("Canonical slot alignment quality (after head alignment)")
plt.grid(True, axis="y", alpha=0.25)
plt.tight_layout()
plt.show()

print(f"Ref layer: L{ref_layer} | H={hkinfo['H']} K={hkinfo['K']} | slot_desc_dim={hkinfo['D']} | head_feat_dim={len(FEATURE_KEYS)}")
print("Done.")

In [ ]:

#@title Analysis 2B (DROP-IN): Chunkwise slot retention curves + slot-slot confusion heatmaps (baseline vs cyclic), OOM-safe
# Assumes you already ran the prior "Analysis 2" cell and have:
#   - c_train, c_base, c_cyc (dicts with rw_by_layer, head_feat_by_layer, etc.)
#   - ref_layer (int)
#   - match_rows_by_cosine (row matcher)
#
# Produces:
#   - df_chunk_slot: chunkwise slot match quality (train↔baseline, train↔cyclic)
#   - heatmaps: slot-slot similarity matrices (baseline vs cyclic) after head alignment
#
# Notes:
#   - Uses the SAME slot descriptor as Analysis 2 (usage+time-bin context), but computed per-chunk.
#   - Head alignment uses your existing head features at ref_layer (robust + canonical).
#   - Slot alignment is recomputed per chunk to show drift (baseline) vs stability (cyclic).

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

# ----------------------------
# knobs
# ----------------------------
CHUNK_TOKENS = 1024      # analyze drift per ~train length
USE_REF_LAYER = ref_layer

# If your rw was time-strided, this cell automatically maps chunk boundaries in the strided index space.
# (so chunk count will be approx T_LONG / CHUNK_TOKENS, but based on rw.shape[2] effective T_eff)

# ----------------------------
# utilities
# ----------------------------
def _np_l2norm(x, axis=-1, eps=1e-9):
    return x / (np.linalg.norm(x, axis=axis, keepdims=True) + eps)

def _cosine_sim_matrix(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    A = _np_l2norm(A.astype(np.float64), axis=1)
    B = _np_l2norm(B.astype(np.float64), axis=1)
    return A @ B.T

def slot_desc_from_rw_chunk(rw_bhtk: torch.Tensor) -> np.ndarray:
    """
    Same spirit as Analysis 2:
      descriptor(slot k) = [usage_k, usage_vector (K dims), timebin_usage_vector (K dims)]
    where timebin_usage_vector gives a weak 'temporal context' for the chunk.
    Returns: [H,K,D]
    """
    eps = 1e-9
    rw = rw_bhtk.float()
    B, H, T, K = rw.shape

    # probs
    p = rw / rw.sum(dim=-1, keepdim=True).clamp_min(eps)    # [B,H,T,K]

    # usage within chunk
    u = p.mean(dim=(0,2))                                   # [H,K]
    u = u / u.sum(dim=-1, keepdim=True).clamp_min(eps)

    # a tiny temporal "shape": split the chunk into 4 bins (in strided time)
    nb = 4
    edges = torch.linspace(0, T, nb+1, device=rw.device).long()
    tb = []
    for i in range(nb):
        a = int(edges[i].item()); b = int(edges[i+1].item())
        if b <= a:
            tb.append(u)  # fallback
        else:
            uu = p[:, :, a:b, :].mean(dim=(0,2))            # [H,K]
            uu = uu / uu.sum(dim=-1, keepdim=True).clamp_min(eps)
            tb.append(uu)
    tb = torch.stack(tb, dim=0).mean(dim=0)                 # [H,K] (avg of per-bin usages)

    u_np  = u.detach().cpu().numpy()
    tb_np = tb.detach().cpu().numpy()

    # build slot descriptors
    D = 1 + K + K
    desc = np.zeros((H, K, D), dtype=np.float32)
    for h in range(H):
        for k in range(K):
            desc[h,k,0] = u_np[h,k]
            desc[h,k,1:1+K] = u_np[h]
            desc[h,k,1+K:]  = tb_np[h]
    desc = _np_l2norm(desc, axis=-1)
    return desc

def align_slots_per_head(descA_hkd: np.ndarray, descB_hkd: np.ndarray):
    """
    descA, descB: [H,K,D], align B slots to A slots per head.
    Returns:
      perms_hk: [H,K], q_mean, q_min
    """
    H, K, D = descA_hkd.shape
    perms = np.zeros((H, K), dtype=np.int64)
    sims_all = []
    for h in range(H):
        S = _cosine_sim_matrix(descA_hkd[h], descB_hkd[h])  # [K,K]
        # reuse your assignment routine via match_rows_by_cosine if available
        perm, q, qmin = match_rows_by_cosine(descA_hkd[h], descB_hkd[h])
        perms[h] = perm
        sims_all.extend([S[i, perm[i]] for i in range(K)])
    sims_all = np.asarray(sims_all, dtype=np.float64)
    return perms, float(sims_all.mean()), float(sims_all.min())

def apply_head_perm_desc(desc_hkd: np.ndarray, head_perm_A_to_B: np.ndarray) -> np.ndarray:
    # reorder B heads to line up with A heads
    return desc_hkd[head_perm_A_to_B]

def head_perm_at_layer(condA, condB, layer: int) -> np.ndarray:
    A = condA["head_feat_by_layer"][layer]
    B = condB["head_feat_by_layer"][layer]
    perm, q, qmin = match_rows_by_cosine(A, B)
    return perm

# ----------------------------
# gather tensors
# ----------------------------
rw_train = c_train["rw_by_layer"][USE_REF_LAYER]  # [B,H,Ttrain_eff,K]
rw_base  = c_base["rw_by_layer"][USE_REF_LAYER]   # [B,H,Tlong_eff,K]
rw_cyc   = c_cyc["rw_by_layer"][USE_REF_LAYER]    # [B,H,Tlong_eff,K]

# sanity
B0, H, Tt_eff, K = rw_train.shape
B1, H1, Tb_eff, K1 = rw_base.shape
B2, H2, Tc_eff, K2 = rw_cyc.shape
assert H == H1 == H2 and K == K1 == K2, "H/K mismatch across conditions at ref layer."

print(f"[ref layer L{USE_REF_LAYER}] rw shapes:")
print(f"  train: {tuple(rw_train.shape)}  baseline: {tuple(rw_base.shape)}  cyclic: {tuple(rw_cyc.shape)}")

# ----------------------------
# head alignment (train -> long)
# ----------------------------
perm_tb = head_perm_at_layer(c_train, c_base, USE_REF_LAYER)
perm_tc = head_perm_at_layer(c_train, c_cyc,  USE_REF_LAYER)

# ----------------------------
# chunk mapping in effective time coordinates
# ----------------------------
# Each rw time index corresponds to original tokens spaced by INFO_CFG['time_stride'].
time_stride = int(INFO_CFG.get("time_stride", 1))
chunk_eff = max(1, CHUNK_TOKENS // time_stride)  # chunk size in rw time axis
n_chunks = int(np.ceil(Tb_eff / chunk_eff))

print(f"time_stride={time_stride} => chunk_eff={chunk_eff} steps (~{CHUNK_TOKENS} tokens). n_chunks={n_chunks}")

# ----------------------------
# per-chunk slot retention curves
# ----------------------------
rows = []
for ci in range(n_chunks):
    t0 = ci * chunk_eff
    t1 = min(Tb_eff, (ci + 1) * chunk_eff)
    if t1 - t0 < 2:
        continue

    # train descriptor: always computed on full train window (reference basis)
    # (this is intentional: measure how well long-window chunk maps back to canonical train slots)
    desc_train = slot_desc_from_rw_chunk(rw_train)  # [H,K,D] small

    # baseline chunk
    desc_base = slot_desc_from_rw_chunk(rw_base[:, :, t0:t1, :])
    desc_base = apply_head_perm_desc(desc_base, perm_tb)
    _, q_b, qmin_b = align_slots_per_head(desc_train, desc_base)

    # cyclic chunk
    desc_cyc = slot_desc_from_rw_chunk(rw_cyc[:, :, t0:t1, :])
    desc_cyc = apply_head_perm_desc(desc_cyc, perm_tc)
    _, q_c, qmin_c = align_slots_per_head(desc_train, desc_cyc)

    center_tok = int((0.5 * (t0 + t1)) * time_stride)

    rows.append(dict(
        chunk=ci,
        t0_eff=t0, t1_eff=t1,
        center_token=center_tok,
        slot_match_mean_train_base=q_b,
        slot_match_min_train_base=qmin_b,
        slot_match_mean_train_cyc=q_c,
        slot_match_min_train_cyc=qmin_c,
    ))

df_chunk_slot = pd.DataFrame(rows)
display(df_chunk_slot)

# Plot: mean slot retention vs position
if not df_chunk_slot.empty:
    plt.figure(figsize=(9.0, 4.4))
    plt.plot(df_chunk_slot["center_token"].values, df_chunk_slot["slot_match_mean_train_base"].values,
             marker="o", linewidth=2, label="train↔baseline (per-chunk)")
    plt.plot(df_chunk_slot["center_token"].values, df_chunk_slot["slot_match_mean_train_cyc"].values,
             marker="o", linewidth=2, label="train↔cyclic (per-chunk)")
    plt.ylim(-0.2, 1.02)
    plt.xlabel("Absolute position (chunk center, token index)")
    plt.ylabel("Slot match quality (mean cosine)")
    plt.title(f"Chunkwise slot identity retention @ L{USE_REF_LAYER} (after head alignment)")
    plt.grid(True, alpha=0.25)
    plt.legend(frameon=True)
    plt.tight_layout()
    plt.show()

# Plot: min slot retention vs position (highlights worst-case flips)
if not df_chunk_slot.empty:
    plt.figure(figsize=(9.0, 4.4))
    plt.plot(df_chunk_slot["center_token"].values, df_chunk_slot["slot_match_min_train_base"].values,
             marker="o", linewidth=2, label="train↔baseline min")
    plt.plot(df_chunk_slot["center_token"].values, df_chunk_slot["slot_match_min_train_cyc"].values,
             marker="o", linewidth=2, label="train↔cyclic min")
    plt.ylim(-1.02, 1.02)
    plt.xlabel("Absolute position (chunk center, token index)")
    plt.ylabel("Slot match quality (min cosine)")
    plt.title(f"Chunkwise WORST-CASE slot match @ L{USE_REF_LAYER} (after head alignment)")
    plt.grid(True, alpha=0.25)
    plt.legend(frameon=True)
    plt.tight_layout()
    plt.show()

# ----------------------------
# slot-slot confusion heatmaps (global, at ref layer)
# ----------------------------
# Build global descriptors (whole sequences)
desc_train_full = slot_desc_from_rw_chunk(rw_train)  # [H,K,D]

desc_base_full = slot_desc_from_rw_chunk(rw_base)
desc_base_full = apply_head_perm_desc(desc_base_full, perm_tb)

desc_cyc_full = slot_desc_from_rw_chunk(rw_cyc)
desc_cyc_full = apply_head_perm_desc(desc_cyc_full, perm_tc)

# Compute averaged slot-slot similarity across heads:
def mean_slot_confusion(descA_hkd, descB_hkd):
    H, K, D = descA_hkd.shape
    mats = []
    for h in range(H):
        mats.append(_cosine_sim_matrix(descA_hkd[h], descB_hkd[h]))  # [K,K]
    return np.mean(np.stack(mats, axis=0), axis=0)  # [K,K]

S_base = mean_slot_confusion(desc_train_full, desc_base_full)
S_cyc  = mean_slot_confusion(desc_train_full, desc_cyc_full)

def plot_confusion(S, title):
    plt.figure(figsize=(6.2, 5.3))
    plt.imshow(S, aspect="auto", interpolation="nearest", vmin=-1.0, vmax=1.0)
    plt.colorbar(label="Cosine similarity")
    plt.xticks(range(K), [f"slot{j}" for j in range(K)], rotation=45, ha="right")
    plt.yticks(range(K), [f"slot{i}" for i in range(K)])
    plt.xlabel("Long-window slot (after head align)")
    plt.ylabel("Train slot")
    plt.title(title)
    plt.tight_layout()
    plt.show()

plot_confusion(S_base, f"Slot-slot confusion @ L{USE_REF_LAYER}: train vs baseline (mean over heads)")
plot_confusion(S_cyc,  f"Slot-slot confusion @ L{USE_REF_LAYER}: train vs cyclic RoPE (mean over heads)")

# Quick scalar summaries (diagonal dominance)
def diag_stats(S):
    diag = np.diag(S)
    off = S.copy()
    np.fill_diagonal(off, np.nan)
    return dict(
        diag_mean=float(np.nanmean(diag)),
        diag_min=float(np.nanmin(diag)),
        off_mean=float(np.nanmean(off)),
        off_max=float(np.nanmax(off)),
        margin=float(np.nanmean(diag) - np.nanmean(off)),
    )

stats = pd.DataFrame([
    dict(condition="train vs baseline", **diag_stats(S_base)),
    dict(condition="train vs cyclic",   **diag_stats(S_cyc)),
])
display(stats)

print("Done.")

In [ ]:


#@title ASA-aware Sampling Lab v2 (DROP-IN): stable jerk metric + ASA margin gating + bidirectional controller

import math
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# User knobs
# ----------------------------
USE_XB_PROMPT = True

PROMPT_B = 1
PROMPT_LEN = 256
MAX_NEW_TOKENS = 256
CTX_WINDOW = 2048

ASA_SAMPLER_INFO_CFG = dict(
    store_read_weights=True,
    store_read_logits=False,
    store_write_logits=False,
    store_slot_state_norm=False,
    detach_to_cpu=False,
    time_stride=1,
    batch_stride=1,
)
INFO_LEVEL = "basic"

LAYER_PICK = "mid"   # "mid" | "first" | "last" | int

# ----------------------------
# Baseline sampling params
# ----------------------------
BASE_TEMP = 0.9
BASE_TOP_P = 0.9
BASE_TOP_K = 0

# ----------------------------
# ASA-adaptive bounds
# ----------------------------
TEMP_MIN = 0.35
TEMP_MAX = 1.25
TOPP_MIN = 0.60
TOPP_MAX = 0.97

# ----------------------------
# Controller weights
# ----------------------------
# Uncertainty signals (cool down)
W_ENT   = 0.9     # routing entropy
W_FLIP  = 1.0     # argmax flip rate
W_EFF   = 0.6     # effective slots proxy
W_JERK  = 0.35    # jerk metric (bounded JSD)

# Confidence signals (warm up)
W_INERT = 0.9     # inertia (cosine)
W_MARG  = 1.1     # routing margin (max - second max)

ADAPT_STRENGTH = 1.15

# EMA smoothing
EMA_BETA = 0.92

# Jerk metric config (bounded + cheap)
JERK_TOPM = 256     # compute JSD over top-M tokens only
JERK_CAP = 0.50     # cap jerk contribution (JSD is in [0, ln2] ~ [0,0.693])

# ----------------------------
# Utilities: nucleus/top-k
# ----------------------------
def top_k_logits(logits, k: int):
    if k <= 0:
        return logits
    v, _ = torch.topk(logits, k)
    thresh = v[..., -1, None]
    return logits.masked_fill(logits < thresh, float("-inf"))

def top_p_logits(logits, p: float):
    if p >= 1.0:
        return logits
    probs = torch.softmax(logits, dim=-1)
    sorted_probs, sorted_idx = torch.sort(probs, descending=True, dim=-1)
    cdf = torch.cumsum(sorted_probs, dim=-1)
    mask = cdf > p
    mask[..., 0] = False
    sorted_logits = logits.gather(-1, sorted_idx)
    sorted_logits = sorted_logits.masked_fill(mask, float("-inf"))
    unsorted = torch.empty_like(sorted_logits).scatter_(-1, sorted_idx, sorted_logits)
    return unsorted

# ----------------------------
# ASA layer picker + metrics
# ----------------------------
def pick_layer_index(infos, pick="mid"):
    valid = [i for i, info in enumerate(infos)
             if (info is not None and isinstance(info, dict) and info.get("read_weights", None) is not None)]
    if not valid:
        return None
    if isinstance(pick, int):
        return pick if pick in valid else valid[len(valid)//2]
    if pick == "first":
        return valid[0]
    if pick == "last":
        return valid[-1]
    return valid[len(valid)//2]

@torch.no_grad()
def asa_lasttoken_stats(rw_bhtk, prev_p_bhk=None, prev_arg_bh=None, eps=1e-9):
    """
    rw: [B,H,T,K]
    returns dict of scalars + updated prevs
    """
    p = rw_bhtk[:, :, -1, :].float()                    # [B,H,K]
    p = p / p.sum(dim=-1, keepdim=True).clamp_min(eps)
    B, H, K = p.shape

    # entropy (normalized)
    ent = -(p * torch.log(p.clamp_min(eps))).sum(dim=-1)  # [B,H]
    ent_norm = (ent / math.log(K)).mean().item()

    # effective slots proxy: exp(E[H(p)]) (mean over BH)
    eff = torch.exp(ent).mean().item()

    # margin: max - 2nd max (mean over BH)
    top2 = torch.topk(p, k=min(2, K), dim=-1).values  # [B,H,2]
    margin = (top2[..., 0] - top2[..., 1]).mean().item() if K >= 2 else 1.0

    # inertia: cosine(p_t, p_{t-1})
    if prev_p_bhk is None:
        inertia = 1.0
        prev_p_bhk = p
    else:
        a = p / p.norm(dim=-1, keepdim=True).clamp_min(eps)
        b = prev_p_bhk / prev_p_bhk.norm(dim=-1, keepdim=True).clamp_min(eps)
        inertia = (a * b).sum(dim=-1).mean().item()
        prev_p_bhk = p

    # flip rate: argmax changes across heads
    arg = p.argmax(dim=-1)  # [B,H]
    if prev_arg_bh is None:
        flip = 0.0
        prev_arg_bh = arg
    else:
        flip = (arg != prev_arg_bh).float().mean().item()
        prev_arg_bh = arg

    return dict(ent_norm=ent_norm, eff_slots=eff, margin=margin, inertia=inertia, flip=flip,
                H=H, K=K), prev_p_bhk, prev_arg_bh

# ----------------------------
# Bounded jerk: JSD over top-M support
# ----------------------------
@torch.no_grad()
def jsd_topm(logits_new_bv, logits_old_bv, topm=256, eps=1e-9):
    """
    Jensen-Shannon divergence between distributions induced by logits, computed on union top-M support.
    Returns scalar in [0, ln2] ~ [0,0.693]
    """
    B, V = logits_new_bv.shape
    m = min(int(topm), V)

    # union support indices per batch: take topm of each, then union
    idx_new = torch.topk(logits_new_bv, k=m, dim=-1).indices
    idx_old = torch.topk(logits_old_bv, k=m, dim=-1).indices
    idx = torch.cat([idx_new, idx_old], dim=-1)  # [B,2m]
    # unique per row (small), but torch.unique on each row is annoying; we'll just keep 2m and accept duplicates via gather+scatter
    # We'll build dense probs on this set by softmax then renorm on gathered entries.
    ln_new = logits_new_bv.gather(-1, idx)
    ln_old = logits_old_bv.gather(-1, idx)

    p = torch.softmax(ln_new, dim=-1)
    q = torch.softmax(ln_old, dim=-1)

    m_mix = 0.5 * (p + q)
    # KL(p||m) + KL(q||m)
    kl_pm = (p * (torch.log(p.clamp_min(eps)) - torch.log(m_mix.clamp_min(eps)))).sum(dim=-1)
    kl_qm = (q * (torch.log(q.clamp_min(eps)) - torch.log(m_mix.clamp_min(eps)))).sum(dim=-1)
    jsd = 0.5 * (kl_pm + kl_qm)
    return jsd.mean().item()

# ----------------------------
# Forward helper
# ----------------------------
@torch.no_grad()
def forward_full(model, x_bt):
    model.eval()
    try:
        logits, infos = model(
            x_bt,
            attention_mask=None,
            return_info=True,
            info_level=INFO_LEVEL,
            info_cfg=ASA_SAMPLER_INFO_CFG,
        )
    except TypeError:
        logits, infos = model(x_bt, attention_mask=None, return_info=True)
    return logits, infos

# ----------------------------
# Controller
# ----------------------------
def ema(old, new, beta=0.92):
    return float(new) if old is None else float(beta * old + (1.0 - beta) * new)

def clamp01(x):
    return float(max(0.0, min(1.0, x)))

def adapt_params_v2(base_temp, base_top_p, s):
    """
    s is dict of EMA'd signals:
      ent_norm_ema in [0,1] (higher => uncertainty)
      inertia_ema in [-1,1] (higher => stable)
      flip_ema in [0,1] (higher => unstable)
      margin_ema in [0,1-ish] (higher => confident)
      eff_slots_ema in [1,K] (higher => uncertain / spread)
      jerk_ema in [0, ln2]
    """
    ent = clamp01(s["ent_norm_ema"])
    # inertia [-1,1] -> [0,1]
    inert = clamp01(0.5 * (s["inertia_ema"] + 1.0))
    flip = clamp01(s["flip_ema"])
    # margin is usually small-ish; map through a saturating function
    margin = float(s["margin_ema"])
    marg01 = clamp01(margin / 0.35)  # 0.35 is a decent "pretty confident" heuristic; adjust later

    eff = float(s["eff_slots_ema"])
    K = max(2.0, float(s.get("K", 16)))
    eff01 = clamp01((eff - 1.0) / (K - 1.0))  # 1 => deterministic, K => uniform

    jerk = float(s.get("jerk_ema", 0.0))
    jerk01 = clamp01(min(jerk, JERK_CAP) / JERK_CAP)

    warm = (W_INERT * inert + W_MARG * marg01)
    cool = (W_ENT * ent + W_FLIP * flip + W_EFF * eff01 + W_JERK * jerk01)
    score = ADAPT_STRENGTH * (warm - cool)

    # Turn score into a smooth multiplier.
    # Positive => warm up; Negative => cool down.
    m = math.tanh(score)  # [-1,1]

    temp = base_temp * (1.0 + 0.55 * m)
    topp = base_top_p * (1.0 + 0.22 * m)

    temp = float(np.clip(temp, TEMP_MIN, TEMP_MAX))
    topp = float(np.clip(topp, TOPP_MIN, TOPP_MAX))
    return temp, topp, dict(ent=ent, inert=inert, flip=flip, marg01=marg01, eff01=eff01, jerk01=jerk01, score=score, m=m)

# ----------------------------
# Sampler
# ----------------------------
@torch.no_grad()
def generate(model, prompt_bt, *, max_new, ctx_window, mode="baseline",
             base_temp=0.9, base_top_p=0.9, base_top_k=0):
    x = prompt_bt.to(DEVICE)
    prev_p_bhk = None
    prev_arg_bh = None
    prev_logits = None

    st = dict(ent=None, inertia=None, flip=None, margin=None, eff=None, jerk=None)
    logs = []

    for step in range(int(max_new)):
        x_ctx = x[:, -ctx_window:].contiguous()
        logits_btv, infos = forward_full(model, x_ctx)
        next_logits = logits_btv[:, -1, :]  # [B,V]

        layer = pick_layer_index(infos, LAYER_PICK)
        if layer is None:
            # no ASA info; fall back to baseline
            temp = base_temp
            topp = base_top_p
            ctrl = {}
            ent = inert = flip = margin = eff = np.nan
            H = K = -1
        else:
            rw = infos[layer]["read_weights"]
            stats, prev_p_bhk, prev_arg_bh = asa_lasttoken_stats(rw, prev_p_bhk, prev_arg_bh)
            ent, inert, flip = stats["ent_norm"], stats["inertia"], stats["flip"]
            margin, eff = stats["margin"], stats["eff_slots"]
            H, K = stats["H"], stats["K"]

            # jerk via bounded JSD on top-M support of logits
            if prev_logits is None:
                jerk = 0.0
            else:
                jerk = jsd_topm(next_logits, prev_logits, topm=JERK_TOPM)
            prev_logits = next_logits.detach()

            # EMA
            st["ent"] = ema(st["ent"], ent, EMA_BETA)
            st["inertia"] = ema(st["inertia"], inert, EMA_BETA)
            st["flip"] = ema(st["flip"], flip, EMA_BETA)
            st["margin"] = ema(st["margin"], margin, EMA_BETA)
            st["eff"] = ema(st["eff"], eff, EMA_BETA)
            st["jerk"] = ema(st["jerk"], jerk, EMA_BETA)

            if mode == "baseline":
                temp = base_temp
                topp = base_top_p
                ctrl = {}
            else:
                s = dict(
                    ent_norm_ema=st["ent"],
                    inertia_ema=st["inertia"],
                    flip_ema=st["flip"],
                    margin_ema=st["margin"],
                    eff_slots_ema=st["eff"],
                    jerk_ema=st["jerk"],
                    K=K,
                )
                temp, topp, ctrl = adapt_params_v2(base_temp, base_top_p, s)

        # sample
        logits = next_logits / max(1e-6, float(temp))
        logits = top_k_logits(logits, int(base_top_k))
        logits = top_p_logits(logits, float(topp))
        probs = torch.softmax(logits, dim=-1)
        nxt = torch.multinomial(probs, num_samples=1)
        x = torch.cat([x, nxt], dim=1)

        logs.append(dict(
            step=step, layer=int(layer if layer is not None else -1),
            H=int(H), K=int(K),
            ent_norm=float(ent) if not np.isnan(ent) else np.nan,
            inertia=float(inert) if not np.isnan(inert) else np.nan,
            flip=float(flip) if not np.isnan(flip) else np.nan,
            margin=float(margin) if not np.isnan(margin) else np.nan,
            eff_slots=float(eff) if not np.isnan(eff) else np.nan,
            jerk=float(st["jerk"]) if st["jerk"] is not None else np.nan,
            ent_ema=float(st["ent"]) if st["ent"] is not None else np.nan,
            inertia_ema=float(st["inertia"]) if st["inertia"] is not None else np.nan,
            flip_ema=float(st["flip"]) if st["flip"] is not None else np.nan,
            margin_ema=float(st["margin"]) if st["margin"] is not None else np.nan,
            eff_ema=float(st["eff"]) if st["eff"] is not None else np.nan,
            temp=float(temp), top_p=float(topp),
            ctrl_score=float(ctrl.get("score", np.nan)),
            ctrl_m=float(ctrl.get("m", np.nan)),
            ctrl_marg01=float(ctrl.get("marg01", np.nan)),
            ctrl_ent=float(ctrl.get("ent", np.nan)),
            ctrl_inert=float(ctrl.get("inert", np.nan)),
            ctrl_flip=float(ctrl.get("flip", np.nan)),
            ctrl_eff01=float(ctrl.get("eff01", np.nan)),
            ctrl_jerk01=float(ctrl.get("jerk01", np.nan)),
        ))

    return x, pd.DataFrame(logs)

# ----------------------------
# Prompt
# ----------------------------
if USE_XB_PROMPT:
    try:
        xb
    except NameError:
        raise NameError("USE_XB_PROMPT=True but xb is not defined. Define xb or set USE_XB_PROMPT=False and pass prompt_ids.")
    prompt = xb[:PROMPT_B, :PROMPT_LEN].contiguous().long()
else:
    raise ValueError("Set USE_XB_PROMPT=True or define prompt_ids.")

print(f"Prompt: B={prompt.shape[0]} T0={prompt.shape[1]} | CTX_WINDOW={CTX_WINDOW} | MAX_NEW_TOKENS={MAX_NEW_TOKENS}")

tok_base, df_base = generate(model, prompt, max_new=MAX_NEW_TOKENS, ctx_window=CTX_WINDOW,
                             mode="baseline", base_temp=BASE_TEMP, base_top_p=BASE_TOP_P, base_top_k=BASE_TOP_K)
tok_adap, df_adap = generate(model, prompt, max_new=MAX_NEW_TOKENS, ctx_window=CTX_WINDOW,
                             mode="asa_adaptive", base_temp=BASE_TEMP, base_top_p=BASE_TOP_P, base_top_k=BASE_TOP_K)

print("Done sampling.")
display(df_base.head(5))
display(df_adap.head(5))

# ----------------------------
# Plots
# ----------------------------
def plot_series(dfA, dfB, col, title):
    plt.figure(figsize=(9.5, 3.4))
    plt.plot(dfA["step"], dfA[col], linewidth=2, label=f"baseline")
    plt.plot(dfB["step"], dfB[col], linewidth=2, label=f"ASA-adaptive")
    plt.title(title)
    plt.xlabel("step")
    plt.grid(True, alpha=0.25)
    plt.legend(frameon=True)
    plt.tight_layout()
    plt.show()

plot_series(df_base, df_adap, "temp", "Temperature schedule")
plot_series(df_base, df_adap, "top_p", "Top-p schedule")
plot_series(df_base, df_adap, "ent_ema", "Routing entropy (EMA, normalized)")
plot_series(df_base, df_adap, "inertia_ema", "Routing inertia (EMA)")
plot_series(df_base, df_adap, "flip_ema", "Routing argmax flip rate (EMA)")
plot_series(df_base, df_adap, "margin_ema", "Routing margin (EMA)")
plot_series(df_base, df_adap, "eff_ema", "Effective slots proxy (EMA)")
plot_series(df_base, df_adap, "jerk", "Bounded jerk (EMA JSD over top-M logits)")

# Controller internals
plot_series(df_base, df_adap, "ctrl_score", "Controller score (warm - cool)")
plot_series(df_base, df_adap, "ctrl_m", "Controller tanh(score)")

# Summary stats
def summarize(df, name):
    return dict(
        name=name,
        temp_mean=float(np.nanmean(df["temp"])),
        temp_min=float(np.nanmin(df["temp"])),
        temp_max=float(np.nanmax(df["temp"])),
        topp_mean=float(np.nanmean(df["top_p"])),
        ent_mean=float(np.nanmean(df["ent_norm"])),
        inert_mean=float(np.nanmean(df["inertia"])),
        flip_mean=float(np.nanmean(df["flip"])),
        margin_mean=float(np.nanmean(df["margin"])),
        eff_mean=float(np.nanmean(df["eff_slots"])),
        jerk_mean=float(np.nanmean(df["jerk"])),
    )

df_sum = pd.DataFrame([summarize(df_base, "baseline"), summarize(df_adap, "asa_adaptive")])
display(df_sum)

# If you have a tokenizer:
# print(tokenizer.decode(tok_adap[0].tolist()))

In [ ]:


#@title Qualitative sampling check (baseline vs RoPE-reset) — side-by-side + light ASA stats

import math, numpy as np, pandas as pd, torch
import torch.nn.functional as F
from contextlib import nullcontext
from dataclasses import dataclass

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Assumptions:
#   - model, cfg exist
#   - tokenizer available as `tokenizer` OR provide your own decode_fn
#   - You have ASA info plumbing (optional) for read_weights stats. If not, this still works.
# ----------------------------

# ----------------------------
# Config
# ----------------------------
TRAIN_T = int(getattr(cfg, "max_seq_len", getattr(model, "training_max_seq_len", getattr(model, "max_seq_len", 256))))
print("TRAIN_T =", TRAIN_T)

N_SAMPLES       = 4
PROMPT_TOKENS   = 4096
CTX_WINDOW      = 2048            # sliding window
MAX_NEW_TOKENS  = 256
TOP_P_BASE      = 0.90
TEMP_BASE       = 0.90
SEED            = 1337

# For the "rope reset" condition:
ROPE_MODE = "reset"               # "reset" or "rand_block"
DISABLE_ALIBI = False             # try True as a 3rd condition if you want

# Internals logging (optional)
LOG_EVERY = 16                    # steps between metric logs
CAPTURE_INFO = True               # set False if your wrapper doesn't support return_info
INFO_LEVEL = "basic"
INFO_CFG = dict(
    store_read_weights=True,
    store_read_logits=False,
    store_write_logits=False,
    store_slot_state_norm=False,
    detach_to_cpu=True,
    time_stride=16,
    batch_stride=1,
)

# ----------------------------
# Token decode
# ----------------------------
def default_decode(ids):
    if "tokenizer" in globals() and tokenizer is not None:
        return tokenizer.decode(ids, skip_special_tokens=True)
    # fallback: show token ids
    return " ".join(map(str, ids))

decode_fn = default_decode

# ----------------------------
# RoPE + ALiBi patchers (duck-typed)
# ----------------------------
def _is_rope_module(m) -> bool:
    return hasattr(m, "inv_freq") and hasattr(m, "get_cos_sin") and callable(getattr(m, "get_cos_sin"))

def _make_pos_map(T: int, period: int, mode: str, seed: int, device):
    t = torch.arange(T, device=device, dtype=torch.long)
    if mode == "baseline":
        return t
    if mode == "reset":
        return t % period
    if mode == "rand_block":
        rng = np.random.default_rng(seed)
        out = torch.empty((T,), device=device, dtype=torch.long)
        n_blocks = (T + period - 1) // period
        for b in range(n_blocks):
            off = int(rng.integers(low=0, high=period))
            s = b * period
            e = min(T, (b + 1) * period)
            out[s:e] = (torch.arange(e - s, device=device, dtype=torch.long) + off) % period
        return out
    raise ValueError(f"Unknown mode={mode}")

class RopeRemapPatch:
    def __init__(self, model, *, mode: str, period: int, seed: int):
        self.model = model
        self.mode = mode
        self.period = int(period)
        self.seed = int(seed)
        self._saved = []

    def __enter__(self):
        rope_modules = [m for m in self.model.modules() if _is_rope_module(m)]
        if len(rope_modules) == 0:
            print("⚠ No RoPE-like modules found to patch.")
            return self

        for rm in rope_modules:
            old = rm.get_cos_sin

            def make_wrapped(rm_ref, old_fn):
                def wrapped(T: int, device, dtype):
                    t_map = _make_pos_map(T, self.period, self.mode, self.seed, device=device)
                    inv = rm_ref.inv_freq.to(device=device)
                    freqs = torch.einsum("t,f->tf", t_map.to(inv.dtype), inv)  # [T, d/2]
                    emb = torch.cat([freqs, freqs], dim=-1)                    # [T, d]
                    cos = emb.cos()[None, None, :, :].to(dtype=dtype)
                    sin = emb.sin()[None, None, :, :].to(dtype=dtype)
                    return cos, sin
                return wrapped

            rm.get_cos_sin = make_wrapped(rm, old)
            self._saved.append((rm, old))
        return self

    def __exit__(self, exc_type, exc, tb):
        for rm, old in self._saved:
            rm.get_cos_sin = old
        self._saved.clear()
        return False

class TempSetAlibiZero:
    def __init__(self, model):
        self.model = model
        self.saved = []

    def __enter__(self):
        for m in self.model.modules():
            if hasattr(m, "_alibi_strength_param") and isinstance(getattr(m, "_alibi_strength_param"), torch.nn.Parameter):
                p = m._alibi_strength_param
                self.saved.append(("param", p, p.data.clone()))
                p.data.fill_(-30.0)
            elif hasattr(m, "alibi_strength") and isinstance(getattr(m, "alibi_strength"), (float, int)):
                self.saved.append(("attr", m, float(m.alibi_strength)))
                m.alibi_strength = 0.0
        return self

    def __exit__(self, exc_type, exc, tb):
        for kind, obj, old in self.saved:
            if kind == "param":
                obj.data.copy_(old)
            else:
                obj.alibi_strength = old
        self.saved.clear()
        return False

# ----------------------------
# Helpers: top-p sampling
# ----------------------------
def sample_top_p(logits_1v, top_p: float, temperature: float):
    logits = logits_1v / max(1e-6, float(temperature))
    probs = torch.softmax(logits, dim=-1)

    # sort
    p_sorted, idx = torch.sort(probs, descending=True)
    cdf = torch.cumsum(p_sorted, dim=-1)
    keep = cdf <= top_p
    keep[..., 0] = True

    p_keep = p_sorted * keep
    p_keep = p_keep / p_keep.sum(dim=-1, keepdim=True).clamp_min(1e-9)

    # sample in sorted space then map back
    j = torch.multinomial(p_keep, num_samples=1)
    tok = idx.gather(-1, j)
    return tok.squeeze(-1)

# ----------------------------
# Optional: compute light ASA stats from infos
# ----------------------------
def light_asa_stats(infos):
    """
    Returns dict with:
      ent_norm, inertia, flip, margin, eff_slots
    Computed on the *last* layer that has read_weights.
    Uses the time-strided read_weights (already downsampled/offloaded).
    """
    if infos is None:
        return {}
    last = None
    for info in reversed(infos):
        if info is None:
            continue
        rw = info.get("read_weights", None)
        if rw is not None and rw.dim() == 4:
            last = rw.float()
            break
    if last is None:
        return {}

    # last: [B,H,T,K]
    eps = 1e-9
    p = last / last.sum(dim=-1, keepdim=True).clamp_min(eps)
    B,H,T,K = p.shape

    # entropy normalized by log K
    ent = -(p * torch.log(p.clamp_min(eps))).sum(dim=-1)              # [B,H,T]
    ent_norm = float((ent / math.log(K)).mean().cpu())

    # inertia: mean cosine between successive time points
    if T >= 2:
        a = p[:, :, :-1, :]
        b = p[:, :, 1:, :]
        a = a / a.norm(dim=-1, keepdim=True).clamp_min(eps)
        b = b / b.norm(dim=-1, keepdim=True).clamp_min(eps)
        inertia = float((a*b).sum(dim=-1).mean().cpu())
    else:
        inertia = float("nan")

    # flip: argmax slot changes
    arg = p.argmax(dim=-1)  # [B,H,T]
    if T >= 2:
        flip = float((arg[:, :, 1:] != arg[:, :, :-1]).float().mean().cpu())
    else:
        flip = float("nan")

    # margin: mean (top1 - top2)
    top2 = torch.topk(p, k=min(2, K), dim=-1).values  # [B,H,T,2]
    if top2.shape[-1] == 2:
        margin = float((top2[..., 0] - top2[..., 1]).mean().cpu())
    else:
        margin = float("nan")

    # eff slots: exp(entropy)
    eff_slots = float(torch.exp(ent.mean()).cpu())

    return dict(ent_norm=ent_norm, inertia=inertia, flip=flip, margin=margin, eff_slots=eff_slots)

# ----------------------------
# Model forward wrappers
# ----------------------------
@torch.no_grad()
def forward_logits_and_infos(x_ids):
    """
    x_ids: [1,T] token ids
    returns logits [1,T,V] and infos (or None)
    """
    model.eval()
    try:
        out = model(x_ids, attention_mask=None, return_info=CAPTURE_INFO, info_level=INFO_LEVEL, info_cfg=INFO_CFG)
        if isinstance(out, (tuple, list)) and len(out) == 2:
            logits, infos = out
        else:
            logits, infos = out, None
        return logits, infos
    except TypeError:
        out = model(x_ids, attention_mask=None, return_info=CAPTURE_INFO)
        if isinstance(out, (tuple, list)) and len(out) == 2:
            logits, infos = out
        else:
            logits, infos = out, None
        return logits, infos

# ----------------------------
# Sampling routine
# ----------------------------
@dataclass
class SampleResult:
    name: str
    prompt_ids: list
    gen_ids: list
    logs: pd.DataFrame

@torch.no_grad()
def generate_one(name: str, prompt_ids: torch.Tensor, *, rope_patch=False, rope_mode="reset", alibi_zero=False):
    # running sequence
    x = prompt_ids.clone().to(DEVICE)  # [1,T0]
    logs = []
    remap_hits = 0

    rope_ctx = RopeRemapPatch(model, mode=rope_mode, period=TRAIN_T, seed=SEED) if rope_patch else nullcontext()
    alibi_ctx = TempSetAlibiZero(model) if alibi_zero else nullcontext()

    with rope_ctx, alibi_ctx:
        for step in range(MAX_NEW_TOKENS):
            x_ctx = x[:, -CTX_WINDOW:]
            T_ctx = int(x_ctx.shape[1])

            # only count "remap needed" if we *would* exceed TRAIN_T in this call
            did_remap = bool(rope_patch and (T_ctx > TRAIN_T))
            remap_hits += int(did_remap)

            logits, infos = forward_logits_and_infos(x_ctx)
            next_logits = logits[:, -1, :]  # [1,V]
            tok = sample_top_p(next_logits, top_p=TOP_P_BASE, temperature=TEMP_BASE)
            x = torch.cat([x, tok.view(1,1)], dim=1)

            if (step % LOG_EVERY) == 0:
                stats = light_asa_stats(infos) if CAPTURE_INFO else {}
                logs.append(dict(
                    step=step,
                    t_ctx=T_ctx,
                    did_rope_remap=did_remap,
                    ent_norm=stats.get("ent_norm", np.nan),
                    inertia=stats.get("inertia", np.nan),
                    flip=stats.get("flip", np.nan),
                    margin=stats.get("margin", np.nan),
                    eff_slots=stats.get("eff_slots", np.nan),
                    temp=TEMP_BASE,
                    top_p=TOP_P_BASE,
                ))

    df = pd.DataFrame(logs)
    return SampleResult(
        name=name,
        prompt_ids=prompt_ids.squeeze(0).tolist(),
        gen_ids=x.squeeze(0).tolist(),
        logs=df,
    )

# ----------------------------
# Build deterministic prompts from same-window data
# ----------------------------
torch.manual_seed(SEED)
np.random.seed(SEED)

@torch.no_grad()
def get_prompts_from_data(cfg, *, T: int, B: int, split="val"):
    gen = make_batch_generator(cfg, split=split, device=DEVICE, seq_len=int(T), batches_per_epoch=1, infinite=False, num_workers=0)
    xb, yb = next(gen)
    xb = xb[:B].contiguous()
    return xb

prompts = get_prompts_from_data(cfg, T=PROMPT_TOKENS, B=N_SAMPLES, split="val")

# ----------------------------
# Run both conditions
# ----------------------------
results = []
all_logs = []

for i in range(N_SAMPLES):
    prompt = prompts[i:i+1]  # [1,T0]
    r0 = generate_one(f"sample{i}_baseline", prompt, rope_patch=False, alibi_zero=False)
    r1 = generate_one(f"sample{i}_rope_reset", prompt, rope_patch=True, rope_mode=ROPE_MODE, alibi_zero=DISABLE_ALIBI)
    results.append((r0, r1))
    all_logs.append(r0.logs.assign(sample=i, condition="baseline"))
    all_logs.append(r1.logs.assign(sample=i, condition=f"rope_{ROPE_MODE}"))

df_logs = pd.concat(all_logs, ignore_index=True)
display(df_logs.head(12))

# ----------------------------
# Print side-by-side text
# ----------------------------
def show_pair(r_base: SampleResult, r_rope: SampleResult, n_chars=1200):
    base_txt = decode_fn(r_base.gen_ids)
    rope_txt = decode_fn(r_rope.gen_ids)
    print("="*110)
    print(f"[{r_base.name}]")
    print(base_txt[:n_chars])
    print("-"*110)
    print(f"[{r_rope.name}]")
    print(rope_txt[:n_chars])
    print("="*110)

for r0, r1 in results:
    show_pair(r0, r1, n_chars=1200)

# ----------------------------
# Quick aggregate view of remap actually triggering
# ----------------------------
if not df_logs.empty:
    agg = df_logs.groupby(["condition"]).agg(
        ent_mean=("ent_norm", "mean"),
        inert_mean=("inertia", "mean"),
        flip_mean=("flip", "mean"),
        margin_mean=("margin", "mean"),
        eff_mean=("eff_slots", "mean"),
        tctx_max=("t_ctx", "max"),
        remap_frac=("did_rope_remap", "mean"),
    ).reset_index()
    display(agg)

print("Done.")

In [ ]:

#@title Semantic Token Group Definitions & Classification

import numpy as np
import torch
from collections import defaultdict
from typing import Dict, List, Set
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# ============================
# Comprehensive Semantic Groups
# ============================

SEMANTIC_GROUPS = {
    'colors': {
        'red', 'blue', 'green', 'yellow', 'orange', 'purple', 'pink', 'brown',
        'black', 'white', 'gray', 'grey', 'gold', 'silver', 'crimson', 'azure',
        'violet', 'indigo', 'scarlet', 'emerald', 'turquoise', 'cyan', 'magenta'
    },

    'numbers': {
        'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight',
        'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen',
        'twenty', 'thirty', 'forty', 'fifty', 'hundred', 'thousand', 'million',
        'first', 'second', 'third', 'fourth', 'fifth', 'once', 'twice'
    },

    'animals': {
        'dog', 'cat', 'bird', 'fish', 'lion', 'tiger', 'bear', 'wolf', 'fox',
        'elephant', 'giraffe', 'zebra', 'horse', 'cow', 'pig', 'sheep', 'goat',
        'chicken', 'duck', 'rabbit', 'mouse', 'rat', 'snake', 'frog', 'monkey',
        'whale', 'dolphin', 'shark', 'eagle', 'owl', 'parrot', 'penguin'
    },

    'action_verbs': {
        'run', 'running', 'ran', 'walk', 'walking', 'walked', 'jump', 'jumping',
        'eat', 'eating', 'ate', 'drink', 'drinking', 'sleep', 'sleeping', 'slept',
        'write', 'writing', 'wrote', 'read', 'reading', 'think', 'thinking',
        'speak', 'speaking', 'spoke', 'listen', 'listening', 'watched', 'watching'
    },

    'ownership': {
        'my', 'mine', 'your', 'yours', 'his', 'her', 'hers', 'their', 'theirs',
        'our', 'ours', 'its', 'own', 'owns', 'owned', 'have', 'has', 'had',
        'possess', 'possesses', 'possessed', 'belong', 'belongs', 'belonged'
    },

    'places': {
        'city', 'town', 'village', 'country', 'state', 'nation', 'home', 'house',
        'building', 'school', 'hospital', 'park', 'street', 'road', 'mountain',
        'river', 'ocean', 'sea', 'lake', 'forest', 'desert', 'island', 'beach',
        'restaurant', 'store', 'shop', 'office', 'church', 'temple', 'museum'
    },

    'dates_time': {
        'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',
        'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
        'september', 'october', 'november', 'december', 'morning', 'afternoon',
        'evening', 'night', 'today', 'tomorrow', 'yesterday', 'week', 'month', 'year'
    },

    'emotions': {
        'happy', 'sad', 'angry', 'fear', 'fearful', 'joy', 'joyful', 'love',
        'hate', 'excited', 'calm', 'anxious', 'proud', 'ashamed', 'surprised',
        'disgusted', 'confused', 'confident', 'nervous', 'jealous', 'grateful'
    },

    'quantifiers': {
        'all', 'some', 'many', 'few', 'several', 'most', 'each', 'every',
        'any', 'no', 'none', 'both', 'either', 'neither', 'much', 'little',
        'more', 'less', 'fewer', 'enough', 'plenty'
    },
}

def classify_token_semantic_group(token_id: int, tokenizer) -> str:
    """
    Classify a token into semantic groups.
    Returns group name or 'other'.
    """
    token_str = tokenizer.decode([token_id]).strip().lower()

    # Remove leading/trailing spaces and special characters
    token_str = ''.join(c for c in token_str if c.isalnum())

    # Check numeric tokens
    if token_str.isdigit():
        return 'numbers'

    # Check each semantic group
    for group_name, group_set in SEMANTIC_GROUPS.items():
        if token_str in group_set:
            return group_name

    return 'other'

def build_token_to_group_map(tokenizer, vocab_size: int = 50257) -> Dict[int, str]:
    """
    Pre-compute token ID -> semantic group mapping.
    """
    token_map = {}
    for token_id in range(vocab_size):
        group = classify_token_semantic_group(token_id, tokenizer)
        token_map[token_id] = group

    return token_map

# Build mapping
print("Building semantic token map...")
token_to_group = build_token_to_group_map(tokenizer)

# Count tokens per group
group_counts = defaultdict(int)
for group in token_to_group.values():
    group_counts[group] += 1

print("\nSemantic Group Statistics:")
print("="*60)
for group, count in sorted(group_counts.items(), key=lambda x: -x[1]):
    if group != 'other':
        print(f"  {group:>15s}: {count:>4d} tokens")
print(f"  {'other':>15s}: {group_counts['other']:>4d} tokens")

print("\n✓ Token classification ready")

In [ ]:

#@title Experiment 1: Semantic Token Routing Trajectories Across Layers

import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
import seaborn as sns

# ============================
# Collect Routing for Semantic Groups
# ============================

def analyze_semantic_routing_trajectories(
    model, cfg, tokenizer, token_to_group, device, num_batches=30
):
    """
    Track routing patterns for different semantic groups across layers.
    Returns canonicalized routing distributions per group.
    """

    # Step 1: Establish canonical roles
    gen = make_batch_generator(cfg, split='val', device=device, batches_per_epoch=1, batch_size=32, infinite=False)
    xb_canon, _ = next(gen)
    canon_perm, _, valid_layers = compute_canonical_permutations(model, xb_canon, device)
    #canon_perm = xb_canon
    #valid_layers = list(range(cfg.num_heads))
    print(f"Analyzing semantic routing across {len(valid_layers)} layers...")

    H = cfg.num_heads
    K = cfg.num_slots

    # Track routing: group_routing[group][layer][canonical_head] = [K] distribution
    group_routing = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: np.zeros(K))))
    group_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

    gen = make_batch_generator(cfg, split='val', device=device,
                               batches_per_epoch=num_batches, infinite=False)

    for batch_idx, (xb, yb) in enumerate(gen):
        B, T = xb.shape

        with torch.no_grad():
            _, infos = model(xb, return_info=True, info_level="basic",
                           info_cfg={'store_read_weights': True})

        # For each valid layer
        for layer_idx in valid_layers:
            info = infos[layer_idx]
            if info is None:
                continue

            rw = info.get('read_weights', None)  # [B,H,T,K]
            if rw is None:
                continue

            perm = canon_perm[layer_idx]

            # For each token
            for b in range(B):
                for t in range(T):
                    token_id = xb[b, t].item()
                    group = token_to_group.get(token_id, 'other')

                    if group == 'other':
                        continue

                    # For each canonical head
                    for canon_h in range(H):
                        phys_h = perm[canon_h]
                        routing = rw[b, phys_h, t, :].cpu().numpy()  # [K]

                        group_routing[group][layer_idx][canon_h] += routing
                        group_counts[group][layer_idx][canon_h] += 1

        if (batch_idx + 1) % 10 == 0:
            print(f"  Processed {batch_idx + 1}/{num_batches} batches")

    # Normalize
    for group in group_routing:
        for layer_idx in group_routing[group]:
            for canon_h in group_routing[group][layer_idx]:
                count = group_counts[group][layer_idx][canon_h]
                if count > 0:
                    group_routing[group][layer_idx][canon_h] /= count

    return group_routing, group_counts, valid_layers, canon_perm

# Run analysis
semantic_routing, semantic_counts, valid_layers, canon_perm = \
    analyze_semantic_routing_trajectories(model, cfg, tokenizer, token_to_group, DEVICE, num_batches=30)

# ============================
# Visualization 1: Slot Preference Heatmaps
# ============================

def plot_semantic_slot_preferences(semantic_routing, valid_layers, groups_to_plot=None):
    """
    For each semantic group, show preferred slots across layers.
    """
    if groups_to_plot is None:
        groups_to_plot = [g for g in semantic_routing.keys() if g != 'other']

    n_groups = len(groups_to_plot)
    n_layers = len(valid_layers)
    K = list(list(semantic_routing.values())[0].values())[0][0].shape[0]

    fig, axes = plt.subplots(3, 3, figsize=(16, 12))
    axes = axes.flatten()

    for idx, group in enumerate(groups_to_plot[:9]):
        ax = axes[idx]

        # Build matrix: [layers, slots] = average routing weight across all heads
        matrix = np.zeros((n_layers, K))

        for li, layer_idx in enumerate(valid_layers):
            if layer_idx in semantic_routing[group]:
                # Average across canonical heads
                for canon_h in semantic_routing[group][layer_idx]:
                    matrix[li] += semantic_routing[group][layer_idx][canon_h]
                matrix[li] /= len(semantic_routing[group][layer_idx])

        im = ax.imshow(matrix.T, aspect='auto', cmap='YlOrRd', interpolation='nearest')
        ax.set_xlabel('Layer', fontsize=10)
        ax.set_ylabel('Slot', fontsize=10)
        ax.set_title(f'{group.upper()}', fontweight='bold', fontsize=11)
        ax.set_xticks(range(n_layers))
        ax.set_xticklabels([f'L{l}' for l in valid_layers], rotation=45)
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    # Hide unused subplots
    for idx in range(len(groups_to_plot), 9):
        axes[idx].axis('off')

    fig.suptitle('Semantic Group Slot Preferences Across Depth',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

plot_semantic_slot_preferences(semantic_routing, valid_layers)

# ============================
# Visualization 2: Trajectory Flow Diagrams
# ============================

def plot_semantic_trajectory_sankey(semantic_routing, valid_layers, group='colors', head_idx=0):
    """
    Sankey-style diagram showing how a semantic group flows through slots across layers.
    """
    n_layers = min(4, len(valid_layers))  # Show first 4 layers
    K = list(list(semantic_routing[group].values())[0].values())[0].shape[0]

    fig, ax = plt.subplots(figsize=(14, 8))

    # Get routing weights for this group, this canonical head
    weights = []
    for li in range(n_layers):
        layer_idx = valid_layers[li]
        if layer_idx in semantic_routing[group] and head_idx in semantic_routing[group][layer_idx]:
            w = semantic_routing[group][layer_idx][head_idx]
            weights.append(w)
        else:
            weights.append(np.ones(K) / K)

    # Draw flow
    layer_spacing = 3.0
    slot_spacing = 0.15

    colors_cmap = plt.cm.YlOrRd(np.linspace(0.3, 0.9, K))

    for li in range(n_layers - 1):
        x_from = li * layer_spacing
        x_to = (li + 1) * layer_spacing

        w_from = weights[li]
        w_to = weights[li + 1]

        # Draw flows from each slot to next layer
        for slot_from in range(K):
            y_from = slot_from * slot_spacing

            # This slot's weight distributes to next layer
            for slot_to in range(K):
                y_to = slot_to * slot_spacing

                # Flow width proportional to: w_from[slot_from] * w_to[slot_to]
                flow_weight = w_from[slot_from] * w_to[slot_to] * 10

                if flow_weight > 0.01:
                    ax.plot([x_from, x_to], [y_from, y_to],
                           alpha=flow_weight, linewidth=flow_weight*5,
                           color=colors_cmap[slot_from])

    # Draw slot nodes
    for li in range(n_layers):
        x = li * layer_spacing
        for slot in range(K):
            y = slot * slot_spacing
            size = weights[li][slot] * 300
            ax.scatter([x], [y], s=size, color=colors_cmap[slot],
                      edgecolors='black', linewidths=1, zorder=10)

    ax.set_xlim(-0.5, (n_layers-1) * layer_spacing + 0.5)
    ax.set_ylim(-0.5, K * slot_spacing)
    ax.set_xticks([i * layer_spacing for i in range(n_layers)])
    ax.set_xticklabels([f'Layer {valid_layers[i]}' for i in range(n_layers)])
    ax.set_ylabel('Slot ID', fontsize=11)
    ax.set_title(f'Routing Trajectory: {group.upper()} tokens (Canonical Head {head_idx})',
                 fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.2, axis='y')

    plt.tight_layout()
    plt.show()

for group in ['colors', 'numbers', 'animals', 'action_verbs']:
    if group in semantic_routing:
        plot_semantic_trajectory_sankey(semantic_routing, valid_layers, group=group, head_idx=0)

print("\n✓ Trajectory analysis complete")

In [ ]:

#@title Experiment 2: Second-Order Attention — Does token T affect routing of token T+1?

import numpy as np
import torch
import matplotlib.pyplot as plt
from collections import defaultdict

# ============================
# Analyze Context-Dependent Routing
# ============================

def analyze_second_order_effects(
    model, cfg, tokenizer, token_to_group, device, num_batches=30
):
    """
    For each token at position t, measure:
    1. Its own routing
    2. The routing of token at t+1
    3. Does the semantic group at t influence routing at t+1?
    """

    # Establish canonicalization
    gen = make_batch_generator(cfg, split='val', device=device, batches_per_epoch=1, infinite=False)
    xb_canon, _ = next(gen)
    canon_perm, _, valid_layers = compute_canonical_permutations(model, xb_canon, device)

    print("Analyzing second-order routing effects...")

    H = cfg.num_heads
    K = cfg.num_slots

    # Track: given group at t, what's the routing distribution at t+1?
    # second_order[group_t][group_t1][layer][canonical_head] = routing distribution at t+1
    second_order = defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(
                lambda: defaultdict(lambda: np.zeros(K))
            )
        )
    )
    second_order_counts = defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(
                lambda: defaultdict(int)
            )
        )
    )

    gen = make_batch_generator(cfg, split='val', device=device,
                               batches_per_epoch=num_batches, infinite=False)

    for batch_idx, (xb, yb) in enumerate(gen):
        B, T = xb.shape

        with torch.no_grad():
            _, infos = model(xb, return_info=True, info_level="basic",
                           info_cfg={'store_read_weights': True})

        # Focus on middle layers (where patterns are most interesting)
        target_layer = valid_layers[len(valid_layers) // 2]

        info = infos[target_layer]
        if info is None:
            continue

        rw = info.get('read_weights', None)  # [B,H,T,K]
        if rw is None:
            continue

        perm = canon_perm[target_layer]

        # For each sequence
        for b in range(B):
            # For each position (except last)
            for t in range(T - 1):
                token_t = xb[b, t].item()
                token_t1 = xb[b, t + 1].item()

                group_t = token_to_group.get(token_t, 'other')
                group_t1 = token_to_group.get(token_t1, 'other')

                if group_t == 'other' or group_t1 == 'other':
                    continue

                # For each canonical head, get routing at t+1
                for canon_h in range(H):
                    phys_h = perm[canon_h]
                    routing_t1 = rw[b, phys_h, t + 1, :].cpu().numpy()

                    second_order[group_t][group_t1][target_layer][canon_h] += routing_t1
                    second_order_counts[group_t][group_t1][target_layer][canon_h] += 1

        if (batch_idx + 1) % 10 == 0:
            print(f"  Processed {batch_idx + 1}/{num_batches} batches")

    # Normalize
    for group_t in second_order:
        for group_t1 in second_order[group_t]:
            for layer_idx in second_order[group_t][group_t1]:
                for canon_h in second_order[group_t][group_t1][layer_idx]:
                    count = second_order_counts[group_t][group_t1][layer_idx][canon_h]
                    if count > 0:
                        second_order[group_t][group_t1][layer_idx][canon_h] /= count

    return second_order, second_order_counts

second_order_routing, second_order_counts = \
    analyze_second_order_effects(model, cfg, tokenizer, token_to_group, DEVICE, num_batches=30)

# ============================
# Visualization: Context Effect Matrix
# ============================

def plot_second_order_matrix(second_order_routing, second_order_counts, layer_idx):
    """
    Matrix showing: given group at t (rows), routing change at t+1 for group at t+1 (cols).
    """
    # Get all groups that appear
    groups_t = list(second_order_routing.keys())
    groups_t1 = set()
    for g in groups_t:
        groups_t1.update(second_order_routing[g].keys())
    groups_t1 = sorted(list(groups_t1))

    if len(groups_t) == 0 or len(groups_t1) == 0:
        print("No second-order data available")
        return

    K = list(list(list(second_order_routing.values())[0].values())[0].values())[0][0].shape[0]

    # Compute "influence score" = KL divergence from baseline
    # Baseline = average routing for group_t1 regardless of context

    # First, get baseline routing for each group_t1
    baseline = {}
    for group_t1 in groups_t1:
        all_routing = []
        total_count = 0
        for group_t in groups_t:
            if group_t1 in second_order_routing[group_t]:
                if layer_idx in second_order_routing[group_t][group_t1]:
                    for canon_h in second_order_routing[group_t][group_t1][layer_idx]:
                        routing = second_order_routing[group_t][group_t1][layer_idx][canon_h]
                        count = second_order_counts[group_t][group_t1][layer_idx][canon_h]
                        all_routing.append(routing * count)
                        total_count += count

        if total_count > 0:
            baseline[group_t1] = np.sum(all_routing, axis=0) / total_count
        else:
            baseline[group_t1] = np.ones(K) / K

    # Build influence matrix
    influence_matrix = np.zeros((len(groups_t), len(groups_t1)))

    for i, group_t in enumerate(groups_t):
        for j, group_t1 in enumerate(groups_t1):
            if group_t1 in second_order_routing[group_t]:
                if layer_idx in second_order_routing[group_t][group_t1]:
                    # Average across heads
                    kls = []
                    for canon_h in second_order_routing[group_t][group_t1][layer_idx]:
                        p = second_order_routing[group_t][group_t1][layer_idx][canon_h]
                        q = baseline[group_t1]

                        # KL(p||q)
                        kl = np.sum(p * np.log((p + 1e-10) / (q + 1e-10)))
                        kls.append(kl)

                    if kls:
                        influence_matrix[i, j] = np.mean(kls)

    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))

    im = ax.imshow(influence_matrix, cmap='RdBu_r', aspect='auto',
                  vmin=-influence_matrix.max(), vmax=influence_matrix.max())

    ax.set_xticks(range(len(groups_t1)))
    ax.set_xticklabels(groups_t1, rotation=45, ha='right')
    ax.set_yticks(range(len(groups_t)))
    ax.set_yticklabels(groups_t)

    ax.set_xlabel('Token Group at t+1', fontsize=11, fontweight='bold')
    ax.set_ylabel('Token Group at t', fontsize=11, fontweight='bold')
    ax.set_title(f'Second-Order Context Effect (Layer {layer_idx})\nKL divergence from baseline',
                 fontsize=13, fontweight='bold')

    plt.colorbar(im, ax=ax, label='Influence (KL divergence)')

    # Add text annotations
    for i in range(len(groups_t)):
        for j in range(len(groups_t1)):
            val = influence_matrix[i, j]
            if abs(val) > 0.01:
                ax.text(j, i, f'{val:.2f}', ha='center', va='center',
                       fontsize=8, color='white' if abs(val) > influence_matrix.max()/2 else 'black')

    plt.tight_layout()
    plt.show()

    # Print strongest effects
    print(f"\nStrongest Second-Order Effects (Layer {layer_idx}):")
    print("="*70)

    effects = []
    for i, group_t in enumerate(groups_t):
        for j, group_t1 in enumerate(groups_t1):
            effects.append((influence_matrix[i, j], group_t, group_t1))

    effects.sort(key=lambda x: abs(x[0]), reverse=True)

    for influence, group_t, group_t1 in effects[:10]:
        direction = "increases" if influence > 0 else "decreases"
        print(f"  {group_t:>15s} → {group_t1:>15s}: {direction} divergence by {abs(influence):.3f}")

target_layer = valid_layers[len(valid_layers) // 2]
plot_second_order_matrix(second_order_routing, second_order_counts, target_layer)

In [ ]:

#@title Experiment 3: Semantic Priming — Does "red" prime routing for "apple"?

import numpy as np
import torch
import matplotlib.pyplot as plt
from collections import defaultdict

# ============================
# Semantic Pair Analysis
# ============================

SEMANTIC_PAIRS = {
    'color_object': [
        (['red', 'crimson', 'scarlet'], ['apple', 'rose', 'blood', 'fire']),
        (['blue', 'azure'], ['sky', 'ocean', 'water', 'sea']),
        (['green'], ['grass', 'tree', 'leaf', 'forest']),
        (['white'], ['snow', 'cloud', 'milk', 'paper']),
        (['yellow', 'gold'], ['sun', 'lemon', 'banana']),
    ],

    'number_object': [
        (['two', 'three', 'four'], ['people', 'person', 'men', 'women', 'children']),
        (['first', 'second', 'third'], ['place', 'position', 'time']),
    ],

    'action_object': [
        (['eat', 'eating'], ['food', 'meal', 'dinner', 'lunch', 'breakfast']),
        (['drink', 'drinking'], ['water', 'coffee', 'tea', 'beer', 'wine']),
        (['read', 'reading'], ['book', 'books', 'novel', 'newspaper', 'article']),
        (['write', 'writing'], ['letter', 'essay', 'story', 'document']),
    ],
}

def find_semantic_pairs_in_batch(xb, tokenizer, pair_type='color_object'):
    """
    Find instances where a prime token is followed (within 5 tokens) by a target.
    Returns: list of (batch_idx, prime_pos, target_pos, prime_word, target_word)
    """
    pairs = SEMANTIC_PAIRS[pair_type]
    found_pairs = []

    B, T = xb.shape

    for b in range(B):
        tokens = xb[b].cpu().numpy()
        token_strs = [tokenizer.decode([t]).strip().lower() for t in tokens]

        for primes, targets in pairs:
            for t in range(T - 1):
                token_t = token_strs[t]

                # Check if this is a prime
                if any(prime in token_t for prime in primes):
                    # Look ahead up to 5 positions
                    for offset in range(1, min(6, T - t)):
                        token_ahead = token_strs[t + offset]

                        # Check if this is a target
                        if any(target in token_ahead for target in targets):
                            found_pairs.append((b, t, t + offset, token_t, token_ahead))

    return found_pairs

def analyze_priming_effect(model, cfg, tokenizer, device, pair_type='color_object', num_batches=50):
    """
    Compare routing when:
    1. Target appears after prime (primed condition)
    2. Target appears without prime (unprimed condition)
    """

    # Establish canonicalization
    gen = make_batch_generator(cfg, split='val', device=device, batches_per_epoch=1, infinite=False)
    xb_canon, _ = next(gen)
    canon_perm, _, valid_layers = compute_canonical_permutations(model, xb_canon, device)

    H = cfg.num_heads
    K = cfg.num_slots
    target_layer = valid_layers[len(valid_layers) // 2]

    # Storage
    primed_routing = defaultdict(lambda: np.zeros(K))
    unprimed_routing = defaultdict(lambda: np.zeros(K))
    primed_counts = defaultdict(int)
    unprimed_counts = defaultdict(int)

    print(f"Analyzing {pair_type} priming effects...")

    gen = make_batch_generator(cfg, split='val', device=device,
                               batches_per_epoch=num_batches, infinite=False)

    for batch_idx, (xb, yb) in enumerate(gen):
        # Find semantic pairs in this batch
        pairs = find_semantic_pairs_in_batch(xb, tokenizer, pair_type=pair_type)

        if not pairs:
            continue

        # Run model
        with torch.no_grad():
            _, infos = model(xb, return_info=True, info_level="basic",
                           info_cfg={'store_read_weights': True})

        info = infos[target_layer]
        if info is None:
            continue

        rw = info.get('read_weights', None)
        if rw is None:
            continue

        perm = canon_perm[target_layer]

        # For each found pair
        for b, prime_pos, target_pos, prime_word, target_word in pairs:
            # Get routing at target position (primed)
            for canon_h in range(H):
                phys_h = perm[canon_h]
                routing = rw[b, phys_h, target_pos, :].cpu().numpy()

                primed_routing[canon_h] += routing
                primed_counts[canon_h] += 1

        # Also collect unprimed targets (targets without recent primes)
        B, T = xb.shape
        tokens = xb.cpu().numpy()
        token_strs = [[tokenizer.decode([t]).strip().lower() for t in tokens[b]] for b in range(B)]

        pairs_dict = SEMANTIC_PAIRS[pair_type]
        all_targets = set()
        for _, targets in pairs_dict:
            all_targets.update(targets)

        for b in range(B):
            for t in range(T):
                token_t = token_strs[b][t]

                # Is this a target?
                is_target = any(target in token_t for target in all_targets)

                if is_target:
                    # Check if there's a prime in the last 5 positions
                    has_prime = False
                    for lookback in range(1, min(6, t + 1)):
                        token_prev = token_strs[b][t - lookback]
                        for primes, _ in pairs_dict:
                            if any(prime in token_prev for prime in primes):
                                has_prime = True
                                break
                        if has_prime:
                            break

                    if not has_prime:
                        # Unprimed target
                        for canon_h in range(H):
                            phys_h = perm[canon_h]
                            routing = rw[b, phys_h, t, :].cpu().numpy()

                            unprimed_routing[canon_h] += routing
                            unprimed_counts[canon_h] += 1

        if (batch_idx + 1) % 10 == 0:
            print(f"  Batch {batch_idx + 1}/{num_batches}: Found {len(pairs)} primed pairs")

    # Normalize
    for canon_h in range(H):
        if primed_counts[canon_h] > 0:
            primed_routing[canon_h] /= primed_counts[canon_h]
        if unprimed_counts[canon_h] > 0:
            unprimed_routing[canon_h] /= unprimed_counts[canon_h]

    return primed_routing, unprimed_routing, primed_counts, unprimed_counts

# Run analysis
for pair_type in ['color_object', 'action_object']:
    primed, unprimed, primed_cnt, unprimed_cnt = \
        analyze_priming_effect(model, cfg, tokenizer, DEVICE, pair_type=pair_type, num_batches=50)

    print(f"\n{pair_type.upper()} Priming Results:")
    print("="*70)
    print(f"Primed instances: {sum(primed_cnt.values())}")
    print(f"Unprimed instances: {sum(unprimed_cnt.values())}")

    # Plot difference
    H = cfg.num_heads
    K = cfg.num_slots

    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()

    for canon_h in range(H):
        ax = axes[canon_h]

        if primed_cnt[canon_h] > 0 and unprimed_cnt[canon_h] > 0:
            diff = primed[canon_h] - unprimed[canon_h]

            ax.bar(range(K), diff, color=['red' if d > 0 else 'blue' for d in diff])
            ax.axhline(0, color='black', linewidth=0.8)
            ax.set_xlabel('Slot', fontsize=9)
            ax.set_ylabel('Δ Routing (primed - unprimed)', fontsize=9)
            ax.set_title(f'Canonical Head {canon_h}', fontweight='bold')
            ax.grid(True, alpha=0.3, axis='y')

    fig.suptitle(f'Priming Effect: {pair_type.replace("_", " ").title()}',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

In [ ]:

#@title Experiment 4: Semantic Clustering in Canonical Slot State Space (t-SNE)

import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import seaborn as sns
from collections import defaultdict

# ============================
# Collect Slot States by Semantic Group
# ============================

def collect_canonical_slot_states(
    model, cfg, tokenizer, token_to_group, device,
    num_batches=20, layer_idx=6, max_samples_per_group=500
):
    """
    Collect actual slot state vectors for tokens, grouped by semantics.
    Uses canonicalization so we track SEMANTIC ROLES, not physical heads.

    Returns:
        states_by_group: dict[group][canonical_head] = [N_samples, d] numpy array
        metadata: dict with token info
    """

    # Step 1: Establish canonicalization
    gen = make_batch_generator(cfg, split='val', device=device, batches_per_epoch=1, infinite=False)
    xb_canon, _ = next(gen)
    canon_perm, _, valid_layers = compute_canonical_permutations(model, xb_canon, device)

    if layer_idx not in valid_layers:
        layer_idx = valid_layers[len(valid_layers) // 2]
        print(f"Using layer {layer_idx} instead")

    perm = canon_perm[layer_idx]
    H = cfg.num_heads
    d = cfg.embed_dim // cfg.num_heads
    K = cfg.num_slots

    print(f"Collecting canonical slot states from Layer {layer_idx}...")
    print(f"Head dim: {d}, Num slots: {K}")

    # We need to hook into the model to extract actual slot states
    # For now, we'll use a simpler approach: collect read weights weighted slot combinations
    # This approximates the "active slot state" for each token

    states_by_group = defaultdict(lambda: defaultdict(list))
    token_info = defaultdict(lambda: defaultdict(list))
    group_sample_counts = defaultdict(int)

    gen = make_batch_generator(cfg, split='val', device=device,
                               batches_per_epoch=num_batches, infinite=False)

    for batch_idx, (xb, yb) in enumerate(gen):
        B, T = xb.shape

        with torch.no_grad():
            # We need to extract actual slot states during forward pass
            # For this, we'll modify the forward to store intermediate states
            # Simplified: use read weights as proxy for now

            _, infos = model(xb, return_info=True, info_level="basic",
                           info_cfg={'store_read_weights': True})

        info = infos[layer_idx]
        if info is None:
            continue

        rw = info.get('read_weights', None)  # [B,H,T,K]
        if rw is None:
            continue

        # For each token
        for b in range(B):
            for t in range(T):
                token_id = xb[b, t].item()
                group = token_to_group.get(token_id, 'other')

                if group == 'other':
                    continue

                # Check if we have enough samples
                if group_sample_counts[group] >= max_samples_per_group:
                    continue

                # For each CANONICAL head
                for canon_h in range(H):
                    phys_h = perm[canon_h]

                    # Get routing weights for this token at this head
                    routing = rw[b, phys_h, t, :].cpu().numpy()  # [K]

                    # Store as "slot state approximation"
                    # In reality, we'd want the actual V @ slot_state vectors
                    # For now, use routing weights as a K-dimensional representation
                    states_by_group[group][canon_h].append(routing)
                    token_info[group][canon_h].append({
                        'token_id': token_id,
                        'token_str': tokenizer.decode([token_id]),
                        'position': t,
                        'batch': batch_idx
                    })

                group_sample_counts[group] += 1

        if (batch_idx + 1) % 5 == 0:
            print(f"  Batch {batch_idx + 1}/{num_batches} | Samples: {dict(group_sample_counts)}")

    # Convert to numpy arrays
    for group in states_by_group:
        for canon_h in states_by_group[group]:
            states_by_group[group][canon_h] = np.array(states_by_group[group][canon_h])

    print("\nCollection complete:")
    for group in sorted(states_by_group.keys()):
        total = sum(len(states_by_group[group][h]) for h in states_by_group[group])
        print(f"  {group:>15s}: {total:>5d} samples across {len(states_by_group[group])} heads")

    return states_by_group, token_info, layer_idx, perm

# Collect states
states_by_group, token_info, layer_idx, canon_perm = collect_canonical_slot_states(
    model, cfg, tokenizer, token_to_group, DEVICE,
    num_batches=30, layer_idx=6, max_samples_per_group=400
)

# ============================
# t-SNE Analysis by Canonical Head
# ============================

def tsne_semantic_clustering_per_head(states_by_group, layer_idx, canon_head_idx):
    """
    For a specific CANONICAL head, perform t-SNE on all semantic groups.
    Color by semantic group to see if they cluster.
    """

    # Collect all states for this canonical head
    all_states = []
    all_groups = []
    all_tokens = []

    semantic_groups = [g for g in states_by_group.keys() if g != 'other']

    for group in semantic_groups:
        if canon_head_idx in states_by_group[group]:
            states = states_by_group[group][canon_head_idx]
            all_states.append(states)
            all_groups.extend([group] * len(states))

    if len(all_states) == 0:
        print(f"No data for canonical head {canon_head_idx}")
        return None, None, None

    all_states = np.vstack(all_states)

    print(f"\nCanonical Head {canon_head_idx}:")
    print(f"  Total samples: {len(all_states)}")
    print(f"  Feature dim: {all_states.shape[1]}")

    # PCA first (for speed and denoising)
    print("  Running PCA...")
    n_components_pca = min(20, all_states.shape[1] - 1)
    pca = PCA(n_components=n_components_pca)
    states_pca = pca.fit_transform(all_states)
    print(f"  PCA variance explained: {pca.explained_variance_ratio_.sum():.2%}")

    # t-SNE
    print("  Running t-SNE...")
    n_samples = len(states_pca)
    perplexity = min(30, n_samples // 4)

    tsne = TSNE(
        n_components=2,
        perplexity=perplexity,
        n_iter=1000,
        random_state=42,
        verbose=0
    )
    coords = tsne.fit_transform(states_pca)

    return coords, all_groups, states_pca

def plot_tsne_semantic_clusters(coords, groups, layer_idx, canon_head_idx):
    """
    Plot t-SNE with points colored by semantic group.
    """

    # Color map
    unique_groups = sorted(set(groups))
    color_palette = sns.color_palette("husl", len(unique_groups))
    group_to_color = {g: color_palette[i] for i, g in enumerate(unique_groups)}

    fig, ax = plt.subplots(figsize=(12, 10))

    # Plot each group
    for group in unique_groups:
        mask = np.array([g == group for g in groups])
        ax.scatter(
            coords[mask, 0],
            coords[mask, 1],
            c=[group_to_color[group]],
            label=group,
            alpha=0.6,
            s=30,
            edgecolors='none'
        )

    ax.set_xlabel('t-SNE 1', fontsize=12)
    ax.set_ylabel('t-SNE 2', fontsize=12)
    ax.set_title(
        f'Semantic Clustering in Slot State Space\n'
        f'Layer {layer_idx}, Canonical Head {canon_head_idx}',
        fontsize=14, fontweight='bold'
    )
    ax.legend(title='Semantic Group', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.2)

    plt.tight_layout()
    plt.show()

# Run t-SNE for multiple canonical heads
for canon_h in range(min(4, cfg.num_heads)):
    coords, groups, states_pca = tsne_semantic_clustering_per_head(
        states_by_group, layer_idx, canon_h
    )

    if coords is not None:
        plot_tsne_semantic_clusters(coords, groups, layer_idx, canon_h)

# ============================
# Quantitative Clustering Analysis
# ============================

def measure_semantic_clustering_quality(coords, groups):
    """
    Measure how well semantic groups cluster using silhouette score.
    """
    from sklearn.metrics import silhouette_score
    from scipy.spatial.distance import pdist, squareform

    # Encode groups as integers
    unique_groups = sorted(set(groups))
    group_to_int = {g: i for i, g in enumerate(unique_groups)}
    labels = np.array([group_to_int[g] for g in groups])

    # Silhouette score (-1 to 1, higher is better)
    if len(unique_groups) > 1:
        sil_score = silhouette_score(coords, labels)
    else:
        sil_score = 0.0

    # Compute within-group vs between-group distances
    distances = squareform(pdist(coords))

    within_group_dists = []
    between_group_dists = []

    for i in range(len(groups)):
        for j in range(i + 1, len(groups)):
            dist = distances[i, j]

            if groups[i] == groups[j]:
                within_group_dists.append(dist)
            else:
                between_group_dists.append(dist)

    within_mean = np.mean(within_group_dists) if within_group_dists else 0
    between_mean = np.mean(between_group_dists) if between_group_dists else 0

    separation_ratio = between_mean / (within_mean + 1e-9)

    return sil_score, separation_ratio, within_mean, between_mean

print("\nClustering Quality by Canonical Head:")
print("="*70)
print(f"{'Head':<8} {'Silhouette':<12} {'Sep Ratio':<12} {'Within':<10} {'Between':<10}")
print("-"*70)

for canon_h in range(min(cfg.num_heads, 8)):
    coords, groups, _ = tsne_semantic_clustering_per_head(states_by_group, layer_idx, canon_h)

    if coords is not None:
        sil, sep_ratio, within, between = measure_semantic_clustering_quality(coords, groups)
        print(f"{canon_h:<8} {sil:>11.3f} {sep_ratio:>11.2f} {within:>9.3f} {between:>9.3f}")

print("\nInterpretation:")
print("  • Silhouette > 0.3: Good clustering by semantic group")
print("  • Sep Ratio > 1.5: Between-group distances exceed within-group")
print("  • Higher values = stronger semantic organization in slot space")

In [ ]:

#@title Experiment 5: Long-Range Context Influence on Canonical Routing

import numpy as np
import torch
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import pearsonr

# ============================
# Context Window Analysis
# ============================

def analyze_long_range_context_influence(
    model, cfg, tokenizer, token_to_group, device,
    num_batches=30, max_lookback=32
):
    """
    For each token at position t, measure how its routing is influenced by
    tokens at positions t-k for k=1,2,...,max_lookback.

    Uses canonical roles to ensure meaningful cross-layer comparison.
    """

    # Establish canonicalization
    gen = make_batch_generator(cfg, split='val', device=device, batches_per_epoch=1, infinite=False)
    xb_canon, _ = next(gen)
    canon_perm, _, valid_layers = compute_canonical_permutations(model, xb_canon, device)

    target_layer = valid_layers[len(valid_layers) // 2]
    print(f"Analyzing long-range dependencies at Layer {target_layer}...")

    H = cfg.num_heads
    K = cfg.num_slots
    perm = canon_perm[target_layer]

    # For each semantic group pair and distance
    # influence[group_context][group_target][distance][canon_h] = correlation strength
    influence = defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(
                lambda: defaultdict(list)
            )
        )
    )

    gen = make_batch_generator(cfg, split='val', device=device,
                               batches_per_epoch=num_batches, infinite=False)

    print(f"Processing {num_batches} batches...")

    for batch_idx, (xb, yb) in enumerate(gen):
        B, T = xb.shape

        with torch.no_grad():
            _, infos = model(xb, return_info=True, info_level="basic",
                           info_cfg={'store_read_weights': True})

        info = infos[target_layer]
        if info is None:
            continue

        rw = info.get('read_weights', None)  # [B,H,T,K]
        if rw is None:
            continue

        # For each sequence
        for b in range(B):
            # For each target position
            for t in range(max_lookback, T):
                token_t = xb[b, t].item()
                group_t = token_to_group.get(token_t, 'other')

                if group_t == 'other':
                    continue

                # Get routing at position t
                routing_t = {}
                for canon_h in range(H):
                    phys_h = perm[canon_h]
                    routing_t[canon_h] = rw[b, phys_h, t, :].cpu().numpy()

                # Look back at different distances
                for distance in range(1, min(max_lookback + 1, t + 1)):
                    pos_context = t - distance
                    token_context = xb[b, pos_context].item()
                    group_context = token_to_group.get(token_context, 'other')

                    if group_context == 'other':
                        continue

                    # Get routing at context position
                    routing_context = {}
                    for canon_h in range(H):
                        phys_h = perm[canon_h]
                        routing_context[canon_h] = rw[b, phys_h, pos_context, :].cpu().numpy()

                    # Compute correlation between context and target routing
                    for canon_h in range(H):
                        # Pearson correlation of slot distributions
                        if routing_t[canon_h].std() > 1e-6 and routing_context[canon_h].std() > 1e-6:
                            corr, _ = pearsonr(routing_t[canon_h], routing_context[canon_h])
                            influence[group_context][group_t][distance][canon_h].append(corr)

        if (batch_idx + 1) % 10 == 0:
            print(f"  Batch {batch_idx + 1}/{num_batches}")

    # Average correlations
    avg_influence = defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(
                lambda: defaultdict(float)
            )
        )
    )

    for group_ctx in influence:
        for group_tgt in influence[group_ctx]:
            for distance in influence[group_ctx][group_tgt]:
                for canon_h in influence[group_ctx][group_tgt][distance]:
                    corrs = influence[group_ctx][group_tgt][distance][canon_h]
                    if corrs:
                        avg_influence[group_ctx][group_tgt][distance][canon_h] = np.mean(corrs)

    return avg_influence, target_layer

influence_data, target_layer = analyze_long_range_context_influence(
    model, cfg, tokenizer, token_to_group, DEVICE,
    num_batches=30, max_lookback=32
)

# ============================
# Visualization 1: Influence Decay Curves
# ============================

def plot_influence_decay_curves(influence_data, target_layer, group_pairs=None):
    """
    Plot how context influence decays with distance for different semantic pairs.
    """

    if group_pairs is None:
        # Select interesting pairs
        group_pairs = [
            ('ownership', 'ownership'),  # Same group
            ('colors', 'places'),         # Different groups
            ('action_verbs', 'places'),
            ('numbers', 'quantifiers'),
        ]

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()

    for idx, (group_ctx, group_tgt) in enumerate(group_pairs):
        if idx >= 4:
            break

        ax = axes[idx]

        if group_ctx not in influence_data or group_tgt not in influence_data[group_ctx]:
            ax.text(0.5, 0.5, 'No data', ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{group_ctx} → {group_tgt}', fontweight='bold')
            continue

        # Get data for this pair
        distances = sorted(influence_data[group_ctx][group_tgt].keys())

        # Plot curve for each canonical head (average across heads for clarity)
        H = cfg.num_heads

        # Average across all heads
        avg_by_distance = []
        for dist in distances:
            head_values = []
            for canon_h in range(H):
                val = influence_data[group_ctx][group_tgt][dist].get(canon_h, 0)
                if val != 0:
                    head_values.append(val)
            avg_by_distance.append(np.mean(head_values) if head_values else 0)

        ax.plot(distances, avg_by_distance, marker='o', linewidth=2, markersize=6, label='Mean')

        # Also plot top 2 heads separately
        for canon_h in [0, 1]:  # Just show first two canonical heads
            values = [influence_data[group_ctx][group_tgt][dist].get(canon_h, 0) for dist in distances]
            ax.plot(distances, values, marker='s', linewidth=1, markersize=4,
                   alpha=0.5, label=f'Canon H{canon_h}')

        ax.axhline(0, color='black', linewidth=0.8, linestyle='--', alpha=0.3)
        ax.set_xlabel('Distance (tokens)', fontsize=11)
        ax.set_ylabel('Routing Correlation', fontsize=11)
        ax.set_title(f'{group_ctx} → {group_tgt}', fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=8)

        # Fit exponential decay
        from scipy.optimize import curve_fit

        def exp_decay(x, a, b):
            return a * np.exp(-b * x)

        try:
            x_data = np.array(distances)
            y_data = np.array(avg_by_distance)

            # Only fit on positive values
            mask = y_data > 0
            if mask.sum() > 3:
                popt, _ = curve_fit(exp_decay, x_data[mask], y_data[mask], p0=[0.5, 0.1])

                x_fit = np.linspace(min(distances), max(distances), 100)
                y_fit = exp_decay(x_fit, *popt)
                ax.plot(x_fit, y_fit, '--', color='red', alpha=0.5, linewidth=1.5,
                       label=f'Exp fit: λ={popt[1]:.3f}')
                ax.legend(fontsize=8)
        except:
            pass

    fig.suptitle(f'Context Influence Decay (Layer {target_layer})',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

plot_influence_decay_curves(influence_data, target_layer)

# ============================
# Visualization 2: Context Memory Heatmap
# ============================

def plot_context_memory_heatmap(influence_data, target_layer, max_distance=16):
    """
    Heatmap showing which semantic group pairs have longest-range dependencies.
    """

    # Get all semantic groups
    groups_ctx = sorted(influence_data.keys())
    groups_tgt_all = set()
    for gc in groups_ctx:
        groups_tgt_all.update(influence_data[gc].keys())
    groups_tgt = sorted(groups_tgt_all)

    # Compute "memory strength" = average correlation at distance 8-16
    memory_matrix = np.zeros((len(groups_ctx), len(groups_tgt)))

    for i, group_ctx in enumerate(groups_ctx):
        for j, group_tgt in enumerate(groups_tgt):
            if group_tgt in influence_data[group_ctx]:
                # Average over long distances
                long_range_corrs = []
                for dist in range(8, min(max_distance + 1, 33)):
                    if dist in influence_data[group_ctx][group_tgt]:
                        for canon_h in influence_data[group_ctx][group_tgt][dist]:
                            val = influence_data[group_ctx][group_tgt][dist][canon_h]
                            if val != 0:
                                long_range_corrs.append(val)

                if long_range_corrs:
                    memory_matrix[i, j] = np.mean(long_range_corrs)

    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))

    im = ax.imshow(memory_matrix, cmap='RdYlGn', aspect='auto', vmin=-0.2, vmax=0.5)

    ax.set_xticks(range(len(groups_tgt)))
    ax.set_xticklabels(groups_tgt, rotation=45, ha='right')
    ax.set_yticks(range(len(groups_ctx)))
    ax.set_yticklabels(groups_ctx)

    ax.set_xlabel('Target Token Group (position t)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Context Token Group (position t-k)', fontsize=12, fontweight='bold')
    ax.set_title(
        f'Long-Range Context Memory Strength (Layer {target_layer})\n'
        f'Average routing correlation at distances 8-16 tokens',
        fontsize=13, fontweight='bold'
    )

    plt.colorbar(im, ax=ax, label='Correlation')

    # Annotate strong effects
    for i in range(len(groups_ctx)):
        for j in range(len(groups_tgt)):
            val = memory_matrix[i, j]
            if abs(val) > 0.2:
                ax.text(j, i, f'{val:.2f}', ha='center', va='center',
                       fontsize=8, color='white' if abs(val) > 0.35 else 'black')

    plt.tight_layout()
    plt.show()

    # Print strongest long-range effects
    print(f"\nStrongest Long-Range Dependencies (distance 8-16):")
    print("="*70)

    effects = []
    for i, group_ctx in enumerate(groups_ctx):
        for j, group_tgt in enumerate(groups_tgt):
            effects.append((memory_matrix[i, j], group_ctx, group_tgt))

    effects.sort(key=lambda x: abs(x[0]), reverse=True)

    for corr, group_ctx, group_tgt in effects[:10]:
        if abs(corr) > 0.1:
            print(f"  {group_ctx:>15s} → {group_tgt:>15s}: r = {corr:+.3f}")

plot_context_memory_heatmap(influence_data, target_layer, max_distance=16)

# ============================
# Visualization 3: Per-Head Context Sensitivity
# ============================

def plot_head_context_sensitivity(influence_data, target_layer):
    """
    Which canonical heads are most sensitive to long-range context?
    """
    H = cfg.num_heads

    # For each canonical head, compute average long-range correlation
    head_sensitivity = np.zeros(H)
    head_counts = np.zeros(H)

    for group_ctx in influence_data:
        for group_tgt in influence_data[group_ctx]:
            for dist in influence_data[group_ctx][group_tgt]:
                if dist >= 8:  # Long-range
                    for canon_h in influence_data[group_ctx][group_tgt][dist]:
                        val = influence_data[group_ctx][group_tgt][dist][canon_h]
                        if val != 0:
                            head_sensitivity[canon_h] += abs(val)
                            head_counts[canon_h] += 1

    # Average
    for h in range(H):
        if head_counts[h] > 0:
            head_sensitivity[h] /= head_counts[h]

    # Plot
    fig, ax = plt.subplots(figsize=(10, 6))

    colors = ['red' if s > head_sensitivity.mean() else 'blue' for s in head_sensitivity]
    bars = ax.bar(range(H), head_sensitivity, color=colors, alpha=0.7)

    ax.axhline(head_sensitivity.mean(), color='black', linestyle='--',
              linewidth=1.5, label=f'Mean: {head_sensitivity.mean():.3f}')

    ax.set_xlabel('Canonical Head', fontsize=12, fontweight='bold')
    ax.set_ylabel('Average Long-Range Correlation (|r|)', fontsize=12, fontweight='bold')
    ax.set_title(
        f'Canonical Head Sensitivity to Long-Range Context (Layer {target_layer})',
        fontsize=13, fontweight='bold'
    )
    ax.set_xticks(range(H))
    ax.set_xticklabels([f'H{i}' for i in range(H)])
    ax.grid(True, alpha=0.3, axis='y')
    ax.legend()

    plt.tight_layout()
    plt.show()

    print(f"\nCanonical Head Context Sensitivity Rankings:")
    print("="*60)

    ranked = sorted(enumerate(head_sensitivity), key=lambda x: -x[1])
    for rank, (h, sens) in enumerate(ranked, 1):
        print(f"  {rank:>2d}. Canonical Head {h}: {sens:.4f}")

plot_head_context_sensitivity(influence_data, target_layer)

In [ ]:

#@title Experiment 6: How Long Do Semantic Groups "Persist" in Slot Memory?

import numpy as np
import torch
import matplotlib.pyplot as plt
from collections import defaultdict

# ============================
# Persistence Measurement
# ============================

def measure_semantic_persistence(
    model, cfg, tokenizer, token_to_group, device, num_batches=30
):
    """
    For each semantic group, measure how long its "signature" persists
    in the routing patterns after the token has passed.

    Method: At position t where group G appears, measure how long
    subsequent positions show routing similar to G's characteristic pattern.
    """

    # Establish canonicalization
    gen = make_batch_generator(cfg, split='val', device=device, batches_per_epoch=1, infinite=False)
    xb_canon, _ = next(gen)
    canon_perm, _, valid_layers = compute_canonical_permutations(model, xb_canon, device)

    target_layer = valid_layers[len(valid_layers) // 2]
    print(f"Measuring semantic persistence at Layer {target_layer}...")

    H = cfg.num_heads
    K = cfg.num_slots
    perm = canon_perm[target_layer]

    # First pass: build "signature" routing pattern for each group
    group_signatures = defaultdict(lambda: defaultdict(lambda: np.zeros(K)))
    group_sig_counts = defaultdict(lambda: defaultdict(int))

    gen = make_batch_generator(cfg, split='val', device=device,
                               batches_per_epoch=15, infinite=False)

    print("Building semantic signatures...")
    for batch_idx, (xb, yb) in enumerate(gen):
        B, T = xb.shape

        with torch.no_grad():
            _, infos = model(xb, return_info=True, info_level="basic",
                           info_cfg={'store_read_weights': True})

        info = infos[target_layer]
        if info is None:
            continue

        rw = info.get('read_weights', None)
        if rw is None:
            continue

        for b in range(B):
            for t in range(T):
                token_id = xb[b, t].item()
                group = token_to_group.get(token_id, 'other')

                if group == 'other':
                    continue

                # Record routing for this group
                for canon_h in range(H):
                    phys_h = perm[canon_h]
                    routing = rw[b, phys_h, t, :].cpu().numpy()

                    group_signatures[group][canon_h] += routing
                    group_sig_counts[group][canon_h] += 1

    # Normalize signatures
    for group in group_signatures:
        for canon_h in group_signatures[group]:
            count = group_sig_counts[group][canon_h]
            if count > 0:
                group_signatures[group][canon_h] /= count

    print(f"Signatures built for {len(group_signatures)} groups")

    # Second pass: measure persistence
    # persistence[group][lookahead] = list of similarity scores
    persistence = defaultdict(lambda: defaultdict(list))
    max_lookahead = 20

    gen = make_batch_generator(cfg, split='val', device=device,
                               batches_per_epoch=num_batches, infinite=False)

    print("Measuring persistence...")
    for batch_idx, (xb, yb) in enumerate(gen):
        B, T = xb.shape

        with torch.no_grad():
            _, infos = model(xb, return_info=True, info_level="basic",
                           info_cfg={'store_read_weights': True})

        info = infos[target_layer]
        if info is None:
            continue

        rw = info.get('read_weights', None)
        if rw is None:
            continue

        for b in range(B):
            for t in range(T - max_lookahead):
                token_id = xb[b, t].item()
                group = token_to_group.get(token_id, 'other')

                if group == 'other' or group not in group_signatures:
                    continue

                # Measure similarity at future positions
                for lookahead in range(1, max_lookahead + 1):
                    future_pos = t + lookahead
                    if future_pos >= T:
                        break

                    # Compute cosine similarity between future routing and group signature
                    similarities = []
                    for canon_h in range(H):
                        phys_h = perm[canon_h]
                        future_routing = rw[b, phys_h, future_pos, :].cpu().numpy()
                        signature = group_signatures[group][canon_h]

                        # Cosine similarity
                        dot = np.dot(future_routing, signature)
                        norm_future = np.linalg.norm(future_routing)
                        norm_sig = np.linalg.norm(signature)

                        if norm_future > 1e-9 and norm_sig > 1e-9:
                            sim = dot / (norm_future * norm_sig)
                            similarities.append(sim)

                    if similarities:
                        persistence[group][lookahead].append(np.mean(similarities))

        if (batch_idx + 1) % 10 == 0:
            print(f"  Batch {batch_idx + 1}/{num_batches}")

    # Average persistence curves
    persistence_curves = {}
    for group in persistence:
        curve = []
        for lookahead in range(1, max_lookahead + 1):
            if lookahead in persistence[group]:
                curve.append(np.mean(persistence[group][lookahead]))
            else:
                curve.append(0)
        persistence_curves[group] = curve

    return persistence_curves, max_lookahead, target_layer

persistence_curves, max_lookahead, target_layer = measure_semantic_persistence(
    model, cfg, tokenizer, token_to_group, DEVICE, num_batches=30
)

# ============================
# Visualization
# ============================

def plot_persistence_curves(persistence_curves, max_lookahead, target_layer):
    """
    Plot persistence decay curves for each semantic group.
    """

    fig, ax = plt.subplots(figsize=(12, 7))

    lookaheads = range(1, max_lookahead + 1)

    # Color palette
    groups = sorted(persistence_curves.keys())
    colors = sns.color_palette("husl", len(groups))

    for group, color in zip(groups, colors):
        curve = persistence_curves[group]
        ax.plot(lookaheads, curve, marker='o', label=group,
               linewidth=2, markersize=4, color=color)

    ax.axhline(0, color='black', linewidth=0.8, linestyle='--', alpha=0.3)
    ax.set_xlabel('Lookahead (tokens)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Similarity to Signature', fontsize=12, fontweight='bold')
    ax.set_title(
        f'Semantic Persistence in Routing Patterns (Layer {target_layer})\n'
        f'How long does a semantic group\'s "signature" persist?',
        fontsize=13, fontweight='bold'
    )
    ax.legend(title='Semantic Group', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Compute half-life for each group
    print(f"\nSemantic Persistence Half-Lives:")
    print("="*60)

    for group in sorted(persistence_curves.keys()):
        curve = np.array(persistence_curves[group])

        # Find half-life (position where similarity drops to 50% of initial)
        if curve[0] > 0:
            target = curve[0] * 0.5
            half_life_idx = np.where(curve < target)[0]

            if len(half_life_idx) > 0:
                half_life = half_life_idx[0] + 1
            else:
                half_life = max_lookahead
        else:
            half_life = 0

        print(f"  {group:>15s}: {half_life:>3d} tokens")

    print("\nInterpretation:")
    print("  • Longer half-life = semantic group maintains influence longer")
    print("  • Ownership/quantifiers expected to have long persistence")
    print("  • Colors/numbers might have shorter, local influence")

plot_persistence_curves(persistence_curves, max_lookahead, target_layer)

In [ ]:

#@title Refined Experiment: What DOES Cluster in Slot Space?

import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import seaborn as sns
from collections import defaultdict

# ============================
# Alternative Grouping Schemes
# ============================

def classify_token_by_frequency(token_id, tokenizer, freq_map):
    """
    Group tokens by frequency (common vs rare).
    """
    # This is a placeholder - in practice you'd use actual corpus frequencies
    # For now, use token_id as proxy (lower IDs are often more frequent in BPE)
    if token_id < 1000:
        return 'very_common'
    elif token_id < 5000:
        return 'common'
    elif token_id < 20000:
        return 'uncommon'
    else:
        return 'rare'

def classify_token_by_length(token_id, tokenizer):
    """
    Group by subword length.
    """
    token_str = tokenizer.decode([token_id])
    length = len(token_str.strip())

    if length == 1:
        return 'single_char'
    elif length <= 3:
        return 'short'
    elif length <= 6:
        return 'medium'
    else:
        return 'long'

def classify_token_by_position(position, total_length):
    """
    Group by position in sequence.
    """
    relative_pos = position / total_length

    if relative_pos < 0.2:
        return 'early'
    elif relative_pos < 0.4:
        return 'early_mid'
    elif relative_pos < 0.6:
        return 'middle'
    elif relative_pos < 0.8:
        return 'late_mid'
    else:
        return 'late'

def classify_token_syntactic(token_id, tokenizer):
    """
    Rough syntactic classification.
    """
    token_str = tokenizer.decode([token_id]).strip().lower()

    # Function words
    function = {'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been',
                'and', 'or', 'but', 'if', 'when', 'where', 'who', 'what',
                'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'}

    # Pronouns
    pronouns = {'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her',
                'us', 'them', 'my', 'your', 'his', 'its', 'our', 'their'}

    # Modals
    modals = {'can', 'could', 'may', 'might', 'must', 'shall', 'should', 'will', 'would'}

    if token_str in function:
        return 'function'
    elif token_str in pronouns:
        return 'pronoun'
    elif token_str in modals:
        return 'modal'
    elif token_str.endswith(('ing', 'ed', 'es', 's')):
        return 'inflected'
    else:
        return 'content'

# ============================
# Re-collect with Multiple Grouping Schemes
# ============================

def collect_states_multiple_schemes(
    model, cfg, tokenizer, device, num_batches=20, layer_idx=6, max_per_group=300
):
    """
    Collect routing states grouped by MULTIPLE classification schemes.
    """

    # Establish canonicalization
    gen = make_batch_generator(cfg, split='val', device=device, batches_per_epoch=1, infinite=False)
    xb_canon, _ = next(gen)
    canon_perm, _, valid_layers = compute_canonical_permutations(model, xb_canon, device)

    if layer_idx not in valid_layers:
        layer_idx = valid_layers[len(valid_layers) // 2]

    perm = canon_perm[layer_idx]
    H = cfg.num_heads
    K = cfg.num_slots

    print(f"Collecting with multiple classification schemes (Layer {layer_idx})...")

    # Storage for different schemes
    schemes = {
        'frequency': defaultdict(lambda: defaultdict(list)),
        'length': defaultdict(lambda: defaultdict(list)),
        'position': defaultdict(lambda: defaultdict(list)),
        'syntactic': defaultdict(lambda: defaultdict(list)),
    }

    counts = {
        'frequency': defaultdict(int),
        'length': defaultdict(int),
        'position': defaultdict(int),
        'syntactic': defaultdict(int),
    }

    gen = make_batch_generator(cfg, split='val', device=device,
                               batches_per_epoch=num_batches, infinite=False)

    for batch_idx, (xb, yb) in enumerate(gen):
        B, T = xb.shape

        with torch.no_grad():
            _, infos = model(xb, return_info=True, info_level="basic",
                           info_cfg={'store_read_weights': True})

        info = infos[layer_idx]
        if info is None:
            continue

        rw = info.get('read_weights', None)
        if rw is None:
            continue

        for b in range(B):
            for t in range(T):
                token_id = xb[b, t].item()

                # Classify by multiple schemes
                freq_group = classify_token_by_frequency(token_id, tokenizer, None)
                len_group = classify_token_by_length(token_id, tokenizer)
                pos_group = classify_token_by_position(t, T)
                syn_group = classify_token_syntactic(token_id, tokenizer)

                # Check limits
                if (counts['frequency'][freq_group] >= max_per_group and
                    counts['length'][len_group] >= max_per_group and
                    counts['position'][pos_group] >= max_per_group and
                    counts['syntactic'][syn_group] >= max_per_group):
                    continue

                # Collect routing for canonical head 0 (can do others later)
                canon_h = 0
                phys_h = perm[canon_h]
                routing = rw[b, phys_h, t, :].cpu().numpy()

                # Store in each scheme
                if counts['frequency'][freq_group] < max_per_group:
                    schemes['frequency'][freq_group][canon_h].append(routing)
                    counts['frequency'][freq_group] += 1

                if counts['length'][len_group] < max_per_group:
                    schemes['length'][len_group][canon_h].append(routing)
                    counts['length'][len_group] += 1

                if counts['position'][pos_group] < max_per_group:
                    schemes['position'][pos_group][canon_h].append(routing)
                    counts['position'][pos_group] += 1

                if counts['syntactic'][syn_group] < max_per_group:
                    schemes['syntactic'][syn_group][canon_h].append(routing)
                    counts['syntactic'][syn_group] += 1

        if (batch_idx + 1) % 5 == 0:
            print(f"  Batch {batch_idx + 1}/{num_batches}")
            print(f"    Frequency: {dict(counts['frequency'])}")
            print(f"    Syntactic: {dict(counts['syntactic'])}")

    # Convert to numpy
    for scheme in schemes:
        for group in schemes[scheme]:
            for canon_h in schemes[scheme][group]:
                schemes[scheme][group][canon_h] = np.array(schemes[scheme][group][canon_h])

    return schemes, layer_idx

schemes_data, layer_idx = collect_states_multiple_schemes(
    model, cfg, tokenizer, DEVICE, num_batches=25, layer_idx=6, max_per_group=300
)

# ============================
# Compare Clustering Quality Across Schemes
# ============================

def test_clustering_quality(scheme_data, scheme_name, canon_h=0):
    """
    Run t-SNE and compute clustering metrics for a grouping scheme.
    """

    # Collect all states
    all_states = []
    all_groups = []

    for group in sorted(scheme_data.keys()):
        if canon_h in scheme_data[group]:
            states = scheme_data[group][canon_h]
            all_states.append(states)
            all_groups.extend([group] * len(states))

    if len(all_states) == 0:
        return None

    all_states = np.vstack(all_states)

    print(f"\n{scheme_name.upper()} Scheme:")
    print(f"  Samples: {len(all_states)}, Groups: {len(set(all_groups))}")

    # PCA
    n_comp = min(20, all_states.shape[1] - 1)
    pca = PCA(n_components=n_comp)
    states_pca = pca.fit_transform(all_states)

    # t-SNE
    perplexity = min(30, len(states_pca) // 4)
    tsne = TSNE(n_components=2, perplexity=perplexity, max_iter=1000, random_state=42)
    coords = tsne.fit_transform(states_pca)

    # Clustering metrics
    unique_groups = sorted(set(all_groups))
    group_to_int = {g: i for i, g in enumerate(unique_groups)}
    labels = np.array([group_to_int[g] for g in all_groups])

    sil_score = silhouette_score(coords, labels) if len(unique_groups) > 1 else 0

    # Separation ratio
    from scipy.spatial.distance import pdist, squareform
    distances = squareform(pdist(coords))

    within = []
    between = []
    for i in range(len(all_groups)):
        for j in range(i + 1, len(all_groups)):
            if all_groups[i] == all_groups[j]:
                within.append(distances[i, j])
            else:
                between.append(distances[i, j])

    within_mean = np.mean(within) if within else 0
    between_mean = np.mean(between) if between else 0
    sep_ratio = between_mean / (within_mean + 1e-9)

    print(f"  Silhouette: {sil_score:.3f}")
    print(f"  Separation ratio: {sep_ratio:.2f}")

    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))

    color_palette = sns.color_palette("husl", len(unique_groups))
    group_to_color = {g: color_palette[i] for i, g in enumerate(unique_groups)}

    for group in unique_groups:
        mask = np.array([g == group for g in all_groups])
        ax.scatter(coords[mask, 0], coords[mask, 1],
                  c=[group_to_color[group]], label=group,
                  alpha=0.6, s=25, edgecolors='none')

    ax.set_xlabel('t-SNE 1', fontsize=11)
    ax.set_ylabel('t-SNE 2', fontsize=11)
    ax.set_title(f'{scheme_name.title()} Clustering (Layer {layer_idx})\n'
                f'Silhouette: {sil_score:.3f}, Sep Ratio: {sep_ratio:.2f}',
                fontsize=13, fontweight='bold')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax.grid(True, alpha=0.2)
    plt.tight_layout()
    plt.show()

    return sil_score, sep_ratio

# Test each scheme
results = {}
for scheme_name, scheme_data in schemes_data.items():
    result = test_clustering_quality(scheme_data, scheme_name, canon_h=0)
    if result:
        results[scheme_name] = result

# Summary
print("\n" + "="*70)
print("CLUSTERING QUALITY COMPARISON")
print("="*70)
print(f"{'Scheme':<15} {'Silhouette':<15} {'Separation Ratio':<20}")
print("-"*70)
for scheme_name in ['frequency', 'length', 'position', 'syntactic']:
    if scheme_name in results:
        sil, sep = results[scheme_name]
        print(f"{scheme_name:<15} {sil:>14.3f} {sep:>19.2f}")

print("\nInterpretation:")
print("  Best clustering scheme reveals the PRIMARY organizing principle")
print("  Positive silhouette + high separation = strong organization")

In [ ]:

#@title What Information Do Slots Actually Encode?

import numpy as np
import torch
from scipy.stats import entropy

# ============================
# Mutual Information Analysis
# ============================

def compute_slot_mutual_information(
    model, cfg, tokenizer, token_to_group, device, num_batches=30, layer_idx=6
):
    """
    Compute mutual information between slot selection and various token properties.
    Higher MI = slot selection is more predictive of that property.
    """

    # Establish canonicalization
    gen = make_batch_generator(cfg, split='val', device=device, batches_per_epoch=1, infinite=False)
    xb_canon, _ = next(gen)
    canon_perm, _, valid_layers = compute_canonical_permutations(model, xb_canon, device)

    if layer_idx not in valid_layers:
        layer_idx = valid_layers[len(valid_layers) // 2]

    perm = canon_perm[layer_idx]
    H = cfg.num_heads
    K = cfg.num_slots

    print(f"Computing mutual information (Layer {layer_idx})...")

    # Joint distributions: P(slot, property)
    # We'll track for canonical head 0
    canon_h = 0

    joint_semantic = np.zeros((K, len(SEMANTIC_GROUPS)))
    joint_frequency = np.zeros((K, 4))  # 4 frequency bins
    joint_syntactic = np.zeros((K, 5))  # 5 syntactic categories
    joint_position = np.zeros((K, 5))   # 5 position bins

    semantic_groups_list = sorted([g for g in SEMANTIC_GROUPS.keys()])

    gen = make_batch_generator(cfg, split='val', device=device,
                               batches_per_epoch=num_batches, infinite=False)

    for batch_idx, (xb, yb) in enumerate(gen):
        B, T = xb.shape

        with torch.no_grad():
            _, infos = model(xb, return_info=True, info_level="basic",
                           info_cfg={'store_read_weights': True})

        info = infos[layer_idx]
        if info is None:
            continue

        rw = info.get('read_weights', None)
        if rw is None:
            continue

        phys_h = perm[canon_h]

        for b in range(B):
            for t in range(T):
                token_id = xb[b, t].item()

                # Get selected slot (argmax)
                slot = rw[b, phys_h, t, :].argmax().item()

                # Semantic group
                sem_group = token_to_group.get(token_id, 'other')
                if sem_group in semantic_groups_list:
                    sem_idx = semantic_groups_list.index(sem_group)
                    joint_semantic[slot, sem_idx] += 1

                # Frequency
                freq_group = classify_token_by_frequency(token_id, tokenizer, None)
                freq_map = {'very_common': 0, 'common': 1, 'uncommon': 2, 'rare': 3}
                if freq_group in freq_map:
                    joint_frequency[slot, freq_map[freq_group]] += 1

                # Syntactic
                syn_group = classify_token_syntactic(token_id, tokenizer)
                syn_map = {'function': 0, 'pronoun': 1, 'modal': 2, 'inflected': 3, 'content': 4}
                if syn_group in syn_map:
                    joint_syntactic[slot, syn_map[syn_group]] += 1

                # Position
                pos_group = classify_token_by_position(t, T)
                pos_map = {'early': 0, 'early_mid': 1, 'middle': 2, 'late_mid': 3, 'late': 4}
                if pos_group in pos_map:
                    joint_position[slot, pos_map[pos_group]] += 1

        if (batch_idx + 1) % 10 == 0:
            print(f"  Batch {batch_idx + 1}/{num_batches}")

    # Compute mutual information: MI(X;Y) = H(X) + H(Y) - H(X,Y)
    def mutual_info(joint):
        """Compute MI from joint distribution."""
        joint = joint / joint.sum()  # Normalize

        marginal_x = joint.sum(axis=1)  # P(slot)
        marginal_y = joint.sum(axis=0)  # P(property)

        H_X = entropy(marginal_x)
        H_Y = entropy(marginal_y)
        H_XY = entropy(joint.flatten())

        MI = H_X + H_Y - H_XY

        # Normalized MI
        MI_norm = MI / min(H_X, H_Y) if min(H_X, H_Y) > 0 else 0

        return MI, MI_norm, H_X, H_Y

    mi_semantic, mi_semantic_norm, H_slot_sem, H_sem = mutual_info(joint_semantic)
    mi_frequency, mi_frequency_norm, H_slot_freq, H_freq = mutual_info(joint_frequency)
    mi_syntactic, mi_syntactic_norm, H_slot_syn, H_syn = mutual_info(joint_syntactic)
    mi_position, mi_position_norm, H_slot_pos, H_pos = mutual_info(joint_position)

    return {
        'semantic': (mi_semantic, mi_semantic_norm),
        'frequency': (mi_frequency, mi_frequency_norm),
        'syntactic': (mi_syntactic, mi_syntactic_norm),
        'position': (mi_position, mi_position_norm),
    }

mi_results = compute_slot_mutual_information(
    model, cfg, tokenizer, token_to_group, DEVICE, num_batches=30, layer_idx=6
)

print("\n" + "="*70)
print("MUTUAL INFORMATION: Slot Selection vs Token Properties")
print("="*70)
print(f"{'Property':<15} {'MI (bits)':<15} {'Normalized MI':<15}")
print("-"*70)

for prop_name in ['semantic', 'frequency', 'syntactic', 'position']:
    mi, mi_norm = mi_results[prop_name]
    print(f"{prop_name:<15} {mi:>14.4f} {mi_norm:>14.4f}")

print("\nInterpretation:")
print("  Higher MI = slot selection is more informative about this property")
print("  Normalized MI ∈ [0,1]: 0 = independent, 1 = deterministic")
print("  This reveals what slots ACTUALLY encode!")

In [ ]:

#@title Experiment: Identify and Characterize the Discrete Routing Modes

import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import seaborn as sns
from collections import defaultdict, Counter

# ============================
# Step 1: Re-collect t-SNE embeddings with metadata
# ============================

def collect_routing_with_full_metadata(
    model, cfg, tokenizer, token_to_group, device,
    num_batches=30, layer_idx=6, max_samples_per_group=400
):
    """
    Collect routing states WITH full token metadata for mode analysis.
    """

    # Establish canonicalization
    gen = make_batch_generator(cfg, split='val', device=device, batches_per_epoch=1, infinite=False)
    xb_canon, _ = next(gen)
    canon_perm, _, valid_layers = compute_canonical_permutations(model, xb_canon, device)

    if layer_idx not in valid_layers:
        layer_idx = valid_layers[len(valid_layers) // 2]

    perm = canon_perm[layer_idx]
    H = cfg.num_heads
    K = cfg.num_slots

    print(f"Collecting routing with metadata (Layer {layer_idx})...")

    # Storage: canonical_head -> list of (routing, metadata)
    data_by_head = defaultdict(list)
    group_counts = defaultdict(int)

    gen = make_batch_generator(cfg, split='val', device=device,
                               batches_per_epoch=num_batches, infinite=False)

    for batch_idx, (xb, yb) in enumerate(gen):
        B, T = xb.shape

        with torch.no_grad():
            _, infos = model(xb, return_info=True, info_level="basic",
                           info_cfg={'store_read_weights': True})

        info = infos[layer_idx]
        if info is None:
            continue

        rw = info.get('read_weights', None)
        if rw is None:
            continue

        for b in range(B):
            for t in range(T):
                token_id = xb[b, t].item()
                group = token_to_group.get(token_id, 'other')

                # Limit samples per group for balance
                if group != 'other' and group_counts[group] >= max_samples_per_group:
                    continue

                token_str = tokenizer.decode([token_id]).strip()

                # Classify additional properties
                syntactic = classify_token_syntactic(token_id, tokenizer)
                freq_class = classify_token_by_frequency(token_id, tokenizer, None)
                pos_class = classify_token_by_position(t, T)

                # For each canonical head
                for canon_h in range(H):
                    phys_h = perm[canon_h]
                    routing = rw[b, phys_h, t, :].cpu().numpy()

                    # Store routing with metadata
                    metadata = {
                        'token_id': token_id,
                        'token_str': token_str,
                        'semantic_group': group,
                        'syntactic': syntactic,
                        'frequency': freq_class,
                        'position': pos_class,
                        'seq_position': t,
                        'seq_length': T,
                        'relative_pos': t / T,
                        'batch_idx': batch_idx,
                    }

                    data_by_head[canon_h].append((routing, metadata))

                if group != 'other':
                    group_counts[group] += 1

        if (batch_idx + 1) % 10 == 0:
            print(f"  Batch {batch_idx + 1}/{num_batches}")

    print(f"\nCollected {sum(len(data_by_head[h]) for h in data_by_head)} samples")
    return data_by_head, layer_idx

# Collect data
print("Collecting routing data with full metadata...")
data_by_head, layer_idx = collect_routing_with_full_metadata(
    model, cfg, tokenizer, token_to_group, DEVICE,
    num_batches=30, layer_idx=6, max_samples_per_group=400
)

# ============================
# Step 2: Compute t-SNE and Cluster
# ============================

def cluster_routing_modes(data_by_head, canon_h, n_clusters_range=range(4, 12)):
    """
    Perform t-SNE and cluster to identify routing modes.
    Tests multiple cluster counts to find optimal.
    """

    if canon_h not in data_by_head or len(data_by_head[canon_h]) == 0:
        return None

    # Extract routing vectors and metadata
    routings = []
    metadata_list = []

    for routing, metadata in data_by_head[canon_h]:
        routings.append(routing)
        metadata_list.append(metadata)

    routings = np.array(routings)

    print(f"\nCanonical Head {canon_h}:")
    print(f"  Samples: {len(routings)}")

    # PCA first
    n_pca = min(20, routings.shape[1] - 1)
    pca = PCA(n_components=n_pca)
    routings_pca = pca.fit_transform(routings)
    print(f"  PCA variance: {pca.explained_variance_ratio_.sum():.2%}")

    # t-SNE
    print("  Running t-SNE...")
    perplexity = min(30, len(routings) // 4)
    tsne = TSNE(n_components=2, perplexity=perplexity, max_iter=1000, random_state=42)
    coords = tsne.fit_transform(routings_pca)

    # Try different cluster counts
    print("  Testing cluster counts...")
    best_n = None
    best_score = -1
    scores = {}

    for n in n_clusters_range:
        kmeans = KMeans(n_clusters=n, random_state=42, n_init=10)
        labels = kmeans.fit_predict(coords)
        score = silhouette_score(coords, labels)
        scores[n] = score

        if score > best_score:
            best_score = score
            best_n = n

    print(f"  Best cluster count: {best_n} (silhouette: {best_score:.3f})")

    # Cluster with best n
    kmeans = KMeans(n_clusters=best_n, random_state=42, n_init=10)
    mode_labels = kmeans.fit_predict(coords)

    # Also try agglomerative for comparison
    agg = AgglomerativeClustering(n_clusters=best_n)
    mode_labels_agg = agg.fit_predict(coords)

    return {
        'coords': coords,
        'metadata': metadata_list,
        'routings': routings,
        'routings_pca': routings_pca,
        'mode_labels': mode_labels,
        'mode_labels_agg': mode_labels_agg,
        'n_modes': best_n,
        'silhouette': best_score,
        'cluster_scores': scores,
        'kmeans': kmeans,
    }

# Cluster all heads
print("\n" + "="*70)
print("CLUSTERING ROUTING MODES")
print("="*70)

clustered_heads = {}
for canon_h in range(cfg.num_heads):
    result = cluster_routing_modes(data_by_head, canon_h, n_clusters_range=range(5, 10))
    if result:
        clustered_heads[canon_h] = result

# ============================
# Step 3: Visualize Modes
# ============================

def plot_routing_modes(result, canon_h, layer_idx):
    """
    Plot t-SNE colored by discovered modes.
    """

    coords = result['coords']
    mode_labels = result['mode_labels']
    n_modes = result['n_modes']
    metadata = result['metadata']

    fig, axes = plt.subplots(1, 2, figsize=(16, 7))

    # Left: Modes
    ax = axes[0]
    colors = sns.color_palette("husl", n_modes)

    for mode_id in range(n_modes):
        mask = mode_labels == mode_id
        ax.scatter(coords[mask, 0], coords[mask, 1],
                  c=[colors[mode_id]], label=f'Mode {mode_id}',
                  alpha=0.6, s=25, edgecolors='none')

    # Plot cluster centers
    centers = result['kmeans'].cluster_centers_
    ax.scatter(centers[:, 0], centers[:, 1],
              c='black', marker='X', s=200, edgecolors='white', linewidths=2,
              label='Centers', zorder=10)

    ax.set_xlabel('t-SNE 1', fontsize=12)
    ax.set_ylabel('t-SNE 2', fontsize=12)
    ax.set_title(f'Routing Modes (Layer {layer_idx}, Canon Head {canon_h})\n'
                f'{n_modes} modes, silhouette={result["silhouette"]:.3f}',
                fontsize=13, fontweight='bold')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax.grid(True, alpha=0.2)

    # Right: Semantic groups overlay
    ax = axes[1]
    semantic_groups = sorted(set(m['semantic_group'] for m in metadata if m['semantic_group'] != 'other'))
    group_colors = sns.color_palette("Set2", len(semantic_groups))
    group_to_color = {g: group_colors[i] for i, g in enumerate(semantic_groups)}

    for group in semantic_groups:
        mask = np.array([m['semantic_group'] == group for m in metadata])
        ax.scatter(coords[mask, 0], coords[mask, 1],
                  c=[group_to_color[group]], label=group,
                  alpha=0.5, s=20, edgecolors='none')

    ax.set_xlabel('t-SNE 1', fontsize=12)
    ax.set_ylabel('t-SNE 2', fontsize=12)
    ax.set_title(f'Same Space: Semantic Groups Overlay',
                fontsize=13, fontweight='bold')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax.grid(True, alpha=0.2)

    plt.tight_layout()
    plt.show()

# Plot for key heads
for canon_h in [0, 1, 3, 7]:  # Different heads including star pattern head 3
    if canon_h in clustered_heads:
        plot_routing_modes(clustered_heads[canon_h], canon_h, layer_idx)

# ============================
# Step 4: Characterize Each Mode
# ============================

def characterize_routing_modes(result, canon_h):
    """
    For each mode, analyze what tokens/properties it contains.
    """

    mode_labels = result['mode_labels']
    metadata = result['metadata']
    n_modes = result['n_modes']

    print(f"\n{'='*70}")
    print(f"MODE CHARACTERIZATION: Canonical Head {canon_h}")
    print(f"{'='*70}")

    mode_stats = {}

    for mode_id in range(n_modes):
        mask = mode_labels == mode_id
        mode_metadata = [m for m, include in zip(metadata, mask) if include]

        n_samples = len(mode_metadata)

        # Gather statistics
        semantic_dist = Counter(m['semantic_group'] for m in mode_metadata)
        syntactic_dist = Counter(m['syntactic'] for m in mode_metadata)
        position_dist = Counter(m['position'] for m in mode_metadata)
        freq_dist = Counter(m['frequency'] for m in mode_metadata)

        # Relative position stats
        rel_positions = [m['relative_pos'] for m in mode_metadata]
        mean_rel_pos = np.mean(rel_positions)
        std_rel_pos = np.std(rel_positions)

        # Sample tokens
        sample_tokens = [m['token_str'] for m in mode_metadata[:30]]

        mode_stats[mode_id] = {
            'n_samples': n_samples,
            'semantic': semantic_dist,
            'syntactic': syntactic_dist,
            'position': position_dist,
            'frequency': freq_dist,
            'mean_rel_pos': mean_rel_pos,
            'std_rel_pos': std_rel_pos,
            'sample_tokens': sample_tokens,
        }

        # Print summary
        print(f"\n{'─'*70}")
        print(f"MODE {mode_id}: {n_samples} samples ({n_samples/len(metadata)*100:.1f}%)")
        print(f"{'─'*70}")

        # Top semantic groups
        print(f"  Semantic (top 5):")
        for group, count in semantic_dist.most_common(5):
            pct = count / n_samples * 100
            print(f"    {group:>15s}: {count:>4d} ({pct:>5.1f}%)")

        # Top syntactic categories
        print(f"  Syntactic (top 3):")
        for cat, count in syntactic_dist.most_common(3):
            pct = count / n_samples * 100
            print(f"    {cat:>15s}: {count:>4d} ({pct:>5.1f}%)")

        # Position bias
        print(f"  Position:")
        for pos, count in position_dist.most_common():
            pct = count / n_samples * 100
            print(f"    {pos:>15s}: {count:>4d} ({pct:>5.1f}%)")

        print(f"  Avg relative position: {mean_rel_pos:.2f} ± {std_rel_pos:.2f}")

        # Sample tokens
        print(f"  Sample tokens: {', '.join(sample_tokens[:15])}...")

    return mode_stats

# Characterize modes for interesting heads
mode_characterizations = {}
for canon_h in [0, 1, 3, 7]:
    if canon_h in clustered_heads:
        mode_characterizations[canon_h] = characterize_routing_modes(
            clustered_heads[canon_h], canon_h
        )

# ============================
# Step 5: Mode Comparison Matrix
# ============================

def compare_modes_across_heads(clustered_heads, mode_characterizations):
    """
    Compare what each mode represents across different heads.
    """

    print(f"\n{'='*70}")
    print("CROSS-HEAD MODE COMPARISON")
    print(f"{'='*70}")

    # For each head, find the most distinctive characteristic of each mode
    head_mode_signatures = {}

    for canon_h in sorted(clustered_heads.keys()):
        stats = mode_characterizations.get(canon_h, {})
        n_modes = clustered_heads[canon_h]['n_modes']

        signatures = []

        for mode_id in range(n_modes):
            if mode_id not in stats:
                continue

            # Find most distinctive feature
            semantic_top = stats[mode_id]['semantic'].most_common(1)[0] if stats[mode_id]['semantic'] else ('none', 0)
            syntactic_top = stats[mode_id]['syntactic'].most_common(1)[0] if stats[mode_id]['syntactic'] else ('none', 0)
            position_top = stats[mode_id]['position'].most_common(1)[0] if stats[mode_id]['position'] else ('none', 0)

            signature = {
                'semantic': semantic_top[0],
                'semantic_pct': semantic_top[1] / stats[mode_id]['n_samples'] * 100,
                'syntactic': syntactic_top[0],
                'syntactic_pct': syntactic_top[1] / stats[mode_id]['n_samples'] * 100,
                'position': position_top[0],
                'position_pct': position_top[1] / stats[mode_id]['n_samples'] * 100,
                'mean_pos': stats[mode_id]['mean_rel_pos'],
            }

            signatures.append(signature)

        head_mode_signatures[canon_h] = signatures

    # Print comparison table
    print(f"\n{'Head':<6} {'Mode':<6} {'Primary Characteristic':<30} {'Strength':<10}")
    print("─" * 70)

    for canon_h in sorted(head_mode_signatures.keys()):
        for mode_id, sig in enumerate(head_mode_signatures[canon_h]):
            # Determine primary characteristic
            if sig['syntactic_pct'] > 40:
                primary = f"Syntactic: {sig['syntactic']}"
                strength = f"{sig['syntactic_pct']:.1f}%"
            elif sig['semantic_pct'] > 35:
                primary = f"Semantic: {sig['semantic']}"
                strength = f"{sig['semantic_pct']:.1f}%"
            elif sig['position_pct'] > 40:
                primary = f"Position: {sig['position']}"
                strength = f"{sig['position_pct']:.1f}%"
            else:
                primary = f"Mixed (pos={sig['mean_pos']:.2f})"
                strength = "—"

            print(f"H{canon_h:<5} M{mode_id:<5} {primary:<30} {strength:<10}")

compare_modes_across_heads(clustered_heads, mode_characterizations)

# ============================
# Step 6: Mode Routing Patterns
# ============================

def analyze_mode_routing_patterns(result, canon_h, mode_characterizations):
    """
    For each mode, what does the ROUTING look like?
    Which slots are preferred?
    """

    mode_labels = result['mode_labels']
    routings = result['routings']
    n_modes = result['n_modes']
    K = routings.shape[1]

    print(f"\n{'='*70}")
    print(f"MODE ROUTING PATTERNS: Canonical Head {canon_h}")
    print(f"{'='*70}")

    # Average routing per mode
    mode_avg_routing = np.zeros((n_modes, K))

    for mode_id in range(n_modes):
        mask = mode_labels == mode_id
        mode_avg_routing[mode_id] = routings[mask].mean(axis=0)

    # Plot heatmap
    fig, ax = plt.subplots(figsize=(12, 6))

    im = ax.imshow(mode_avg_routing, aspect='auto', cmap='YlOrRd', interpolation='nearest')

    ax.set_xlabel('Slot', fontsize=12, fontweight='bold')
    ax.set_ylabel('Mode', fontsize=12, fontweight='bold')
    ax.set_title(f'Average Routing Pattern per Mode (Canon Head {canon_h})',
                fontsize=13, fontweight='bold')
    ax.set_yticks(range(n_modes))
    ax.set_yticklabels([f'Mode {i}' for i in range(n_modes)])

    plt.colorbar(im, ax=ax, label='Routing Weight')
    plt.tight_layout()
    plt.show()

    # Print top slots per mode
    print(f"\nTop Slots per Mode:")
    for mode_id in range(n_modes):
        top_slots = np.argsort(-mode_avg_routing[mode_id])[:5]
        top_weights = mode_avg_routing[mode_id][top_slots]

        stats = mode_characterizations[mode_id]
        primary_feature = stats['syntactic'].most_common(1)[0][0] if stats['syntactic'] else 'unknown'

        print(f"  Mode {mode_id} ({primary_feature}):")
        for slot, weight in zip(top_slots, top_weights):
            print(f"    Slot {slot:>2d}: {weight:.3f}")

# Analyze routing patterns
for canon_h in [0, 3, 7]:
    if canon_h in clustered_heads and canon_h in mode_characterizations:
        analyze_mode_routing_patterns(
            clustered_heads[canon_h],
            canon_h,
            mode_characterizations[canon_h]
        )

print("\n✓ Mode analysis complete!")

In [ ]:

#@title Canonical Head Alignment (FOUNDATION / OFFICIAL): heads across layers via read-weights features + Hungarian
# This is the "boringly stable" base cell for canonicalizing HEADS across LAYERS.
# Canonical definition (OFFICIAL):
#   - Choose an anchor layer L0 (default: first valid layer that has read_weights)
#   - For each layer l, store a permutation:
#         canon_perm_to_layer[l][i] = head_index_in_layer_l for canonical_head_i
#   - Any per-head tensor/vector x_h from layer l can be canonicalized by:
#         x_canon[i] = x_layer[ canon_perm_to_layer[l][i] ]
#
# Minimal assumptions:
#   - You already have: model, xb (token batch)
#   - model(xb, return_info=True, info_level="basic", info_cfg=...) returns infos with read_weights per layer
#
# Dependencies:
#   - SciPy optional (Hungarian). If missing, deterministic greedy fallback is used.

import numpy as np
import torch
import torch.nn.functional as F

# ----------------------------
# Capture policy (read_weights only; OOM-friendly)
# ----------------------------
ASA_CANON_HEAD_INFO_CFG = dict(
    store_read_weights=True,
    store_read_logits=False,
    store_write_logits=False,
    store_slot_state_norm=False,
    detach_to_cpu=False,    # keep on GPU for reduction; we move reduced tensors to CPU
    time_stride=1,
    batch_stride=1,
)

# ----------------------------
# Small math helpers
# ----------------------------
def _safe_log(x: torch.Tensor, eps: float = 1e-9) -> torch.Tensor:
    return torch.log(x.clamp_min(eps))

def cosine_sim(a: torch.Tensor, b: torch.Tensor, eps: float = 1e-9) -> torch.Tensor:
    # cosine over last dim
    an = a.norm(dim=-1).clamp_min(eps)
    bn = b.norm(dim=-1).clamp_min(eps)
    return (a * b).sum(dim=-1) / (an * bn)

# ----------------------------
# Hungarian aligner (max cosine) with deterministic greedy fallback
# ----------------------------
def hungarian_align(A_hd: np.ndarray, B_hd: np.ndarray, eps: float = 1e-9):
    """
    Align rows of B to rows of A by maximizing cosine similarity.
    A,B: [H,D] numpy arrays
    Returns:
      perm: array length H where perm[i] is B-row matched to A-row i   (A-head -> B-head)
      q   : mean matched cosine similarity
      diag: mean diagonal cosine similarity (identity match quality)
    """
    A = A_hd.astype(np.float64, copy=False)
    B = B_hd.astype(np.float64, copy=False)

    A = A / (np.linalg.norm(A, axis=1, keepdims=True) + eps)
    B = B / (np.linalg.norm(B, axis=1, keepdims=True) + eps)

    S = A @ B.T  # [H,H]
    H = S.shape[0]
    diag = float(np.diag(S).mean())

    # Hungarian if available
    try:
        from scipy.optimize import linear_sum_assignment
        row, col = linear_sum_assignment(-S)  # maximize
        perm = np.empty((H,), dtype=np.int64)
        perm[row] = col
        q = float(S[row, col].mean())
        return perm, q, diag
    except Exception:
        # deterministic greedy fallback
        used = np.zeros((H,), dtype=bool)
        perm = np.full((H,), -1, dtype=np.int64)
        vals = []
        for i in range(H):
            j = int(np.argmax(np.where(used, -1e18, S[i])))
            used[j] = True
            perm[i] = j
            vals.append(S[i, j])
        q = float(np.mean(vals)) if vals else float("nan")
        return perm, q, diag

# ----------------------------
# Canonical permutation application helper (canonical -> layer)
# ----------------------------
def apply_perm_canon_to_layer(x_h: torch.Tensor, perm_canon_to_layer: np.ndarray) -> torch.Tensor:
    """
    x_h: torch tensor with head axis at dim=0: [H, ...]
    perm_canon_to_layer maps canonical head i -> layer head perm[i]
    output[i] = x_h[perm[i]]
    """
    idx = torch.as_tensor(perm_canon_to_layer, device=x_h.device, dtype=torch.long)
    return x_h.index_select(0, idx)

# ----------------------------
# Head features from read_weights (OFFICIAL CANON FEATURES)
# ----------------------------
@torch.no_grad()
def build_head_features_from_rw(rw_bhtk: torch.Tensor, *, max_lag: int = 16) -> np.ndarray:
    """
    rw_bhtk: [B,H,T,K] probabilities (or unnormalized nonneg weights over slots).
    Returns [H,D] numpy features capturing routing + inertia shape:
      - inertia_lag1 : mean cosine(p_t, p_{t+1})
      - inertia_mean : mean cosine over lags 1..L
      - inertia_slope: standardized slope of cosine vs lag
      - entropy_mean : mean entropy of p_t over slots
      - eff_slots    : exp(entropy_mean)
      - top4_mass    : mean sum of top4 slot probs
    Notes:
      - Uses p = mean_B rw, then normalizes over K.
      - Normalizes feature vectors to unit norm for cosine matching stability.
    """
    p = rw_bhtk.float().mean(dim=0)  # [H,T,K]
    p = p / p.sum(dim=-1, keepdim=True).clamp_min(1e-9)

    ent = -(p * _safe_log(p)).sum(dim=-1)       # [H,T]
    ent_mean = ent.mean(dim=-1)                 # [H]
    eff_slots = torch.exp(ent_mean)             # [H]

    K = p.shape[-1]
    topk = min(4, K)
    top4 = torch.topk(p, k=topk, dim=-1).values.sum(dim=-1)  # [H,T]
    top4_mean = top4.mean(dim=-1)                            # [H]

    T = p.shape[1]
    L = min(int(max_lag), max(1, T - 1))
    cos_lags = []
    for lag in range(1, L + 1):
        a = p[:, :-lag, :]   # [H, T-lag, K]
        b = p[:, lag:, :]    # [H, T-lag, K]
        c = cosine_sim(a, b).mean(dim=-1)  # [H]
        cos_lags.append(c)
    cos_lags = torch.stack(cos_lags, dim=-1)   # [H,L]

    inertia_lag1 = cos_lags[:, 0]
    inertia_mean = cos_lags.mean(dim=-1)

    # standardized slope of cosine vs lag (robust to scale)
    xs = torch.arange(1, L + 1, device=p.device, dtype=torch.float32)
    xs = (xs - xs.mean()) / xs.std().clamp_min(1e-9)
    ys = (cos_lags - cos_lags.mean(dim=-1, keepdim=True)) / cos_lags.std(dim=-1, keepdim=True).clamp_min(1e-9)
    inertia_slope = (ys * xs).mean(dim=-1)

    feats = torch.stack([inertia_lag1, inertia_mean, inertia_slope, ent_mean, eff_slots, top4_mean], dim=-1)  # [H,6]
    feats = feats / feats.norm(dim=-1, keepdim=True).clamp_min(1e-9)
    return feats.detach().cpu().numpy().astype(np.float64)

# ----------------------------
# Get read_weights infos from model
# ----------------------------
@torch.no_grad()
def get_infos_for_head_canon(model, xb: torch.Tensor, *, batch_size: int = 8, info_cfg: dict = None):
    """
    Runs model forward and returns infos capturing read_weights.
    """
    if info_cfg is None:
        info_cfg = ASA_CANON_HEAD_INFO_CFG
    x = xb[:batch_size]
    logits, infos = model(
        x,
        return_info=True,
        info_level="basic",       # read_weights available in "basic" per your ASA
        info_cfg=info_cfg,
        attention_mask=None,
    )
    if not infos:
        raise RuntimeError("Model returned empty infos; check ASA plumbing / forward passthrough.")
    return infos

# ----------------------------
# Main: canonicalize heads across layers (compose Hungarian permutations)
# ----------------------------
def canonicalize_heads_across_layers(
    *,
    model=None,
    xb: torch.Tensor=None,
    infos=None,
    batch_size: int = 8,
    max_lag: int = 16,
    anchor_layer=None,
    info_cfg: dict = None,
):
    """
    Provide either:
      - infos (precomputed list of per-layer info dicts), OR
      - (model, xb) to compute infos internally

    Returns canon dict:
      canon = {
        "valid_layers": [...],
        "anchor_layer": L0,
        "H": num_heads,
        "K": num_slots,
        "T": seq_len,
        "head_features_by_layer": {l: [H,D]},
        "canon_perm_to_layer":    {l: array[H] mapping canonical -> layer head},
        "align_df":               pandas-like list of dicts with diag/best/improvement,
        "align_quality":          {(la,lb): best_cos},
        "feature_dim":            D,
      }
    """
    if infos is None:
        if model is None or xb is None:
            raise ValueError("Provide either infos=... or (model=..., xb=...)")
        infos = get_infos_for_head_canon(model, xb, batch_size=batch_size, info_cfg=info_cfg)

    # collect valid layers with read_weights
    rw_by_layer = {}
    valid_layers = []
    for l, info in enumerate(infos):
        rw = None if (info is None) else info.get("read_weights", None)
        if rw is None or (not torch.is_tensor(rw)) or rw.dim() != 4:
            continue
        rw_by_layer[l] = rw
        valid_layers.append(l)

    if len(valid_layers) < 2:
        raise RuntimeError("Need at least 2 layers with read_weights captured for canonicalization.")

    L0 = valid_layers[0] if anchor_layer is None else int(anchor_layer)
    if L0 not in valid_layers:
        raise ValueError(f"anchor_layer={L0} is not in valid_layers={valid_layers}")

    B, H, T, K = rw_by_layer[L0].shape

    # head features per valid layer
    head_features_by_layer = {}
    for l in valid_layers:
        head_features_by_layer[l] = build_head_features_from_rw(rw_by_layer[l], max_lag=max_lag)  # [H,D]

    D = head_features_by_layer[L0].shape[1]

    # canonical permutation: canonical -> layer head index
    canon_perm_to_layer = {L0: np.arange(H, dtype=np.int64)}
    align_quality = {}
    align_rows = []

    # build a forward chain anchored at L0 by walking through valid_layers order
    # If L0 is not the first, we still canonicalize by chaining from L0 outward both directions.
    idx0 = valid_layers.index(L0)

    # forward direction (increasing within valid_layers)
    for i in range(idx0, len(valid_layers) - 1):
        la = valid_layers[i]
        lb = valid_layers[i + 1]
        A = head_features_by_layer[la]
        Bf = head_features_by_layer[lb]

        perm_ab, best, diag = hungarian_align(A, Bf)  # la-head -> lb-head
        align_quality[(la, lb)] = best
        align_rows.append(dict(layer=la, transition=f"{la}→{lb}", diag=float(diag), best=float(best), improvement=float(best - diag)))

        canon_to_la = canon_perm_to_layer[la]
        canon_to_lb = perm_ab[canon_to_la]            # compose: canon->la then la->lb gives canon->lb
        canon_perm_to_layer[lb] = canon_to_lb

    # backward direction (decreasing within valid_layers)
    # Need perm_ba to map lb-head -> la-head; invert perm_ab.
    for i in range(idx0, 0, -1):
        lb = valid_layers[i]
        la = valid_layers[i - 1]

        # compute alignment la->lb (same orientation as above), then invert it
        A = head_features_by_layer[la]
        Bf = head_features_by_layer[lb]
        perm_ab, best, diag = hungarian_align(A, Bf)  # la-head -> lb-head
        align_quality[(la, lb)] = best
        align_rows.append(dict(layer=la, transition=f"{la}→{lb}", diag=float(diag), best=float(best), improvement=float(best - diag)))

        # invert perm_ab to map lb -> la
        perm_ba = np.empty_like(perm_ab)
        perm_ba[perm_ab] = np.arange(H, dtype=np.int64)  # lb-head -> la-head

        canon_to_lb = canon_perm_to_layer[lb]
        canon_to_la = perm_ba[canon_to_lb]               # canon->lb then lb->la gives canon->la
        canon_perm_to_layer[la] = canon_to_la

    # sanity checks
    for l in valid_layers:
        p = canon_perm_to_layer[l]
        if p.shape != (H,):
            raise AssertionError(f"perm shape wrong at layer {l}: {p.shape}")
        if sorted(p.tolist()) != list(range(H)):
            raise AssertionError(f"perm not a permutation at layer {l}: {p}")

    canon = dict(
        valid_layers=valid_layers,
        anchor_layer=L0,
        H=int(H),
        K=int(K),
        T=int(T),
        head_features_by_layer=head_features_by_layer,
        canon_perm_to_layer=canon_perm_to_layer,
        align_quality=align_quality,
        align_rows=align_rows,
        feature_dim=int(D),
    )
    return canon

# ----------------------------
# Example usage (safe to keep; comment out if you want)
# ----------------------------
canon = canonicalize_heads_across_layers(model=model, xb=xb, batch_size=8, max_lag=16, anchor_layer=None)
print(f"Loaded per-head stats: L={len(canon['valid_layers'])} layers, H={canon['H']} heads")
print(f"Canonical head alignment anchor: layer {canon['anchor_layer']}")
try:
    import pandas as pd
    df = pd.DataFrame(canon["align_rows"]).sort_values(["layer"])
    display(df)
    print(f"Average head-alignment quality over adjacent transitions: best={df['best'].mean():.4f} improvement={df['improvement'].mean():.4f}")
except Exception:
    # no pandas/display available
    rows = canon["align_rows"]
    bests = [r["best"] for r in rows] if rows else []
    improvs = [r["improvement"] for r in rows] if rows else []
    if bests:
        print(f"Average head-alignment quality: best={float(np.mean(bests)):.4f} improvement={float(np.mean(improvs)):.4f}")
print("✓ Canonical heads ready: canon['canon_perm_to_layer'][layer] maps canonical -> layer head index.")